<a href="https://colab.research.google.com/github/ananthvanchip/genai-news/blob/main/Whatsapp_Parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What it does?
This notebook parses the info on the GenAI Whatsapp Group and generates sumamaries for the New Channel

# How does it work.
## Thread Creation
1. We extract the HTML from whatsapp web. We do not whatsapp export as "replies" or "quoted messages" are not preserved in the whatsapp export version.
1. We scrape the HTML to get messages along with their authors, quoted messages and we build a Thread. A thread contains messages which in-turn could have replies and which in-turn could have more replies. (Like Reddit)
1. We apply some herusitics while generating Threads too since all replies are not "quoted" in whatsapp. We use LLMs for this.

## Thread Categorisation
1. For each thread generated, we categorise it into the following.
  1. **Short threads** - Threads which have 4 messages or lower and have no URLs in them. These might be random conversations or messages that are not categorised as a part of some big threads.
  1. **Short threads with URLs** - These are also short threads but have some URL shared in them. It could be a news item or a cool link.
  1. **Long Threads** Threads which have more than 4 messages. Need to see the discussion in it.
2. The threads are categorised and sent for the next stage of processing.

## Thread Processing.
1. Each type of thread is processed in a different way.

  ### Short threads
  This is not processed progamattically. All the short threads are displayed to the user to take a manual judgement of what needs to be done.

  ### Short threads with URLs.
  This is usually a Link which is a News or Cool Link. We generate a one line summary of the link and add the Link information. Categorising it as News or Cool Link is manual.

  ### Long threads.
  These are long discussions. We use GPT-4-Turbo to generate a summary from the thread. We also add any links shared in the threads.

## Summary Generation
1. Once all the types of threads are processed, they are merged together to generate the final Summary.
2. The Short threads are displayed as is for manual review.
3. The entire summary can be reviewd, modified and shared wit the group.


# New Features
1. Understanding images - Today only the `alt-text` of the images are used. We need to process the images too to get better summaries. Most short threads are image threads.
2. Improving summary quality - The summary quality can be defintely tuned and improved.


In [2]:
!pip install pandas
!pip install tqdm
from tqdm import tqdm
import pandas as pd
import json
from lxml import etree
import os



## Step 1: Copy data from Whatsapp web.
1. Open Whatsapp web.
2. Load the messages (by scrolling up) to the dates you want to create the summary for.
3. Copy the elements (messages) by insepcting the javascript. Look for the `div` with attribute `role="application"`.
4. Copy all the content (Using Copy Outer HTML).
5. Paste it in this section below.

In [247]:
#@title Paste the html here.
# Example HTML input
html = """
<div tabindex="-1" class="n5hs2j7m oq31bsqd gx1rr48f qh5tioqs" data-tab="8" role="application"><div class="_1jHIY _2OvAm _2zFLj focusable-list-item" tabindex="-1"><div class="UzMP7 n6BPp"><span dir="auto" aria-label="" class="_11JPr" style="min-height: 0px;">SATURDAY</span></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_ED321AF37FC5769413953E52DCCF10B7_12064993617@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Ajay +1 (206) 499-3617" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno ckklvomo"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">A</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d fyja2idx" role=""><span dir="auto" aria-label="Maybe Ajay" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Ajay</span><span class="WJuYU" testid="author" role="button">+1 (206) 499-3617</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[9:31 pm, 24/02/2024] +1 (206) 499-3617: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g9gonq0i _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Priyesh Srivastava" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Priyesh Srivastava</span><span class="WJuYU" testid="author">+91 97730 65092</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">For us we just FTed a VLM</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Can you share how you did that? Which VLM?</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">9:31 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">9:31 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_14FF3F017540B2A280_919916576150@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Bharat Shetty" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/410251715_3586044468316077_4057854922213032016_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRpFpmLshf1oE7y-iIbgpuzBA-R8WZgzsR5fyq7Haharw&amp;oe=65EAD974&amp;_nc_sid=e6ed6c&amp;_nc_cat=104" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d lr20dbn2" role=""><span dir="auto" aria-label="" class="_3FuDI ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Bharat Shetty</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[10:07 pm, 24/02/2024] Bharat Shetty: "><div class="M6sU5"><div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span><a dir="auto" href="https://lightning.ai/lightning-ai/studios/understanding-using-and-finetuning-gemma" title="https://lightning.ai/lightning-ai/studios/understanding-using-and-finetuning-gemma" target="_blank" rel="noopener noreferrer" class="_11JPr selectable-text copyable-text">https://lightning.ai/lightning-ai/studios/understanding-using-and-finetuning-gemma</a>

By Sebastian Raschka

1) Gemma uses a really large vocabulary and consequently embedding weight matrices.

2) Like OLMo and GPT-2, Gemma also uses weight tying.

3) It even uses GeLU (in the form of GeGLU), similar to GPT-2, unlike Llama 2 and other LLMs.

4) Interestingly, Gemma normalizes the embeddings by the square root of the hidden layer dimension.

5) The RMSNorm layers are in the usual location, different from what was hinted at in the technical paper.

6) However, the RMSNorm implementation comes with the unit offset that wasn't mentioned in the technical paper.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5 hnx8ox4h mw4yctpw gj5xqxfh">Edited</span><span class="tvf2evcx oq44ahr5">10:07 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto"><span class="mw4yctpw gj5xqxfh">Edited</span>10:07 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="reaction 👍"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍🏻" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -40px -60px;"></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A95502A9684E26EB7DA_917737887058@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Nirant K +91 77378 87058" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno g9gonq0i"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">N</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author" role="button">+91 77378 87058</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[10:18 pm, 24/02/2024] +91 77378 87058: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="m8ez62k1 _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d lr20dbn2" role=""><span dir="auto" aria-label="" class="_3FuDI _11JPr" style="min-height: 0px;">Bharat Shetty</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">https://lightning.ai/lightning-ai/studios/understanding-using-and-finetuning-gemma

By Sebastian Raschka

1) Gemma uses a really large vocabulary and consequently embedding weight matrices.

2) Like OLMo and GPT-2, Gemma also uses weight tying.

3) It even uses GeLU (in the form of GeGLU), similar to GPT-2, unlike Llama 2 and other LLMs.

4) Interestingly, Gemma normalizes the embeddings by the square root of the hidden layer dimension.

5) The RMSNorm layers are in the usual location, different from what was hinted at in the technical paper.

6) However, the RMSNorm implementation comes with the unit offset that wasn't mentioned in the technical paper.</span></div></div></div><div class="jrEtJ"><div class="kknmh"><div class="ln8gz9je ppled2lx t3g6t33p qnwaluaf" style="background-image: url(&quot;data:image/jpeg;base64,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAMCAgICAgMCAgIDAwMDBAYEBAQEBAgGBgUGCQgKCgkICQkKDA8MCgsOCwkJDRENDg8QEBEQCgwSExIQEw8QEBD/2wBDAQMDAwQDBAgEBAgQCwkLEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBD/wAARCABdAF0DASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwD8sqKKKACiiigAooooAKKKKACiiigAooooA63/AIQH/qLZ/wC2H/2VH/CA/wDUVx/2w/8Asq+gbf4WaZr39qWOi+E/HulXcMLTaXPq1upgvGRSTDIFhUQs3RSZCue+cK1PxB4Z+F3hGDwxJrD+KL+TW9KstSuks7i3i+ziVWMjIXiIkBOAqHbjacudwx9PPIatNc07KPdtpb27X6rp1vseTHM4T92N2+1jwr/hAf8AqLf+QP8A7Kj/AIQH/qLZ/wC2H/2VfQc3gD4Z6X4+T4X6lqXiE6hJ/o41dWhW2S5lXdAv2bYzugDxBj5yEtuGFX5qo3HhX4a6H8NfDPjXX5NfvNS15b1P7PsbuG3VjFc7BKJHhk2KqDBQoxZnUgqFbOU8mnT5ua3u3vrs00mn6cy2+RUcxhO3LfW1tN73a/JnhX/CBf8AUW/8gf8A2VH/AAgP/UV/8gf/AGVfQvibwj8E/CeqaDHqreNrrTvEdpb6xDPDcWlvJYWE4+RXUxzfaJlZX3YMa424LEnFTS/h34VhuvFuj3mh+P8Axbq/h/Vzp8EPh22SCDyVkZPPmlMdwVLiNyI/L428M2SY4llMoy5bK97fhf8AFdjSONjNXT0PBf8AhAf+ot/5A/8Asqk0Xw54HtdefT/HfiPU7KxNr5sU+n2CzyNLvxtKM4AXaGOc9QOK9i+MHw+sfBF9o+oaLY69Y6br1h9rhstdhWO+tJFkaOSKQKBu5UOrbVyki8ZBJ8nihiufiDosNxI0UTmMSMuni+cLubdst2+WZ8Z2o2FLYDELkjz8ZhvYXi1Zo66FXnfNuiXVvDvwlt5Uk0fxR4huLSQOVlvNO+zswEjBflUSD7mwnDHDMy8gBmzv7L+Gn/QyX/8A37b/AONV0S6bbWcml2ug+KIruS60+e6mWHwy8kcSpbrc+RKrofPw5kViFeNPJjlDsPu9J4au/CukXUHhm+8U6UbeTUWit5F+GdnfSqRJLERK88YnBZYoTGi+fkyyBhG6Et56VjU4Cz0n4TPMy6h4r1aGLypCrQ23mN5gRjGpBRQFZwqs2cqpLAMRtPQXHhz9mlJFW1+KHjWaMxIzNJ4ahjIkP3lAFw2QD0bIyOw6VpX1razeHZ47jVvDdjrcH+krHL4KEcV9Zm3WTdBKlsSSBJvG+KH5X3GRwsZrf8LjwFb3moS3PjrQrGa52SXUOofDG3uJba5DyxvbGJonigKLHG5ERwTNhuV4q4FKa88D2cNzdWeta/qMksMqWdpNEsHkO3CPLKrkPtBLYVQGYDO0ZFM8b+KLPxQdA+yw3Cf2Volrpc3nADdJFv3Mu0n5fmGM4PtX2h/w59sRqFlpbftMeJRPfELG3/Cv7oxJlJ3Hmyi68uIYtpBl2ADGJSQ00IerJ/wSLsbfQofEF7+0Z4vhhm3fuE+HV3cXKbSQd0MN08g6ZHHIIxXuV87daLpqKin0S7O/Vt/p2SOKGB5f3l72/U+Ste8dadq3xXj8eQwXaWSahaXWx0XztsQjB4DEZ+Q4+b0qj4g8VWOq+BPCPhe3juPtOgLfC4kZQI286beuwg5PA5yB+NfZmn/8Ef8AT9U/49P2mfEg/ded/pHgC6t+NiPj97dL8+HA2feBDqQCjhYf+HROmrrEWhyftJeLUuZXWPd/wre9NupaLzfmuBcmFQF4LF8B/kzu4qamezqe0pv7bbendxbt84oUcBCPK19myXyTX5Nnxx4+8XWfiqHw2tnDcR/2L4es9Hm85VAMsW/cVwT8nzDGcH2rd1Tx34Z8WWeuaNq+razpVpd+I7rxDbvDbJcLMJV2+XJFvXbIAq7W3lfncEDg19aTf8Ec/Km02FP2jvEUp1KN5EZfBTbYdqbsSk3oEZPQBsZbilH/AAR2tTFPN/w0z4gAt8lg3gSdWYBnU7AbvL/6tjhckgqQCHXOkM9qRc7xTU901vpbo0+vRmTy6nLls9Y7eWz9D44+I3jvQ/E2k+GdF8O2upQ23h60lswb9ld3UyZRshjjIGSo4TO1cqoNecafELnx/poZb0skDzRvZ30VnLDIiyOkomlGxAjKrnOMhSAykhh+kX/DlOT/AKOkvv8Awlf/ALtr5g/bI/Yts/2OPEfw9a++KVt4uh8WvqH2h9U0me0t7eK1NvlHEE0szK4uGB8vawxxycjgxmO+uSc5LV2/BJfodeHw6w8eSL0/zdzzSx8O+JTZ6f4F1+38dSWut239pTLa6xCYNSijtpJ4hb2zfJIylEBwzsrLsCrJIEGRYaX4yuodN1gaL8T4L66vhaR6hbwTTvdOlxNcTxROFXfIoSOXAbcJIpXbOV8vYXwPoaqstr4g+GmnQ6fp+l3Vyn/CW3by3QkgcXVshQkbpS4SaMLmLdCobC3DGCL4f6Mq23ji08VfDC+083Vw0WmTeIbyKQp50sCO0LlLqNNwjkUEiTEm9sRq+zzzoM2LSfETaJrFjdeDviYPDVnD/aEVtDCY4bC/W1EiXE7eQUaNRLKwAWMiKQEMA7VpaXqHjjTWu4fhlafFuz8m6mt777Cs9ui7ZXeGJoLc4iZUk3Mu7bulbaqjlsqzt/Dvh/QdTutftvAE9006PYwzalqU8jjyxK0MAsZdqIBKgLTEbjIQr7opQubq2ueFdKiSO3bw5r7tdXS+ZDLqmYY1k2oq/aEQCB8NLGAvmASt5p3YVAD+kG18SeHr63S6s9d0+eGXOySO5RlbBIOCDg8gj8KpXHjjwra60vh+XWI/tz232sRqGZfK3Fd28DbnIIxnNcz8BWI+E+jfKT/x8+n/AD8SV4R4AUD4l61gAcXX/pQK5MxrTwmWVsfTtemk7PZ3T/yNckjHM5wjU05ouWnSyX+Z9M+HviB4N8VWcmoaLr0MsMMxgcyboSHCqxGJApPDLzjHNa39s6P/ANBaz5H/AD3Xp+deUfsvtt8D6qApb/idSdP+uEHrXr01zHbxtNcMIo1+8zsqgduSTW8Z3hzy0OPCVJV6Maj3ZCdY0kddUsx/22X/ABobWNI/i1S0Gf8Apuv+NY/g/wAYHxZ/bX/EtNsdH1e40s4l3+Z5W35+QMZ3dOcY610O5v8Anm36f41aakro1hONSPNHVFb+2NJ/6Cln6f65f8a/L/8A4LYPbyal8EZGmhELLr58xwXQDOn/ADEDO4Ac4wc+h6V+nWk+INL1z7Z/ZNx9o+wXT2VxtGPLmTG5DnGcZHTivzX/AOCyUWqXPi74EW+ix6k2oTPr8dtHpshS7kkY6eFSEgEmRicKAMkkAcmmNNSV0fnhpN34HvrWe61nx1pOmtHpqQLb2/gy3nmmZ/LV1XcqxhkVQfNZg5IkAOWDvlTeMNJtJIdNtbjTZrK0jlCyL4P09mZzOSB5kg8yVTGkbb5CGUs8YTb8zdnf+H9e0zSdGvfES+M7FXj8u1vZfF1qIMGwUmFQUAg8yJYE+Zz+7j27ZDhVli+H2t+KtUtf+FmaL471eeHUJNPmuhrlv53764lYrbx3Ct5zJNFqEkm2TaWdizRbHkkBnFWPjLw3/wAI5NoeoPbbblfKbZ4M05poVEqBXS6Eiz7vL3uRuHzoqEsruwzbf4hX2jq0OkWOgzRzMZpDeeFtMlYP935WlhkIXaqHau1QS2FySzdjpvwdW402XxJd+BfE39kQWO9vL1yx81rk2YnQhTGGaNzJCcKhYKWGWcbay77wh4Q8IabpR8ceDPGkN5qtu15Ey6jb2ymNZGhI8o28jqRJFJkOVbkDBAEjgH9BHwHa4Hwp0bZHGV/0nq5B/wCPiT2rwv4f7v8AhZOsbsA7bnOP+vgV2vw1/aR/Z28KaMfC+ofH74RWthY82c0PxEsLx5t5LyeZlYdhV2YDAYFQpyCSq8R/wsf9m/QPE2p654X/AGgvhJeC4VvJa++J1hDvL/vGDIlu/ljzAFBDP8rFsZG08uZ0pYrKMRgqfxzSS7aX/wAy8hksuqUpVvswcXbu0v8AI9X/AGYWuP8AhCdV8uOMj+2ZPvMR/wAsIPauc8RfEzXfEfxRi0GN5bDTbK5u9NntUuC8dy8Jc+aw2jqduBzjb15qz8N/jr+yZ4A8Ptpdl+0t8Nt13N9suEm8baZN5czRorKrqyblGwYJUE8njoOGm+JH7NEfiDWfFlr+0N8J31EXFzfabE/xM09YbqeZpMpMRblrdMeUcr5xBdxg7Az5ZjhPrWU18LD+JKNo62Xnf5BkkvqFSn7b4VGSfXW2n4nuPwfa4z418pIyP+Eu1DO5iO0fsayPGHxRfUtZsfC2lQQm1n1C+07UGljbes1qI3/dnIGCW6kHI9Kw/A/7RX7MPhtdb+0ftGfCqFtU1i41LCePNPuN3mBecny9nKn5Pnx/fOeOX1z4ofspw3MuveHf2kPhle6ml9fapDb3nxC021ge4uVUMrSKkrKg2LjCsRk5zjBeKw0q2WVcPH+I42j287/IMnccJUp/WFeKUr9dbPl/Gx6j8CTONM8UeTHGR/wkt395iP4Y/QV8Lf8ABZSG2uPFXwJh1BtOWFz4gEq38jpbuP8AiX5R3QFkDfd3jG3IOVxuH2P4Q+P37JPg22vrfTf2nPhtINRvpNQm8/xnprYkcKCFw6/L8oxkZr4L/wCCsnxX+E/xM1/4P3Hw/wDiz4M8QQ6RF4hkvptMvbTWVtiUsmjSSGMuu6QxsqCQBSwOSArMN8NTlSoxpy3SRz0YOEOV+f5nxTp19pVpocF4fD3w/cR2V5JJDc3zyOHEMSqHQP5rTt5JK4Zow1wQqxFXCmq+IvCu/U9LvPDfw5S3vZ7iezuljuXntoGuGRbdXtMLlMmVXeIMyJ1ZSkTYcepeGbrLa9r0FlNaWn2SzXT/AArp0yTRSRkOZiroPOCt8shDuGGQyEbguq+ItLvLGPRv7Z0o2jtN58kPg7TYLmPdcSMNkseHbMRDj512sViGFUNW5qW9J8XeE9LuJrf/AIRvwHdi7t47Zrm8s9RlS3JjYNKnO/dkjcNp2kL5fAJMjat4PQC4j8TeHNPjdnRbSz8NtqRjwxO5nvYlYKd3yfvJGKqN+0gA8THpfh7YzTa7Pu2uVWO2RvmCgoCWkUDcxCsRnaAxG/ABt2Oj+B5Dcf2l4s1G2CTMsHk6ZHMZI+zNmddrew3D3oA5qiiigAooooAKKKKACiiigAooooAKKKKAP//Z&quot;);"></div></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>What's GeGLU?</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">10:18 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">10:18 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_59545DE375182C8AD689AF43C5882F44_919986197355@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe TRJ🐍👨‍💻 +91 99861 97355" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://media-tir3-1.cdn.whatsapp.net/v/t61.24694-24/408753155_217372938090286_5222666690277099082_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdQmjYgtUnKM-v0pMaLQbKYrPrrKSWc7hAhKAJyvAYOlUA&amp;oe=65EB0786&amp;_nc_sid=e6ed6c&amp;_nc_cat=102" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d swew6qod" role=""><span dir="auto" aria-label="Maybe TRJ🐍👨‍💻" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">TRJ<img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="🐍" draggable="false" class="b35 emoji apple _11JPr" tabindex="-1" style="background-position: -80px 0px;"><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👨‍💻" draggable="false" class="b43 emoji apple _11JPr" tabindex="-1" style="background-position: -40px 0px;"></span><span class="WJuYU" testid="author" role="button">+91 99861 97355</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[10:18 pm, 24/02/2024] +91 99861 97355: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g9gonq0i _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author">+91 77378 87058</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">What's GeGLU?</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Activation function</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">10:18 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">10:18 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3D9CE0A6455154938874C630F4E882C9_919773065092@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Priyesh Srivastava +91 97730 65092" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno g9gonq0i"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">P</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Priyesh Srivastava" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Priyesh Srivastava</span><span class="WJuYU" testid="author" role="button">+91 97730 65092</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[10:19 pm, 24/02/2024] +91 97730 65092: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g9gonq0i _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author">+91 77378 87058</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">What's GeGLU?</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Sigmoid+relu</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">10:19 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">10:19 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_D8B543C05194E6CD94_919916576150@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Bharat Shetty" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/410251715_3586044468316077_4057854922213032016_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRpFpmLshf1oE7y-iIbgpuzBA-R8WZgzsR5fyq7Haharw&amp;oe=65EAD974&amp;_nc_sid=e6ed6c&amp;_nc_cat=104" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d lr20dbn2" role=""><span dir="auto" aria-label="" class="_3FuDI ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Bharat Shetty</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[10:20 pm, 24/02/2024] Bharat Shetty: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g9gonq0i _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author">+91 77378 87058</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">What's GeGLU?</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="M6sU5"><div class="xzT4z gndfcl4n l8fojup5 paxyh2gw sfeitywo cqsf3vkf k2bacm8l ajgl1lbb p357zi0d sap93d0t ovllcyds kvdvyush gfz4du6o r7fjleex rrq4r3yd"><img alt="" class="tvf2evcx oq44ahr5 lb5m6g5c n43pk08i fpj7ivsd jpthtbts" src="data:image/jpeg;base64,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAMCAgICAgMCAgIDAwMDBAYEBAQEBAgGBgUGCQgKCgkICQkKDA8MCgsOCwkJDRENDg8QEBEQCgwSExIQEw8QEBD/2wBDAQMDAwQDBAgEBAgQCwkLEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBD/wAARCAA2ADYDASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwD9I/APirxv4m0xV8afDO+8G6utha3M0c2oWuoWgmmVy0EU0MgaRoSirIWjjUlx5bSL81TaLr/jaTxBr+k6/wCBJYdPsLqNdJ1a1vbd49St3jDljC0gkheJiYmDZDld64DbVteKfE154buNLY6fZ/2bdXSQX2pXupR2kFirkJGPmBaSWSV4o441GGZjllO0PrSXF9Dp5uTp4mukjDtbQTA7mwCyI7hQT1CltoPGSuTgA4/UPFnxSs7iaztfhJFfOIJHhuYfEMK2rzCG3ZUdpEWVVaaS5TcImwtsHK5kCL5v4f8Aih+1zqXh3xPZ63+zVa6Tr1p4Z1LUfD+oSa/YSWd9rCl/senzWsd0zxBgYg0vn7CQ/wA0YIx7npNxrdxawy61pdpYzvErSRQXjXAjkJbcu4xpkAbOccktx8oLeceKvir8SNLvPEtn4Q+Fui+I7nRL6KzsdO/4TO1tdQ1JSlg8koieMxwBFvm+WWVXOyH5QLhGAA7wx48+O1x4r0fw74v+BKWek3Ph+2vtR8SWPiK0mhttVIzNY/Y2ZZdowQJVZ1yyj5gC1S+JvH/xqsPiRY+FvC/wBk1bwrcafJc3Pim48T2dslrdLHcMtsbT5pmDPHbp5q5C/aC2D5RVt3x94q8deGQknhXwDZeI0lV1RX16KwcTC2umRWEse3D3KWFspVmObxnKgQ4k1NO8Tvc+EbbxVe6ekCy2hu5livoJYUQIX3rPuEbxsBlXyuQylgnzbQCn4kk+JF3q6aT4POkaZaJB9pm1fVLJr2NmJ2rax28VzFIX4d3lZlVQYlVZCztEVv2MmqSNdf2nZ2tuqTlbUwXLTGWHapDuGjTY24uNo3DCqd2SVUrojiZQioxUf/AU/wA0zN003dt/ezE8Xf2xJqOhWdnd2C2d5epHLDdeHrjUd0sckdyr+bFKiWgWK3uVWWVSonktiCWUQzz6frF5qnhm48TWt4FttQtV1DShNotzFPbQPboyrcWzsJnkD72KbYnAYRlA6lmxPixqOqadZ6A2i2Ws3d/NrAjgh0bWLOyvXItbh2EUV662158qNuglPCb5l+eBCNfwXL5Xg+BV1S71T7H9ot1ury9guJpRDK8Y8yaIBGYbMEn5hj58uGNc5oclpap4f8eaDb6DZ6VpsXiOTVW1cWfw51GKS+aKa4mQyagjiCyIlnmkzdK/2l5ZWi2mXNbq+IINDvvFV9cfNDbatAJI9P8AC9810wNlbk58sOb5iMYuIk2KqiEgvA5rA+HV7rGteNpdb1QeILGOTQh9ntLvxHY3lpNuvZy8kdvbEnKARBZ2PKSqhAZDV7xpoN48l4JNY13+ztW1mwe6h/4SX+zI40KRQCK2niUXEZaQROIEkjEsgILhZZEkAOi8YaBoniWBdM8RaLpWq2Btbwta6ho/25CzwNA3B+XDQzTxtHjdIkrqCF3hreqatI3hS51zSJ3t2Onvd28k2lXFy6fuyyFrRCk8hHBMK7ZDgqMMeOJ+LWvHTdc0O2/tvVNLjs7K/wBaubrTNe06zkt4IXtoJJ7i3vSEntIo7ySWSQ7hEYkwrSPEKsatcXXh34T6RYQz300p0cWZurjxNHHdKV0+VvNa/l4mkzGCZsE5JlIIU0AdZ4d1O61KfWftF0s0dpqT2sCf2XPZtCixx5RjKx+0fOXYTRhUKsqgEoWYqr4LMMa65Zr4ifVZ7bWrr7RG+oR3ZsGkKzJbZVFaMCKWJxFICyCQAMyBGJQBz3xm8O/8JbZ+GvDsml2N/b3mtss8WpeF49d08KthdsGu4mdDDHuVdsqMG80xRn5ZGroPD/8AasXhC4bXJri4vVe+MrtpixF/38u0x2yM2U242KWZ2TZvJctWV8U7q10tPDWvTeE5fEN1pWtG5s7a21iGwuo5DZXUbywCeWGG4ZYZJt8UkigRGWQbmiVW2fDdhov/AAjdzaaRo76fbXV3qMk1qtyu8XEt1M9y3mRSMFZpnlY7XypYj5SNoAOJ+DuhnQG01buzjhbVNBS8tYbfwSNGFoTMXuPPZC6xXEvn2oa3ZlObd3VTiTZR8SeI4bzULuST4V+Ivttv4rmsrW5XwdFcuWttOExvfNaUq1vPFG9mk/yPuuEiwm1nGj8KNT8M/wBrf8If4c8J3Wnv4TsZtMu5IfEdtqENkwnUx2t0qXT3DXDJ++RpoiUVpRvUuyvpeMtU8QXV9q3h+T4OWviSxK2TaebjU7MJfvuZp3lim5iitz5OHAkdnk4RQoZgDnfjhdWtxNe2+v8Aw71XxZ4f0rRllv8ASf8AhC4tdttZ86489YYVLhzPGNLZNuPKWS/s5X3BPkf411mHxd8GpPE1r8PfEi2WlT38D+HbrwjBdand28JubA+RYzsqhZAfPj7vAQNmJClanxR8deHfh34k0rxPq2gQX01tpF/5l1/b9jYz2Nm1zYrI3lXtxBG8LSGDzJN+UKRLhjKoqHxMvgb/AIVn4Sh03TdJ1TTnigTwzp9t4mFpbX8n2CX7LbQXG9VukkjBRVO5WRvMKkLQB0Xw5ltJI/EEi6NcWOoTa9fyX8k2gtppu2Wd4IJs8rc/6NBbos4Zi8aRsQmRGpWr4X1h9WXVo28OT6QLDVLizXzJ7aQXYUhvtKeRI+1XLn5ZQkoIbcgG0sUAalxY2V3NbXF1ZwTS2UpntnkjDNBIUaMuhIyrbJHXI52uw6E1V8P+HfD/AIT0e38PeFtC0/RtKswy29jp9qlvbwgsWISNAFXLMScDqSe9FFAD7bQ9Fs9Ql1az0eygvp4Y7aW5jt0WWSFCxSNnAyVUuxCk4G446mqdx4L8G3msN4iu/CejT6qy7WvpLCJrgjy2iwZCu7HlyOnX7rsOhIoooAXxN4N8H+NLGTTfGPhTR9ds5oWt5LfU7GK6jeJnjkaMrIpBUvDExB4LRoeqjGhDpmm29nb6bb6fbRWloixW8CQqI4kVdqqigYUBeAB0HFFFABY6bp2lxyxabp9taJPPJcyrBEsYeaRi0kjBQMszEszHkkknmiiigD//2Q=="><div class="k5z04t9s cm280p3y p357zi0d ggj6brxn m0h2a7mj lb5m6g5c f8m0rgwh ac2vgrno idwf4z32 onmzirdk gfz4du6o r7fjleex n1yiu2zv itegkywt eb4rp10x rppts313 ln8gz9je"><div class="aoi073rw suguakab jjtagnqy c32ccnay tvf2evcx m0h2a7mj lb5m6g5c j4enbv94 gfz4du6o r7fjleex lhj4utae ljrqcn24" title="Papers with Code - GeGLU Explained"><span dir="ltr" aria-label="" class="ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae i033jvx7 cw3vfol9 _11JPr" style="min-height: 0px;">Papers with Code - GeGLU Explained</span></div><div class="jjtagnqy tvf2evcx ttu8nud2 lb5m6g5c dsh4tgtl gfz4du6o r7fjleex lhj4utae"><span class="g0rxnol2 gfz4du6o r7fjleex" style="display: -webkit-box; overflow: hidden; -webkit-box-orient: vertical; -webkit-line-clamp: 2;"><span dir="ltr" title="GeGLU is an activation function which is a variant of GLU. The definition is as follows:

$$ \text{GeGLU}\left(x, W, V, b, c\right) = \text{GELU}\left(xW + b\right) \otimes \left(xV + c\right) $$" aria-label="" class="i033jvx7 _11JPr" style="min-height: 0px;">GeGLU is an activation function which is a variant of GLU. The definition is as follows:

$$ \text{GeGLU}\left(x, W, V, b, c\right) = \text{GELU}\left(xW + b\right) \otimes \left(xV + c\right) $$</span></span></div><div class="j8h4s3v0 tvf2evcx oq44ahr5 lb5m6g5c dsh4tgtl qbqilfqo"><span dir="ltr" aria-label="" class="_11JPr" style="min-height: 0px;">paperswithcode.com</span></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span><a dir="auto" href="https://paperswithcode.com/method/geglu" title="https://paperswithcode.com/method/geglu" target="_blank" rel="noopener noreferrer" class="_11JPr selectable-text copyable-text">https://paperswithcode.com/method/geglu</a></span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">10:20 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">10:20 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_D5DC10D185A9DFA2A4_919148574393@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Adithya S K +91 91485 74393" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno ckklvomo"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">A</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d fyja2idx" role=""><span dir="auto" aria-label="Maybe Adithya S K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Adithya S K</span><span class="WJuYU" testid="author" role="button">+91 91485 74393</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[10:23 pm, 24/02/2024] +91 91485 74393: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="m8ez62k1 _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d lr20dbn2" role=""><span dir="auto" aria-label="" class="_3FuDI _11JPr" style="min-height: 0px;">Bharat Shetty</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">https://lightning.ai/lightning-ai/studios/understanding-using-and-finetuning-gemma

By Sebastian Raschka

1) Gemma uses a really large vocabulary and consequently embedding weight matrices.

2) Like OLMo and GPT-2, Gemma also uses weight tying.

3) It even uses GeLU (in the form of GeGLU), similar to GPT-2, unlike Llama 2 and other LLMs.

4) Interestingly, Gemma normalizes the embeddings by the square root of the hidden layer dimension.

5) The RMSNorm layers are in the usual location, different from what was hinted at in the technical paper.

6) However, the RMSNorm implementation comes with the unit offset that wasn't mentioned in the technical paper.</span></div></div></div><div class="jrEtJ"><div class="kknmh"><div class="ln8gz9je ppled2lx t3g6t33p qnwaluaf" style="background-image: url(&quot;data:image/jpeg;base64,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAMCAgICAgMCAgIDAwMDBAYEBAQEBAgGBgUGCQgKCgkICQkKDA8MCgsOCwkJDRENDg8QEBEQCgwSExIQEw8QEBD/2wBDAQMDAwQDBAgEBAgQCwkLEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBD/wAARCABdAF0DASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwD8sqKKKACiiigAooooAKKKKACiiigAooooA63/AIQH/qLZ/wC2H/2VH/CA/wDUVx/2w/8Asq+gbf4WaZr39qWOi+E/HulXcMLTaXPq1upgvGRSTDIFhUQs3RSZCue+cK1PxB4Z+F3hGDwxJrD+KL+TW9KstSuks7i3i+ziVWMjIXiIkBOAqHbjacudwx9PPIatNc07KPdtpb27X6rp1vseTHM4T92N2+1jwr/hAf8AqLf+QP8A7Kj/AIQH/qLZ/wC2H/2VfQc3gD4Z6X4+T4X6lqXiE6hJ/o41dWhW2S5lXdAv2bYzugDxBj5yEtuGFX5qo3HhX4a6H8NfDPjXX5NfvNS15b1P7PsbuG3VjFc7BKJHhk2KqDBQoxZnUgqFbOU8mnT5ua3u3vrs00mn6cy2+RUcxhO3LfW1tN73a/JnhX/CBf8AUW/8gf8A2VH/AAgP/UV/8gf/AGVfQvibwj8E/CeqaDHqreNrrTvEdpb6xDPDcWlvJYWE4+RXUxzfaJlZX3YMa424LEnFTS/h34VhuvFuj3mh+P8Axbq/h/Vzp8EPh22SCDyVkZPPmlMdwVLiNyI/L428M2SY4llMoy5bK97fhf8AFdjSONjNXT0PBf8AhAf+ot/5A/8Asqk0Xw54HtdefT/HfiPU7KxNr5sU+n2CzyNLvxtKM4AXaGOc9QOK9i+MHw+sfBF9o+oaLY69Y6br1h9rhstdhWO+tJFkaOSKQKBu5UOrbVyki8ZBJ8nihiufiDosNxI0UTmMSMuni+cLubdst2+WZ8Z2o2FLYDELkjz8ZhvYXi1Zo66FXnfNuiXVvDvwlt5Uk0fxR4huLSQOVlvNO+zswEjBflUSD7mwnDHDMy8gBmzv7L+Gn/QyX/8A37b/AONV0S6bbWcml2ug+KIruS60+e6mWHwy8kcSpbrc+RKrofPw5kViFeNPJjlDsPu9J4au/CukXUHhm+8U6UbeTUWit5F+GdnfSqRJLERK88YnBZYoTGi+fkyyBhG6Et56VjU4Cz0n4TPMy6h4r1aGLypCrQ23mN5gRjGpBRQFZwqs2cqpLAMRtPQXHhz9mlJFW1+KHjWaMxIzNJ4ahjIkP3lAFw2QD0bIyOw6VpX1razeHZ47jVvDdjrcH+krHL4KEcV9Zm3WTdBKlsSSBJvG+KH5X3GRwsZrf8LjwFb3moS3PjrQrGa52SXUOofDG3uJba5DyxvbGJonigKLHG5ERwTNhuV4q4FKa88D2cNzdWeta/qMksMqWdpNEsHkO3CPLKrkPtBLYVQGYDO0ZFM8b+KLPxQdA+yw3Cf2Volrpc3nADdJFv3Mu0n5fmGM4PtX2h/w59sRqFlpbftMeJRPfELG3/Cv7oxJlJ3Hmyi68uIYtpBl2ADGJSQ00IerJ/wSLsbfQofEF7+0Z4vhhm3fuE+HV3cXKbSQd0MN08g6ZHHIIxXuV87daLpqKin0S7O/Vt/p2SOKGB5f3l72/U+Ste8dadq3xXj8eQwXaWSahaXWx0XztsQjB4DEZ+Q4+b0qj4g8VWOq+BPCPhe3juPtOgLfC4kZQI286beuwg5PA5yB+NfZmn/8Ef8AT9U/49P2mfEg/ded/pHgC6t+NiPj97dL8+HA2feBDqQCjhYf+HROmrrEWhyftJeLUuZXWPd/wre9NupaLzfmuBcmFQF4LF8B/kzu4qamezqe0pv7bbendxbt84oUcBCPK19myXyTX5Nnxx4+8XWfiqHw2tnDcR/2L4es9Hm85VAMsW/cVwT8nzDGcH2rd1Tx34Z8WWeuaNq+razpVpd+I7rxDbvDbJcLMJV2+XJFvXbIAq7W3lfncEDg19aTf8Ec/Km02FP2jvEUp1KN5EZfBTbYdqbsSk3oEZPQBsZbilH/AAR2tTFPN/w0z4gAt8lg3gSdWYBnU7AbvL/6tjhckgqQCHXOkM9qRc7xTU901vpbo0+vRmTy6nLls9Y7eWz9D44+I3jvQ/E2k+GdF8O2upQ23h60lswb9ld3UyZRshjjIGSo4TO1cqoNecafELnx/poZb0skDzRvZ30VnLDIiyOkomlGxAjKrnOMhSAykhh+kX/DlOT/AKOkvv8Awlf/ALtr5g/bI/Yts/2OPEfw9a++KVt4uh8WvqH2h9U0me0t7eK1NvlHEE0szK4uGB8vawxxycjgxmO+uSc5LV2/BJfodeHw6w8eSL0/zdzzSx8O+JTZ6f4F1+38dSWut239pTLa6xCYNSijtpJ4hb2zfJIylEBwzsrLsCrJIEGRYaX4yuodN1gaL8T4L66vhaR6hbwTTvdOlxNcTxROFXfIoSOXAbcJIpXbOV8vYXwPoaqstr4g+GmnQ6fp+l3Vyn/CW3by3QkgcXVshQkbpS4SaMLmLdCobC3DGCL4f6Mq23ji08VfDC+083Vw0WmTeIbyKQp50sCO0LlLqNNwjkUEiTEm9sRq+zzzoM2LSfETaJrFjdeDviYPDVnD/aEVtDCY4bC/W1EiXE7eQUaNRLKwAWMiKQEMA7VpaXqHjjTWu4fhlafFuz8m6mt777Cs9ui7ZXeGJoLc4iZUk3Mu7bulbaqjlsqzt/Dvh/QdTutftvAE9006PYwzalqU8jjyxK0MAsZdqIBKgLTEbjIQr7opQubq2ueFdKiSO3bw5r7tdXS+ZDLqmYY1k2oq/aEQCB8NLGAvmASt5p3YVAD+kG18SeHr63S6s9d0+eGXOySO5RlbBIOCDg8gj8KpXHjjwra60vh+XWI/tz232sRqGZfK3Fd28DbnIIxnNcz8BWI+E+jfKT/x8+n/AD8SV4R4AUD4l61gAcXX/pQK5MxrTwmWVsfTtemk7PZ3T/yNckjHM5wjU05ouWnSyX+Z9M+HviB4N8VWcmoaLr0MsMMxgcyboSHCqxGJApPDLzjHNa39s6P/ANBaz5H/AD3Xp+deUfsvtt8D6qApb/idSdP+uEHrXr01zHbxtNcMIo1+8zsqgduSTW8Z3hzy0OPCVJV6Maj3ZCdY0kddUsx/22X/ABobWNI/i1S0Gf8Apuv+NY/g/wAYHxZ/bX/EtNsdH1e40s4l3+Z5W35+QMZ3dOcY610O5v8Anm36f41aakro1hONSPNHVFb+2NJ/6Cln6f65f8a/L/8A4LYPbyal8EZGmhELLr58xwXQDOn/ADEDO4Ac4wc+h6V+nWk+INL1z7Z/ZNx9o+wXT2VxtGPLmTG5DnGcZHTivzX/AOCyUWqXPi74EW+ix6k2oTPr8dtHpshS7kkY6eFSEgEmRicKAMkkAcmmNNSV0fnhpN34HvrWe61nx1pOmtHpqQLb2/gy3nmmZ/LV1XcqxhkVQfNZg5IkAOWDvlTeMNJtJIdNtbjTZrK0jlCyL4P09mZzOSB5kg8yVTGkbb5CGUs8YTb8zdnf+H9e0zSdGvfES+M7FXj8u1vZfF1qIMGwUmFQUAg8yJYE+Zz+7j27ZDhVli+H2t+KtUtf+FmaL471eeHUJNPmuhrlv53764lYrbx3Ct5zJNFqEkm2TaWdizRbHkkBnFWPjLw3/wAI5NoeoPbbblfKbZ4M05poVEqBXS6Eiz7vL3uRuHzoqEsruwzbf4hX2jq0OkWOgzRzMZpDeeFtMlYP935WlhkIXaqHau1QS2FySzdjpvwdW402XxJd+BfE39kQWO9vL1yx81rk2YnQhTGGaNzJCcKhYKWGWcbay77wh4Q8IabpR8ceDPGkN5qtu15Ey6jb2ymNZGhI8o28jqRJFJkOVbkDBAEjgH9BHwHa4Hwp0bZHGV/0nq5B/wCPiT2rwv4f7v8AhZOsbsA7bnOP+vgV2vw1/aR/Z28KaMfC+ofH74RWthY82c0PxEsLx5t5LyeZlYdhV2YDAYFQpyCSq8R/wsf9m/QPE2p654X/AGgvhJeC4VvJa++J1hDvL/vGDIlu/ljzAFBDP8rFsZG08uZ0pYrKMRgqfxzSS7aX/wAy8hksuqUpVvswcXbu0v8AI9X/AGYWuP8AhCdV8uOMj+2ZPvMR/wAsIPauc8RfEzXfEfxRi0GN5bDTbK5u9NntUuC8dy8Jc+aw2jqduBzjb15qz8N/jr+yZ4A8Ptpdl+0t8Nt13N9suEm8baZN5czRorKrqyblGwYJUE8njoOGm+JH7NEfiDWfFlr+0N8J31EXFzfabE/xM09YbqeZpMpMRblrdMeUcr5xBdxg7Az5ZjhPrWU18LD+JKNo62Xnf5BkkvqFSn7b4VGSfXW2n4nuPwfa4z418pIyP+Eu1DO5iO0fsayPGHxRfUtZsfC2lQQm1n1C+07UGljbes1qI3/dnIGCW6kHI9Kw/A/7RX7MPhtdb+0ftGfCqFtU1i41LCePNPuN3mBecny9nKn5Pnx/fOeOX1z4ofspw3MuveHf2kPhle6ml9fapDb3nxC021ge4uVUMrSKkrKg2LjCsRk5zjBeKw0q2WVcPH+I42j287/IMnccJUp/WFeKUr9dbPl/Gx6j8CTONM8UeTHGR/wkt395iP4Y/QV8Lf8ABZSG2uPFXwJh1BtOWFz4gEq38jpbuP8AiX5R3QFkDfd3jG3IOVxuH2P4Q+P37JPg22vrfTf2nPhtINRvpNQm8/xnprYkcKCFw6/L8oxkZr4L/wCCsnxX+E/xM1/4P3Hw/wDiz4M8QQ6RF4hkvptMvbTWVtiUsmjSSGMuu6QxsqCQBSwOSArMN8NTlSoxpy3SRz0YOEOV+f5nxTp19pVpocF4fD3w/cR2V5JJDc3zyOHEMSqHQP5rTt5JK4Zow1wQqxFXCmq+IvCu/U9LvPDfw5S3vZ7iezuljuXntoGuGRbdXtMLlMmVXeIMyJ1ZSkTYcepeGbrLa9r0FlNaWn2SzXT/AArp0yTRSRkOZiroPOCt8shDuGGQyEbguq+ItLvLGPRv7Z0o2jtN58kPg7TYLmPdcSMNkseHbMRDj512sViGFUNW5qW9J8XeE9LuJrf/AIRvwHdi7t47Zrm8s9RlS3JjYNKnO/dkjcNp2kL5fAJMjat4PQC4j8TeHNPjdnRbSz8NtqRjwxO5nvYlYKd3yfvJGKqN+0gA8THpfh7YzTa7Pu2uVWO2RvmCgoCWkUDcxCsRnaAxG/ABt2Oj+B5Dcf2l4s1G2CTMsHk6ZHMZI+zNmddrew3D3oA5qiiigAooooAKKKKACiiigAooooAKKKKAP//Z&quot;);"></div></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>I feel continual pretraining on gemma for different languages will be more effective when compared to llama+extended vocabulary</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">10:23 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">10:23 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_7BB095D2C0CD51C2C78C09336B1354F0_919773065092@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Priyesh Srivastava +91 97730 65092" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno g9gonq0i"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">P</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Priyesh Srivastava" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Priyesh Srivastava</span><span class="WJuYU" testid="author" role="button">+91 97730 65092</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[10:23 pm, 24/02/2024] +91 97730 65092: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Why? I didnt seem tonfigure this out</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">10:23 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">10:23 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj _3sxvM" data-id="false_120363049558306142@g.us_2B510A776FE161B836_919916576150@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Bharat Shetty" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/410251715_3586044468316077_4057854922213032016_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRpFpmLshf1oE7y-iIbgpuzBA-R8WZgzsR5fyq7Haharw&amp;oe=65EAD974&amp;_nc_sid=e6ed6c&amp;_nc_cat=104" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d lr20dbn2" role=""><span dir="auto" aria-label="" class="_3FuDI ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Bharat Shetty</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[10:23 pm, 24/02/2024] Bharat Shetty: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="ckklvomo _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d fyja2idx" role=""><span dir="auto" aria-label="Maybe Adithya S K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Adithya S K</span><span class="WJuYU" testid="author">+91 91485 74393</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">I feel continual pretraining on gemma for different languages will be more effective when compared to llama+extended vocabulary</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>yeah. for sure - we can experiment and see how they are compared to llama2 etc</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">10:23 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">10:23 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_1007F9592329AB9443_919916576150@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a"><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d lr20dbn2" role=""><span dir="auto" aria-label="" class="_3FuDI ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Bharat Shetty</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[10:24 pm, 24/02/2024] Bharat Shetty: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g9gonq0i _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Priyesh Srivastava" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Priyesh Srivastava</span><span class="WJuYU" testid="author">+91 97730 65092</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Why? I didnt seem tonfigure this out</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>The large vocabulary size of 256,000 words (in contrast, Llama has a vocabulary of 32,000 words);
The extensive 6 trillion token training dataset (Llama was trained on only one-third of that amount).

this could be a factor</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">10:24 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">10:24 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_9BC0C0A74F3FADB0A4_919148574393@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Adithya S K +91 91485 74393" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno ckklvomo"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">A</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d fyja2idx" role=""><span dir="auto" aria-label="Maybe Adithya S K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Adithya S K</span><span class="WJuYU" testid="author" role="button">+91 91485 74393</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[10:38 pm, 24/02/2024] +91 91485 74393: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="m8ez62k1 _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d lr20dbn2" role=""><span dir="auto" aria-label="" class="_3FuDI _11JPr" style="min-height: 0px;">Bharat Shetty</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">yeah. for sure - we can experiment and see how they are compared to llama2 etc</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>in my opinion vocabular extension in the tokenizer is a problem
even though the tokenizer efficience goes up significantly
these new tokens will be seen by the model for the first time

when I was training ambari (kannada bilingual) I did a version with vocabular extension and the other without

even tho the tokenisation was bad that model significantly performed better than the one with tokenizer extension

with gemma we don't have to worry about adding new tokens to the vocabular as its already 50 to 60% more efficient than llamas tokenizer</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">10:38 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">10:38 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="Reactions 👍 3 in total"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk o2es7gts i2cterl7" style="flex-shrink: 0;"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 f8m0rgwh r15c9g6i ilf8vifs gfz4du6o r7fjleex qnz2jpws kjemk6od"><div class="dn5wc90o ilf8vifs gndfcl4n p357zi0d"><span class="f8jlpxt4 qfejxiq4 ce5kru2g">3</span></div></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_28162171741708796157_919405887831@c.us"><div class="_2OvAm focusable-list-item _1jHIY"><span class=""></span><div class="UzMP7 n6BPp"><div class="_1BOF7"><div class="_1sykI"><div class="Mk0Bp _30scZ"><span dir="auto" aria-label="" class="l7jjieqr ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">+91 95501 64716</span></div> added <div class="Mk0Bp _30scZ"><span dir="auto" aria-label="" class="l7jjieqr ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">+91 94058 87831</span></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"></div></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB0D53A47728494E83A47_919638627147@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Apurva Bhatt +91 96386 27147" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://media-tir3-1.cdn.whatsapp.net/v/t61.24694-24/168561917_4135544339809174_1916088753789893833_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdQbzm7iMNw4UcRa_M_1QpPBPrABI-kM8Nq47XZ8lqeeIQ&amp;oe=65EADE48&amp;_nc_sid=e6ed6c&amp;_nc_cat=103" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d oyzge2kb" role=""><span dir="auto" aria-label="Maybe Apurva Bhatt" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Apurva Bhatt</span><span class="WJuYU" testid="author" role="button">+91 96386 27147</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[11:47 pm, 24/02/2024] +91 96386 27147: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>I have a question about how autoregressive models (AR) work, let's say GPT-2. We have an input string of 500 words(or 740 tokens).
<ol class="cw3vfol9 hmy10g0s g0rxnol2 duniunel r6x3u63k qt60bha0 inww9tbj" dir="ltr"><li dir="auto" value="1" class=""><span class="_11JPr selectable-text copyable-text" data-pre-plain-text="1. ">AR model reads one window of words and starts predicting the output without reading the whole input, the model keeps on sequentially reading the windows and predicting the output.</span></li>
<li dir="auto" value="2" class=""><span class="_11JPr selectable-text copyable-text" data-pre-plain-text="2. ">AR model reads the whole input in one go and start predicting the output until EOS token is predicted.</span></li></ol></span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">11:47 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">11:47 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="_1jHIY _2OvAm _2zFLj focusable-list-item" tabindex="-1"><div class="UzMP7 n6BPp"><span dir="auto" aria-label="" class="_11JPr" style="min-height: 0px;">SUNDAY</span></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A0D639EEBC5C9EA90D5_917737887058@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Nirant K +91 77378 87058" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno g9gonq0i"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">N</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author" role="button">+91 77378 87058</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[0:45 am, 25/02/2024] +91 77378 87058: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="k4182h19 _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d oyzge2kb" role=""><span dir="auto" aria-label="Maybe Apurva Bhatt" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Apurva Bhatt</span><span class="WJuYU" testid="author">+91 96386 27147</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">I have a question about how autoregressive models (AR) work, let's say GPT-2. We have an input string of 500 words(or 740 tokens).
<ol class="cw3vfol9 hmy10g0s g0rxnol2 duniunel r6x3u63k qt60bha0 inww9tbj" dir="ltr"><li dir="auto" value="1" class=""><span class="_11JPr">AR model reads one window of words and starts predicting the output without reading the whole input, the model keeps on sequentially reading the windows and predicting the output.</span></li>
<li dir="auto" value="2" class=""><span class="_11JPr">AR model reads the whole input in one go and start predicting the output until EOS token is predicted.</span></li></ol></span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>The second. That's why the input tokens can't be streamed easily.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">12:45 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">12:45 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="Reactions ❤ 2 in total"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="❤️" draggable="false" class="bd96tnyg eipqoq80 b8 emoji apple _11JPr" tabindex="-1" style="background-position: -20px 0px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk o2es7gts i2cterl7" style="flex-shrink: 0;"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 f8m0rgwh r15c9g6i ilf8vifs gfz4du6o r7fjleex qnz2jpws kjemk6od"><div class="dn5wc90o ilf8vifs gndfcl4n p357zi0d"><span class="f8jlpxt4 qfejxiq4 ce5kru2g">2</span></div></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A4C31A67D7374C8FE32_919915123897@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Harveen +91 99151 23897" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/55964039_402243816992965_6637152673499971584_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdQtkZ88AbZaodP4QnfDlPwkgUdBcxY7iFDH1wr_oFccKQ&amp;oe=65EAFA23&amp;_nc_sid=e6ed6c&amp;_nc_cat=102" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d a7s2a5ab" role=""><span dir="auto" aria-label="Maybe Harveen" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Harveen</span><span class="WJuYU" testid="author" role="button">+91 99151 23897</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[10:34 am, 25/02/2024] +91 99151 23897: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="ckklvomo _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d fyja2idx" role=""><span dir="auto" aria-label="Maybe Adithya S K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Adithya S K</span><span class="WJuYU" testid="author">+91 91485 74393</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">in my opinion vocabular extension in the tokenizer is a problem
even though the tokenizer efficience goes up significantly
these new tokens will be seen by the model for the first time

when I was training ambari (kannada bilingual) I did a version with vocabular extension and the other without

even tho the tokenisation was bad that model significantly performed better than the one with tokenizer extension

with gemma we don't have to worry about adding new tokens to the vocabular as its already 50 to 60% more efficient than llamas tokenizer</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="M6sU5"><div class="xzT4z gndfcl4n l8fojup5 paxyh2gw sfeitywo cqsf3vkf k2bacm8l ajgl1lbb p357zi0d sap93d0t ovllcyds kvdvyush gfz4du6o r7fjleex rrq4r3yd"><img alt="" class="tvf2evcx oq44ahr5 lb5m6g5c n43pk08i fpj7ivsd jpthtbts" src="data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAASABIAAD/4QCMRXhpZgAATU0AKgAAAAgABQESAAMAAAABAAEAAAEaAAUAAAABAAAASgEbAAUAAAABAAAAUgEoAAMAAAABAAIAAIdpAAQAAAABAAAAWgAAAAAAAABIAAAAAQAAAEgAAAABAAOgAQADAAAAAQABAACgAgAEAAAAAQAAAIygAwAEAAAAAQAAAIwAAAAA/+0AOFBob3Rvc2hvcCAzLjAAOEJJTQQEAAAAAAAAOEJJTQQlAAAAAAAQ1B2M2Y8AsgTpgAmY7PhCfv/AABEIAIwAjAMBIgACEQEDEQH/xAAfAAABBQEBAQEBAQAAAAAAAAAAAQIDBAUGBwgJCgv/xAC1EAACAQMDAgQDBQUEBAAAAX0BAgMABBEFEiExQQYTUWEHInEUMoGRoQgjQrHBFVLR8CQzYnKCCQoWFxgZGiUmJygpKjQ1Njc4OTpDREVGR0hJSlNUVVZXWFlaY2RlZmdoaWpzdHV2d3h5eoOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4eLj5OXm5+jp6vHy8/T19vf4+fr/xAAfAQADAQEBAQEBAQEBAAAAAAAAAQIDBAUGBwgJCgv/xAC1EQACAQIEBAMEBwUEBAABAncAAQIDEQQFITEGEkFRB2FxEyIygQgUQpGhscEJIzNS8BVictEKFiQ04SXxFxgZGiYnKCkqNTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqCg4SFhoeIiYqSk5SVlpeYmZqio6Slpqeoqaqys7S1tre4ubrCw8TFxsfIycrS09TV1tfY2dri4+Tl5ufo6ery8/T19vf4+fr/2wBDAAYGBgYGBgoGBgoOCgoKDhIODg4OEhcSEhISEhccFxcXFxcXHBwcHBwcHBwiIiIiIiInJycnJywsLCwsLCwsLCz/2wBDAQcHBwsKCxMKChMuHxofLi4uLi4uLi4uLi4uLi4uLi4uLi4uLi4uLi4uLi4uLi4uLi4uLi4uLi4uLi4uLi4uLi7/3QAEAAn/2gAMAwEAAhEDEQA/AOb8k0nlGrxGKaRxXpJnK0ZzR4zVOUYPFa7rmsycYqJOw4xMqViOlUGuSp5q1M4zg1myqGNcdSvbY2jTua9ncRs2DW2gUjKkj8a4hA0bAqa6WxuSygN1rajiObRkVKVtUagDjoxFa9mZvss7ZzgVTjTeMgV0umabcywSoIyA44JBrolJbGKict9omB5UGnfa8feQ1r3miXVlgupYH0BrKMNFkIBdRHgginefAeM0ww98VE1tnnFS4opIsFoz90inebMo/dyMPoaz2sz2qu1tcj7jGoaQ7Eeu6rq1raboLl15HftWlpuo6hJZRu07EkdTXNahpGsXCDe+6Jj0xWxZNLa2yQGJjtGM1z7u6NOVpH//0MwijbxUpFG3ivQRzFRhWZcLwa2mWqU0O4cVMloOL1OQul5qmRW1dWzZrU0zw695A19dEx26ZGR95yOy/wCNeVX91ts7aeuiOThtpp22wqWPf2+tb+nwQW0ge7/eKOqq2P1qvezBf3KkRKOin7oH07n1J/Cuckurdj5aySyn/Y4FcSrTb9w6fYx+0e36Z4p8MWLLv05uOpEgY163oni7wrqqLFaTJE//ADzlAQ/h2/WvjEKs52W6tkdSTnH45FalnutZFa8uljAOfkyTx/WumjVm3qYVsPC2h9xzW0U0ZVQvI64zXjPi3wyNKIvEfcJW5GMAE+lT+CPFDNttzMZrfoCSSyfXpxXX+O0WbSI5FOQHBFepHmVrnmK6bTR4qIs0eRV0RkCpVi5q2zRIoC3z2p6WmT0rq9O021uf9fLs4qssCR3BQHcqnGfasXI0US3/AGFNLYQskTHPoDVhPB2ouoYWzc+or2618oWcW3ATYuPpirAZPUVzmp//0a7LTCOKssKixXejBkeKayAipCOaXjFXYgorYNdzLDGMs5Cj6mup8RMdK0qO1tQJHC7VwOmOpP1qDT3gso2vJsbz8kQzzluCfwzXGatq1xqMs9vCcsWMY56AV4WY1E58nY9LB03a5wTwvdXBDkuSSWY8/lW/ZaK0zBFXArotP8PSRiNJF+Zj83869BsNJitwvHIryald9D2qWHjuznbPwlahFMqhsdAabceE7IkuIcYBxXoqKFGKnZFZcYrJVpJ8zZs6MWrWPM9C0m80+6MwcQhvuAe39fSu9OsRanoE9r53nPbyDBGMYz2x1xUl3psN7ps9seCwIBHUcV5fpcE2k3Zsv4Gyv+FfR4bFKUYxsfOYvC8snI6ZSPSp1KelRhamVOa7mjjTJ1CEcVLFHzmhI60IIckAisWi07nekn+wIDHIcjbxn9KqRvPtHzn86oQ58tY/Q9K244gEHFcvLY3uf//SeyP6VCVYdRVOyvY7xiiMQR61ZaZEbaz813RTOdyQfhS4yOaBLCePMFToqno4NaJMhyRY0y3iuboQzDIKtj6gcVjeHNBW31OXz+SGJwfrmui07bDexTEggNyPY8VbsHW4vJbhRgxjDeua8PNabU1LuengJpqyNlo4I/3kmF9zTY72yc7I5FY+xrktRubPcZdUcneeI8noPYVQt7mGSQJYW8kSnozLgcfXmvCmux9DS2O8ku44VLHJx6VS/t5N5iW3mY+oXIp20yWY4AYgVzbJrkcxEbiOPthC3Huag1a00PQdJuYrxzGwKMR91q5vXNINre5Yc9Qf1qfRm1CK4SW6AkAYfMo2nHuDXY+JYRILa5XlG+Un37V7eB5XTfdHi5gpKdu5wKpmrSR054HhfEmOc4wc/hx3qeIc9RXsKV1dHjOLTsyeKGtaGHpVOJSeBitaJXI4FZSZcUTR8TAVvp92ucUstwFYYOK2lk+WsSz/0+PFkyNuRyDSG3lJyZDWv5L4+9TfsznndXeqcjlckZsVg00qhpDXVWmmiHJZi2ao2luUnVnPArqUKEcGuilG25lUd9iqtoMjHX61vaFZqs1yqkP5gDAg5HvVWMJwa0NEVLW5MKLjeHwR+BHFeTnUmlBW0PTyqmmpu+qIZ9EhaQyNkt61VMVrZnylGGf8Sa1Lq+bLY4C5rkLmF7xHkkyC/C5ODg/yr5h72PqKOsTpHubaK33SzIqjHU8fnTItRtjIsRYEEZDA5Uj61xsFhDDtjllj8oDkFgR+ta6yaSkf2dZ4l9MEDB9qUo9jeMbLU7yLYcFGz6Vv3s/l6ILjYX8tgcDkjnGfwrzfSLiQzmFnEqrghl6emD716VCGutJaHHDkJ+Z5rvwG07b2PKx/xRvtc82155LKJWiUtulJ/BlBJrnxf3hIbaa73VoAzpCcZiGDnrngfyArJW0jAyQtfRYShKnQimfP42rGpXlKJiDU5UiaRgylTwPWsF/iAi3C2cJbzS22ul1aFREMY/CvB2THiYY/56j+dFf3Yp2Mqauz6U0DU7i+Je5OWXiu0WUY615p4YbBkB9TXdCTiuWK0Nz/1AQoTyKebePHFOHNSjpXfzM57IgMIHIqRFIqYDinBRVKbE4oRS1aul3EcU6Gc4CkkE/TGKzguBTTxWVemqsXGRdKo6cuaJrsFaZs9MkGuX1DQ2mvo7qWd5beM5aI9D7cVpTzyQlJ3+5JxntkVPFfRtiMjOf0r5GtTlRqOMkfU4aspwTQ2zm0KOPAsIyRnqQe2O9Ur+z0/WNkCWyoiNnK9cema1ha27He/U+9W4ViQhEHJ6YrG6v7p280UrkcEUdmnl26hVGAAPaurv5p7Pw/D5DFJXkGCOowCSazbLTJprkK/wB1Tk1q+JY2QWsf8AUkfXNe5lWGnC9SaPn8zxMZ2hFnCmO7cli2SaakNyWG5sD61sIBSNgDivbdaWx4vs0ZN6oaLb6V4gY/+KoP/XVa9ylG5iPavG5U2+JmH/TVawxX8NPzNKXxM9Z0A7Z5V/2q7HzMVxOknZfTL711geskjW5//9V61MDUA4PNSd67bGBYXmpBUCmpAc1SQmSioXp+e1QyHFMR1lpZRX/h+aKRc/KSPUEc5FeSySXlnKwU7gPWvcvDCeZphT+9kV5df2g81mA5BINeNncVeEvI9PKJN88TnR4gutwRgQM8133h27glkEpyWH8T44+g/wAa5hNDMxDoMA10NhprWY4NeRQrRpyu0evWpSnG1z0C31GEShBwAfzqbxM4mt7WdenzD+VcpDEync3U10WqSR/2HaBj87S7V/EGvYw2Ye0k1LZHm4jAcsVyatnOKcU12NQwXEc6F48/KSrBgQQR2IPIokavSTT1R5TVtGQqN0jfSvIblceKWH/TRa9gtuZW+leSagNvip/99P50YpfuYvzJpP8AeP0PQrI7NTmH0/lXRbzXNQtt1eQeoH8q3d9Zpas0T0R//9Y3ZbNSg1WPDYqRTxXoWMCcGpAahUM33RmmXN3aWSB7qQKSCQo6mpnUjHWTBRb2LoNVJ54o2CO4UmuUvvEZc+XDhI92DjqR9axXvmeb7dM/IDLGv4da4543pFG0cP1Z6Jpuv6hcS/ZrWcwWkRLMU4ZznaOeuCfTsK0bjH2iXv8AMa8r0/UHgtp9pyI4UbH+62TXolpeRXSLeQtuSTn8+x968TG1ZzfvM9zCYeNOnGS3Zr6fOm8wuMEcj6Vs7wz4UYArnTDlgQdpByK27YqFCk5NefI7kX3IABrG1iaR7eOIk/fG0Vf3F229q4PxbriWYCWjgzowHrtJGcn34p0lJytE0pyjGXPPpqZvinW2i1UQWMjK0KBZGU4y4/niqlr4v1CIbbpVuF9T8rfmOP0risuWLOcseST3J707cQa9ejOVNJRZ8/WSqScmtz1XT/FemNL+/DxE8cjcPzH+Fcdqi58R/aU5ikZNrjoefWue8wjBFXrbUZbdt0Z4znB5Bx7V2PFynD2czk+rpS54nphO3WXz3Vf5VsFq4q01yzurgXM5McuMEdj9K6aO9tJl3pKmPc4/Q11xnF3aZi4tWR//14CTmpFNU7e8kuY1aRVBK5OBjmrkABlANdsKikroxcbaHIeINVlhuPsqHAQqcevfmuUub2e6kMszbjS6rPJPqEzyHJ3mqGecV41SblJtnfCNoolLluKRmY8EnAphJFHXGazKNbShv+0xn+KFv0qvp2pXunNutnwp6oeVP4Vb0MZvWU9GRgf0rHXmosm2md9VtUKUl5/oel2njq38sLe2jbh1KEEfkat/8J3p6HMFtKcDocDn9a8p3sOhpUdjkVk8PBmSxNRdTuNR8b6pdI0VsFtkbqV5cj69vwrlZuIIN5JaTfISeTycA/pVOQcZq5qA/fRr2WJAPyz/AFq4wUbKJcZOVOc5Py/EZkY5NBZQKhXng08gAAirOQaWx0o3kHnvQ3HNRsT+VMRbV8DikM0q4CnP1qqGbFTBjRcLH//Z"><div class="k5z04t9s cm280p3y p357zi0d ggj6brxn m0h2a7mj lb5m6g5c f8m0rgwh ac2vgrno idwf4z32 onmzirdk gfz4du6o r7fjleex n1yiu2zv itegkywt eb4rp10x rppts313 ln8gz9je"><div class="aoi073rw suguakab jjtagnqy c32ccnay tvf2evcx m0h2a7mj lb5m6g5c j4enbv94 gfz4du6o r7fjleex lhj4utae ljrqcn24" title="John Hewitt (@johnhewtt) on X"><span dir="ltr" aria-label="" class="ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae i033jvx7 cw3vfol9 _11JPr" style="min-height: 0px;">John Hewitt (@johnhewtt) on X</span></div><div class="jjtagnqy tvf2evcx ttu8nud2 lb5m6g5c dsh4tgtl gfz4du6o r7fjleex lhj4utae"><span class="g0rxnol2 gfz4du6o r7fjleex" style="display: -webkit-box; overflow: hidden; -webkit-box-orient: vertical; -webkit-line-clamp: 2;"><span dir="ltr" title="If you're adding new tokens to Gemma, you're likely running into the &quot;all logits are negative, so randomly init embedding with a logit of ~0 dominates the softmax&quot; problem! Averaging existing embeddings solves this by bounding KL from initial model. See:

https://t.co/EFwc66OOSy" aria-label="" class="i033jvx7 _11JPr" style="min-height: 0px;">If you're adding new tokens to Gemma, you're likely running into the "all logits are negative, so randomly init embedding with a logit of ~0 dominates the softmax" problem! Averaging existing embeddings solves this by bounding KL from initial model. See:

https://t.co/EFwc66OOSy</span></span></div><div class="j8h4s3v0 tvf2evcx oq44ahr5 lb5m6g5c dsh4tgtl qbqilfqo"><span dir="ltr" aria-label="" class="_11JPr" style="min-height: 0px;">x.com</span></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>If I had GPU’s I would try the following:

<ol class="cw3vfol9 hmy10g0s g0rxnol2 duniunel r6x3u63k qt60bha0 inww9tbj" dir="ltr"><li dir="auto" value="1" class=""><span class="_11JPr selectable-text copyable-text" data-pre-plain-text="1. ">Train tokenizer on your text of intended language.</span></li>
<li dir="auto" value="2" class=""><span class="_11JPr selectable-text copyable-text" data-pre-plain-text="2. ">⁠find tokens not common with gemmas vocab.</span></li>
<li dir="auto" value="3" class=""><span class="_11JPr selectable-text copyable-text" data-pre-plain-text="3. ">⁠replace unused tokens in gemma vocab with tokens from step 2</span></li>
<li dir="auto" value="4" class=""><span class="_11JPr selectable-text copyable-text" data-pre-plain-text="4. ">⁠continual pretrain</span></li>
<li dir="auto" value="5" class=""><span class="_11JPr selectable-text copyable-text" data-pre-plain-text="5. ">⁠sft</span></li>
</ol>
There is already some discussion here:

<a dir="auto" href="https://x.com/johnhewtt/status/1761056178988073319?s=46" title="https://x.com/johnhewtt/status/1761056178988073319?s=46" target="_blank" rel="noopener noreferrer" class="_11JPr selectable-text copyable-text">https://x.com/johnhewtt/status/1761056178988073319?s=46</a></span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">10:34 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">10:34 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="Reactions ❤, 💯 3 in total"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="❤️" draggable="false" class="bd96tnyg eipqoq80 b8 emoji apple _11JPr" tabindex="-1" style="background-position: -20px 0px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="💯" draggable="false" class="bd96tnyg p83aruvr b75 emoji apple _11JPr" tabindex="-1" style="background-position: -60px -20px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk o2es7gts i2cterl7" style="flex-shrink: 0;"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 f8m0rgwh r15c9g6i ilf8vifs gfz4du6o r7fjleex qnz2jpws kjemk6od"><div class="dn5wc90o ilf8vifs gndfcl4n p357zi0d"><span class="f8jlpxt4 qfejxiq4 ce5kru2g">3</span></div></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3AA55ACD579944E07E82_918050098772@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Shan Shah +91 80500 98772" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno exofpctk"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">S</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d a1q56urb" role=""><span dir="auto" aria-label="Maybe Shan Shah" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Shan Shah</span><span class="WJuYU" testid="author" role="button">+91 80500 98772</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[10:36 am, 25/02/2024] +91 80500 98772: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="jc7tlmys _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d sthcsby1" role=""><span dir="auto" aria-label="Maybe Ankit Kumar Pandey" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Ankit Kumar Pandey</span><span class="WJuYU" testid="author">+91 94317 95264</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Is anyone using any agent framework (Crew, Autogen) in production or even internally?</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>What’s the specific question</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">10:36 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">10:36 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_FF430BB586795DC9D1E13320BCFBA782_918867705880@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Adithya +91 88677 05880" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/319947062_1208068306490361_762498494962268271_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdSLcJL6jiRgAQdZQsPwsIjZiltHgg3MgrJf_L012-6J_Q&amp;oe=65EAE599&amp;_nc_sid=e6ed6c&amp;_nc_cat=101" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d dud7p2xx" role=""><span dir="auto" aria-label="Maybe Adithya" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Adithya</span><span class="WJuYU" testid="author" role="button">+91 88677 05880</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[10:45 am, 25/02/2024] +91 88677 05880: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="ckklvomo _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d fyja2idx" role=""><span dir="auto" aria-label="Maybe Adithya S K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Adithya S K</span><span class="WJuYU" testid="author">+91 91485 74393</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">in my opinion vocabular extension in the tokenizer is a problem
even though the tokenizer efficience goes up significantly
these new tokens will be seen by the model for the first time

when I was training ambari (kannada bilingual) I did a version with vocabular extension and the other without

even tho the tokenisation was bad that model significantly performed better than the one with tokenizer extension

with gemma we don't have to worry about adding new tokens to the vocabular as its already 50 to 60% more efficient than llamas tokenizer</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>I feel like if the newly added tokens don't see as much pre training data as the already present tokens, the correlations dont occure and the efficiency reduces severely</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">10:45 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">10:45 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="Reactions 💯 2 in total"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="💯" draggable="false" class="bd96tnyg p83aruvr b75 emoji apple _11JPr" tabindex="-1" style="background-position: -60px -20px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk o2es7gts i2cterl7" style="flex-shrink: 0;"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 f8m0rgwh r15c9g6i ilf8vifs gfz4du6o r7fjleex qnz2jpws kjemk6od"><div class="dn5wc90o ilf8vifs gndfcl4n p357zi0d"><span class="f8jlpxt4 qfejxiq4 ce5kru2g">2</span></div></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_2C29C9DEB442D11520138D79712507C6_919199120076@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Anukriti +91 91991 20076" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno k4182h19"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">A</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d oyzge2kb" role=""><span dir="auto" aria-label="Maybe Anukriti" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Anukriti</span><span class="WJuYU" testid="author" role="button">+91 91991 20076</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[11:10 am, 25/02/2024] +91 91991 20076: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>i am doing entity extraction from streaming voice data, the current pipeline is STT (whisper) , make an API call to mistral/gpt3.5 every n words collected .
is there a more efficient way ?</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">11:10 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">11:10 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_0E378D9745C6407832_919916576150@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Bharat Shetty" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/410251715_3586044468316077_4057854922213032016_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRpFpmLshf1oE7y-iIbgpuzBA-R8WZgzsR5fyq7Haharw&amp;oe=65EAD974&amp;_nc_sid=e6ed6c&amp;_nc_cat=104" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d lr20dbn2" role=""><span dir="auto" aria-label="" class="_3FuDI ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Bharat Shetty</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[11:16 am, 25/02/2024] Bharat Shetty: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="k4182h19 _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d oyzge2kb" role=""><span dir="auto" aria-label="Maybe Anukriti" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Anukriti</span><span class="WJuYU" testid="author">+91 91991 20076</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">i am doing entity extraction from streaming voice data, the current pipeline is STT (whisper) , make an API call to mistral/gpt3.5 every n words collected .
is there a more efficient way ?</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>how does mistral entity extraction compare to normal entity ml extractors - say using spacy entity detectors ?</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">11:16 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">11:16 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_1CC972AF798613B0416DA48FD77BC92C_919199120076@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Anukriti +91 91991 20076" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno k4182h19"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">A</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d oyzge2kb" role=""><span dir="auto" aria-label="Maybe Anukriti" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Anukriti</span><span class="WJuYU" testid="author" role="button">+91 91991 20076</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[11:22 am, 25/02/2024] +91 91991 20076: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="m8ez62k1 _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d lr20dbn2" role=""><span dir="auto" aria-label="" class="_3FuDI _11JPr" style="min-height: 0px;">Bharat Shetty</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">how does mistral entity extraction compare to normal entity ml extractors - say using spacy entity detectors ?</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>currently providing the expected  schema in prompt (entities are product id, category name etc. ) in case of mistral, tbh, have not explored spacy for this , gpt3.5 function calling works well</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">11:22 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">11:22 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="reaction 👍"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3AD75BB4C711296181D7_919971004124@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Ojasvi Yadav +91 99710 04124" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://media-tir3-1.cdn.whatsapp.net/v/t61.24694-24/368264585_747754717061201_6147702637079399948_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdS82xqa9NAHlYHRL-2raI_BQvvDGLytDcImH1m-oDj2Nw&amp;oe=65EAEE94&amp;_nc_sid=e6ed6c&amp;_nc_cat=107" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d c7ayxzn8" role=""><span dir="auto" aria-label="Maybe Ojasvi Yadav" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Ojasvi Yadav</span><span class="WJuYU" testid="author" role="button">+91 99710 04124</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[1:35 pm, 25/02/2024] +91 99710 04124: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Has anyone tried building multi-modal projects with Quest 3?
please share any experience/resource/demo</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:35 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:35 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj _3sxvM" data-id="false_120363049558306142@g.us_3EB06E454DCFB56FC04646_917737887058@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Nirant K +91 77378 87058" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno g9gonq0i"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">N</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author" role="button">+91 77378 87058</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[1:38 pm, 25/02/2024] +91 77378 87058: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g755h7wn _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d a7s2a5ab" role=""><span dir="auto" aria-label="Maybe Harveen" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Harveen</span><span class="WJuYU" testid="author">+91 99151 23897</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">If I had GPU’s I would try the following:

<ol class="cw3vfol9 hmy10g0s g0rxnol2 duniunel r6x3u63k qt60bha0 inww9tbj" dir="ltr"><li dir="auto" value="1" class=""><span class="_11JPr">Train tokenizer on your text of intended language.</span></li>
<li dir="auto" value="2" class=""><span class="_11JPr">⁠find tokens not common with gemmas vocab.</span></li>
<li dir="auto" value="3" class=""><span class="_11JPr">⁠replace unused tokens in gemma vocab with tokens from step 2</span></li>
<li dir="auto" value="4" class=""><span class="_11JPr">⁠continual pretrain</span></li>
<li dir="auto" value="5" class=""><span class="_11JPr">⁠sft</span></li>
</ol>
There is already some discussion here:

https://x.com/johnhewtt/status/1761056178988073319?s=46</span></div></div></div><div class="jrEtJ"><div class="kknmh"><div class="ln8gz9je ppled2lx t3g6t33p qnwaluaf" style="background-image: url(&quot;data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAASABIAAD/4QCMRXhpZgAATU0AKgAAAAgABQESAAMAAAABAAEAAAEaAAUAAAABAAAASgEbAAUAAAABAAAAUgEoAAMAAAABAAIAAIdpAAQAAAABAAAAWgAAAAAAAABIAAAAAQAAAEgAAAABAAOgAQADAAAAAQABAACgAgAEAAAAAQAAAIygAwAEAAAAAQAAAIwAAAAA/+0AOFBob3Rvc2hvcCAzLjAAOEJJTQQEAAAAAAAAOEJJTQQlAAAAAAAQ1B2M2Y8AsgTpgAmY7PhCfv/AABEIAIwAjAMBIgACEQEDEQH/xAAfAAABBQEBAQEBAQAAAAAAAAAAAQIDBAUGBwgJCgv/xAC1EAACAQMDAgQDBQUEBAAAAX0BAgMABBEFEiExQQYTUWEHInEUMoGRoQgjQrHBFVLR8CQzYnKCCQoWFxgZGiUmJygpKjQ1Njc4OTpDREVGR0hJSlNUVVZXWFlaY2RlZmdoaWpzdHV2d3h5eoOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4eLj5OXm5+jp6vHy8/T19vf4+fr/xAAfAQADAQEBAQEBAQEBAAAAAAAAAQIDBAUGBwgJCgv/xAC1EQACAQIEBAMEBwUEBAABAncAAQIDEQQFITEGEkFRB2FxEyIygQgUQpGhscEJIzNS8BVictEKFiQ04SXxFxgZGiYnKCkqNTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqCg4SFhoeIiYqSk5SVlpeYmZqio6Slpqeoqaqys7S1tre4ubrCw8TFxsfIycrS09TV1tfY2dri4+Tl5ufo6ery8/T19vf4+fr/2wBDAAYGBgYGBgoGBgoOCgoKDhIODg4OEhcSEhISEhccFxcXFxcXHBwcHBwcHBwiIiIiIiInJycnJywsLCwsLCwsLCz/2wBDAQcHBwsKCxMKChMuHxofLi4uLi4uLi4uLi4uLi4uLi4uLi4uLi4uLi4uLi4uLi4uLi4uLi4uLi4uLi4uLi4uLi7/3QAEAAn/2gAMAwEAAhEDEQA/AOb8k0nlGrxGKaRxXpJnK0ZzR4zVOUYPFa7rmsycYqJOw4xMqViOlUGuSp5q1M4zg1myqGNcdSvbY2jTua9ncRs2DW2gUjKkj8a4hA0bAqa6WxuSygN1rajiObRkVKVtUagDjoxFa9mZvss7ZzgVTjTeMgV0umabcywSoIyA44JBrolJbGKict9omB5UGnfa8feQ1r3miXVlgupYH0BrKMNFkIBdRHgginefAeM0ww98VE1tnnFS4opIsFoz90inebMo/dyMPoaz2sz2qu1tcj7jGoaQ7Eeu6rq1raboLl15HftWlpuo6hJZRu07EkdTXNahpGsXCDe+6Jj0xWxZNLa2yQGJjtGM1z7u6NOVpH//0MwijbxUpFG3ivQRzFRhWZcLwa2mWqU0O4cVMloOL1OQul5qmRW1dWzZrU0zw695A19dEx26ZGR95yOy/wCNeVX91ts7aeuiOThtpp22wqWPf2+tb+nwQW0ge7/eKOqq2P1qvezBf3KkRKOin7oH07n1J/Cuckurdj5aySyn/Y4FcSrTb9w6fYx+0e36Z4p8MWLLv05uOpEgY163oni7wrqqLFaTJE//ADzlAQ/h2/WvjEKs52W6tkdSTnH45FalnutZFa8uljAOfkyTx/WumjVm3qYVsPC2h9xzW0U0ZVQvI64zXjPi3wyNKIvEfcJW5GMAE+lT+CPFDNttzMZrfoCSSyfXpxXX+O0WbSI5FOQHBFepHmVrnmK6bTR4qIs0eRV0RkCpVi5q2zRIoC3z2p6WmT0rq9O021uf9fLs4qssCR3BQHcqnGfasXI0US3/AGFNLYQskTHPoDVhPB2ouoYWzc+or2618oWcW3ATYuPpirAZPUVzmp//0a7LTCOKssKixXejBkeKayAipCOaXjFXYgorYNdzLDGMs5Cj6mup8RMdK0qO1tQJHC7VwOmOpP1qDT3gso2vJsbz8kQzzluCfwzXGatq1xqMs9vCcsWMY56AV4WY1E58nY9LB03a5wTwvdXBDkuSSWY8/lW/ZaK0zBFXArotP8PSRiNJF+Zj83869BsNJitwvHIryald9D2qWHjuznbPwlahFMqhsdAabceE7IkuIcYBxXoqKFGKnZFZcYrJVpJ8zZs6MWrWPM9C0m80+6MwcQhvuAe39fSu9OsRanoE9r53nPbyDBGMYz2x1xUl3psN7ps9seCwIBHUcV5fpcE2k3Zsv4Gyv+FfR4bFKUYxsfOYvC8snI6ZSPSp1KelRhamVOa7mjjTJ1CEcVLFHzmhI60IIckAisWi07nekn+wIDHIcjbxn9KqRvPtHzn86oQ58tY/Q9K244gEHFcvLY3uf//SeyP6VCVYdRVOyvY7xiiMQR61ZaZEbaz813RTOdyQfhS4yOaBLCePMFToqno4NaJMhyRY0y3iuboQzDIKtj6gcVjeHNBW31OXz+SGJwfrmui07bDexTEggNyPY8VbsHW4vJbhRgxjDeua8PNabU1LuengJpqyNlo4I/3kmF9zTY72yc7I5FY+xrktRubPcZdUcneeI8noPYVQt7mGSQJYW8kSnozLgcfXmvCmux9DS2O8ku44VLHJx6VS/t5N5iW3mY+oXIp20yWY4AYgVzbJrkcxEbiOPthC3Huag1a00PQdJuYrxzGwKMR91q5vXNINre5Yc9Qf1qfRm1CK4SW6AkAYfMo2nHuDXY+JYRILa5XlG+Un37V7eB5XTfdHi5gpKdu5wKpmrSR054HhfEmOc4wc/hx3qeIc9RXsKV1dHjOLTsyeKGtaGHpVOJSeBitaJXI4FZSZcUTR8TAVvp92ucUstwFYYOK2lk+WsSz/0+PFkyNuRyDSG3lJyZDWv5L4+9TfsznndXeqcjlckZsVg00qhpDXVWmmiHJZi2ao2luUnVnPArqUKEcGuilG25lUd9iqtoMjHX61vaFZqs1yqkP5gDAg5HvVWMJwa0NEVLW5MKLjeHwR+BHFeTnUmlBW0PTyqmmpu+qIZ9EhaQyNkt61VMVrZnylGGf8Sa1Lq+bLY4C5rkLmF7xHkkyC/C5ODg/yr5h72PqKOsTpHubaK33SzIqjHU8fnTItRtjIsRYEEZDA5Uj61xsFhDDtjllj8oDkFgR+ta6yaSkf2dZ4l9MEDB9qUo9jeMbLU7yLYcFGz6Vv3s/l6ILjYX8tgcDkjnGfwrzfSLiQzmFnEqrghl6emD716VCGutJaHHDkJ+Z5rvwG07b2PKx/xRvtc82155LKJWiUtulJ/BlBJrnxf3hIbaa73VoAzpCcZiGDnrngfyArJW0jAyQtfRYShKnQimfP42rGpXlKJiDU5UiaRgylTwPWsF/iAi3C2cJbzS22ul1aFREMY/CvB2THiYY/56j+dFf3Yp2Mqauz6U0DU7i+Je5OWXiu0WUY615p4YbBkB9TXdCTiuWK0Nz/1AQoTyKebePHFOHNSjpXfzM57IgMIHIqRFIqYDinBRVKbE4oRS1aul3EcU6Gc4CkkE/TGKzguBTTxWVemqsXGRdKo6cuaJrsFaZs9MkGuX1DQ2mvo7qWd5beM5aI9D7cVpTzyQlJ3+5JxntkVPFfRtiMjOf0r5GtTlRqOMkfU4aspwTQ2zm0KOPAsIyRnqQe2O9Ur+z0/WNkCWyoiNnK9cema1ha27He/U+9W4ViQhEHJ6YrG6v7p280UrkcEUdmnl26hVGAAPaurv5p7Pw/D5DFJXkGCOowCSazbLTJprkK/wB1Tk1q+JY2QWsf8AUkfXNe5lWGnC9SaPn8zxMZ2hFnCmO7cli2SaakNyWG5sD61sIBSNgDivbdaWx4vs0ZN6oaLb6V4gY/+KoP/XVa9ylG5iPavG5U2+JmH/TVawxX8NPzNKXxM9Z0A7Z5V/2q7HzMVxOknZfTL711geskjW5//9V61MDUA4PNSd67bGBYXmpBUCmpAc1SQmSioXp+e1QyHFMR1lpZRX/h+aKRc/KSPUEc5FeSySXlnKwU7gPWvcvDCeZphT+9kV5df2g81mA5BINeNncVeEvI9PKJN88TnR4gutwRgQM8133h27glkEpyWH8T44+g/wAa5hNDMxDoMA10NhprWY4NeRQrRpyu0evWpSnG1z0C31GEShBwAfzqbxM4mt7WdenzD+VcpDEync3U10WqSR/2HaBj87S7V/EGvYw2Ye0k1LZHm4jAcsVyatnOKcU12NQwXEc6F48/KSrBgQQR2IPIokavSTT1R5TVtGQqN0jfSvIblceKWH/TRa9gtuZW+leSagNvip/99P50YpfuYvzJpP8AeP0PQrI7NTmH0/lXRbzXNQtt1eQeoH8q3d9Zpas0T0R//9Y3ZbNSg1WPDYqRTxXoWMCcGpAahUM33RmmXN3aWSB7qQKSCQo6mpnUjHWTBRb2LoNVJ54o2CO4UmuUvvEZc+XDhI92DjqR9axXvmeb7dM/IDLGv4da4543pFG0cP1Z6Jpuv6hcS/ZrWcwWkRLMU4ZznaOeuCfTsK0bjH2iXv8AMa8r0/UHgtp9pyI4UbH+62TXolpeRXSLeQtuSTn8+x968TG1ZzfvM9zCYeNOnGS3Zr6fOm8wuMEcj6Vs7wz4UYArnTDlgQdpByK27YqFCk5NefI7kX3IABrG1iaR7eOIk/fG0Vf3F229q4PxbriWYCWjgzowHrtJGcn34p0lJytE0pyjGXPPpqZvinW2i1UQWMjK0KBZGU4y4/niqlr4v1CIbbpVuF9T8rfmOP0risuWLOcseST3J707cQa9ejOVNJRZ8/WSqScmtz1XT/FemNL+/DxE8cjcPzH+Fcdqi58R/aU5ikZNrjoefWue8wjBFXrbUZbdt0Z4znB5Bx7V2PFynD2czk+rpS54nphO3WXz3Vf5VsFq4q01yzurgXM5McuMEdj9K6aO9tJl3pKmPc4/Q11xnF3aZi4tWR//14CTmpFNU7e8kuY1aRVBK5OBjmrkABlANdsKikroxcbaHIeINVlhuPsqHAQqcevfmuUub2e6kMszbjS6rPJPqEzyHJ3mqGecV41SblJtnfCNoolLluKRmY8EnAphJFHXGazKNbShv+0xn+KFv0qvp2pXunNutnwp6oeVP4Vb0MZvWU9GRgf0rHXmosm2md9VtUKUl5/oel2njq38sLe2jbh1KEEfkat/8J3p6HMFtKcDocDn9a8p3sOhpUdjkVk8PBmSxNRdTuNR8b6pdI0VsFtkbqV5cj69vwrlZuIIN5JaTfISeTycA/pVOQcZq5qA/fRr2WJAPyz/AFq4wUbKJcZOVOc5Py/EZkY5NBZQKhXng08gAAirOQaWx0o3kHnvQ3HNRsT+VMRbV8DikM0q4CnP1qqGbFTBjRcLH//Z&quot;);"></div></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Yeah the pipeline makes a ton of sense — needs patience more than GPU. You can go pretty far with A100s I think, which most orgs can afford.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:38 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:38 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="reaction 👍"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj _3sxvM" data-id="false_120363049558306142@g.us_3EB04181B3548F4F9D0BC8_917737887058@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a"><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author" role="button">+91 77378 87058</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[1:39 pm, 25/02/2024] +91 77378 87058: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="k4182h19 _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d oyzge2kb" role=""><span dir="auto" aria-label="Maybe Anukriti" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Anukriti</span><span class="WJuYU" testid="author">+91 91991 20076</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">i am doing entity extraction from streaming voice data, the current pipeline is STT (whisper) , make an API call to mistral/gpt3.5 every n words collected .
is there a more efficient way ?</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Function Calling can be streamed from OpenAI Models — assuming you meant faster by efficient, and not token reduction, which is a different thing altogether</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:39 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:39 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="reaction 👍"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB06FC72145DB78344F62_917737887058@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a"><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author" role="button">+91 77378 87058</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[1:41 pm, 25/02/2024] +91 77378 87058: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="m8ez62k1 _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d lr20dbn2" role=""><span dir="auto" aria-label="" class="_3FuDI _11JPr" style="min-height: 0px;">Bharat Shetty</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">how does mistral entity extraction compare to normal entity ml extractors - say using spacy entity detectors ?</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="M6sU5"><div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>spaCy allows you to use LLMs for NER via their APIs fwiw: <a dir="auto" href="https://spacy.io/api/large-language-models#ner" title="https://spacy.io/api/large-language-models#ner" target="_blank" rel="noopener noreferrer" class="_11JPr selectable-text copyable-text">https://spacy.io/api/large-language-models#ner</a>

By their own, quite robust eval suite — it's 15% absolute better, that's quite big for NER where the annual progress upto T5 used to be like 1-2%</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:41 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:41 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="Reactions 👍, 😮, 💯 3 in total"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="😮" draggable="false" class="bd96tnyg p83aruvr b87 emoji apple _11JPr" tabindex="-1" style="background-position: -40px -40px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="💯" draggable="false" class="bd96tnyg p83aruvr b75 emoji apple _11JPr" tabindex="-1" style="background-position: -60px -20px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk o2es7gts i2cterl7" style="flex-shrink: 0;"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 f8m0rgwh r15c9g6i ilf8vifs gfz4du6o r7fjleex qnz2jpws kjemk6od"><div class="dn5wc90o ilf8vifs gndfcl4n p357zi0d"><span class="f8jlpxt4 qfejxiq4 ce5kru2g">3</span></div></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_C6E733C57BE2E4C7E07CBA1670308CE4_12064993617@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Ajay +1 (206) 499-3617" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno ckklvomo"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">A</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d fyja2idx" role=""><span dir="auto" aria-label="Maybe Ajay" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Ajay</span><span class="WJuYU" testid="author" role="button">+1 (206) 499-3617</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:12 pm, 25/02/2024] +1 (206) 499-3617: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span><span role="button" tabindex="0" class=""><span dir="auto" data-jid="917737887058@c.us" data-display="~Nirant K" aria-label="Maybe Nirant K" class="ajgl1lbb o0rubyzf _11JPr selectable-text select-all copyable-text" data-plain-text="@~Nirant K" data-app-text-template="​917737887058@c.us​">@<span dir="ltr">~Nirant K</span></span></span> thank you for your suggestion on using the Azure AI for Document markdown output for pdf files with ChatGPT/GPT-4 It clearly did best in my tests compared with unstructured ( JSON ) and Llama Parse. ( Also thank you to <span role="button" tabindex="0" class=""><span dir="auto" data-jid="32486634341@c.us" data-display="~Vignesh Baskaran Hexo" aria-label="Maybe Vignesh Baskaran Hexo" class="ajgl1lbb o0rubyzf _11JPr selectable-text select-all copyable-text" data-plain-text="@~Vignesh Baskaran Hexo" data-app-text-template="​32486634341@c.us​">@<span dir="ltr">~Vignesh Baskaran Hexo</span></span></span> for pointing out how in isolation to best convert the table to markdown )

Follow up question - does the same file format/conversion work best with llama-2-70B-chat as well or are there any specific quirks on the llama-2 side?</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5 hnx8ox4h mw4yctpw gj5xqxfh">Edited</span><span class="tvf2evcx oq44ahr5">2:12 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto"><span class="mw4yctpw gj5xqxfh">Edited</span>2:12 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="Reactions 👍 2 in total"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk o2es7gts i2cterl7" style="flex-shrink: 0;"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 f8m0rgwh r15c9g6i ilf8vifs gfz4du6o r7fjleex qnz2jpws kjemk6od"><div class="dn5wc90o ilf8vifs gndfcl4n p357zi0d"><span class="f8jlpxt4 qfejxiq4 ce5kru2g">2</span></div></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB016053F8E08B2249223_917737887058@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Nirant K +91 77378 87058" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno g9gonq0i"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">N</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author" role="button">+91 77378 87058</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[3:44 pm, 25/02/2024] +91 77378 87058: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="ckklvomo _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d fyja2idx" role=""><span dir="auto" aria-label="Maybe Ajay" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Ajay</span><span class="WJuYU" testid="author">+1 (206) 499-3617</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;"><span class=""><span dir="auto" data-jid="917737887058@c.us" data-display="~Nirant K" aria-label="Maybe Nirant K" class="_11JPr">@<span dir="ltr">~Nirant K</span></span></span> thank you for your suggestion on using the Azure AI for Document markdown output for pdf files with ChatGPT/GPT-4 It clearly did best in my tests compared with unstructured ( JSON ) and Llama Parse. ( Also thank you to <span class=""><span dir="auto" data-jid="32486634341@c.us" data-display="~Vignesh Baskaran Hexo" aria-label="Maybe Vignesh Baskaran Hexo" class="_11JPr">@<span dir="ltr">~Vignesh Baskaran Hexo</span></span></span> for pointing out how in isolation to best convert the table to markdown )

Follow up question - does the same file format/conversion work best with llama-2-70B-chat as well or are there any specific quirks on the llama-2 side?</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Do you mean the chat completion format or something else?</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">3:44 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">3:44 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_D41A059A98F2C279B5488CEBE30F6EAA_12064993617@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Ajay +1 (206) 499-3617" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno ckklvomo"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">A</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d fyja2idx" role=""><span dir="auto" aria-label="Maybe Ajay" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Ajay</span><span class="WJuYU" testid="author" role="button">+1 (206) 499-3617</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[3:51 pm, 25/02/2024] +1 (206) 499-3617: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g9gonq0i _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author">+91 77378 87058</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Do you mean the chat completion format or something else?</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>I mean to chat "with my pdf" using GPT-4, I converted it into the markdown format using Azure AI for Documents ( the pdf contains tables, figures, reports, etc. ). And that worked really well. Can I do exactly the same thing and chat with llama-2-chat? Or have you seen the conversion from pdf to markdown with Azure AI for Document not be the best way to do it?</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">3:51 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">3:51 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="reaction 👍"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍🏻" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -40px -60px;"></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_18820561251708848459_917737397925@c.us"><div class="_2OvAm focusable-list-item _1jHIY"><span class=""></span><div class="UzMP7 n6BPp"><div class="_1BOF7"><div class="_1sykI"><div class="Mk0Bp _30scZ"><span dir="auto" aria-label="" class="l7jjieqr ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">+91 99404 74056</span></div> added <div class="Mk0Bp _30scZ"><span dir="auto" aria-label="" class="l7jjieqr ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">+91 77373 97925</span></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"></div></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB0744DFC51E28A086E08_919632834013@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Kartik Mandaville +91 96328 34013" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://media-tir3-1.cdn.whatsapp.net/v/t61.24694-24/238839000_4525164770861873_3812610583520059178_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRI0xFfzqU2qdwQflFbVM2JukE12YLkI5egUb57euXyJQ&amp;oe=65EB02B6&amp;_nc_sid=e6ed6c&amp;_nc_cat=104" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d giysludt" role=""><span dir="auto" aria-label="Maybe Kartik Mandaville" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Kartik Mandaville</span><span class="WJuYU" testid="author" role="button">+91 96328 34013</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[9:04 pm, 25/02/2024] +91 96328 34013: "><div class="M6sU5"><div class="xzT4z gndfcl4n l8fojup5 paxyh2gw sfeitywo cqsf3vkf k2bacm8l ajgl1lbb f804f6gw sap93d0t ovllcyds kvdvyush gfz4du6o r7fjleex rrq4r3yd"><div class="k5z04t9s cm280p3y f804f6gw ggj6brxn m0h2a7mj lb5m6g5c f8m0rgwh ac2vgrno idwf4z32 o3plsq22 ora14ekb nv3qcefw n1yiu2zv itegkywt eb4rp10x rppts313 ln8gz9je"><div class="aoi073rw suguakab jjtagnqy c32ccnay tvf2evcx m0h2a7mj lb5m6g5c j4enbv94 gfz4du6o r7fjleex lhj4utae ljrqcn24" title="twitter.com"><span dir="ltr" aria-label="" class="ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae i033jvx7 cw3vfol9 _11JPr" style="min-height: 0px;">twitter.com</span></div><div class="jjtagnqy tvf2evcx ttu8nud2 lb5m6g5c dsh4tgtl ora14ekb nv3qcefw lhj4utae"><span class="g0rxnol2 gfz4du6o r7fjleex" style="display: -webkit-box; overflow: hidden; -webkit-box-orient: vertical; -webkit-line-clamp: 2;"><span dir="ltr" title="https://twitter.com/The_AI_Investor/status/1760921566408507529?s=20" aria-label="" class="i033jvx7 _11JPr" style="min-height: 0px;">https://twitter.com/The_AI_Investor/status/1760921566408507529?s=20</span></span></div><div class="j8h4s3v0 tvf2evcx oq44ahr5 lb5m6g5c dsh4tgtl qbqilfqo"><span dir="ltr" aria-label="" class="_11JPr" style="min-height: 0px;">twitter.com</span></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span><a dir="auto" href="https://twitter.com/The_AI_Investor/status/1760921566408507529?s=20" title="https://twitter.com/The_AI_Investor/status/1760921566408507529?s=20" target="_blank" rel="noopener noreferrer" class="_11JPr selectable-text copyable-text">https://twitter.com/The_AI_Investor/status/1760921566408507529?s=20</a>
Anyone understands what is meant by a new way of doing computing?</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">9:04 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">9:04 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_1C2AB4F981B9EF177C11446E230E3FAC_919003135354@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Vetrivel PS +91 90031 35354" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno b2boiwr0"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">V</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d mglqvgej" role=""><span dir="auto" aria-label="Maybe Vetrivel PS" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Vetrivel PS</span><span class="WJuYU" testid="author" role="button">+91 90031 35354</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[9:21 pm, 25/02/2024] +91 90031 35354: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Anyone worked on Google's VERTEX AI.. or have any idea on that ?

My friend is planning to build a search which pulls data out of Salesforce.

He needs suggestions and ideas how to do it ?<img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="😀" draggable="false" class="b85 emoji apple _11JPr selectable-text copyable-text" data-plain-text="😀" tabindex="-1" style="background-position: -20px -60px;"></span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">9:21 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">9:21 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A20B3014B14D39AF081_918890571387@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Hasan +91 88905 71387" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno akr8p6hy"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">H</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d jake6gi8" role=""><span dir="auto" aria-label="Maybe Hasan" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Hasan</span><span class="WJuYU" testid="author" role="button">+91 88905 71387</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[9:38 pm, 25/02/2024] +91 88905 71387: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="a82mwcnm _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d giysludt" role=""><span dir="auto" aria-label="Maybe Kartik Mandaville" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Kartik Mandaville</span><span class="WJuYU" testid="author">+91 96328 34013</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">https://twitter.com/The_AI_Investor/status/1760921566408507529?s=20
Anyone understands what is meant by a new way of doing computing?</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>I have also been trying to make sense of shift. From what I have understood till now is that the technologies we invented solved transportation, processing and storage of information. I can communicate with anyone anywhere instantly but still the technology works like a dumb delivery mechanism. WhatsApp doesn’t know the context of the conversation we are having, gmail handles mail. Sheets manage tables and images/videos all are just different containers to store and transfer information. Now we are moving to a paradigm where the system’s understand the context of the information being stored and transferred. The mediums are becoming interchangeable without human effort and changing specific parameters like style, length, mood etc are changeable individually.…</span></span><div role="button" tabindex="0" class="o0rubyzf le5p0ye3 ajgl1lbb l7jjieqr read-more-button"> Read more</div><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">9:38 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">9:38 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3AC8D9BDE8E20272F541_919868221372@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Paras Chopra +91 98682 21372" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/162690799_131241352786128_2676048871288803711_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRLabAEX9bmDVVjQ8t15GgUazuMiCZOrKVtAXDMdkl8Pw&amp;oe=65EAF085&amp;_nc_sid=e6ed6c&amp;_nc_cat=103" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d jake6gi8" role=""><span dir="auto" aria-label="Maybe Paras Chopra" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Paras Chopra</span><span class="WJuYU" testid="author" role="button">+91 98682 21372</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[10:06 pm, 25/02/2024] +91 98682 21372: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="a82mwcnm _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d giysludt" role=""><span dir="auto" aria-label="Maybe Kartik Mandaville" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Kartik Mandaville</span><span class="WJuYU" testid="author">+91 96328 34013</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">https://twitter.com/The_AI_Investor/status/1760921566408507529?s=20
Anyone understands what is meant by a new way of doing computing?</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>It probably just means computers used to follow their instructions, now they’re trying to guess intent and execute on it</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">10:06 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">10:06 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_315565F1BDEFCD0C1298A90D85EC7A16_919920599420@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Rakesh Waghela +91 99205 99420" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno m8ez62k1"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">R</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d lr20dbn2" role=""><span dir="auto" aria-label="Maybe Rakesh Waghela" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Rakesh Waghela</span><span class="WJuYU" testid="author" role="button">+91 99205 99420</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[10:12 pm, 25/02/2024] +91 99205 99420: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="akr8p6hy _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d jake6gi8" role=""><span dir="auto" aria-label="Maybe Paras Chopra" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Paras Chopra</span><span class="WJuYU" testid="author">+91 98682 21372</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">It probably just means computers used to follow their instructions, now they’re trying to guess intent and execute on it</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>More like they found a blue ocean and capitalised on it well ahead of others ?</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">10:12 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">10:12 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_F7BC9525A493E37DC04DFF111D87C55A_919591374000@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Sayan +91 95913 74000" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno ro7tsnc9"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">S</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d pfvzdcj5" role=""><span dir="auto" aria-label="Maybe Sayan" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Sayan</span><span class="WJuYU" testid="author" role="button">+91 95913 74000</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[10:38 pm, 25/02/2024] +91 95913 74000: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="b2boiwr0 _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d mglqvgej" role=""><span dir="auto" aria-label="Maybe Vetrivel PS" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Vetrivel PS</span><span class="WJuYU" testid="author">+91 90031 35354</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Anyone worked on Google's VERTEX AI.. or have any idea on that ?

My friend is planning to build a search which pulls data out of Salesforce.

He needs suggestions and ideas how to do it ?<img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="😀" draggable="false" class="b85 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Looking at Coveo functionalities might be a good starting point.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">10:38 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">10:38 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_465D6C0EEF84F83EA9444DF62B96D3E4_919003135354@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Vetrivel PS +91 90031 35354" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno b2boiwr0"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">V</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d mglqvgej" role=""><span dir="auto" aria-label="Maybe Vetrivel PS" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Vetrivel PS</span><span class="WJuYU" testid="author" role="button">+91 90031 35354</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[10:46 pm, 25/02/2024] +91 90031 35354: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="ro7tsnc9 _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d pfvzdcj5" role=""><span dir="auto" aria-label="Maybe Sayan" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Sayan</span><span class="WJuYU" testid="author">+91 95913 74000</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Looking at Coveo functionalities might be a good starting point.</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Sure thanks a ton <img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="😀" draggable="false" class="b85 emoji apple _11JPr selectable-text copyable-text" data-plain-text="😀" tabindex="-1" style="background-position: -20px -60px;"></span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">10:46 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">10:46 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_8250CBADC8B18D232A3C60D3D3171C6F_918128536843@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Darshil Jariwala +91 81285 36843" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno b2boiwr0"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">D</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d mglqvgej" role=""><span dir="auto" aria-label="Maybe Darshil Jariwala" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Darshil Jariwala</span><span class="WJuYU" testid="author" role="button">+91 81285 36843</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[11:47 pm, 25/02/2024] +91 81285 36843: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Does anyone have a good reading list for Image Generation Models, looking for something like Road to Stable Diffusion</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">11:47 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">11:47 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="_1jHIY _2OvAm _2zFLj focusable-list-item" tabindex="-1"><div class="UzMP7 n6BPp"><span dir="auto" aria-label="" class="_11JPr" style="min-height: 0px;">YESTERDAY</span></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3AC71313A76C58465534_919820234828@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Abhinav Verma +91 98202 34828" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://media-tir3-1.cdn.whatsapp.net/v/t61.24694-24/56894619_289600311939327_2356728484656578560_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRFDWpEuer3cJRMCvXGRIyIsyOaZGC2iFMBusqiuPsg9Q&amp;oe=65EAD5C7&amp;_nc_sid=e6ed6c&amp;_nc_cat=109" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d lr20dbn2" role=""><span dir="auto" aria-label="Maybe Abhinav Verma" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Abhinav Verma</span><span class="WJuYU" testid="author" role="button">+91 98202 34828</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[0:05 am, 26/02/2024] +91 98202 34828: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Chatgpt helps in devops better than Gemini for now</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">12:05 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">12:05 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_76B170E8ACF014AE4C4383E31C2C984E_32486634341@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _27hEJ _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Vignesh Baskaran Hexo +32 486 63 43 41" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://media-tir3-1.cdn.whatsapp.net/v/t61.24694-24/370347531_1963172344054421_636883848033854865_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdSFU49o1u5QVkKhkGT8YD4W626Y7ak8CwvYXrVMtAFVTQ&amp;oe=65EAF84A&amp;_nc_sid=e6ed6c&amp;_nc_cat=103" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div class="cm280p3y eu4mztcy ocd2b0bc folpon7g aa0kojfi snweb893 g0rxnol2 jnl3jror copyable-text" data-pre-plain-text="[1:02 am, 26/02/2024] +32 486 63 43 41: "><div><div class="bxcbqipq mhcwslh8 ocd2b0bc jfqm35v0"><div class="_3IzYj _6rIWC p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe Vignesh Baskaran Hexo" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Vignesh Baskaran Hexo</span><span class="WJuYU" testid="author" role="button">+32 486 63 43 41</span></div></div><div class="cm280p3y czh1irh3 kzyzudjh"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="b2boiwr0 _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d mglqvgej" role=""><span dir="auto" aria-label="Maybe Darshil Jariwala" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Darshil Jariwala</span><span class="WJuYU" testid="author">+91 81285 36843</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Does anyone have a good reading list for Image Generation Models, looking for something like Road to Stable Diffusion</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div role="button" tabindex="0" aria-label="Open Picture" class="gndfcl4n l8fojup5 paxyh2gw sfeitywo cqsf3vkf p357zi0d ac2vgrno laorhtua gfz4du6o r7fjleex g0rxnol2" style="width: 240px; height: 338.028px;"><span class=""></span><div class="g0rxnol2 ln8gz9je ppled2lx"><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><img alt="At Hexo previously we were working on Image Generation models. This is a study guide that all our engineers followed: https://hexoai.notion.site/Image-Generation-Study-Guide-97b81731c2c64117a54e50cda6149ed8?pvs=4" src="data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEABsbGxscGx4hIR4qLSgtKj04MzM4PV1CR0JHQl2NWGdYWGdYjX2Xe3N7l33gsJycsOD/2c7Z//////////////8BGxsbGxwbHiEhHiotKC0qPTgzMzg9XUJHQkdCXY1YZ1hYZ1iNfZd7c3uXfeCwnJyw4P/Zztn////////////////CABEIAEgAHgMBIgACEQEDEQH/xAArAAADAQEAAAAAAAAAAAAAAAAAAQIDBgEBAQAAAAAAAAAAAAAAAAAAAAH/2gAMAwEAAhADEAAAAOicMubmxOXLZNmSsGxErQAJKAEwP//EACEQAQADAAEEAwEBAAAAAAAAAAEAAhEhAxASMUFRYSKh/9oACAEBAAE/AH/YXuGFZ5OOzptFf6mW87KEG485GoiMr0aUdCXtjKXpb2Zk0+JlvuXLNuJ4j8kDOD12sbeHTocg9tIttwjXq/ky0KMs2LSjd3bR9+5p9xBhUr6iV39h06nbc5yYD5Zz2//EABYRAAMAAAAAAAAAAAAAAAAAAAAhMP/aAAgBAgEBPwAUf//EABYRAAMAAAAAAAAAAAAAAAAAAAAhMP/aAAgBAwEBPwAcf//Z" class="jciay5ix tvf2evcx oq44ahr5 lb5m6g5c fsmudgz7" style="width: 100%;"></div><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><img alt="At Hexo previously we were working on Image Generation models. This is a study guide that all our engineers followed: https://hexoai.notion.site/Image-Generation-Study-Guide-97b81731c2c64117a54e50cda6149ed8?pvs=4" src="blob:https://web.whatsapp.com/c11d3470-d040-4b56-9769-9e03c41a1612" class="jciay5ix tvf2evcx oq44ahr5 lb5m6g5c osz0hll6 nq7eualt em5jvqoa a21kwdn3" style="width: 100%;"></div></div></div><div class="cm280p3y czh1irh3 esbfogcw jfqm35v0 bxcbqipq mhcwslh8"><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>At Hexo previously we were working on Image Generation models. This is a study guide that all our engineers followed: <a dir="auto" href="https://hexoai.notion.site/Image-Generation-Study-Guide-97b81731c2c64117a54e50cda6149ed8?pvs=4" title="https://hexoai.notion.site/Image-Generation-Study-Guide-97b81731c2c64117a54e50cda6149ed8?pvs=4" target="_blank" rel="noopener noreferrer" class="_11JPr selectable-text copyable-text">https://hexoai.notion.site/Image-Generation-Study-Guide-97b81731c2c64117a54e50cda6149ed8?pvs=4</a></span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:02 am</span></span></span></div></div><div class="lrw9n60e lhggkp7q fz4q5utg b9fczbqn"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:02 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 cvsrm5e1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="Reactions ❤, 👍, 🙏 9 in total"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="❤️" draggable="false" class="bd96tnyg eipqoq80 b8 emoji apple _11JPr" tabindex="-1" style="background-position: -20px 0px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="🙏🏼" draggable="false" class="bd96tnyg p83aruvr b93 emoji apple _11JPr" tabindex="-1" style="background-position: -40px -20px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk o2es7gts i2cterl7" style="flex-shrink: 0;"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 f8m0rgwh r15c9g6i ilf8vifs gfz4du6o r7fjleex qnz2jpws kjemk6od"><div class="dn5wc90o ilf8vifs gndfcl4n p357zi0d"><span class="f8jlpxt4 qfejxiq4 ce5kru2g">9</span></div></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_C0604D955E84A408B3D86E80B89996D3_918128536843@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Darshil Jariwala +91 81285 36843" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno b2boiwr0"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">D</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d mglqvgej" role=""><span dir="auto" aria-label="Maybe Darshil Jariwala" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Darshil Jariwala</span><span class="WJuYU" testid="author" role="button">+91 81285 36843</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[1:03 am, 26/02/2024] +91 81285 36843: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="po56q9xr _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe Vignesh Baskaran Hexo" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Vignesh Baskaran Hexo</span><span class="WJuYU" testid="author">+32 486 63 43 41</span></div><div class="_37DQv" dir="ltr" role="button"><div class="l7jjieqr tvf2evcx oq44ahr5 lb5m6g5c g97rt3p3 fewfhwl7 jdwybkuq kjemk6od"><span data-icon="status-image" class=""><svg viewBox="0 0 16 20" height="20" width="16" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 16 20"><title>status-image</title><path fill="currentColor" d="M13.822,4.668H7.14l-1.068-1.09C5.922,3.425,5.624,3.3,5.409,3.3H3.531 c-0.214,0-0.51,0.128-0.656,0.285L1.276,5.296C1.13,5.453,1.01,5.756,1.01,5.971v1.06c0,0.001-0.001,0.002-0.001,0.003v6.983 c0,0.646,0.524,1.17,1.17,1.17h11.643c0.646,0,1.17-0.524,1.17-1.17v-8.18C14.992,5.191,14.468,4.668,13.822,4.668z M7.84,13.298 c-1.875,0-3.395-1.52-3.395-3.396c0-1.875,1.52-3.395,3.395-3.395s3.396,1.52,3.396,3.395C11.236,11.778,9.716,13.298,7.84,13.298z  M7.84,7.511c-1.321,0-2.392,1.071-2.392,2.392s1.071,2.392,2.392,2.392s2.392-1.071,2.392-2.392S9.161,7.511,7.84,7.511z"></path></svg></span></div><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">At Hexo previously we were working on Image Generation models. This is a study guide that all our engineers followed: https://hexoai.notion.site/Image-Generation-Study-Guide-97b81731c2c64117a54e50cda6149ed8?pvs=4</span></div></div></div><div class="jrEtJ"><div class="kknmh"><div class="g0rxnol2 ln8gz9je ppled2lx"><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><div class="lhggkp7q ln8gz9je ppled2lx t3g6t33p qnwaluaf" style="background-image: url(&quot;data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEABsbGxscGx4hIR4qLSgtKj04MzM4PV1CR0JHQl2NWGdYWGdYjX2Xe3N7l33gsJycsOD/2c7Z//////////////8BGxsbGxwbHiEhHiotKC0qPTgzMzg9XUJHQkdCXY1YZ1hYZ1iNfZd7c3uXfeCwnJyw4P/Zztn////////////////CABEIAEgAHgMBIgACEQEDEQH/xAArAAADAQEAAAAAAAAAAAAAAAAAAQIDBgEBAQAAAAAAAAAAAAAAAAAAAAH/2gAMAwEAAhADEAAAAOicMubmxOXLZNmSsGxErQAJKAEwP//EACEQAQADAAEEAwEBAAAAAAAAAAEAAhEhAxASMUFRYSKh/9oACAEBAAE/AH/YXuGFZ5OOzptFf6mW87KEG485GoiMr0aUdCXtjKXpb2Zk0+JlvuXLNuJ4j8kDOD12sbeHTocg9tIttwjXq/ky0KMs2LSjd3bR9+5p9xBhUr6iV39h06nbc5yYD5Zz2//EABYRAAMAAAAAAAAAAAAAAAAAAAAhMP/aAAgBAgEBPwAUf//EABYRAAMAAAAAAAAAAAAAAAAAAAAhMP/aAAgBAwEBPwAcf//Z&quot;);"></div></div><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><div class="lhggkp7q ln8gz9je ppled2lx t3g6t33p qnwaluaf" style="background-image: url(&quot;blob:https://web.whatsapp.com/c11d3470-d040-4b56-9769-9e03c41a1612&quot;);"></div></div></div></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>This is great, thank you for sharing</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:03 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:03 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj _3sxvM" data-id="false_120363049558306142@g.us_A9B29B701A254309F276C60825A67966_918072173935@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Anubhav Mishra +91 80721 73935" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/422474244_2207779056257205_6380017147338436444_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdQcBWFoU7N5fP-tIl9Il6cyVmnu6_LWDTkkQQWClawzfg&amp;oe=65EAE6EE&amp;_nc_sid=e6ed6c&amp;_nc_cat=104" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d gvktkc8w" role=""><span dir="auto" aria-label="Maybe Anubhav Mishra" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Anubhav Mishra</span><span class="WJuYU" testid="author" role="button">+91 80721 73935</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:22 am, 26/02/2024] +91 80721 73935: "><div class="M6sU5"><div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span><a dir="auto" href="https://feather.openai.com/" title="https://feather.openai.com/" target="_blank" rel="noopener noreferrer" class="_11JPr selectable-text copyable-text">https://feather.openai.com/</a></span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">2:22 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:22 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_64AEAA271F14DD5C90E37D6E95FBF913_918072173935@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a"><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="copyable-text" data-pre-plain-text="[2:22 am, 26/02/2024] +91 80721 73935: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>What's this ?Any idea?</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">2:22 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:22 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB07478AB5B31DDA2F83A_918527011187@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Ashish Anand +91 85270 11187" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno po56q9xr"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">A</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe Ashish Anand" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Ashish Anand</span><span class="WJuYU" testid="author" role="button">+91 85270 11187</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:29 am, 26/02/2024] +91 85270 11187: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="j3zl77sr _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d gvktkc8w" role=""><span dir="auto" aria-label="Maybe Anubhav Mishra" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Anubhav Mishra</span><span class="WJuYU" testid="author">+91 80721 73935</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">What's this ?Any idea?</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="M6sU5"><div class="xzT4z gndfcl4n l8fojup5 paxyh2gw sfeitywo cqsf3vkf k2bacm8l ajgl1lbb p357zi0d sap93d0t ovllcyds kvdvyush gfz4du6o r7fjleex rrq4r3yd"><div class="k5z04t9s cm280p3y p357zi0d ggj6brxn m0h2a7mj lb5m6g5c f8m0rgwh ac2vgrno idwf4z32 onmzirdk gfz4du6o r7fjleex n1yiu2zv itegkywt eb4rp10x rppts313 ln8gz9je"><div class="aoi073rw suguakab jjtagnqy c32ccnay tvf2evcx m0h2a7mj lb5m6g5c j4enbv94 gfz4du6o r7fjleex lhj4utae ljrqcn24" title="www.reddit.com"><span dir="ltr" aria-label="" class="ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae i033jvx7 cw3vfol9 _11JPr" style="min-height: 0px;">www.reddit.com</span></div><div class="jjtagnqy tvf2evcx ttu8nud2 lb5m6g5c dsh4tgtl gfz4du6o r7fjleex lhj4utae"><span class="g0rxnol2 gfz4du6o r7fjleex" style="display: -webkit-box; overflow: hidden; -webkit-box-orient: vertical; -webkit-line-clamp: 2;"><span dir="ltr" title="https://www.reddit.com/r/OpenAI/comments/17ybes0/comment/k9tc04x/" aria-label="" class="i033jvx7 _11JPr" style="min-height: 0px;">https://www.reddit.com/r/OpenAI/comments/17ybes0/comment/k9tc04x/</span></span></div><div class="j8h4s3v0 tvf2evcx oq44ahr5 lb5m6g5c dsh4tgtl qbqilfqo"><span dir="ltr" aria-label="" class="_11JPr" style="min-height: 0px;">www.reddit.com</span></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span><a dir="auto" href="https://www.reddit.com/r/OpenAI/comments/17ybes0/comment/k9tc04x/" title="https://www.reddit.com/r/OpenAI/comments/17ybes0/comment/k9tc04x/" target="_blank" rel="noopener noreferrer" class="_11JPr selectable-text copyable-text">https://www.reddit.com/r/OpenAI/comments/17ybes0/comment/k9tc04x/</a></span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">2:29 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:29 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_7189B0A80A8C0BF74B4A32E0244E1128_919334372044@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _27hEJ _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Sid +91 93343 72044" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno akr8p6hy"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">S</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div class="cm280p3y eu4mztcy ocd2b0bc folpon7g aa0kojfi snweb893 g0rxnol2 jnl3jror copyable-text" data-pre-plain-text="[2:30 am, 26/02/2024] +91 93343 72044: "><div><div class="bxcbqipq mhcwslh8 ocd2b0bc jfqm35v0"><div class="_3IzYj _6rIWC p357zi0d jake6gi8" role=""><span dir="auto" aria-label="Maybe Sid" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Sid</span><span class="WJuYU" testid="author" role="button">+91 93343 72044</span></div></div><div class="cm280p3y czh1irh3 kzyzudjh"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="po56q9xr _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe Ashish Anand" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Ashish Anand</span><span class="WJuYU" testid="author">+91 85270 11187</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">https://www.reddit.com/r/OpenAI/comments/17ybes0/comment/k9tc04x/</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div role="button" tabindex="0" aria-label="Open Picture" class="gndfcl4n l8fojup5 paxyh2gw sfeitywo cqsf3vkf p357zi0d ac2vgrno laorhtua gfz4du6o r7fjleex g0rxnol2" style="width: 240px; height: 338.028px;"><span class=""></span><div class="g0rxnol2 ln8gz9je ppled2lx"><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><img alt="Interesting" src="data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEABsbGxscGx4hIR4qLSgtKj04MzM4PV1CR0JHQl2NWGdYWGdYjX2Xe3N7l33gsJycsOD/2c7Z//////////////8BGxsbGxwbHiEhHiotKC0qPTgzMzg9XUJHQkdCXY1YZ1hYZ1iNfZd7c3uXfeCwnJyw4P/Zztn////////////////CABEIAEgAJgMBIgACEQEDEQH/xAAsAAADAQEBAAAAAAAAAAAAAAAAAgMBBAUBAQEAAAAAAAAAAAAAAAAAAAAB/9oADAMBAAIQAxAAAADxmk9lEQEFCuLUllkJGg9o3CdEIADuUJpdTnNCuAAAgB//xAAiEAACAQIGAwEAAAAAAAAAAAAAAQIRMQMQEiEyQRMiUWL/2gAIAQEAAT8Aw+I6tl1ctsT5Mjqoexqad0Tk2rrKG65UNUa3Z6j09ZQVWaI/DQl0OP5GYfLPEl1lDkMpZk9+hqhCLuj3Gp0HCSVcsOw2J7j3T3GknsYcZNVQ4SbPHL6zxP6yUdLof//EABQRAQAAAAAAAAAAAAAAAAAAADD/2gAIAQIBAT8Af//EABQRAQAAAAAAAAAAAAAAAAAAADD/2gAIAQMBAT8Af//Z" class="jciay5ix tvf2evcx oq44ahr5 lb5m6g5c fsmudgz7" style="width: 100%;"></div><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><img alt="Interesting" src="blob:https://web.whatsapp.com/11c71d5b-8fb5-4f22-aeb6-a78490acf277" class="jciay5ix tvf2evcx oq44ahr5 lb5m6g5c osz0hll6 nq7eualt em5jvqoa a21kwdn3" style="width: 100%;"></div></div></div><div class="cm280p3y czh1irh3 esbfogcw jfqm35v0 bxcbqipq mhcwslh8"><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Interesting</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">2:30 am</span></span></span></div></div><div class="lrw9n60e lhggkp7q fz4q5utg b9fczbqn"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:30 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 cvsrm5e1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj _3sxvM" data-id="false_120363049558306142@g.us_3EB0D5EC01A95DC0834A3E_19377081307@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Dr. Pratik Desai +1 (937) 708-1307" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/407052186_368942458989574_5837140328940210822_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRT7N2iCE5tWNPEgrn8zUMppMyQOh2yI-vDe0xjKk_mmA&amp;oe=65EAD8D6&amp;_nc_sid=e6ed6c&amp;_nc_cat=110" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe Dr. Pratik Desai" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Dr. Pratik Desai</span><span class="WJuYU" testid="author" role="button">+1 (937) 708-1307</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[3:00 am, 26/02/2024] +1 (937) 708-1307: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Is it a platform or consulting service?</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">3:00 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">3:00 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB01FC210C6A0E3417230_19377081307@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a"><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="copyable-text" data-pre-plain-text="[3:00 am, 26/02/2024] +1 (937) 708-1307: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>If a Platform, this is amazing, as I haven't found a single good curation tool.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">3:00 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">3:00 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_D22D3AC373D2602608DDEA23C1395D51_919315659680@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Hemant Mohapatra +91 93156 59680" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno g9gonq0i"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">H</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Hemant Mohapatra" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Hemant Mohapatra</span><span class="WJuYU" testid="author" role="button">+91 93156 59680</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[6:55 am, 26/02/2024] +91 93156 59680: "><div class="M6sU5"><div class="xzT4z gndfcl4n l8fojup5 paxyh2gw sfeitywo cqsf3vkf k2bacm8l ajgl1lbb p357zi0d sap93d0t ovllcyds kvdvyush gfz4du6o r7fjleex rrq4r3yd"><div class="k5z04t9s cm280p3y p357zi0d ggj6brxn m0h2a7mj lb5m6g5c f8m0rgwh ac2vgrno idwf4z32 onmzirdk gfz4du6o r7fjleex n1yiu2zv itegkywt eb4rp10x rppts313 ln8gz9je"><div class="aoi073rw suguakab jjtagnqy c32ccnay tvf2evcx m0h2a7mj lb5m6g5c j4enbv94 gfz4du6o r7fjleex lhj4utae ljrqcn24" title="The Shift from Models to Compound AI Systems"><span dir="ltr" aria-label="" class="ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae i033jvx7 cw3vfol9 _11JPr" style="min-height: 0px;">The Shift from Models to Compound AI Systems</span></div><div class="jjtagnqy tvf2evcx ttu8nud2 lb5m6g5c dsh4tgtl gfz4du6o r7fjleex lhj4utae"><span class="g0rxnol2 gfz4du6o r7fjleex" style="display: -webkit-box; overflow: hidden; -webkit-box-orient: vertical; -webkit-line-clamp: 2;"><span dir="ltr" title="The BAIR Blog" aria-label="" class="i033jvx7 _11JPr" style="min-height: 0px;">The BAIR Blog</span></span></div><div class="j8h4s3v0 tvf2evcx oq44ahr5 lb5m6g5c dsh4tgtl qbqilfqo"><span dir="ltr" aria-label="" class="_11JPr" style="min-height: 0px;">bair.berkeley.edu</span></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span><a dir="auto" href="http://bair.berkeley.edu/blog/2024/02/18/compound-ai-systems/" title="http://bair.berkeley.edu/blog/2024/02/18/compound-ai-systems/" target="_blank" rel="noopener noreferrer" class="_11JPr selectable-text copyable-text">http://bair.berkeley.edu/blog/2024/02/18/compound-ai-systems/</a> a pretty interesting paper with some real implications on how to use LLMs in production.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">6:55 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">6:55 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="Reactions ❤, 👍 4 in total"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="❤️" draggable="false" class="bd96tnyg eipqoq80 b8 emoji apple _11JPr" tabindex="-1" style="background-position: -20px 0px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk o2es7gts i2cterl7" style="flex-shrink: 0;"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 f8m0rgwh r15c9g6i ilf8vifs gfz4du6o r7fjleex qnz2jpws kjemk6od"><div class="dn5wc90o ilf8vifs gndfcl4n p357zi0d"><span class="f8jlpxt4 qfejxiq4 ce5kru2g">4</span></div></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj _3sxvM" data-id="false_120363049558306142@g.us_3AB24057DB65D6F51764_917338007553@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Neha Nupoor +91 73380 07553" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno ckklvomo"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">N</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d fyja2idx" role=""><span dir="auto" aria-label="Maybe Neha Nupoor" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Neha Nupoor</span><span class="WJuYU" testid="author" role="button">+91 73380 07553</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[7:04 am, 26/02/2024] +91 73380 07553: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="po56q9xr _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe Dr. Pratik Desai" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Dr. Pratik Desai</span><span class="WJuYU" testid="author">+1 (937) 708-1307</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">If a Platform, this is amazing, as I haven't found a single good curation tool.</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>We are building it. Support datagen for 1000of rows in minutes. 8-10 type of labelled data pairs. Im not sure if itll be a platform tbh, and what capabilities they will give. A full foedged platform is in our roadmap. Ill ping you separately, i our chat is long overdue</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">7:04 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">7:04 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="Reactions 👍 4 in total"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk o2es7gts i2cterl7" style="flex-shrink: 0;"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 f8m0rgwh r15c9g6i ilf8vifs gfz4du6o r7fjleex qnz2jpws kjemk6od"><div class="dn5wc90o ilf8vifs gndfcl4n p357zi0d"><span class="f8jlpxt4 qfejxiq4 ce5kru2g">4</span></div></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A88BECDDF34FC41AC21_917338007553@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a"><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="copyable-text" data-pre-plain-text="[7:04 am, 26/02/2024] +91 73380 07553: "><div class="_21Ahp"><span dir="auto" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>:)</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">7:04 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">7:04 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_23978571071708890050_14168364476@c.us"><div class="_2OvAm focusable-list-item _1jHIY"><span class=""></span><div class="UzMP7 n6BPp"><div class="_1BOF7"><div class="_1sykI"><div class="Mk0Bp _30scZ"><span dir="auto" aria-label="" class="l7jjieqr ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">+91 95501 64716</span></div> added <div class="Mk0Bp _30scZ"><span dir="auto" aria-label="" class="l7jjieqr ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">+1 (416) 836-4476</span></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"></div></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A6957BF17D0B4D704EC_917892563038@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe ashish +91 78925 63038" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno po56q9xr"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">A</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe ashish" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">ashish</span><span class="WJuYU" testid="author" role="button">+91 78925 63038</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[9:05 am, 26/02/2024] +91 78925 63038: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g9gonq0i _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Hemant Mohapatra" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Hemant Mohapatra</span><span class="WJuYU" testid="author">+91 93156 59680</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">http://bair.berkeley.edu/blog/2024/02/18/compound-ai-systems/ a pretty interesting paper with some real implications on how to use LLMs in production.</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Great read !
tl/dr : stand alone models alone are not the binding constraint on great ai systems,
model + system architecture + ops(monitoring/observability) + sw engg will make ai systems <strong class="_11JPr selectable-text copyable-text" data-app-text-template="*${appText}*">work well</strong>.

Maybe a <strong class="_11JPr selectable-text copyable-text" data-app-text-template="*${appText}*">great model</strong>, with mediocre system arch &amp; other components &lt;&lt;&lt; (could be beaten by) a decent model + good system arch + good ops + good sw engg

<span role="button" tabindex="0" class=""><span dir="auto" data-jid="919315659680@c.us" data-display="~Hemant Mohapatra" aria-label="Maybe Hemant Mohapatra" class="ajgl1lbb o0rubyzf _11JPr selectable-text select-all copyable-text" data-plain-text="@~Hemant Mohapatra" data-app-text-template="​919315659680@c.us​">@<span dir="ltr">~Hemant Mohapatra</span></span></span> what were your takeaways from the blog ?
Do you think perplexity (or some other system) embodies the qualities of a <strong class="_11JPr selectable-text copyable-text" data-app-text-template="*${appText}*">good</strong> compound ai system ?

maybe naive thinking, but IMHO, in really good compound ai systems e.g. like google maps real time travel prediction, the model actually fades away from the core ux and becomes transparent.
(Good tech fades into the background - Alan Kay ?)
So i would add t…</span></span><div role="button" tabindex="0" class="o0rubyzf le5p0ye3 ajgl1lbb l7jjieqr read-more-button"> Read more</div><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">9:05 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">9:05 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_83DAB13E48484CFC9D5D25BA2A71B09A_919315659680@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Hemant Mohapatra +91 93156 59680" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno g9gonq0i"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">H</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Hemant Mohapatra" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Hemant Mohapatra</span><span class="WJuYU" testid="author" role="button">+91 93156 59680</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[9:07 am, 26/02/2024] +91 93156 59680: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="po56q9xr _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe ashish" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">ashish</span><span class="WJuYU" testid="author">+91 78925 63038</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Great read !
tl/dr : stand alone models alone are not the binding constraint on great ai systems,
model + system architecture + ops(monitoring/observability) + sw engg will make ai systems <strong class="_11JPr">work well</strong>.

Maybe a <strong class="_11JPr">great model</strong>, with mediocre system arch &amp; other components &lt;&lt;&lt; (could be beaten by) a decent model + good system arch + good ops + good sw engg

<span class=""><span dir="auto" data-jid="919315659680@c.us" data-display="~Hemant Mohapatra" aria-label="Maybe Hemant Mohapatra" class="_11JPr">@<span dir="ltr">~Hemant Mohapatra</span></span></span> what were your takeaways from the blog ?
Do you think perplexity (or some other system) embodies the qualities of a <strong class="_11JPr">good</strong> compound ai system ?

maybe naive thinking, but IMHO, in really good compound ai systems e.g. like google maps real time travel prediction, the model actually fades away from the core ux and becomes transparent.
(Good tech fades into the background - Alan Kay ?)
So i would add t</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>I had the same takeaways. A dumb LLM + smart scaffolding may be better than trying to fine tune or train a smart LLM to use without the same scaffolding.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">9:07 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">9:07 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="reaction ❤"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="❤️" draggable="false" class="bd96tnyg eipqoq80 b8 emoji apple _11JPr" tabindex="-1" style="background-position: -20px 0px;"></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj _3sxvM" data-id="false_120363049558306142@g.us_3EB0ECA7725093BBBF262C_919538104545@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Prakash +91 95381 04545" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://media-tir3-1.cdn.whatsapp.net/v/t61.24694-24/401465275_3600251153635990_7908427480903878831_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRNfXvODk8oqYeXargUy-TM8FGqnGtTr7lxDIBpn_IvSg&amp;oe=65EB0194&amp;_nc_sid=e6ed6c&amp;_nc_cat=104" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d dud7p2xx" role=""><span dir="auto" aria-label="Maybe Prakash" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Prakash</span><span class="WJuYU" testid="author" role="button">+91 95381 04545</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[9:11 am, 26/02/2024] +91 95381 04545: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="po56q9xr _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe ashish" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">ashish</span><span class="WJuYU" testid="author">+91 78925 63038</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Great read !
tl/dr : stand alone models alone are not the binding constraint on great ai systems,
model + system architecture + ops(monitoring/observability) + sw engg will make ai systems <strong class="_11JPr">work well</strong>.

Maybe a <strong class="_11JPr">great model</strong>, with mediocre system arch &amp; other components &lt;&lt;&lt; (could be beaten by) a decent model + good system arch + good ops + good sw engg

<span class=""><span dir="auto" data-jid="919315659680@c.us" data-display="~Hemant Mohapatra" aria-label="Maybe Hemant Mohapatra" class="_11JPr">@<span dir="ltr">~Hemant Mohapatra</span></span></span> what were your takeaways from the blog ?
Do you think perplexity (or some other system) embodies the qualities of a <strong class="_11JPr">good</strong> compound ai system ?

maybe naive thinking, but IMHO, in really good compound ai systems e.g. like google maps real time travel prediction, the model actually fades away from the core ux and becomes transparent.
(Good tech fades into the background - Alan Kay ?)
So i would add t</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>I don't understand why this is a great article. Isn't it basically saying that LLMs are one part of your system, build your product around the LLM and do w/e it takes to give your user a good product experience.

I mean with the stage even GPT-4 is at and given LLMs are inherently probabilistic functions isn't this always going to be the case?</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">9:11 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">9:11 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB07A8EE5DCCE77EAF8AD_919538104545@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a"><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="copyable-text" data-pre-plain-text="[9:12 am, 26/02/2024] +91 95381 04545: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>if you want to use them to replace processes in an intelligent way, then ya, you need to build around them - but I feel like anybody building products with them could tell you this</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">9:12 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">9:12 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A2F62D9C23E1CFF1D9E_918763968157@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Pratyush Choudhury (PC) +91 87639 68157" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno oj03u76w"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">P</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d dud7p2xx" role=""><span dir="auto" aria-label="Maybe Pratyush Choudhury (PC)" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Pratyush Choudhury (PC)</span><span class="WJuYU" testid="author" role="button">+91 87639 68157</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[9:12 am, 26/02/2024] +91 87639 68157: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="po56q9xr _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe ashish" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">ashish</span><span class="WJuYU" testid="author">+91 78925 63038</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Great read !
tl/dr : stand alone models alone are not the binding constraint on great ai systems,
model + system architecture + ops(monitoring/observability) + sw engg will make ai systems <strong class="_11JPr">work well</strong>.

Maybe a <strong class="_11JPr">great model</strong>, with mediocre system arch &amp; other components &lt;&lt;&lt; (could be beaten by) a decent model + good system arch + good ops + good sw engg

<span class=""><span dir="auto" data-jid="919315659680@c.us" data-display="~Hemant Mohapatra" aria-label="Maybe Hemant Mohapatra" class="_11JPr">@<span dir="ltr">~Hemant Mohapatra</span></span></span> what were your takeaways from the blog ?
Do you think perplexity (or some other system) embodies the qualities of a <strong class="_11JPr">good</strong> compound ai system ?

maybe naive thinking, but IMHO, in really good compound ai systems e.g. like google maps real time travel prediction, the model actually fades away from the core ux and becomes transparent.
(Good tech fades into the background - Alan Kay ?)
So i would add t</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Counter question- beauty of SaaS &amp; API movement has been around thoughtful abstractions for the end user or a dev to perform tasks without caring too much about what's under the hood.

The complex AI architecture turns that on its head and creates more points of failure?</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">9:12 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">9:12 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj _3sxvM" data-id="false_120363049558306142@g.us_E98D2F1A0AF9422EE22DAD40DC695A3B_919731933997@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Rajesh Parikh +91 97319 33997" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno exofpctk"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">R</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d a1q56urb" role=""><span dir="auto" aria-label="Maybe Rajesh Parikh" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Rajesh Parikh</span><span class="WJuYU" testid="author" role="button">+91 97319 33997</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[9:16 am, 26/02/2024] +91 97319 33997: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>That's not the right way to find the binary SaaS vs AI. We called out integrated approach much before in December when Sanjeev and I published the trends for this year. BAIR paper is only a validation of that.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5 hnx8ox4h mw4yctpw gj5xqxfh">Edited</span><span class="tvf2evcx oq44ahr5">9:16 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto"><span class="mw4yctpw gj5xqxfh">Edited</span>9:16 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj _3sxvM" data-id="false_120363049558306142@g.us_3623BABA1D25E28B6291F4B83ED03841_919731933997@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a"><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="copyable-text" data-pre-plain-text="[9:17 am, 26/02/2024] +91 97319 33997: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>There's a good alchemy at play here. Just like sql engines, models are another set of engines that play in an overall system or application architectures</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">9:17 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">9:17 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj _3sxvM" data-id="false_120363049558306142@g.us_C7E38B6AEAA63A70D4636C9EC544807F_919731933997@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a"><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="copyable-text" data-pre-plain-text="[9:18 am, 26/02/2024] +91 97319 33997: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Unlike what is a common belief that AI is eating software , we see most early outcomes are adding layers of software around model.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">9:18 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">9:18 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_7CB14E86CE085052942B7443B8D3D194_919731933997@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a"><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="copyable-text" data-pre-plain-text="[9:18 am, 26/02/2024] +91 97319 33997: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Most likely scenario is it will be finding best of both worlds and both eating each other where their strengths lie</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">9:18 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">9:18 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A74F6369A4C4D12BEFB_917892563038@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe ashish +91 78925 63038" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno po56q9xr"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">A</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe ashish" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">ashish</span><span class="WJuYU" testid="author" role="button">+91 78925 63038</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[9:19 am, 26/02/2024] +91 78925 63038: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="oj03u76w _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d dud7p2xx" role=""><span dir="auto" aria-label="Maybe Prakash" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Prakash</span><span class="WJuYU" testid="author">+91 95381 04545</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">I don't understand why this is a great article. Isn't it basically saying that LLMs are one part of your system, build your product around the LLM and do w/e it takes to give your user a good product experience.

I mean with the stage even GPT-4 is at and given LLMs are inherently probabilistic functions isn't this always going to be the case?</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="M6sU5"><div class="xzT4z gndfcl4n l8fojup5 paxyh2gw sfeitywo cqsf3vkf k2bacm8l ajgl1lbb p357zi0d sap93d0t ovllcyds kvdvyush gfz4du6o r7fjleex rrq4r3yd"><div class="k5z04t9s cm280p3y p357zi0d ggj6brxn m0h2a7mj lb5m6g5c f8m0rgwh ac2vgrno idwf4z32 onmzirdk gfz4du6o r7fjleex n1yiu2zv itegkywt eb4rp10x rppts313 ln8gz9je"><div class="aoi073rw suguakab jjtagnqy c32ccnay tvf2evcx m0h2a7mj lb5m6g5c j4enbv94 gfz4du6o r7fjleex lhj4utae ljrqcn24" title="Natural Language Is an Unnatural Interface"><span dir="ltr" aria-label="" class="ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae i033jvx7 cw3vfol9 _11JPr" style="min-height: 0px;">Natural Language Is an Unnatural Interface</span></div><div class="jjtagnqy tvf2evcx ttu8nud2 lb5m6g5c dsh4tgtl gfz4du6o r7fjleex lhj4utae"><span class="g0rxnol2 gfz4du6o r7fjleex" style="display: -webkit-box; overflow: hidden; -webkit-box-orient: vertical; -webkit-line-clamp: 2;"><span dir="ltr" title="How can we use familiar design patterns to constrain the unwieldy state space of large language models?" aria-label="" class="i033jvx7 _11JPr" style="min-height: 0px;">How can we use familiar design patterns to constrain the unwieldy state space of large language models?</span></span></div><div class="j8h4s3v0 tvf2evcx oq44ahr5 lb5m6g5c dsh4tgtl qbqilfqo"><span dir="ltr" aria-label="" class="_11JPr" style="min-height: 0px;">varunshenoy.substack.com</span></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>maybe i will temper my excitement :)
the piece just articulated &amp; helped reinforce some of my own mental models around what makes a <strong class="_11JPr selectable-text copyable-text" data-app-text-template="*${appText}*">great system</strong> vs a <strong class="_11JPr selectable-text copyable-text" data-app-text-template="*${appText}*">great model</strong> and we shuld be cautious of conflating the two.

on the hci/ux/design on integrating the model with the surrounding <strong class="_11JPr selectable-text copyable-text" data-app-text-template="*${appText}*">system scaffolding</strong>, i would posit that natural language inputs liuse casesLLMs are suboptimal for most usecases, and we need to come up with newer ux, prompted by reading this excellent piece :

<a dir="auto" href="https://varunshenoy.substack.com/p/natural-language-is-an-unnatural" title="https://varunshenoy.substack.com/p/natural-language-is-an-unnatural" target="_blank" rel="noopener noreferrer" class="_11JPr selectable-text copyable-text">https://varunshenoy.substack.com/p/natural-language-is-an-unnatural</a>
cc: <span role="button" tabindex="0" class=""><span dir="auto" data-jid="919315659680@c.us" data-display="~Hemant Mohapatra" aria-label="Maybe Hemant Mohapatra" class="ajgl1lbb o0rubyzf _11JPr selectable-text select-all copyable-text" data-plain-text="@~Hemant Mohapatra" data-app-text-template="​919315659680@c.us​">@<span dir="ltr">~Hemant Mohapatra</span></span></span> <span role="button" tabindex="0" class=""><span dir="auto" data-jid="919868221372@c.us" data-display="~Paras Chopra" aria-label="Maybe Paras Chopra" class="ajgl1lbb o0rubyzf _11JPr selectable-text select-all copyable-text" data-plain-text="@~Paras Chopra" data-app-text-template="​919868221372@c.us​">@<span dir="ltr">~Paras Chopra</span></span></span></span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5 hnx8ox4h mw4yctpw gj5xqxfh">Edited</span><span class="tvf2evcx oq44ahr5">9:19 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto"><span class="mw4yctpw gj5xqxfh">Edited</span>9:19 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="Reactions 👍 3 in total"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk o2es7gts i2cterl7" style="flex-shrink: 0;"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 f8m0rgwh r15c9g6i ilf8vifs gfz4du6o r7fjleex qnz2jpws kjemk6od"><div class="dn5wc90o ilf8vifs gndfcl4n p357zi0d"><span class="f8jlpxt4 qfejxiq4 ce5kru2g">3</span></div></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB04E4F7EBCE33AC5E415_919538104545@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Prakash +91 95381 04545" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://media-tir3-1.cdn.whatsapp.net/v/t61.24694-24/401465275_3600251153635990_7908427480903878831_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRNfXvODk8oqYeXargUy-TM8FGqnGtTr7lxDIBpn_IvSg&amp;oe=65EB0194&amp;_nc_sid=e6ed6c&amp;_nc_cat=104" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d dud7p2xx" role=""><span dir="auto" aria-label="Maybe Prakash" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Prakash</span><span class="WJuYU" testid="author" role="button">+91 95381 04545</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[9:24 am, 26/02/2024] +91 95381 04545: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="po56q9xr _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe ashish" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">ashish</span><span class="WJuYU" testid="author">+91 78925 63038</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">maybe i will temper my excitement :)
the piece just articulated &amp; helped reinforce some of my own mental models around what makes a <strong class="_11JPr">great system</strong> vs a <strong class="_11JPr">great model</strong> and we shuld be cautious of conflating the two.

on the hci/ux/design on integrating the model with the surrounding <strong class="_11JPr">system scaffolding</strong>, i would posit that natural language inputs liuse casesLLMs are suboptimal for most usecases, and we need to come up with newer ux, prompted by reading this excellent piece :

https://varunshenoy.substack.com/p/natural-language-is-an-unnatural
cc: <span class=""><span dir="auto" data-jid="919315659680@c.us" data-display="~Hemant Mohapatra" aria-label="Maybe Hemant Mohapatra" class="_11JPr">@<span dir="ltr">~Hemant Mohapatra</span></span></span> <span class=""><span dir="auto" data-jid="919868221372@c.us" data-display="~Paras Chopra" aria-label="Maybe Paras Chopra" class="_11JPr">@<span dir="ltr">~Paras Chopra</span></span></span></span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>doesn't every product builder in this space know this? if you're leveraging LLMs to actually replace a process with some level of success</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">9:24 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">9:24 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj _3sxvM" data-id="false_120363049558306142@g.us_A317F580E080607C790EF193638F1B24_919731933997@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Rajesh Parikh +91 97319 33997" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno exofpctk"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">R</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d a1q56urb" role=""><span dir="auto" aria-label="Maybe Rajesh Parikh" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Rajesh Parikh</span><span class="WJuYU" testid="author" role="button">+91 97319 33997</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[9:25 am, 26/02/2024] +91 97319 33997: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Easiest way to find value on where we are heading in one line as Tim Cook called out is 'thought to action' whatever it takes to get there. Integrated data and AI platform aand AI Agents/Agencies (compound ai systems as Berkeley calls it) are 2 possible systemic architectures to reduce friction to deployment.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5 hnx8ox4h mw4yctpw gj5xqxfh">Edited</span><span class="tvf2evcx oq44ahr5">9:25 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto"><span class="mw4yctpw gj5xqxfh">Edited</span>9:25 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="reaction 👍"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj _3sxvM" data-id="false_120363049558306142@g.us_1C10BC4C9E1837EE24ADD5C3C10D4925_919731933997@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a"><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="copyable-text" data-pre-plain-text="[9:26 am, 26/02/2024] +91 97319 33997: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>In an isolated approach, the same thing looks much harder to accomplish</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">9:26 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">9:26 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_B0D3BFF383D1DAE6BFC1D98EB0714A95_919731933997@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a"><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="copyable-text" data-pre-plain-text="[9:27 am, 26/02/2024] +91 97319 33997: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Success will be seen on how creators find a perfect alchemy of sorts with all tools at hand</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">9:27 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">9:27 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj _3sxvM" data-id="false_120363049558306142@g.us_4E75CC2A884EC6B76C25547DFABA3E66_918898020185@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Sourasis Roy +91 88980 20185" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno m8ez62k1"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">S</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d lr20dbn2" role=""><span dir="auto" aria-label="Maybe Sourasis Roy" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Sourasis Roy</span><span class="WJuYU" testid="author" role="button">+91 88980 20185</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[9:28 am, 26/02/2024] +91 88980 20185: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g9gonq0i _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Hemant Mohapatra" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Hemant Mohapatra</span><span class="WJuYU" testid="author">+91 93156 59680</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">I had the same takeaways. A dumb LLM + smart scaffolding may be better than trying to fine tune or train a smart LLM to use without the same scaffolding.</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>True. No point falling in love with LLMs blindly. Their usage needs to justify gtm speed, cost, roi etc and timely compromises are critically important here

But lately I have been thinking what happens if 1million token inferencing cost comes down to let's say $0.003 . Do we still need a retrieval for most of the applications</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">9:28 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">9:28 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_EB72BD9B019505373F32FDEDB772FAAB_918898020185@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a"><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="copyable-text" data-pre-plain-text="[9:30 am, 26/02/2024] +91 88980 20185: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>so certain  pillars of compounding maybe temporary</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">9:30 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">9:30 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="reaction 👍"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A23DD3AFEB16D0E74D7_917892563038@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe ashish +91 78925 63038" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno po56q9xr"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">A</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe ashish" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">ashish</span><span class="WJuYU" testid="author" role="button">+91 78925 63038</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[9:39 am, 26/02/2024] +91 78925 63038: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="m8ez62k1 _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d lr20dbn2" role=""><span dir="auto" aria-label="Maybe Sourasis Roy" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Sourasis Roy</span><span class="WJuYU" testid="author">+91 88980 20185</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">so certain  pillars of compounding maybe temporary</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>and does scaling continue to hold up ? till when ?</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">9:39 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">9:39 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj _3sxvM" data-id="false_120363049558306142@g.us_ADE3D9FE94AE419068919546F7802849_918898020185@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Sourasis Roy +91 88980 20185" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno m8ez62k1"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">S</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d lr20dbn2" role=""><span dir="auto" aria-label="Maybe Sourasis Roy" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Sourasis Roy</span><span class="WJuYU" testid="author" role="button">+91 88980 20185</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[9:43 am, 26/02/2024] +91 88980 20185: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Not sure. maybe the underlying hardware iterates and becomes drastically cheaper. But so far nothing suggests that the current pricing is anywhere close to final.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">9:43 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">9:43 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_29CF070ABEBDC67A8D5AABBBDBA8FB80_918898020185@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a"><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="copyable-text" data-pre-plain-text="[9:44 am, 26/02/2024] +91 88980 20185: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>but definitely it will take several years to reach there if at all</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">9:44 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">9:44 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_28011827631708922938_16503059563@c.us"><div class="_2OvAm focusable-list-item _1jHIY"><span class=""></span><div class="UzMP7 n6BPp"><div class="_1BOF7"><div class="_1sykI"><div class="Mk0Bp _30scZ"><span dir="auto" aria-label="" class="l7jjieqr ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">+91 77378 87058</span></div> added <div class="Mk0Bp _30scZ"><span dir="auto" aria-label="" class="l7jjieqr ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">+1 (650) 305-9563</span></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"></div></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj _3sxvM" data-id="false_120363049558306142@g.us_3EB0410C1D1BBAA1526356_917737887058@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Nirant K +91 77378 87058" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno g9gonq0i"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">N</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author" role="button">+91 77378 87058</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[10:20 am, 26/02/2024] +91 77378 87058: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="oj03u76w _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d dud7p2xx" role=""><span dir="auto" aria-label="Maybe Prakash" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Prakash</span><span class="WJuYU" testid="author">+91 95381 04545</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">I don't understand why this is a great article. Isn't it basically saying that LLMs are one part of your system, build your product around the LLM and do w/e it takes to give your user a good product experience.

I mean with the stage even GPT-4 is at and given LLMs are inherently probabilistic functions isn't this always going to be the case?</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>It's a great article because it makes people feel heard — and it gives names to the idea you just described. Humans like nouns — that's why we've a name for the colour "blue", which we didn't have 5K years ago.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5 hnx8ox4h mw4yctpw gj5xqxfh">Edited</span><span class="tvf2evcx oq44ahr5">10:20 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto"><span class="mw4yctpw gj5xqxfh">Edited</span>10:20 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="reaction ❤"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="❤️" draggable="false" class="bd96tnyg eipqoq80 b8 emoji apple _11JPr" tabindex="-1" style="background-position: -20px 0px;"></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB0586E9C8EADF4E2F98D_917737887058@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a"><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo hbm07cg7 n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author" role="button">+91 77378 87058</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[10:24 am, 26/02/2024] +91 77378 87058: "><div class="M6sU5"><div class="xzT4z gndfcl4n l8fojup5 paxyh2gw sfeitywo cqsf3vkf k2bacm8l ajgl1lbb p357zi0d f8m0rgwh ovllcyds kvdvyush gfz4du6o r7fjleex rrq4r3yd"><div class="gndfcl4n p357zi0d ac2vgrno gfz4du6o r7fjleex g0rxnol2 ln8gz9je" style="height: 173.25px;"><img alt="" class="tvf2evcx oq44ahr5 lb5m6g5c ppled2lx ln8gz9je jpthtbts" src="..."></div><div class="k5z04t9s cm280p3y p357zi0d ggj6brxn m0h2a7mj lb5m6g5c f8m0rgwh ac2vgrno idwf4z32 onmzirdk gfz4du6o r7fjleex n1yiu2zv itegkywt eb4rp10x rppts313 ln8gz9je"><div class="aoi073rw suguakab jjtagnqy c32ccnay tvf2evcx m0h2a7mj lb5m6g5c j4enbv94 gfz4du6o r7fjleex lhj4utae ljrqcn24" title="Lamini Startup Program Application"><span dir="ltr" aria-label="" class="ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae i033jvx7 cw3vfol9 _11JPr" style="min-height: 0px;">Lamini Startup Program Application</span></div><div class="jjtagnqy tvf2evcx ttu8nud2 lb5m6g5c dsh4tgtl gfz4du6o r7fjleex lhj4utae"><span class="g0rxnol2 gfz4du6o r7fjleex" style="display: -webkit-box; overflow: hidden; -webkit-box-orient: vertical; -webkit-line-clamp: 2;"><span dir="ltr" title=" Thank you for your interest in Lamini's LLM Compute Startup Program! Lamini offers an all-in-one LLM pods for every developer to build LLM experts efficiently. We'll review your application and get" aria-label="" class="i033jvx7 _11JPr" style="min-height: 0px;"> Thank you for your interest in Lamini's LLM Compute Startup Program! Lamini offers an all-in-one LLM pods for every developer to build LLM experts efficiently. We'll review your application and get</span></span></div><div class="j8h4s3v0 tvf2evcx oq44ahr5 lb5m6g5c dsh4tgtl qbqilfqo"><span dir="ltr" aria-label="" class="_11JPr" style="min-height: 0px;">docs.google.com</span></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Lamini AI has a startup credits offer program: <a dir="auto" href="https://docs.google.com/forms/d/e/1FAIpQLSdiUGjIkzJfplThk1285D3RFyvXS8OlWeJ2Hg-HAcBciGcNxw/viewform" title="https://docs.google.com/forms/d/e/1FAIpQLSdiUGjIkzJfplThk1285D3RFyvXS8OlWeJ2Hg-HAcBciGcNxw/viewform" target="_blank" rel="noopener noreferrer" class="_11JPr selectable-text copyable-text">https://docs.google.com/forms/d/e/1FAIpQLSdiUGjIkzJfplThk1285D3RFyvXS8OlWeJ2Hg-HAcBciGcNxw/viewform</a>

Minimum 4 GPU, with DPO implementations in a box as well</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">10:24 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">10:24 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="reaction 👍"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB04F73776FB6FA88176C_919334372044@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _27hEJ _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Sid +91 93343 72044" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno akr8p6hy"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">S</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div class="cm280p3y ktbp76dp ocd2b0bc folpon7g aa0kojfi snweb893 g0rxnol2 jnl3jror copyable-text" data-pre-plain-text="[10:38 am, 26/02/2024] +91 93343 72044: "><div><div class="bxcbqipq mhcwslh8 ocd2b0bc jfqm35v0"><div class="_3IzYj _6rIWC p357zi0d jake6gi8" role=""><span dir="auto" aria-label="Maybe Sid" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Sid</span><span class="WJuYU" testid="author" role="button">+91 93343 72044</span></div></div><div role="button" tabindex="0" aria-label="Open Picture" class="gndfcl4n l8fojup5 paxyh2gw sfeitywo cqsf3vkf p357zi0d ac2vgrno laorhtua gfz4du6o r7fjleex g0rxnol2" style="width: 330px; height: 121.481px;"><span class=""></span><div class="g0rxnol2 ln8gz9je ppled2lx"><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><img alt="Pretty interesting work on the reasoning side of things:
https://arxiv.org/pdf/2402.10200.pdf" src="data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAASABIAAD/4QBMRXhpZgAATU0AKgAAAAgAAYdpAAQAAAABAAAAGgAAAAAAA6ABAAMAAAABAAEAAKACAAQAAAABAAAAZKADAAQAAAABAAAAJAAAAAD/wAARCAAkAGQDASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9sAQwACAgICAgIDAgIDBQMDAwUGBQUFBQYIBgYGBgYICggICAgICAoKCgoKCgoKDAwMDAwMDg4ODg4PDw8PDw8PDw8P/9sAQwECAgIEBAQHBAQHEAsJCxAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQ/90ABAAH/9oADAMBAAIRAxEAPwD9uPEM/jpJb2PQ9VsrQSGRbUz6Jd3whZYY5B5phvIfNGVmOVEYYskYIdD5vB6VL8Z9c0jX7h/FlhZPHNc6fbv/AMIjfW0sEtvHcW5uI4rjUZDco100E8cgHlSQRsi580Tx9LdaPp158Rn8QXdnp0stmjwySy+H52v3itIobi1jh1Jn2OIZbieRQkbhjIyRhJIJ2fxvxQPgbod14h0fxJo+gReE9G0uzXWNLk8G3UrRWGj213fWcs8qqYfsMEUe2zJtzEs8NxBC5uG8qEAk134+eKdHs/CN7c6Xrtqdds11e5to/AGuanNb2l28j29lObC5kS0vreMLFcK7S5k/eCOONgq8p4j/AGifHdp4Zs5tBXWpNWnuI45S3wq8U3CwoEnklLwC6tztJMCIyTPtZX4kEm6DtU8K+I9cvYbXWrHwrqeo+PdE8y/1SfwVqUCzNaymIQ39tPcSDy5dNvZbdILu9hmikEpWO4jmlht8bwh4O0H4a3ya1p3hHw1pMWpaZpus297oXgLUra9e4sBM+ri5tYA728stpcCPT4pJftSyzToYrgo0cgB1yfGDxFqHiIafp8erWkUupRWEa3PgbWvKH+kW8cxe581ECKrsBdHbBh/NTzUt5gbev+PPiFrPiay8G+A9WXQ9Snnvw02reBddvbBoraaeNAb2O8srWE4t2KvJNi4Vo5IlCyxhuN8PfD34SeJfA/h210Twv4Uv5bWTRtQF9F4EnGj3FvdSGaCTT4S+2N9l25WZbmYWrSSSSrgyLU+peBvhvZTWEB8H6F/wh+nz3ssOlp8PL+4uIl0k3klx5TxZSN3mmaS1cWv+kF5fs6ztcq6gHRP4z+I+jrqjza5LqCW48Q6jHC3gfV5LlbWCYxWlossFzHBNNbSh1UIN+oQeW8Chc3Mt/U/E/wATrv4bWfi/TtbXT5roaXLtPg3U5rxPOvCsySaYb37SgeJ4o5F+/bhZZmbayiLnrrwl4Z0zULrQ9I0HQ7OPVYZr2VF8A6hNDLY6zHHFeQzzRSJC1xc30EU9zGxEnkRos0JKrciPX/BfgXWvE+reJm8J6EPEmsR3OkzX934Dvr6dy0lpprpcXn7lri0MM3lEkpFLBulDi3tp8AFvxX8TPF/h68juTqWp2bataRxWtqfAes63DBNbXF0kl3I+lOxjF0jQFYZplZEjDbVMjhdx/i/q2iz6tfa7a6te2GmLJOLex8G6y1zLCuqXlgEhdJJxNIFW2bCxhmiV7sIILhBb+R6V4H+EXhXVT4x8H+BdE0PUzp2oLpV1afC7VYrq0kFm9yjTmFUkZV0+aG2MGIZJ51nt4nSV2tYbFx8OfC2r+KLPTPAngrwlpfhjR9FSyuF1T4e6gtxHcalL9o0mKyLfZYHs457h576JCWtpEYSmF5nktwDX0vWP2nviZ4WK+GvEFr4Qkube4tjqOoeB7i1uBP8Abb21W8trO91/fCEiginSO5gk3JLFIVYO8EOUf2h/Gd1rUf8AZVn4hktJtX0+2EMvw08QxYsJ76aOY+dNcwBWW2khaS5cbIxbyyC3dp4oEW6+HnwpR4/AvivQ/DHiHVvDzLrV1bt4GvtR8pNTuru9uJbaOSe6aH7VNZ6sYAruEklgVUf92l3R07wTpsa6DpD6X4J8O+OH1A2dnPN4F1FtOnvdFlvI9Gnso5rizEUttpWnXIwk8hRTC0M5tvs/2kA+tPh94huPFXhS28QXLyyG7luin2jSb3Q5lhW4kWJZLHUP9JjZYwqszhRKR5qKiOqjtK4v4eWenaf4M0yx0yzgsYrdGjkjtdMl0e3NwrsJ3isZsyQxyTb3UMz5DBg7gh27SgD/0P1s8YX3h6zuvFWr6lfWrvpmv6JDaWdtr2pxb9ZvorazsoNVSzDi2iuZLqBPsht5oDHsu5g5k2w84PiL8H9Ct9f8BWnjrRJLPwWsaXRuvHt3Jq1rAbxTqb6jJI8t1C9rtAh82Zy7E27NbRsSdD4ueKF8K+MH/tD4haj4d0qTT3u76WLUPDdpb6FFvjhtZZItUg810v5w8KSu0qrMNg2Kxx4pYeJ/Fy+MfF+t6L4k8Qvcaheabpunxf8ACV+GLqz1aWx0u5urZrOznhljsJNUjW2kmjiSJnF1HcKiRo7AA9p0q8m8NeCorvVTYSWlvc2+gW0i+MtZ1SOW91jVYbLUba4uJbeSVprS9ItraRw7hg0R+wRvIi0fFvjjwDrFv4l8SeFfFWjaydE0OXzJ5/EeoaLp1tpms3ptXkfWbV7mGORbjTr1Vmij8+B444lMAy7+OWX7SNp4X8C3ieIPiTFe6nqmjX1/pV9qfirwXAZ5Hu5bKKO2lt7aGAyIto89rI9tJbs7zQ3TBogqdP4Y8beKfhr4W8NeGvij4o1Xw4dP0a2udZudf8R+GLi/0cSM+lQ/bZnto/tEEsrefBefNJPPCqSqpaWCQA77UvEOjva3uoeLfFGhBH1O5ub24tfGmo6TDa2bX2mrpwWNGZVdtKntZp4w8UL3UqKq+XqDzhdBtNGXV/hvoUd7FqfiyTSpbW/S18Wa69nDD4cvLUaobUnz47yaDUmht5RdtDcTRF4ZnaNZoa8Fv/jJZ/D7Xbc698YL68Xxe1kllqOpeKvBVtFFoH2OXGrRWhtYIdy3l7Oo8iGczvp0RklMH7mX2268R6/p9xp3iax+Iesavo2r+ItJiic3/hKPSmgvkiljt7af7EtxNbT/AGtFVSReyLDugYb4zcAGfqnjLQbXwpqHhz4i+JdGtvFEGhTS6zDaeNNY0ryJNL1AxWlyu0G7sLG4nkkF5e5zCojt3a9hVWXqrTwadC0OfxJ4LvIILrT9R1aO1vtR8Za5qenh/sZ06Ga/jlkC3Ui3FvBDLZyybYG82WKf7RnzOZ8VfGZ/CepeH7QarqesaRqtvJrFlq8uteE7G01fT9N02K9mez3SxTTxN5W+d1jiRBcSyealsIgnmXiX48tqNpeal8KPiNceK7W5Fqb2ex8T+CY30VbmSfULhEWa3mgaawsLK4wJWZJYS7GV2ia5gAPfvE//AAiev+D28R+H9WGsaf4a027vLWay8Z6xbpPpl3ZyYurmXTjLJPvnhkjhkdbho1R5oH80GKsLSPiD4Tj+JPirQY5Lmey8FRPda1PBrPiK/ng1i/kt746fb2n2Q2t0pCRiCG3uHlUM9rFaIkssbecaF+0F4R8M+I/GHhPxJ8aoNZXQYyNKuLzxH4OS+1m7uYNr2sdpb2toLZ7OeMJEbgqJJZW8zdEqhed8PfHvVPC+iqt94nvdV8Q+Gdf1C3utA1LxZ4MOpajaO8st3PqLRpHFarp8cNw0NtbSxOJImilxAhlQA+ntN8YpqGjW2uxa3oGs2Og6hbTRalaeILm2s5NOl09J5ri6Ci4RpI7K5a4itpppop1EN2ZYSytD55p+q/D7wV4v1vQPif4httC1qbw1BHFpcPj/AFrWdRnsoLHffyx6VceRIkkAtm8q9t0e6uFWSdzFK0ilYPiFb+GLHUP+E6+JtzF4TvtNayi8X3er+F4rO01G4luJIgGjtbXbfC1eB0Bhmt9oUbXKvLNwun+M/HmseJNMjtvGGtP4P0uyXxJqPih9f8ICODS76CK4ijvrS2snT7HiC6SO6iwxeNilw0TNLCAfavgy7k1Dwvp2oyG2b7ZF56NZ30mp2zxykvG8V3KkbSoyEMp2gYOFyoBPT15P8Dtel8T/AAt0PXpddl8Sm8Fw66jNdaZeyXCC4kCFp9HihsXwoC/uUAAG1izhmPrFAH//0f25ufBfiSfxPqOvxePtbt7G9t2hh0mOHSTZWcjRLGJ4XfT2ujIrAyATXEke9iChTCDWfw3rLQvEvirUkZoNQiEgjsNyveSiSGYZtCu+yUGK3BBRkYm4Sd8OOtooA+efjHJ40+G3wV8eeO9G8Zald6p4Z8L61eWpuoNOaNru3sVkhnkWOzjy0cluzhRhCZ5QyMohWLu9O8CeKbKygtbn4ja9fywwvE088GjLJK7facSuItNjQOvnx4Cqqf6NDlTuuPP5P9qb/k2P4vf9ifr/AP6b5693oA+dbCTxTrHxm8d/DW/8T3kmlWGm+FvEVkxt9PM1pJd3+prNaxk2pVrdhpsWDKrzoXkZJlbyzHn/AAQh8e/Ej4C/D3xtrvxA1iHWfE3hjSr+7nt7bSV/0q9sJZJJUR9PdFIkukdV27QbaAbSpuFuNvw5/wAnTfEP/sTPBv8A6cfEdL+yh/ya38HP+xN8Pf8ApugoA9A0Twf4h0rS7zT77xzrOsz3VqLeO7u4dLSe2lBlJuYhbWMMRlIkUESRvFiKPEYJlMmePBfii0m1K5b4ha5cJqFjHYwxSQaQEsZykcRvoCmnKzXBZTKVmaW33uwEIjCovp1Vrv8A1S/9dI//AENaAPELubxd4k+Kvjv4f2niu/0Kxh8P+G7+ymsYbBrixuLu81WO5eE3VrcI/nJaRKwnSUKMmMITmum0yLxD4d8a6J4evvEd7r1te6PP5pvorNHNxp8sK/ac2lvbgSTi5IlUDyv3aeVHGfM34eg/8nIeOv8AsU/Cn/pfr9dZq3/JUfDX/YK1j/0dYUAehUUUUAFFFFAH/9k=" class="jciay5ix tvf2evcx oq44ahr5 lb5m6g5c fsmudgz7" style="width: 100%;"></div><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><img alt="Pretty interesting work on the reasoning side of things:
https://arxiv.org/pdf/2402.10200.pdf" src="blob:https://web.whatsapp.com/2db7afea-0dc4-424f-ab37-700f4ce275ed" class="jciay5ix tvf2evcx oq44ahr5 lb5m6g5c osz0hll6 nq7eualt em5jvqoa a21kwdn3" style="width: 100%;"></div></div></div><div class="cm280p3y bkoknyjm esbfogcw jfqm35v0 bxcbqipq mhcwslh8"><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Pretty interesting work on the reasoning side of things:
<a dir="auto" href="https://arxiv.org/pdf/2402.10200.pdf" title="https://arxiv.org/pdf/2402.10200.pdf" target="_blank" rel="noopener noreferrer" class="_11JPr selectable-text copyable-text">https://arxiv.org/pdf/2402.10200.pdf</a></span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">10:38 am</span></span></span></div></div><div class="lrw9n60e lhggkp7q fz4q5utg b9fczbqn"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">10:38 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 cvsrm5e1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A945204BDA2644D3533_919819739552@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Rahul Deora +91 98197 39552" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/424643744_950879756689408_5500617842927232288_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdTfGuI1ycSPTskdKddpc_-NRgesc1W-1A31K6kHxI7DTA&amp;oe=65EAF644&amp;_nc_sid=e6ed6c&amp;_nc_cat=101" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d a7s2a5ab" role=""><span dir="auto" aria-label="Maybe Rahul Deora" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Rahul Deora</span><span class="WJuYU" testid="author" role="button">+91 98197 39552</span></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t r15c9g6i"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l mhp1pqu9 nu7pwgvd dnb887gk gjuq5ydh i2cterl7 l7jjieqr a6r0u4sv bx0vhl82 ntr8esoy"><span data-icon="recalled" class=""><svg viewBox="0 0 24 24" height="24" width="24" preserveAspectRatio="xMidYMid meet" class="" fill="none"><title>recalled</title><path fill-rule="evenodd" clip-rule="evenodd" d="M7.75897 6.43054C8.93584 5.533 10.4057 5 12 5C15.866 5 19 8.13401 19 12C19 13.5943 18.467 15.0642 17.5695 16.241L7.75897 6.43054ZM6.35707 7.85707C5.50399 9.01706 5 10.4497 5 12C5 15.866 8.13401 19 12 19C13.5503 19 14.9829 18.496 16.1429 17.6429L6.35707 7.85707ZM12 3C7.02944 3 3 7.02944 3 12C3 16.9706 7.02944 21 12 21C16.9706 21 21 16.9706 21 12C21 7.02944 16.9706 3 12 3Z" fill="currentColor"></path></svg></span></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 kiiy14zj j4enbv94"><div class="_21Ahp c4mcV">This message was deleted<span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">10:44 am</span></span></span></div></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">10:44 am</span></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"></div></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB0423C8080C6B3DC9B00_917737887058@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Nirant K +91 77378 87058" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno g9gonq0i"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">N</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author" role="button">+91 77378 87058</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[10:46 am, 26/02/2024] +91 77378 87058: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g755h7wn _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d a7s2a5ab" role=""><span dir="auto" aria-label="Maybe Rahul Deora" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Rahul Deora</span><span class="WJuYU" testid="author">+91 98197 39552</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Hey guys I’m trying to launch a microsaas and want to accept international monthly recurring payments in a subscription model. How can I do this as an individual? Don’t have a company set up.

Am using no-code tools to build as my service is an email service so any recommendations on that would also be helpful.</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>bit off topic for here, perhaps folks in the startup group or watercooler can be more helpful</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">10:46 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">10:46 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="reaction 👍"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍🏻" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -40px -60px;"></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A58A302217906669202_919008752566@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Mrigesh Parashar +91 90087 52566" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://media-tir3-1.cdn.whatsapp.net/v/t61.24694-24/361665663_1233851381343825_4378859487165230079_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRbjEYxymmW-NY5Fk1Xs3hs-3gmzcojhE0RlxoNv6bMRw&amp;oe=65EAD007&amp;_nc_sid=e6ed6c&amp;_nc_cat=109" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo hbm07cg7 n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d a1q56urb" role=""><span dir="auto" aria-label="Maybe Mrigesh Parashar" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Mrigesh Parashar</span><span class="WJuYU" testid="author" role="button">+91 90087 52566</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[10:51 am, 26/02/2024] +91 90087 52566: "><div class="M6sU5"><div class="xzT4z gndfcl4n l8fojup5 paxyh2gw sfeitywo cqsf3vkf k2bacm8l ajgl1lbb p357zi0d f8m0rgwh ovllcyds kvdvyush gfz4du6o r7fjleex rrq4r3yd"><div class="gndfcl4n p357zi0d ac2vgrno gfz4du6o r7fjleex g0rxnol2 ln8gz9je" style="height: 172.412px;"><img alt="" class="tvf2evcx oq44ahr5 lb5m6g5c ppled2lx ln8gz9je jpthtbts" src="..."></div><div class="k5z04t9s cm280p3y p357zi0d ggj6brxn m0h2a7mj lb5m6g5c f8m0rgwh ac2vgrno idwf4z32 onmzirdk gfz4du6o r7fjleex n1yiu2zv itegkywt eb4rp10x rppts313 ln8gz9je"><div class="aoi073rw suguakab jjtagnqy c32ccnay tvf2evcx m0h2a7mj lb5m6g5c j4enbv94 gfz4du6o r7fjleex lhj4utae ljrqcn24" title="GraphRAG: A new approach for discovery using complex information"><span dir="ltr" aria-label="" class="ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae i033jvx7 cw3vfol9 _11JPr" style="min-height: 0px;">GraphRAG: A new approach for discovery using complex information</span></div><div class="jjtagnqy tvf2evcx ttu8nud2 lb5m6g5c dsh4tgtl gfz4du6o r7fjleex lhj4utae"><span class="g0rxnol2 gfz4du6o r7fjleex" style="display: -webkit-box; overflow: hidden; -webkit-box-orient: vertical; -webkit-line-clamp: 2;"><span dir="ltr" title="Microsoft is transforming retrieval-augmented generation with GraphRAG, using LLM-generated knowledge graphs to significantly improve Q&amp;A when analyzing complex information and consistently outperforming baseline RAG. Get the details." aria-label="" class="i033jvx7 _11JPr" style="min-height: 0px;">Microsoft is transforming retrieval-augmented generation with GraphRAG, using LLM-generated knowledge graphs to significantly improve Q&amp;A when analyzing complex information and consistently outperforming baseline RAG. Get the details.</span></span></div><div class="j8h4s3v0 tvf2evcx oq44ahr5 lb5m6g5c dsh4tgtl qbqilfqo"><span dir="ltr" aria-label="" class="_11JPr" style="min-height: 0px;">www.microsoft.com</span></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span><a dir="auto" href="https://www.microsoft.com/en-us/research/blog/graphrag-unlocking-llm-discovery-on-narrative-private-data/" title="https://www.microsoft.com/en-us/research/blog/graphrag-unlocking-llm-discovery-on-narrative-private-data/" target="_blank" rel="noopener noreferrer" class="_11JPr selectable-text copyable-text">https://www.microsoft.com/en-us/research/blog/graphrag-unlocking-llm-discovery-on-narrative-private-data/</a>
Very interesting ! Baseline RAG struggles to integrate information from various sources, while GraphRAG performs significantly better.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">10:51 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">10:51 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="Reactions 👍 4 in total"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍🏻" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -40px -60px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk o2es7gts i2cterl7" style="flex-shrink: 0;"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 f8m0rgwh r15c9g6i ilf8vifs gfz4du6o r7fjleex qnz2jpws kjemk6od"><div class="dn5wc90o ilf8vifs gndfcl4n p357zi0d"><span class="f8jlpxt4 qfejxiq4 ce5kru2g">4</span></div></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj _3sxvM" data-id="false_120363049558306142@g.us_3EB099F41734AF96F60698_917737887058@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _27hEJ _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Nirant K +91 77378 87058" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno g9gonq0i"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">N</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div class="cm280p3y ktbp76dp ocd2b0bc folpon7g aa0kojfi snweb893 g0rxnol2 jnl3jror copyable-text" data-pre-plain-text="[10:57 am, 26/02/2024] +91 77378 87058: "><div><div class="bxcbqipq mhcwslh8 ocd2b0bc jfqm35v0"><div class="_3IzYj _6rIWC p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author" role="button">+91 77378 87058</span></div></div><div role="button" tabindex="0" aria-label="Open Picture" class="gndfcl4n l8fojup5 paxyh2gw sfeitywo cqsf3vkf p357zi0d ac2vgrno laorhtua gfz4du6o r7fjleex g0rxnol2" style="width: 330px; height: 325.417px;"><span class=""></span><div class="g0rxnol2 ln8gz9je ppled2lx"><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><img alt="I love to see how graphs are an idea virus in AI circles.

It's such as catch all idea, and so tempting to model everything as a graph — and I've seen people apply graphs to everything from recommendation systems to image classifiers and claim that &quot;This works&quot;" src="data:image/jpeg;base64,/9j/2wCEAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDIBCQkJDAsMGA0NGDIhHCEyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMv/AABEIAEcASAMBIgACEQEDEQH/xAGiAAABBQEBAQEBAQAAAAAAAAAAAQIDBAUGBwgJCgsQAAIBAwMCBAMFBQQEAAABfQECAwAEEQUSITFBBhNRYQcicRQygZGhCCNCscEVUtHwJDNicoIJChYXGBkaJSYnKCkqNDU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6g4SFhoeIiYqSk5SVlpeYmZqio6Slpqeoqaqys7S1tre4ubrCw8TFxsfIycrS09TV1tfY2drh4uPk5ebn6Onq8fLz9PX29/j5+gEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoLEQACAQIEBAMEBwUEBAABAncAAQIDEQQFITEGEkFRB2FxEyIygQgUQpGhscEJIzNS8BVictEKFiQ04SXxFxgZGiYnKCkqNTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqCg4SFhoeIiYqSk5SVlpeYmZqio6Slpqeoqaqys7S1tre4ubrCw8TFxsfIycrS09TV1tfY2dri4+Tl5ufo6ery8/T19vf4+fr/2gAMAwEAAhEDEQA/ANcJlOM9Tzj3q3o0lvFqVxFd3UVvb3FnJCrzsEjErMgQ7t6HcAXICsGI3YIIFS+Hr7wjZ6S0GvaWJdQFzcGR5NGlnLKZ3KfOI2DDYUxyeMDtineGrzwhpfhCO61/S4I53uZ5JfM0wzOimV2i3mOLAARlA4AH3QBjAzhGzvc6KlRtWsaN54q0KTWFhTVfDqW3myiC4aeMoh8hdjOEl+dd5lG1wgOB8ysqeZ5jpXiW9vL6zsL37JfoWWNppIMlx03DcAeevIB9QK6Pw14p+GkWj6dZavpFj9ot9OtfOuZNJeRpbjaRKrYiOSpVSWJIYvxyKr+Gdc8J32kpBc22jw6v59nIC+hqoMaRW7XKjbDjczLcqo6liAvVa1Tsc5z58WQw3IJ0HTd0bhg0Uewgg9cir2h6no8+swNDof2e5ZjtlF07YODzg8VR1y78PXPhjwzDpcEUWo21mE1MrZtCzTbIwSzFR5h3B+QT1PrzQ0GULrtp/v4H5VS8wO2tBjwNEvu3/ow1iFa6bQEhm8PwQTR70LyJjdtAJLYz/L6kVHpmjvD4iAkBMMCmUMyEq3tnH3hnI757Dty1q6hJrsbU7cpzTDjNR5PrW54mtFttUcx52yqJMY6E5z+oz+NYeD71VOanFSXUo7W9nitLZ553CRoMsxryvVNUutXvCWLFS2I4hyBnoB711/xAldLazgBGx3Zj9QBj/wBCNcxplikdzLJqUptTAmUjZSJGYg7SB7Yzn6UQtFczJm3J2I9H0G61l5BEUjVFDbpM4OSRx+R/KoLqzvNG1EbwEmhkBSQYI3DBBGfqD+Ndta2+oaRHbSLCBbsg+WKZG3gEAgApu6tn72Mk881Dqcsl5f6f9pg2RTXSCKN9pzhwrbhsOf8AvvoRxSjVk5eQShFRv1Ocn8P6226U2Fw67jyE5OD6DNS6Po+pxazaSTafdxqJVJZ4WAAz64qjLc3Npeyw21xLGqOVXDkYAPFa2lavqkuqWsb39w8ZkXIaUkGuhK5iddppI8NS7SQyTnBHY7s5q4moSXccswXypbbZITHjL84Y4PcAce+M9BVeyXOh3YAx/pDH9RWe6e1cmKpKctTopK8R+syPPd4e7iulVcK8ZGMe+O9Znkj0NWmWm7adOPLFRNLGp4ptFLWOoOT5NpLmTHVQxAD/AIEDjFWLFE8yaTY4LgcuO3Py++CW/Mc1o3sUFxB5FxHvR2A2cncQcjgdemfp1rA0Own02O6tZWfdDcFVDtn5NoIHBPY5+prKotLhSavYu/YbW2hRZJ5lgjZSqPcNsUg8dT9OtU7mwiM0dw5PnpeRHaJSVBJXHy8AHBB6Z9zWo3lXCGKQckcqeD+H59RWXNZrG+n2kU9pAUuEmkBbYXwegXknP17UqV3NF1UlBnCX0QGoXOevmtn86s6fazm5tpEBjV5AiSkcbvb1xW/D4b83XHllubWaBZGd0icsevAPHHP8jTr+LU5dZt5Tbv8AY4Zl2BBkBQeTgc9K7nUSlynHyPlubuoxaPqmhalFaeENAsZHjkt0lSBDJExUqHVggwQSpHpsxnkFaukx6dplpYW9x4d02+itbbypI5VjCTSFYlMpHlfe/dMfm3NmVhuAAFWYZPJsrkSIyq8rSBjgALx171CrLJGHXoc1nOav7rNadNW95GSdL04aqlz/AGBpht47z7QkDIPuES5jZggyA0qleMBYo1IOM1u/a9A/6EnRv++Y/wD41VUp3pNn0qedmns4l3XLaTUrZIIwFIkDBnyF6Yx068n8vcVz1hfy6FeyW9xAjxbv3qYG7oOVP0x7fTrXXxgANbsSwUDBYkkqfUnqeD69s9aztT0lNTjKsxW5iHySlcBh6H1/oee/NTo6XiRSrr4Z7FWHUYL3UZrSCMm3RcpJI245UgZGR07jJJ/PA3YtP0vUo4pGijNxD0kXh0IJGT68g9eK4ixZtH1dVvUkjXG1yozwe/uMjtXWxXFswW6sroSMRtXvjkE5H0xnPqOlc0Z8k9Ua4imrJRGW9nFZNcJFJ5gaZ3ZsY5J6fh0/CpiwAphIjjZmYd2Zjgc9Sa529n1dtWtRJbSQ2RmXBAyCMgfMR0+lEYOpJ2BzVOKTNNbc3yXDJIPKU7fMHJOSCQCfb8OR6VJIYoVQOyICQq5IGT2A96fDKYNMv5AM7ZmIH5VwWvX93eXKC5VECr8ipnGM9frXQ6Ki7Ix9tpdm7f8AiKKCV4IYmeZXCkMMA+tV/wDhI5/+fFP+/n/1q56zG0FnCkdQe4q150f+RVezRk68rnoYvi8zJAssrws0cinaASADnJ79Me55x1FS/wBU0vSfENndapdmzjRlcXKReY0SCG5RmUbWBxJLb8EHJ2nBCnDtM/5Ceof9fDf+gpXI/E/7if8AXo//AKNirZ7GZ1+l654X0TVJLbxDLYT2qCRlnfS1JkDJAY2wqEqCRdEL/COOAEFZ/hnU/Dz6LY2FxqEdt9lkt7y5sjY+YlzHHbQRzMW2kqVljl6H5gG4YMDXDeNP+Pn/ALdrb/0CSo9I/wCQnP8A9gi7/wDRklS0hpnodlrngXT/AAXrNrZS51G7hksrZnt28xj5KxQfvAg2mURLNzgKznJBFWJfGPw4l0goDYWtz/aSynOnkFoEuw55jhAw0IHykZw21iTknyK3/wBXZ/8AYQsv/RZrEufu/wDAf/ZVqU7opqx7ZbeKPh/ZfCy4s4JbdNfuNHWCX/RJC5uvs6qAH24HzxKcggbhu6nJpeKVtvFF1FdeGbVPsFvCkAZVWEybBgOUwoGV2AY6BQMDoPI2/wBUf+u8X8nr1vwJ/wAi5L/n+BalyYJanJpJtj5bkDn6U3zx/fb8qYekn+7/AFFQ1oiGtT//2Q==" class="jciay5ix tvf2evcx oq44ahr5 lb5m6g5c fsmudgz7" style="height: 100%;"></div><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><img alt="I love to see how graphs are an idea virus in AI circles.

It's such as catch all idea, and so tempting to model everything as a graph — and I've seen people apply graphs to everything from recommendation systems to image classifiers and claim that &quot;This works&quot;" src="blob:https://web.whatsapp.com/c180d661-ffab-4511-8a13-3405f8f8fe3a" class="jciay5ix tvf2evcx oq44ahr5 lb5m6g5c osz0hll6 nq7eualt em5jvqoa a21kwdn3" style="height: 100%;"></div></div></div><div class="cm280p3y bkoknyjm esbfogcw jfqm35v0 bxcbqipq mhcwslh8"><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>I love to see how graphs are an idea virus in AI circles.

It's such as catch all idea, and so tempting to model everything as a graph — and I've seen people apply graphs to everything from recommendation systems to image classifiers and claim that "This works"</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">10:57 am</span></span></span></div></div><div class="lrw9n60e lhggkp7q fz4q5utg b9fczbqn"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">10:57 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 cvsrm5e1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="Reactions 😂, 💯 3 in total"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="😂" draggable="false" class="bd96tnyg p83aruvr b85 emoji apple _11JPr" tabindex="-1" style="background-position: -60px -60px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="💯" draggable="false" class="bd96tnyg p83aruvr b75 emoji apple _11JPr" tabindex="-1" style="background-position: -60px -20px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk o2es7gts i2cterl7" style="flex-shrink: 0;"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 f8m0rgwh r15c9g6i ilf8vifs gfz4du6o r7fjleex qnz2jpws kjemk6od"><div class="dn5wc90o ilf8vifs gndfcl4n p357zi0d"><span class="f8jlpxt4 qfejxiq4 ce5kru2g">3</span></div></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB09CD13BF3B5CF1FB040_917737887058@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a"><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="copyable-text" data-pre-plain-text="[10:57 am, 26/02/2024] +91 77378 87058: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>To the point where people model "pipelines" as L to R trees, they call it DAGs (Directed Acyclic Graphs) these days</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5 hnx8ox4h mw4yctpw gj5xqxfh">Edited</span><span class="tvf2evcx oq44ahr5">10:57 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto"><span class="mw4yctpw gj5xqxfh">Edited</span>10:57 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_30A6738928974920F005696AEAA3BADD_919043003225@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Sumba +91 90430 03225" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno ro7tsnc9"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">S</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d pfvzdcj5" role=""><span dir="auto" aria-label="Maybe Sumba" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Sumba</span><span class="WJuYU" testid="author" role="button">+91 90430 03225</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[10:58 am, 26/02/2024] +91 90430 03225: "><div class="_21Ahp"><span dir="auto" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>+1</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">10:58 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">10:58 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB0D8CDD5715A890610_919082776009@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Harsha +91 90827 76009" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://media-tir3-1.cdn.whatsapp.net/v/t61.24694-24/322461936_856881328727760_4977520626474252607_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdT6vyZkUO3Cy5CxH8NWudJxdbjdhx0tMihkdXxCDAQixg&amp;oe=65EAE19B&amp;_nc_sid=e6ed6c&amp;_nc_cat=109" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d gvktkc8w" role=""><span dir="auto" aria-label="Maybe Harsha" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Harsha</span><span class="WJuYU" testid="author" role="button">+91 90827 76009</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[11:45 am, 26/02/2024] +91 90827 76009: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>What’s the best tool to get front-end code (React Native) based on a Figma screen?</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">11:45 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">11:45 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3745E6DF5223C426044EFAFFCF90816D_919665543333@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Palash +91 96655 43333" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno j3zl77sr"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">P</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d gvktkc8w" role=""><span dir="auto" aria-label="Maybe Palash" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Palash</span><span class="WJuYU" testid="author" role="button">+91 96655 43333</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[11:52 am, 26/02/2024] +91 96655 43333: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="j3zl77sr _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d gvktkc8w" role=""><span dir="auto" aria-label="Maybe Harsha" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Harsha</span><span class="WJuYU" testid="author">+91 90827 76009</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">What’s the best tool to get front-end code (React Native) based on a Figma screen?</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Try <a dir="auto" href="http://codeparrot.ai" title="http://codeparrot.ai" target="_blank" rel="noopener noreferrer" class="_11JPr selectable-text copyable-text">codeparrot.ai</a></span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">11:52 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">11:52 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="Reactions 👍 2 in total"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk o2es7gts i2cterl7" style="flex-shrink: 0;"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 f8m0rgwh r15c9g6i ilf8vifs gfz4du6o r7fjleex qnz2jpws kjemk6od"><div class="dn5wc90o ilf8vifs gndfcl4n p357zi0d"><span class="f8jlpxt4 qfejxiq4 ce5kru2g">2</span></div></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A10AF22CA391962485E3D8622DB3AC5_919929636400@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe shobhitic +91 99296 36400" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://media-tir3-1.cdn.whatsapp.net/v/t61.24694-24/131879339_253735113008719_5102242893939211973_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRmvtD5SHTkQfhTvepNdqILsmVyf_aL4H6nYrQqjsczdg&amp;oe=65EB0027&amp;_nc_sid=e6ed6c&amp;_nc_cat=105" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d sthcsby1" role=""><span dir="auto" aria-label="Maybe shobhitic" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">shobhitic</span><span class="WJuYU" testid="author" role="button">+91 99296 36400</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[0:45 pm, 26/02/2024] +91 99296 36400: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="j3zl77sr _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d gvktkc8w" role=""><span dir="auto" aria-label="Maybe Palash" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Palash</span><span class="WJuYU" testid="author">+91 96655 43333</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Try codeparrot.ai</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Strong +1</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">12:45 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">12:45 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A6343961C1D6F29535F_918763968157@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Pratyush Choudhury (PC) +91 87639 68157" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno oj03u76w"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">P</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo hbm07cg7 n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d dud7p2xx" role=""><span dir="auto" aria-label="Maybe Pratyush Choudhury (PC)" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Pratyush Choudhury (PC)</span><span class="WJuYU" testid="author" role="button">+91 87639 68157</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[0:48 pm, 26/02/2024] +91 87639 68157: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="j3zl77sr _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d gvktkc8w" role=""><span dir="auto" aria-label="Maybe Harsha" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Harsha</span><span class="WJuYU" testid="author">+91 90827 76009</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">What’s the best tool to get front-end code (React Native) based on a Figma screen?</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="M6sU5"><div class="xzT4z gndfcl4n l8fojup5 paxyh2gw sfeitywo cqsf3vkf k2bacm8l ajgl1lbb p357zi0d f8m0rgwh ovllcyds kvdvyush gfz4du6o r7fjleex rrq4r3yd"><div class="gndfcl4n p357zi0d ac2vgrno gfz4du6o r7fjleex g0rxnol2 ln8gz9je" style="height: 173.379px;"><img alt="" class="tvf2evcx oq44ahr5 lb5m6g5c ppled2lx ln8gz9je jpthtbts" src="..."></div><div class="k5z04t9s cm280p3y p357zi0d ggj6brxn m0h2a7mj lb5m6g5c f8m0rgwh ac2vgrno idwf4z32 onmzirdk gfz4du6o r7fjleex n1yiu2zv itegkywt eb4rp10x rppts313 ln8gz9je"><div class="aoi073rw suguakab jjtagnqy c32ccnay tvf2evcx m0h2a7mj lb5m6g5c j4enbv94 gfz4du6o r7fjleex lhj4utae ljrqcn24" title="Web and mobile app development platform for high productivity."><span dir="ltr" aria-label="" class="ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae i033jvx7 cw3vfol9 _11JPr" style="min-height: 0px;">Web and mobile app development platform for high productivity.</span></div><div class="jjtagnqy tvf2evcx ttu8nud2 lb5m6g5c dsh4tgtl gfz4du6o r7fjleex lhj4utae"><span class="g0rxnol2 gfz4du6o r7fjleex" style="display: -webkit-box; overflow: hidden; -webkit-box-orient: vertical; -webkit-line-clamp: 2;"><span dir="ltr" title="Build app faster without compromising on the code quality." aria-label="" class="i033jvx7 _11JPr" style="min-height: 0px;">Build app faster without compromising on the code quality.</span></span></div><div class="j8h4s3v0 tvf2evcx oq44ahr5 lb5m6g5c dsh4tgtl qbqilfqo"><span dir="ltr" aria-label="" class="_11JPr" style="min-height: 0px;">dhiwise.com</span></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Might be a little biased but <a dir="auto" href="http://DhiWise.com" title="http://DhiWise.com" target="_blank" rel="noopener noreferrer" class="_11JPr selectable-text copyable-text">DhiWise.com</a> <img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="🙂" draggable="false" class="b88 emoji apple _11JPr selectable-text copyable-text" data-plain-text="🙂" tabindex="-1" style="background-position: 0px -40px;"></span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">12:48 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">12:48 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB089BA1035223B97DA83_919649859146@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Vandit Gandotra +91 96498 59146" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://media-tir3-1.cdn.whatsapp.net/v/t61.24694-24/425907157_1160734085335352_7203533381077955836_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRSUbLUde4dmluj0-i0PHVrw9Cm3vdC--WXOdxrnNyJsg&amp;oe=65EAD0A1&amp;_nc_sid=e6ed6c&amp;_nc_cat=101" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Vandit Gandotra" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Vandit Gandotra</span><span class="WJuYU" testid="author" role="button">+91 96498 59146</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[1:08 pm, 26/02/2024] +91 96498 59146: "><div class="M6sU5"><div class="xzT4z gndfcl4n l8fojup5 paxyh2gw sfeitywo cqsf3vkf k2bacm8l ajgl1lbb f804f6gw sap93d0t ovllcyds kvdvyush gfz4du6o r7fjleex rrq4r3yd"><div class="k5z04t9s cm280p3y f804f6gw ggj6brxn m0h2a7mj lb5m6g5c f8m0rgwh ac2vgrno idwf4z32 o3plsq22 ora14ekb nv3qcefw n1yiu2zv itegkywt eb4rp10x rppts313 ln8gz9je"><div class="aoi073rw suguakab jjtagnqy c32ccnay tvf2evcx m0h2a7mj lb5m6g5c j4enbv94 gfz4du6o r7fjleex lhj4utae ljrqcn24" title="twitter.com"><span dir="ltr" aria-label="" class="ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae i033jvx7 cw3vfol9 _11JPr" style="min-height: 0px;">twitter.com</span></div><div class="jjtagnqy tvf2evcx ttu8nud2 lb5m6g5c dsh4tgtl ora14ekb nv3qcefw lhj4utae"><span class="g0rxnol2 gfz4du6o r7fjleex" style="display: -webkit-box; overflow: hidden; -webkit-box-orient: vertical; -webkit-line-clamp: 2;"><span dir="ltr" title="https://twitter.com/adcock_brett/status/1761814066748670071" aria-label="" class="i033jvx7 _11JPr" style="min-height: 0px;">https://twitter.com/adcock_brett/status/1761814066748670071</span></span></div><div class="j8h4s3v0 tvf2evcx oq44ahr5 lb5m6g5c dsh4tgtl qbqilfqo"><span dir="ltr" aria-label="" class="_11JPr" style="min-height: 0px;">twitter.com</span></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span><a dir="auto" href="https://twitter.com/adcock_brett/status/1761814066748670071" title="https://twitter.com/adcock_brett/status/1761814066748670071" target="_blank" rel="noopener noreferrer" class="_11JPr selectable-text copyable-text">https://twitter.com/adcock_brett/status/1761814066748670071</a></span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:08 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:08 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A13E23375DB1979A6E3_919822381281@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Dilip Thomas Ittyera +91 98223 81281" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/309186150_817654219469995_3888201881878390184_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRh29wTc1XaHX8pEd-BMHein5ijlTG5dv5VX2PQwa0OVw&amp;oe=65EAE41B&amp;_nc_sid=e6ed6c&amp;_nc_cat=110" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d dud7p2xx" role=""><span dir="auto" aria-label="Maybe Dilip Thomas Ittyera" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Dilip Thomas Ittyera</span><span class="WJuYU" testid="author" role="button">+91 98223 81281</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[1:11 pm, 26/02/2024] +91 98223 81281: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g9gonq0i _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author">+91 77378 87058</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">To the point where people model "pipelines" as L to R trees, they call it DAGs (Directed Acyclic Graphs) these days</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Very similar to how some folks think vectors are a solution for everything plaguing LLMs<img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="😊" draggable="false" class="b86 emoji apple _11JPr selectable-text copyable-text" data-plain-text="😊" tabindex="-1" style="background-position: -20px 0px;"></span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:11 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:11 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A7C6586F8C5968DF57D_917892563038@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe ashish +91 78925 63038" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno po56q9xr"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">A</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe ashish" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">ashish</span><span class="WJuYU" testid="author" role="button">+91 78925 63038</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[1:11 pm, 26/02/2024] +91 78925 63038: "><div class="M6sU5"><div class="xzT4z gndfcl4n l8fojup5 paxyh2gw sfeitywo cqsf3vkf k2bacm8l ajgl1lbb p357zi0d sap93d0t ovllcyds kvdvyush gfz4du6o r7fjleex rrq4r3yd"><img alt="" class="tvf2evcx oq44ahr5 lb5m6g5c n43pk08i fpj7ivsd jpthtbts" src="data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAASABIAAD/4QCMRXhpZgAATU0AKgAAAAgABQESAAMAAAABAAEAAAEaAAUAAAABAAAASgEbAAUAAAABAAAAUgEoAAMAAAABAAIAAIdpAAQAAAABAAAAWgAAAAAAAABIAAAAAQAAAEgAAAABAAOgAQADAAAAAQABAACgAgAEAAAAAQAAAIygAwAEAAAAAQAAAIwAAAAA/+0AOFBob3Rvc2hvcCAzLjAAOEJJTQQEAAAAAAAAOEJJTQQlAAAAAAAQ1B2M2Y8AsgTpgAmY7PhCfv/AABEIAIwAjAMBIgACEQEDEQH/xAAfAAABBQEBAQEBAQAAAAAAAAAAAQIDBAUGBwgJCgv/xAC1EAACAQMDAgQDBQUEBAAAAX0BAgMABBEFEiExQQYTUWEHInEUMoGRoQgjQrHBFVLR8CQzYnKCCQoWFxgZGiUmJygpKjQ1Njc4OTpDREVGR0hJSlNUVVZXWFlaY2RlZmdoaWpzdHV2d3h5eoOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4eLj5OXm5+jp6vHy8/T19vf4+fr/xAAfAQADAQEBAQEBAQEBAAAAAAAAAQIDBAUGBwgJCgv/xAC1EQACAQIEBAMEBwUEBAABAncAAQIDEQQFITEGEkFRB2FxEyIygQgUQpGhscEJIzNS8BVictEKFiQ04SXxFxgZGiYnKCkqNTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqCg4SFhoeIiYqSk5SVlpeYmZqio6Slpqeoqaqys7S1tre4ubrCw8TFxsfIycrS09TV1tfY2dri4+Tl5ufo6ery8/T19vf4+fr/2wBDAAICAgICAgMCAgMEAwMDBAUEBAQEBQcFBQUFBQcIBwcHBwcHCAgICAgICAgKCgoKCgoLCwsLCw0NDQ0NDQ0NDQ3/2wBDAQICAgMDAwYDAwYNCQcJDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ3/3QAEAAn/2gAMAwEAAhEDEQA/AP38ooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigD/9D9/KKKKACiiigAooooAK5Hx/400r4ceBPEXxC12OeXTfDGlXusXkdqqvO9vYQvPIsSuyKzlEIUFlBOMkda66vAf2rv+TXvi9/2IviT/wBN1xQB8Ff8Pov2U/8AoAeOv/BbYf8Ayyo/4fRfsp/9ADx1/wCC2w/+WVfy7UUAf1E/8Pov2U/+gB46/wDBbYf/ACyo/wCH0X7Kf/QA8df+C2w/+WVfy7UUAf1E/wDD6L9lP/oAeOv/AAW2H/yyo/4fRfsp/wDQA8df+C2w/wDllX8u1FAH9Xfw6/4K0/s1fE3x/wCHPhz4f0TxlDqfijVbPR7OS7sLJLdLi+mWGNpWS/dlQM4LFVYgdAelfqDX8R37IP8AydZ8Hv8AsevDv/pfBX9uNABRRRQB/9H9/KKKKACiiigAooooAK8B/au/5Ne+L3/Yi+JP/TdcV79XgP7V3/Jr3xe/7EXxJ/6brigD+IGiiigAooooAKKKKAPov9kH/k6z4Pf9j14d/wDS+Cv7ca/iO/ZB/wCTrPg9/wBj14d/9L4K/txoAKKKKAP/0v38ooooAKK5Hxp4/wDAnw40qPXfiF4i0rwxpss62sd5rF7DYW7zurOsSyTuiF2VGIUHJCk44NeX/wDDV37L3/RXvAv/AIUmnf8Ax+gD36ivAf8Ahq79l7/or3gX/wAKTTv/AI/R/wANXfsvf9Fe8C/+FJp3/wAfoA9+rP1fSNK1/SrzQtcs4NQ03UbeW1vLS6jWaC4t5lKSRSxuCro6kqysCCCQeKNJ1fStf0qz1zQry31HTdQgjurS8tZVmt7iCZQ8csUiEq6OpDKykgggg4rQoA+e/wDhkn9lj/oj/gX/AMJzT/8A4xR/wyT+yx/0R/wL/wCE5p//AMYr6EooA+e/+GSf2WP+iP8AgX/wnNP/APjFfzof8Fb/AIeeAfhp+0joGg/Drw3pPhfTZvBljdSWej2UNjbvcPfagjStHAqKXKoqliMkKB2Ff1Z1/L7/AMFpf+Tp/Df/AGIun/8Apw1KgD8haKKKAPov9kH/AJOs+D3/AGPXh3/0vgr+3Gv4jv2Qf+TrPg9/2PXh3/0vgr+3GgAooooA/9P9/KKKKAPyL/4LRf8AJqegf9j1pv8A6QalX8u9f1E/8Fov+TU9A/7HrTf/AEg1Kv5dqACiiigD+3f9kn/k1j4P/wDYi+HP/TfBX0LXz3+yT/yax8H/APsRfDn/AKb4K+hKACiiigAr+X3/AILS/wDJ0/hv/sRdP/8AThqVf1BV/L7/AMFpf+Tp/Df/AGIun/8Apw1KgD8haKKKAPov9kH/AJOs+D3/AGPXh3/0vgr+3Gv4jv2Qf+TrPg9/2PXh3/0vgr+3GgAooooA/9T9/KKKKAPyM/4LRf8AJqegf9j1pv8A6QalX8u1f1E/8Fov+TU9A/7HrTf/AEg1Kv5dqACiiigD+3j9kn/k1j4P/wDYi+HP/TfBX0JX8pnw8/4K3/tI/DTwD4b+HWg6B4Mm03wvpNlo9nJdWN89w9vYwpBG0rJfopcqgLFVUE9AOldj/wAPpf2p/wDoW/Av/gv1D/5ZUAf1BUV/L7/w+l/an/6FvwL/AOC/UP8A5ZUf8Ppf2p/+hb8C/wDgv1D/AOWVAH9QVfy+/wDBaX/k6fw3/wBiLp//AKcNSo/4fS/tT/8AQt+Bf/BfqH/yyr4R/ac/ac8fftXePrL4i/EWy0mw1Kw0mHR449HhmgtzbwTTzqzLPNOxctOwJDAYA46kgHznRRRQB9F/sg/8nWfB7/sevDv/AKXwV/bjX8R37IP/ACdZ8Hv+x68O/wDpfBX9uNABRRRQB//V/fyiiigD8jP+C0X/ACanoH/Y9ab/AOkGpV/LtX9RP/BaL/k1PQP+x603/wBINSr+XagAooooAKKKKACiiigAooooAKKKKAPov9kH/k6z4Pf9j14d/wDS+Cv7ca/iO/ZB/wCTrPg9/wBj14d/9L4K/txoAKKKKAP/1v38ooooA/L7/grT8OvH/wATv2atE8P/AA48N6r4o1OHxlYXclno9nNfXCW6WV+jStHCrsEVnVS2MAsB3Ffzn/8ADIP7Vn/RHvHX/hO3/wD8Zr+3GigD+I7/AIZB/as/6I946/8ACdv/AP4zR/wyD+1Z/wBEe8df+E7f/wDxmv7caKAP4jv+GQf2rP8Aoj3jr/wnb/8A+M0f8Mg/tWf9Ee8df+E7f/8Axmv7caKAP4jv+GQf2rP+iPeOv/Cdv/8A4zR/wyD+1Z/0R7x1/wCE7f8A/wAZr+3GigD+I7/hkH9qz/oj3jr/AMJ2/wD/AIzR/wAMg/tWf9Ee8df+E7f/APxmv7caKAP4jv8AhkH9qz/oj3jr/wAJ2/8A/jNH/DIP7Vn/AER7x1/4Tt//APGa/txooA/kF/Zb/Zb/AGlfDn7Svwr8QeIPhX4y03TNN8ZaFd3l5d6Few29vbw3sLySyyPEFREUFmZiAAMmv6+qKKACiiigD//X/fyiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKAP/2Q=="><div class="k5z04t9s cm280p3y p357zi0d ggj6brxn m0h2a7mj lb5m6g5c f8m0rgwh ac2vgrno idwf4z32 onmzirdk gfz4du6o r7fjleex n1yiu2zv itegkywt eb4rp10x rppts313 ln8gz9je"><div class="aoi073rw suguakab jjtagnqy c32ccnay tvf2evcx m0h2a7mj lb5m6g5c j4enbv94 gfz4du6o r7fjleex lhj4utae ljrqcn24" title="Mamba: The Easy Way"><span dir="ltr" aria-label="" class="ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae i033jvx7 cw3vfol9 _11JPr" style="min-height: 0px;">Mamba: The Easy Way</span></div><div class="jjtagnqy tvf2evcx ttu8nud2 lb5m6g5c dsh4tgtl gfz4du6o r7fjleex lhj4utae"><span class="g0rxnol2 gfz4du6o r7fjleex" style="display: -webkit-box; overflow: hidden; -webkit-box-orient: vertical; -webkit-line-clamp: 2;"><span dir="ltr" title="An overview of the big ideas behind Mamba, a brand-new language model architecture." aria-label="" class="i033jvx7 _11JPr" style="min-height: 0px;">An overview of the big ideas behind Mamba, a brand-new language model architecture.</span></span></div><div class="j8h4s3v0 tvf2evcx oq44ahr5 lb5m6g5c dsh4tgtl qbqilfqo"><span dir="ltr" aria-label="" class="_11JPr" style="min-height: 0px;">jackcook.com</span></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span><a dir="auto" href="https://jackcook.com/2024/02/23/mamba.html" title="https://jackcook.com/2024/02/23/mamba.html" target="_blank" rel="noopener noreferrer" class="_11JPr selectable-text copyable-text">https://jackcook.com/2024/02/23/mamba.html</a>

<a dir="auto" href="https://news.ycombinator.com/item?id=39482428" title="https://news.ycombinator.com/item?id=39482428" target="_blank" rel="noopener noreferrer" class="_11JPr selectable-text copyable-text">https://news.ycombinator.com/item?id=39482428</a></span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:11 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:11 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj _3sxvM" data-id="false_120363049558306142@g.us_3A77C0451478F9F7AEBB_919915123897@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _27hEJ _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Harveen +91 99151 23897" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/55964039_402243816992965_6637152673499971584_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdQtkZ88AbZaodP4QnfDlPwkgUdBcxY7iFDH1wr_oFccKQ&amp;oe=65EAFA23&amp;_nc_sid=e6ed6c&amp;_nc_cat=102" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div class="cm280p3y eu4mztcy ocd2b0bc folpon7g aa0kojfi snweb893 g0rxnol2 jnl3jror copyable-text" data-pre-plain-text="[1:23 pm, 26/02/2024] +91 99151 23897: "><div><div class="bxcbqipq mhcwslh8 ocd2b0bc jfqm35v0"><div class="_3IzYj _6rIWC p357zi0d a7s2a5ab" role=""><span dir="auto" aria-label="Maybe Harveen" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Harveen</span><span class="WJuYU" testid="author" role="button">+91 99151 23897</span></div></div><div role="button" tabindex="0" aria-label="Open Picture" class="gndfcl4n l8fojup5 paxyh2gw sfeitywo cqsf3vkf p357zi0d ac2vgrno laorhtua gfz4du6o r7fjleex g0rxnol2" style="width: 240px; height: 338.028px;"><span class=""></span><div class="g0rxnol2 ln8gz9je ppled2lx"><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><img alt="Krutrim is here" src="data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDABsSFBcUERsXFhceHBsgKEIrKCUlKFE6PTBCYFVlZF9VXVtqeJmBanGQc1tdhbWGkJ6jq62rZ4C8ybqmx5moq6T/2wBDARweHigjKE4rK06kbl1upKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKT/wgARCABIAEgDASIAAhEBAxEB/8QAGQAAAwEBAQAAAAAAAAAAAAAAAAIDAQQG/8QAFQEBAQAAAAAAAAAAAAAAAAAAAAH/2gAMAwEAAhADEAAAAPR4KjCbTtEluQsaAc8uiYmPouPpLXccYExEKvzYdpx6dZxsdQgJOykimktoE2ZzCgc6gLoBgBUC4B//xAAWEQADAAAAAAAAAAAAAAAAAAAAEUD/2gAIAQIBAT8AHR//xAAWEQEBAQAAAAAAAAAAAAAAAAARAED/2gAIAQMBAT8AI0f/xAAwEAABBAECAwUHBQEAAAAAAAABAAIDERITIQRBYRAUMTORICIyQlFTcSNDUmSBof/aAAgBAQABPwB4cT7ppVJXiFUleIVSfUJod8xCr2JXPbWDbQlfzjKM0l7RFCSS6MaMz62jKE0nOMrVlBrT7eJ+JtyFo6K2/dfuiQPd1HWUMSdpXLNlVqOtWzHzXbIOb9xxtFzCfNcom4s+Iu6lTZZDHGudo6m4tnRAyFx3Yg2UgFuCLJrsBiLJSBs2+aDJrIDWgJjHZHNra5V2SRNkIyR4eMgCvBd2i+n/AFMY2MU32uIouFyFv4QIFjWO6BHiZjSsWSJzsiRXnFNcPvFW0fvGzzTGmQnCYqNpa2i7I/VTBxcMQ0/lOzLR7rLR1MfgYTyQ1Btiyijqn5WUhn/FnVOD7NNZXJMMgJxa3/FFqb6ldKUkTZCCb2R4dhN7+q7swGwT6ru0fX1Xd2dfVHh2E3uu7su9/VRxNjJx59nEVY/UwTXA+E3JZf2ESAfP3QcAa190XVtr7ppoYun3UMjQcTJkez///gADAP/Z" class="jciay5ix tvf2evcx oq44ahr5 lb5m6g5c fsmudgz7" style="width: 100%;"></div><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><img alt="Krutrim is here" src="blob:https://web.whatsapp.com/b8eb025b-3de6-4d74-a94f-3d40c0c2af35" class="jciay5ix tvf2evcx oq44ahr5 lb5m6g5c osz0hll6 nq7eualt em5jvqoa a21kwdn3" style="width: 100%;"></div></div></div><div class="cm280p3y czh1irh3 esbfogcw jfqm35v0 bxcbqipq mhcwslh8"><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Krutrim is here</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:23 pm</span></span></span></div></div><div class="lrw9n60e lhggkp7q fz4q5utg b9fczbqn"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:23 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 cvsrm5e1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="Reactions 😂, 👍 2 in total"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="😂" draggable="false" class="bd96tnyg p83aruvr b85 emoji apple _11JPr" tabindex="-1" style="background-position: -60px -60px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍🏻" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -40px -60px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk o2es7gts i2cterl7" style="flex-shrink: 0;"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 f8m0rgwh r15c9g6i ilf8vifs gfz4du6o r7fjleex qnz2jpws kjemk6od"><div class="dn5wc90o ilf8vifs gndfcl4n p357zi0d"><span class="f8jlpxt4 qfejxiq4 ce5kru2g">2</span></div></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3AC34E628BEA87E047F1_919915123897@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a"><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="copyable-text" data-pre-plain-text="[1:25 pm, 26/02/2024] +91 99151 23897: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Getting slow response from server error, not sure how many people are using</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:25 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:25 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3AFF2C27EC30AB0896BF_917987544173@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Abhinash Khare +91 79875 44173" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno akr8p6hy"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">A</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d jake6gi8" role=""><span dir="auto" aria-label="Maybe Abhinash Khare" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Abhinash Khare</span><span class="WJuYU" testid="author" role="button">+91 79875 44173</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[1:29 pm, 26/02/2024] +91 79875 44173: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>is it really a  streaming response from model? <img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="😕" draggable="false" class="b86 emoji apple _11JPr selectable-text copyable-text" data-plain-text="😕" tabindex="-1" style="background-position: -40px -40px;"></span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:29 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:29 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB0E9A180E06ED89333D1_919703944446@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Pratiksha +91 97039 44446" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno fz47docv"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">P</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d swew6qod" role=""><span dir="auto" aria-label="Maybe Pratiksha" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Pratiksha</span><span class="WJuYU" testid="author" role="button">+91 97039 44446</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[1:34 pm, 26/02/2024] +91 97039 44446: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g755h7wn _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d a7s2a5ab" role=""><span dir="auto" aria-label="Maybe Harveen" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Harveen</span><span class="WJuYU" testid="author">+91 99151 23897</span></div><div class="_37DQv" dir="ltr" role="button"><div class="l7jjieqr tvf2evcx oq44ahr5 lb5m6g5c g97rt3p3 fewfhwl7 jdwybkuq kjemk6od"><span data-icon="status-image" class=""><svg viewBox="0 0 16 20" height="20" width="16" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 16 20"><title>status-image</title><path fill="currentColor" d="M13.822,4.668H7.14l-1.068-1.09C5.922,3.425,5.624,3.3,5.409,3.3H3.531 c-0.214,0-0.51,0.128-0.656,0.285L1.276,5.296C1.13,5.453,1.01,5.756,1.01,5.971v1.06c0,0.001-0.001,0.002-0.001,0.003v6.983 c0,0.646,0.524,1.17,1.17,1.17h11.643c0.646,0,1.17-0.524,1.17-1.17v-8.18C14.992,5.191,14.468,4.668,13.822,4.668z M7.84,13.298 c-1.875,0-3.395-1.52-3.395-3.396c0-1.875,1.52-3.395,3.395-3.395s3.396,1.52,3.396,3.395C11.236,11.778,9.716,13.298,7.84,13.298z  M7.84,7.511c-1.321,0-2.392,1.071-2.392,2.392s1.071,2.392,2.392,2.392s2.392-1.071,2.392-2.392S9.161,7.511,7.84,7.511z"></path></svg></span></div><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Krutrim is here</span></div></div></div><div class="jrEtJ"><div class="kknmh"><div class="g0rxnol2 ln8gz9je ppled2lx"><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><div class="lhggkp7q ln8gz9je ppled2lx t3g6t33p qnwaluaf" style="background-image: url(&quot;data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDABsSFBcUERsXFhceHBsgKEIrKCUlKFE6PTBCYFVlZF9VXVtqeJmBanGQc1tdhbWGkJ6jq62rZ4C8ybqmx5moq6T/2wBDARweHigjKE4rK06kbl1upKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKT/wgARCABIAEgDASIAAhEBAxEB/8QAGQAAAwEBAQAAAAAAAAAAAAAAAAIDAQQG/8QAFQEBAQAAAAAAAAAAAAAAAAAAAAH/2gAMAwEAAhADEAAAAPR4KjCbTtEluQsaAc8uiYmPouPpLXccYExEKvzYdpx6dZxsdQgJOykimktoE2ZzCgc6gLoBgBUC4B//xAAWEQADAAAAAAAAAAAAAAAAAAAAEUD/2gAIAQIBAT8AHR//xAAWEQEBAQAAAAAAAAAAAAAAAAARAED/2gAIAQMBAT8AI0f/xAAwEAABBAECAwUHBQEAAAAAAAABAAIDERITIQRBYRAUMTORICIyQlFTcSNDUmSBof/aAAgBAQABPwB4cT7ppVJXiFUleIVSfUJod8xCr2JXPbWDbQlfzjKM0l7RFCSS6MaMz62jKE0nOMrVlBrT7eJ+JtyFo6K2/dfuiQPd1HWUMSdpXLNlVqOtWzHzXbIOb9xxtFzCfNcom4s+Iu6lTZZDHGudo6m4tnRAyFx3Yg2UgFuCLJrsBiLJSBs2+aDJrIDWgJjHZHNra5V2SRNkIyR4eMgCvBd2i+n/AFMY2MU32uIouFyFv4QIFjWO6BHiZjSsWSJzsiRXnFNcPvFW0fvGzzTGmQnCYqNpa2i7I/VTBxcMQ0/lOzLR7rLR1MfgYTyQ1Btiyijqn5WUhn/FnVOD7NNZXJMMgJxa3/FFqb6ldKUkTZCCb2R4dhN7+q7swGwT6ru0fX1Xd2dfVHh2E3uu7su9/VRxNjJx59nEVY/UwTXA+E3JZf2ESAfP3QcAa190XVtr7ppoYun3UMjQcTJkez///gADAP/Z&quot;);"></div></div><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><div class="lhggkp7q ln8gz9je ppled2lx t3g6t33p qnwaluaf" style="background-image: url(&quot;blob:https://web.whatsapp.com/b8eb025b-3de6-4d74-a94f-3d40c0c2af35&quot;);"></div></div></div></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>looks like a half cooked product</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:34 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:34 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A07F86B9A1CE037B16B_919868221372@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Paras Chopra +91 98682 21372" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/162690799_131241352786128_2676048871288803711_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRLabAEX9bmDVVjQ8t15GgUazuMiCZOrKVtAXDMdkl8Pw&amp;oe=65EAF085&amp;_nc_sid=e6ed6c&amp;_nc_cat=103" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d jake6gi8" role=""><span dir="auto" aria-label="Maybe Paras Chopra" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Paras Chopra</span><span class="WJuYU" testid="author" role="button">+91 98682 21372</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[1:35 pm, 26/02/2024] +91 98682 21372: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g755h7wn _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d a7s2a5ab" role=""><span dir="auto" aria-label="Maybe Harveen" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Harveen</span><span class="WJuYU" testid="author">+91 99151 23897</span></div><div class="_37DQv" dir="ltr" role="button"><div class="l7jjieqr tvf2evcx oq44ahr5 lb5m6g5c g97rt3p3 fewfhwl7 jdwybkuq kjemk6od"><span data-icon="status-image" class=""><svg viewBox="0 0 16 20" height="20" width="16" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 16 20"><title>status-image</title><path fill="currentColor" d="M13.822,4.668H7.14l-1.068-1.09C5.922,3.425,5.624,3.3,5.409,3.3H3.531 c-0.214,0-0.51,0.128-0.656,0.285L1.276,5.296C1.13,5.453,1.01,5.756,1.01,5.971v1.06c0,0.001-0.001,0.002-0.001,0.003v6.983 c0,0.646,0.524,1.17,1.17,1.17h11.643c0.646,0,1.17-0.524,1.17-1.17v-8.18C14.992,5.191,14.468,4.668,13.822,4.668z M7.84,13.298 c-1.875,0-3.395-1.52-3.395-3.396c0-1.875,1.52-3.395,3.395-3.395s3.396,1.52,3.396,3.395C11.236,11.778,9.716,13.298,7.84,13.298z  M7.84,7.511c-1.321,0-2.392,1.071-2.392,2.392s1.071,2.392,2.392,2.392s2.392-1.071,2.392-2.392S9.161,7.511,7.84,7.511z"></path></svg></span></div><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Krutrim is here</span></div></div></div><div class="jrEtJ"><div class="kknmh"><div class="g0rxnol2 ln8gz9je ppled2lx"><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><div class="lhggkp7q ln8gz9je ppled2lx t3g6t33p qnwaluaf" style="background-image: url(&quot;data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDABsSFBcUERsXFhceHBsgKEIrKCUlKFE6PTBCYFVlZF9VXVtqeJmBanGQc1tdhbWGkJ6jq62rZ4C8ybqmx5moq6T/2wBDARweHigjKE4rK06kbl1upKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKT/wgARCABIAEgDASIAAhEBAxEB/8QAGQAAAwEBAQAAAAAAAAAAAAAAAAIDAQQG/8QAFQEBAQAAAAAAAAAAAAAAAAAAAAH/2gAMAwEAAhADEAAAAPR4KjCbTtEluQsaAc8uiYmPouPpLXccYExEKvzYdpx6dZxsdQgJOykimktoE2ZzCgc6gLoBgBUC4B//xAAWEQADAAAAAAAAAAAAAAAAAAAAEUD/2gAIAQIBAT8AHR//xAAWEQEBAQAAAAAAAAAAAAAAAAARAED/2gAIAQMBAT8AI0f/xAAwEAABBAECAwUHBQEAAAAAAAABAAIDERITIQRBYRAUMTORICIyQlFTcSNDUmSBof/aAAgBAQABPwB4cT7ppVJXiFUleIVSfUJod8xCr2JXPbWDbQlfzjKM0l7RFCSS6MaMz62jKE0nOMrVlBrT7eJ+JtyFo6K2/dfuiQPd1HWUMSdpXLNlVqOtWzHzXbIOb9xxtFzCfNcom4s+Iu6lTZZDHGudo6m4tnRAyFx3Yg2UgFuCLJrsBiLJSBs2+aDJrIDWgJjHZHNra5V2SRNkIyR4eMgCvBd2i+n/AFMY2MU32uIouFyFv4QIFjWO6BHiZjSsWSJzsiRXnFNcPvFW0fvGzzTGmQnCYqNpa2i7I/VTBxcMQ0/lOzLR7rLR1MfgYTyQ1Btiyijqn5WUhn/FnVOD7NNZXJMMgJxa3/FFqb6ldKUkTZCCb2R4dhN7+q7swGwT6ru0fX1Xd2dfVHh2E3uu7su9/VRxNjJx59nEVY/UwTXA+E3JZf2ESAfP3QcAa190XVtr7ppoYun3UMjQcTJkez///gADAP/Z&quot;);"></div></div><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><div class="lhggkp7q ln8gz9je ppled2lx t3g6t33p qnwaluaf" style="background-image: url(&quot;blob:https://web.whatsapp.com/b8eb025b-3de6-4d74-a94f-3d40c0c2af35&quot;);"></div></div></div></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>What are Indian values?</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:35 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:35 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB053CD12D47F91B9DA20_919550164716@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _27hEJ _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Ravi Theja +91 95501 64716" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno j3zl77sr"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">R</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div class="cm280p3y ktbp76dp ocd2b0bc folpon7g aa0kojfi snweb893 g0rxnol2 jnl3jror copyable-text" data-pre-plain-text="[1:36 pm, 26/02/2024] +91 95501 64716: "><div><div class="bxcbqipq mhcwslh8 ocd2b0bc jfqm35v0"><div class="_3IzYj _6rIWC p357zi0d gvktkc8w" role=""><span dir="auto" aria-label="Maybe Ravi Theja" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Ravi Theja</span><span class="WJuYU" testid="author" role="button">+91 95501 64716</span></div></div><div role="button" tabindex="0" aria-label="Open Picture" class="gndfcl4n l8fojup5 paxyh2gw sfeitywo cqsf3vkf p357zi0d ac2vgrno laorhtua gfz4du6o r7fjleex g0rxnol2" style="width: 330px; height: 155.012px;"><span class=""></span><div class="g0rxnol2 ln8gz9je ppled2lx"><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><img alt="😅" src="data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/4gHYSUNDX1BST0ZJTEUAAQEAAAHIAAAAAAQwAABtbnRyUkdCIFhZWiAH4AABAAEAAAAAAABhY3NwAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAQAA9tYAAQAAAADTLQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAlkZXNjAAAA8AAAACRyWFlaAAABFAAAABRnWFlaAAABKAAAABRiWFlaAAABPAAAABR3dHB0AAABUAAAABRyVFJDAAABZAAAAChnVFJDAAABZAAAAChiVFJDAAABZAAAAChjcHJ0AAABjAAAADxtbHVjAAAAAAAAAAEAAAAMZW5VUwAAAAgAAAAcAHMAUgBHAEJYWVogAAAAAAAAb6IAADj1AAADkFhZWiAAAAAAAABimQAAt4UAABjaWFlaIAAAAAAAACSgAAAPhAAAts9YWVogAAAAAAAA9tYAAQAAAADTLXBhcmEAAAAAAAQAAAACZmYAAPKnAAANWQAAE9AAAApbAAAAAAAAAABtbHVjAAAAAAAAAAEAAAAMZW5VUwAAACAAAAAcAEcAbwBvAGcAbABlACAASQBuAGMALgAgADIAMAAxADb/2wBDAAMCAgICAgMCAgIDAwMDBAYEBAQEBAgGBgUGCQgKCgkICQkKDA8MCgsOCwkJDRENDg8QEBEQCgwSExIQEw8QEBD/2wBDAQMDAwQDBAgEBAgQCwkLEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBD/wAARCAAuAGQDASIAAhEBAxEB/8QAHQABAAIDAAMBAAAAAAAAAAAAAAUGAgQHAQMICf/EAC4QAAICAQQBAwMDAwUAAAAAAAECAwQFAAYREhMHFCIVISMxMlEIF5EkQlJTcf/EABUBAQEAAAAAAAAAAAAAAAAAAAAB/8QAGhEBAQEBAAMAAAAAAAAAAAAAAAERQQISYf/aAAwDAQACEQMRAD8A/Ur3jHJHHipOVWATNP04iUluAnJ/VjwTwvPAHy69k7bOqD6gYLZNmbI5Pd2Vw9cS46GBXyaY8LRRHmY2o2sQO3dfIW4kLRcwpwq8yFqFt7YPpzZlyt3au79t59tyhcslmsm3zPeuVpbAS1+HHBZZFu2BJ5XaTpYUcAdpUleMvak3rvemuA7N/t/j7WdyWx/UzGxtuSUTz0Mda21G1iScWQmQcwVeZVkjQ3FeSSRmSN/xno8T71f0u9Lmyd3KZ+PZd7JYnJnOWknrYaRo7qPJKXmcUEeNlNrv5eRIGk5LDsxe4ruGmvn/AAuO/pys4SPE7sX0muJdaTM3LF2fCzWJ7duBbjzyxx1kgd3hgEplUcvHVjkJcAsI6DY3ohj8Rc3Rl6vp5uPa12SnYs3r426uIqxVKfta1fzR0oizCGy6wh+wWONl8sSP4pGD6R01QdjemeyttvHkcN6X7LxFg2I7fv8AF1K6yTyLXkiS12irx8yeOeaPngcJLKAerEG/agaaaaCD3Js/E7qerJk7eZhNNi0f0/M26AYkg/MV5E8g+I+z8gff+Tzlj9p4vG5KfK17OVeaxBDXdLGVszwhIwQpWGR2jRzz8nVQz/qxYjU1pq7TDTTTUFR3ht9dyXo8dYawie3Lx+z3XfxU7gdhIelUryqkwDv2J/I37eqh4vFUd44+1Try4LFJZu11e5FJv3JWWiVU8UpgWWt+QKqUyH4jJeeZj1b5TdB4B/UD+NOAPuAP41RS6mP3vSjuZSHbuNbKX4EjlrS7xyEtVSteaQCMvWIi/wBVL4i6RKzQ9ZD94o6+tIemZzk1G1uWfP0pUriWyMZv3LqhtPK00kQVGhV4hJI/V+EPRI4wiRgRr0LTU0VjOeneB3BRmx97I7nhjnuR3majufJU5VlSBYQFkgnR1j6qCYgRGz8yMpdix2G2Thno4vHG7n/FiOnt2G4L4lfo6OPPIJu9j7xqD5i/ZS6nlXcNP6aCpJ6X7ajydbLrk92mepZe1GjbvyzQs7SiUh4TZ8cidhwI3UoF5QAISurbppoGmmsS6r9mYD/06DLTWPkj/wCxf868NKoHKDyH/ipHJ/ydBnpppoNb3nOQ+nrUsHiHzNP04iXluFTsT8mPDHhQeAvy69k7bOq1l8fvEbi+r4XKNLR9klf6XPbhgrGbu5afkU5JvIFKAfl8Z4/YCCzeoYfenthRO4bI8NJKS3RZreexI4RZLkiey8azx9GeNVHidpGDoqhAskvak+rVpqqS7d3Pk4pkv7jt0xNAekdZ6c61Z+lYo8flp/cxyQzMrOCOZWJQ8xpBlHs/PJmzlm9UNzPVJdvpjVsZ7b7xdB8hTE3xb8g/J+8cHlPhqqtOmq2+08w+KrY8eom4Es11lD5AV8d7idnUqruDVMQZOeV6ooPHyDDkaxubOuyz5Kxjd7ZnEvkpZZmNOrj+YmaoldCpkquWMZjSVTIXPcBWLxARALNpqPwOHh2/iauGqzNJBTiSCEGGGIJGihVVUhREVQB9gqgD9BwOBqQ0DURndr43cVeetfs5SJLAiDmllLNNx43LL1eB0ZOSfl1I7DhW5UAal9NNwVO76Z7dv/UPPkd0L9TCLN4N0ZKHoFk7jxdJx4Dz9iY+vK/E8r9tSOM2ji8TYjtVbWWd4qUVBRZy1qyhij/axWWRlaU8/KUgyP8A7mbU3pq+1TDTTTUV/9k=" class="jciay5ix tvf2evcx oq44ahr5 lb5m6g5c fsmudgz7" style="width: 100%;"></div><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><img alt="😅" src="blob:https://web.whatsapp.com/e7e2cf80-e070-463c-bc76-8f2da3f8bb74" class="jciay5ix tvf2evcx oq44ahr5 lb5m6g5c osz0hll6 nq7eualt em5jvqoa a21kwdn3" style="width: 100%;"></div></div></div><div class="cm280p3y bkoknyjm esbfogcw jfqm35v0 bxcbqipq mhcwslh8"><div class="_21Ahp"><span dir="auto" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="😅" draggable="false" class="b85 emoji apple _11JPr selectable-text copyable-text" data-plain-text="😅" tabindex="-1" style="background-position: -20px -80px;"></span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:36 pm</span></span></span></div></div><div class="lrw9n60e lhggkp7q fz4q5utg b9fczbqn"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:36 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 cvsrm5e1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="reaction 😂"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="😂" draggable="false" class="bd96tnyg p83aruvr b85 emoji apple _11JPr" tabindex="-1" style="background-position: -60px -60px;"></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB0395EC262A4A5E6AC91_919703944446@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _27hEJ _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Pratiksha +91 97039 44446" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno fz47docv"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">P</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div class="cm280p3y ktbp76dp ocd2b0bc folpon7g aa0kojfi snweb893 g0rxnol2 jnl3jror"><div><div class="bxcbqipq mhcwslh8 ocd2b0bc jfqm35v0"><div class="_3IzYj _6rIWC p357zi0d swew6qod" role=""><span dir="auto" aria-label="Maybe Pratiksha" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Pratiksha</span><span class="WJuYU" testid="author" role="button">+91 97039 44446</span></div></div><div role="button" tabindex="0" aria-label="Open Picture" class="gndfcl4n l8fojup5 paxyh2gw sfeitywo cqsf3vkf p357zi0d ac2vgrno laorhtua gfz4du6o r7fjleex g0rxnol2" style="width: 330px; height: 249.15px;"><span class=""></span><div class="g0rxnol2 ln8gz9je ppled2lx"><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><img src="data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/4gHYSUNDX1BST0ZJTEUAAQEAAAHIAAAAAAQwAABtbnRyUkdCIFhZWiAH4AABAAEAAAAAAABhY3NwAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAQAA9tYAAQAAAADTLQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAlkZXNjAAAA8AAAACRyWFlaAAABFAAAABRnWFlaAAABKAAAABRiWFlaAAABPAAAABR3dHB0AAABUAAAABRyVFJDAAABZAAAAChnVFJDAAABZAAAAChiVFJDAAABZAAAAChjcHJ0AAABjAAAADxtbHVjAAAAAAAAAAEAAAAMZW5VUwAAAAgAAAAcAHMAUgBHAEJYWVogAAAAAAAAb6IAADj1AAADkFhZWiAAAAAAAABimQAAt4UAABjaWFlaIAAAAAAAACSgAAAPhAAAts9YWVogAAAAAAAA9tYAAQAAAADTLXBhcmEAAAAAAAQAAAACZmYAAPKnAAANWQAAE9AAAApbAAAAAAAAAABtbHVjAAAAAAAAAAEAAAAMZW5VUwAAACAAAAAcAEcAbwBvAGcAbABlACAASQBuAGMALgAgADIAMAAxADb/2wBDAAMCAgICAgMCAgIDAwMDBAYEBAQEBAgGBgUGCQgKCgkICQkKDA8MCgsOCwkJDRENDg8QEBEQCgwSExIQEw8QEBD/2wBDAQMDAwQDBAgEBAgQCwkLEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBD/wAARCABLAGQDASIAAhEBAxEB/8QAHAAAAgIDAQEAAAAAAAAAAAAAAAYEBQECBwMJ/8QAMhAAAgIBBAEDAgQFBAMAAAAAAgMBBAUABhESEwcUIRUiFjEyYRcjJEGRCCUzcUJRgf/EABYBAQEBAAAAAAAAAAAAAAAAAAABAv/EABwRAQACAgMBAAAAAAAAAAAAAAABEQIhElGRgf/aAAwDAQACEQMRAD8A+qejWrBklkImQTMTEEPHI/vHMTH+Y1TbPp2MbgFUbeUtZFyX2QKxZS5ZlPnP4iHERyA/pEpIoIREomYmJ1icsucY1qp35Ufd+M3PKq12u9Gsc/8Af+NHOttM6NRq2RoXkTZo21WlC5lcjQXkiGgyVsCZHniRMSEo/wDGRKJ44nWKuRqXPhRMA5NwQtyjSwvEzoZQBxBSPbjgojqUEJDMiQzIStGsajZLJV8VRsZG0u0aqymOMa1VthsiAyUwC1CRmXETwIxJFPEREzMRoJWsTzETMRzP/rWiHhYSuwsWQLBg4hiyAoiY5+RKImJ/aYiY16aCjqXtzHfcvIY3HopixsJam6bWkEePxySySMDJcu7RBl16BxJ956TgsWpGfL1GeZ4gZ5jjn4n8o+eOPj+35fP56naNauOkpDxBXmY9DcnUr1rjFidlVdstWDZGO8CcgEmMTzEFIDMxETIx+WjUzRrMqNchw+0PUO7fJ2TuZGii1mrOQ6ObdiUY8+8e0KV5pg+bi9Y6tAfCM1wkUBKKuuuzMREzM8RH99cE27uP1DmrSsZT1Nrs3DnU07GI2+7O04quDzIa3s76EqwqJR0AuypmJuwv+UZoZFgPeAwe59olnLFkMpn7lpKLksr2Hyh9vow2rrJu5JkVxl0FMKiVLFbUq7nC5MaLdOy86q3Rs7Rx28xcm24ROxuG5bSt1hpANpqpzCIbVTCoYdeYIpB6xUIFDAj0xN7dt2lk7X8TlWtw4y+cXMDQzFC3Tp3XOSVfGNeGLiwpHC4DyGrzCu48i7yCTCTlrPqJe3rjYwG6qEJBa8lZxAZ6sC2pKULIJicU10piBsMAxcszOSHmAmPCECts7fedovDyZnb+SmnNWvZyFu62stJtA0iaKubmTepdcFufBiTTdJCUAbgmbQ9PN20t5bbvo3Dfilga15Be5t5R6rK5QsEQwSyxA5kMa4/JYQ4iFfWJSQAxkg/4mHmq2Bx248fZp0WIJd48/WHI5RSFL9zNmqOLJapmwIKOEnH2WmGJJIVKiaA+siiW/wCnYRxTlRsMQzccwsaZA4DTBDi+xCHFdoj8GRmwSdALiGhj+F9qmq4zDbqyKrftqy8ay9lsxcUh1fmFG9ZX484yAohgxK5cQNNhlLeB8Ml6X7hfex0Y3eza+MrPRFpDreZbYfVVEyIA8cmEC2TbakmmDO4lXEwL24TMLF7d9caZubd3Wd72T7cVFPzVLpkEzUlaCsyvChKShy1M6q7QJWHTJNBa0FijiPXvH2Se3J08mDsONUk29x1wCvdCmrh6/FhBIiZahomRT4xDhgojtFdYQdseiW8sTELznqYzIL8aFTCLm4lFIiS4b9zs06exLFsCX5iZicyfUhPsWuQ38H/qHKnjsfj83W5Uq9Fy+W5aoWGGyyRV56fQTUXjSCgiRFfENbBw8gBxN+3KnqLGdr39zWK66LcewrVOvlF2VIul7cRWr+gSxixhTjhpNGe1hkSoh8XhSG/Ro0agNGjRoMTzxPExE/250kRkfUCkdl9nL0MkC8Q5yqdPaVtDm3PK0lTDW3PFx4ghcoLqUs4OWAJiuHZgAwCWwBISiYISjmJifzidKO2qOxMUvFxjcLs/F5izjIMK+NNPEVYGsLoQcLAmIGPaj26CMxFfmB5GIDQ8lvmzuPP4uldq16i1oDFvsbasEpDyrnJixsWx91EMlLJIAUsR7J8hMkiVVEHrgyy1A722nUkkApAu2JddEPghE3kxeUkPEci0hXMiYCxfcpkC7+FHeXo3GCwO6MyzYWGbhUuq4tljJ0JVQq/etpU7ATMChiapmMD0klriGCuRMQuqO5PRqsx9ahntlJZYo0EvWi3ViWUml4qIlET8qOXQCon7ZlvUOe3zRqMeqMVWYw937cPLxZdMWR2fdGpCCVPgHrN7iSA+pGcN4MYkIFZTBRW1ch6wU8j9Oy2WxGQW90KXfpbMsIr1+UWeZYJ5M2FAvGpMkIxEizqPPZjq8rIWPQV2RejKv2Cy/h6CmPCwVKXU6RMQ1ZFBfctMsVUMZngZJaZj5EJj2fd9HE0UtyrdjpTjGM3MHZ1SVVGs9wx2QGS4gJmRumT/AImerymf18B75C3v4KLrGNzeMN0pyRIVO1rRERlwVCJgrYR/LGYFsEQQ8uepVuJiIVS36q5OrVtrzmHxRWk3bE1b20LLGo7qn2qWSrISHkQyJl0jMi+OBVKuYPVIvI+hvqEFRG9MP6Z3Cy12FVK9q5j8gdzJ1vcVzhUSMwyQWBwsonywBsE1pkSDTLi9/enNdEPRl9t4bKZe0tVnHvyVNdicj4ek1nSkzE7ALqkuYEj+K0xEyIcwFlii3g2yb7+axh1JJrBQOBsIcCidEqCTKwUEYqFoHMD9xkDIEBjod9XshZg5AGhKzkChiyD5iI/LmPuj5/OOY/fSYj1Z2puHHX7mwNybVz0Vkf09r8RVxovtEEkNeXJ8xrmJOrJzKp4C0shhk/ZptxOXxOfxlbNYLKVMlj7qhfWt1Hi5L1lHImBjMiQzHzExPE6gl6NGjQGjRo0GJ+YmJ5/+aSHbi2juwVqOvvhEUbiURIYvOY6SZZUxQzMitflXAsKSOey1FAMKQMAIXVwExLFg40kQyMMCIkgmY/OO0THMfvEx+2uZ/SfSl9m5lR3zgiuxUjIHdgMRLq6UpJhWe/g/TH1M3yZciM3iKOoOmCsBgxec2y/M1AqI3fFm1XqIX7rH5cK8B4rLFyyWhCVn1B0MI+C7zXBs9yrjMXCfhGJldD8c/wC2jGG5vfXPviiXkg+X/wDNJSfE2fumzEdJY2B6xVfSvRbYTLW90bi2ntixFjIvyebBeKpssBLvA9Nh/ij7FWCrCU8wflQgWEU9hOs25i/9PuPx2WbtHeOya2Iwddt3IrxoYUKuNxt1cmwG+NPCqzxStkkcxLIrLmTKBnkL1+1NkZ/cIZJlr1DVcxXLYEMzuGnUPqYW+JULQQ/5tiPWRPtAmjiYrmpdjt3MbWxVXLljU7zNdAlTZ+o0czYMpBS0j7f3IETY6qGShHaCOTYXJsMiUvUDG+hGWTeye7N+7Rx1RQZDEQdocIasfdKJO0wCtIZxYiK8kwDkgmEzJrnp8Md6z6VbbzL0bg3ftupY+ksRToXmY9M47GCsieCI6CyK8xUJjIOTD+mmfgVzEBnbm4dnUrNt+Mr78I7A0bDYyWMzrQiLSx8ULGyuRCYgY8oBESku3mhZSXMi96ubVx9Orefid6Gu55YWKNk5lzB8ZsEu611ZNfMqLr3ge0SBDzBhJb7gx/p3RyeVy+VyOCw946C8hlrDQpA06lMuw2LBtCSlaYIxgynqENPjrMxMKOXL0JbvMV5X1N2ivP321U4qk5mFm1WtyLDpsqwaZcTh965ioKT+bRzAzDS7A90N/wCCyVsaVehuQGEwlwT9tZJC+RNITMmxAjEcvDgpniYFpRMipsgya50ndHoLsJLlL3hsTb+PxVi0U1IuUKlehcWcDcZxHWVu7XVC2Zn4ly4mBlk92rbd3ZlcPwhtC3hVBgqylfS8axQxQr9mKUPhX/xB2ruAY4iOUmMfomIC70aNGoDRo0aDUzhYEcxMwMTPERMz/iPmdcs24FXL+0bl8TuqhbxV8DQtWZ3GSWDbIWCTSclMPiBdPZZia68j45lfSYDqusRoENuxLlpj8jYq5ELlRCyoKVvzMAh7hV+hwjwIj5CMe3RkkMCcjz9g7U9l5JRluttK4O4pUyIofjnLNxkkQTHHUo8cRzMxBe35H4IY5iIh70aBJp7SymPZm7VajYNxRzjQfvTKOXZmGG2PLBiUVY7lET44Z9k9PkQEdTMjs4bNt2VTGTO3ItpguN1ZFCCrPZBOZKwKQFwwTJXMBJB1AQYoZ+xq0aBMPF7vdg6eCdgaQpsSurkCXvDIQ+vWhaxJibEIhzGxMHHyS5LiClkEZcVI+neUq2rwV6uSfXcybymv9Rc1DCsi1wrV06yKk+FvaYEpHvAjIH4gbrpOjVsJsentOi6k7Gszr5x9lb0+63llpg+5LB3lEmHDRFShIFs7ARycfy/IbCuNvbfDCWMlYGbslkrJWm+4zNu8MGRFzCgeUjXDjrMLV1COZiI4GJm60agNGjRoDRrET8z/AN6NB//Z" class="jciay5ix tvf2evcx oq44ahr5 lb5m6g5c fsmudgz7" style="width: 100%;"></div><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><img src="blob:https://web.whatsapp.com/51fcbb39-0a07-489a-aacc-d1c29e243276" class="jciay5ix tvf2evcx oq44ahr5 lb5m6g5c osz0hll6 nq7eualt em5jvqoa a21kwdn3" style="width: 100%;"></div></div><div class="lhggkp7q jxacihee tkdu00h0 b9fczbqn ln8gz9je he7yjufn e1lnay39"></div></div><div class="dpkuihx7 lhggkp7q j2mzdvlq b9fczbqn"><div class="pp8r7oc8 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:36 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 cvsrm5e1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3AEA696CEC60FC23ABEB_918105723869@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe AA +91 81057 23869" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno b2boiwr0"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">A</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d mglqvgej" role=""><span dir="auto" aria-label="Maybe AA" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">AA</span><span class="WJuYU" testid="author" role="button">+91 81057 23869</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[1:37 pm, 26/02/2024] +91 81057 23869: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="akr8p6hy _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d jake6gi8" role=""><span dir="auto" aria-label="Maybe Paras Chopra" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Paras Chopra</span><span class="WJuYU" testid="author">+91 98682 21372</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">What are Indian values?</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>What career should I choose ?
Ask your parent</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:37 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:37 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="Reactions 😂 12 in total"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="😂" draggable="false" class="bd96tnyg p83aruvr b85 emoji apple _11JPr" tabindex="-1" style="background-position: -60px -60px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk o2es7gts i2cterl7" style="flex-shrink: 0;"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 f8m0rgwh r15c9g6i ilf8vifs gfz4du6o r7fjleex qnz2jpws kjemk6od"><div class="dn5wc90o ilf8vifs gndfcl4n p357zi0d"><span class="f8jlpxt4 qfejxiq4 ce5kru2g">12</span></div></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB04F1EC828A8BEF9FACE_919000844590@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Chetan +91 90008 44590" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno b2boiwr0"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">C</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d mglqvgej" role=""><span dir="auto" aria-label="Maybe Chetan" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Chetan</span><span class="WJuYU" testid="author" role="button">+91 90008 44590</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[1:39 pm, 26/02/2024] +91 90008 44590: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="akr8p6hy _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d jake6gi8" role=""><span dir="auto" aria-label="Maybe Paras Chopra" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Paras Chopra</span><span class="WJuYU" testid="author">+91 98682 21372</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">What are Indian values?</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Values which will not be frowned upon by the government.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:39 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:39 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_75C571B741110192E5_917892792975@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _27hEJ _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Adarsh +91 78927 92975" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/363225108_857693822457811_1088711388449985548_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRcT6arSV5WnGI4aSQcbLouECrP13HSczwH_vmBb1fOmA&amp;oe=65EB07E2&amp;_nc_sid=e6ed6c&amp;_nc_cat=109" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div class="cm280p3y ktbp76dp ocd2b0bc folpon7g aa0kojfi snweb893 g0rxnol2 jnl3jror copyable-text" data-pre-plain-text="[1:39 pm, 26/02/2024] +91 78927 92975: "><div><div class="bxcbqipq mhcwslh8 ocd2b0bc jfqm35v0"><div class="_3IzYj _6rIWC p357zi0d gvktkc8w" role=""><span dir="auto" aria-label="Maybe Adarsh" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Adarsh</span><span class="WJuYU" testid="author" role="button">+91 78927 92975</span></div></div><div role="button" tabindex="0" aria-label="Open Picture" class="gndfcl4n l8fojup5 paxyh2gw sfeitywo cqsf3vkf p357zi0d ac2vgrno laorhtua gfz4du6o r7fjleex g0rxnol2" style="width: 330px; height: 216.67px;"><span class=""></span><div class="g0rxnol2 ln8gz9je ppled2lx"><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><img alt="serious question: Is the code indentation bad due to UI or is it inherently bad because of the their dataset?" src="data:image/jpeg;base64,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAMCAgICAgMCAgIDAwMDBAYEBAQEBAgGBgUGCQgKCgkICQkKDA8MCgsOCwkJDRENDg8QEBEQCgwSExIQEw8QEBD/2wBDAQMDAwQDBAgEBAgQCwkLEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBD/wAARCABBAGMDASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwD9U6KyfFf/AAkQ8O3x8JrC2reV/owlIAJyM4JBUPt3bdw27tu7jNHhX/hIT4dsP+EqWFdV8ofaREQQGycZIAUtjG7aNu7dt4xWHt/3/sOV7Xvb3d7Wv3627Ec/v8lnte/T/hzWrIvtU1C1vfs8OkXVxEQp82FogFJkVSCHdTwrb8gHKo2Pm2q2vRXQnYs57/hINW+2RWv/AAi+qeXJIyNcb7Xy4wGKhmHnb8EAMNqk4YZAOQDVNa1+28ldK8Py3rvfwW0he4ihSO3baZbnOSSEUsAm3czqBhVPmDoaKfMuwrGNYapql1Zw3V5pM9nLM3zW7yRO8KnoXKsVyB1Cs3PQt1py6pfMZM6XdrskZFJaH5wFyGGH6E/KM4OeoA5rXoo5vILHOTa54hj8RW+mw+G5pdKktjJNqYuYR5U2Ttj8oncwwp3MOheIAMC7RyTa9qsUipH4a1OYG8FsWR7UAR7c/aDulB8vPGAPMz/Bjmt+ijmXYLGNqmr6hp9i91aaPe6nKn3bW1aBZH+hlkRPzYVVj8Qa3NNLAvhXUotly9vHLPNaiJ1Ebss+UlZxEzKqD5PMBdSYwoZh0dFHMuwWMuPUbyRWLWNxGQ7KAxj5AYgMMMeGADDvgjIByAVqUUc3kFhkkixxl5DhR1rIvLHSL6zlsZrrUljmt/srNDqFzDIE2OmVkRw6viRv3gIfIVs7kUjSvv8Aj1f8P5ivKLP4T+FtF8QaR4k0fwf4EsdUs7V7OG+tvCKrdQlYUitxFMkgaCKO2SaEochg8aqyBdkiSBs9E1DTtH1PVNN1i4utTSfSmka3S31K6ggcuoU+dDG6x3AAHAlVwpyVwTmrl1JY3kMlvNNOqyxPCxilkiYKwwSGQgq3owIYdiKwLaDxAuhi3vNU06XWPsoRruOwdLY3HlgGTyDMziMyZYR+bkKQu8kbzoDOBuwTTsK5JZafo9gtisF1qTDTrZrWHztQuZtyMYyTLvc+c+YlxJJucAuAwDuGp6H4b8O+Hf7Q/s+/16X+08ed9u12/vdmN3+q+0TP5H3j/qtucL/dXFiiiwXKkXhXw3DHpsK6h4hK6UztAW8QagxctMkx84mYm4G6NQBLvAQvGMI7q2jNZ6TPpN3oz3Woi3vfP8x0v7hJ185mZ/LnVxLHgudmxl2AKE2hVAhoosFy8jafHfTais1wZZ4Y4WVppDEFRnIKxk7FYmRssoDMAoYkIoDpJrGRWVpZgGXadsjrxnPGDwffrWfRRYLmr9vtf75/75NWawa3qTVhphRRRSGV77/j1f8AD+YrhdDu9K1O31rQP7atJ5La7ukuFstZlmuIEmeRhvfcJLdxlwEVsJswhUKFXur7/j1f8P5iuF1vxB4tsbi1Gi+ALnU7dpmS7Y6jbwSRxiJXDxIzESZYtHtZo/nUn7hD1SJZd0Xxd4Z8Q6TPruj63aXOn201xbzXKviNHhkaOTJOMAMjfN0IwwJUgmld/EfwXZeJtE8Hza4jap4itLy+06OGGSWOW3tVhaeRpUUxxqouYCC7Lu81du6tXT7zVpNFhvNW0X7PqP2ZJLixt7hZlWbywzxRysEDgMSoZgmcZIUGsmz17xlceM20a58AtaeHRZ3Eq61JqcLO1wkyJHD9mXLASI0kgfdwqKGCs21WI0rXxP4fvNfvvC9rqsEuq6bbw3V3aqfmhjlLhCe2T5bcZyBtJADLmlrnj/wj4f0K78SXurfaLCxuY7O4fT7eW/eOaSVIlTy7dXkJDypuwvyg7mwoJHQUUAFFFFABRRRQAVvVhVu1LKQUUUUhkVxc29pGJrq4jhQuse6Rgo3MwVVye5YgAdyQKzte8XeFPCtnb6l4o8T6To9pdzLb29xf3sdvHLKwLLGjOwDMQrEAckKfStVs7TtBJx26/rxWEsmsR+E7mRoddk1COCdo4z9h+3yMNxVUwRa7zwF3ELyu8j5jQBZ1jxV4X8PSWkOv+JNL0yS/vINPtUvLyOEz3UxYQwRhyN8khVgqDLNtOAcVb1PVdL0Wxk1LWdStbCzh2+ZcXUyxRplgoyzEAZJAGe5ArJ1a61yHTxDa6dr08tuqz+dZtYedc+SEkMJEzLGDMVaE4CgbyQ8XyyLR8S3viq3k0XxFo+j+J7oQ+ZHdaDYy6WizGWNSr3T3DZCwsCP9HmB3McrMuCrA17jxh4TtdP1LV7rxRpENjoreXqV1JexLFZN5aSbZnLbYzskjfDEfK6noRXOp8dvgjJeRafH8ZPAzXU5jWKBfENmZJDJ9wKvmZO7tjr2qOT/hPNUW61JofFGjCYac0NhBcaTI8UgjL3CAvE4EWXWN3aR3LRv5SooV5d3UJNat9Q+1W1v4guoUjdfs9udPETl4xg5kKyZQw4HzAFrnnco/dAFiLxj4Rn1qbw3D4q0eTVreZLaawW+iNzHK8UkyRtGG3BmiilkCkZKxuw4UkSf8JR4ZHiIeET4i0wa61qL4aX9sj+1m3LMgm8nO/wAvcjruxjKMM5BrE0mPxNNqMVxeT+JreEyx6m1tcNprDE6SRtYSeWpIS3ISUtG+5m2gSzLvU4+oat460tNN1+38MfEbWDduJZ9Et5PDqmzDCZvLmaSWMEL9qjTMUznOnxckNK1wNWC51sfjbwbLff2ZF4u0V7wzPb/Z1v4jL5qeVvj27s7l+0QZGMjzo8/fXN/T9X0nVrK01LS9UtLy0v41mtbi3mWSOeNgCrxspIdSCCCMggisCOTxFa+LIdPaDxVd6fcSS6g18x0sWFuCgjWxYArdEA5lVlRjuyGlK4SoNDufFF5Zg6hp/jCwk1G6njxfNo5k01JoBOkn+jsyMkLj7Mv+tcyElxLHiYAGxD408H3Gn2Or2/izRpbDVLhbSxukvomiup2YoscThtruWBUKpJJBHWtmucuo9c1K8XRYZvEelR6f9nlOsRDTmj1HIYPEVcSOuCAXPlR8suxiNwHRKCqhSxYgdT1P5UgFooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKAP/2Q==" class="jciay5ix tvf2evcx oq44ahr5 lb5m6g5c fsmudgz7" style="width: 100%;"></div><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><img alt="serious question: Is the code indentation bad due to UI or is it inherently bad because of the their dataset?" src="blob:https://web.whatsapp.com/ddbaaa4d-ec4f-4297-8fc0-b83c2e11235a" class="jciay5ix tvf2evcx oq44ahr5 lb5m6g5c osz0hll6 nq7eualt em5jvqoa a21kwdn3" style="width: 100%;"></div></div></div><div class="cm280p3y bkoknyjm esbfogcw jfqm35v0 bxcbqipq mhcwslh8"><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>serious question: Is the code indentation bad due to UI or is it inherently bad because of the their dataset?</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:39 pm</span></span></span></div></div><div class="lrw9n60e lhggkp7q fz4q5utg b9fczbqn"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:39 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 cvsrm5e1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_084E740A1B65B2DF13D27C95F3D4A3B9_919334372044@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Sid +91 93343 72044" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno akr8p6hy"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">S</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d jake6gi8" role=""><span dir="auto" aria-label="Maybe Sid" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Sid</span><span class="WJuYU" testid="author" role="button">+91 93343 72044</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[1:41 pm, 26/02/2024] +91 93343 72044: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>I will just say that, whatever the output quality, let's all remember  that at-least it's happening as somewhat of a grassroots level from India. Things will improve</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:41 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:41 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="Reactions 👍, ❤ 7 in total"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="❤️" draggable="false" class="bd96tnyg eipqoq80 b8 emoji apple _11JPr" tabindex="-1" style="background-position: -20px 0px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk o2es7gts i2cterl7" style="flex-shrink: 0;"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 f8m0rgwh r15c9g6i ilf8vifs gfz4du6o r7fjleex qnz2jpws kjemk6od"><div class="dn5wc90o ilf8vifs gndfcl4n p357zi0d"><span class="f8jlpxt4 qfejxiq4 ce5kru2g">7</span></div></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3AA3A0868E93A6323F48_918765534151@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _27hEJ _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Ritik Madan +91 87655 34151" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno ckklvomo"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">R</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div class="cm280p3y ktbp76dp ocd2b0bc folpon7g aa0kojfi snweb893 g0rxnol2 jnl3jror copyable-text" data-pre-plain-text="[1:41 pm, 26/02/2024] +91 87655 34151: "><div><div class="bxcbqipq mhcwslh8 ocd2b0bc jfqm35v0"><div class="_3IzYj _6rIWC p357zi0d fyja2idx" role=""><span dir="auto" aria-label="Maybe Ritik Madan" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Ritik Madan</span><span class="WJuYU" testid="author" role="button">+91 87655 34151</span></div></div><div class="cm280p3y bkoknyjm kzyzudjh"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="j3zl77sr _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d gvktkc8w" role=""><span dir="auto" aria-label="Maybe Ravi Theja" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Ravi Theja</span><span class="WJuYU" testid="author">+91 95501 64716</span></div><div class="_37DQv" role="button"><div class="l7jjieqr tvf2evcx oq44ahr5 lb5m6g5c g97rt3p3 fewfhwl7 jdwybkuq kjemk6od"><span data-icon="status-image" class=""><svg viewBox="0 0 16 20" height="20" width="16" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 16 20"><title>status-image</title><path fill="currentColor" d="M13.822,4.668H7.14l-1.068-1.09C5.922,3.425,5.624,3.3,5.409,3.3H3.531 c-0.214,0-0.51,0.128-0.656,0.285L1.276,5.296C1.13,5.453,1.01,5.756,1.01,5.971v1.06c0,0.001-0.001,0.002-0.001,0.003v6.983 c0,0.646,0.524,1.17,1.17,1.17h11.643c0.646,0,1.17-0.524,1.17-1.17v-8.18C14.992,5.191,14.468,4.668,13.822,4.668z M7.84,13.298 c-1.875,0-3.395-1.52-3.395-3.396c0-1.875,1.52-3.395,3.395-3.395s3.396,1.52,3.396,3.395C11.236,11.778,9.716,13.298,7.84,13.298z  M7.84,7.511c-1.321,0-2.392,1.071-2.392,2.392s1.071,2.392,2.392,2.392s2.392-1.071,2.392-2.392S9.161,7.511,7.84,7.511z"></path></svg></span></div><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;"><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="😅" draggable="false" class="b85 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -80px;"></span></div></div></div><div class="jrEtJ"><div class="kknmh"><div class="g0rxnol2 ln8gz9je ppled2lx"><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><div class="lhggkp7q ln8gz9je ppled2lx t3g6t33p qnwaluaf" style="background-image: url(&quot;data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/4gHYSUNDX1BST0ZJTEUAAQEAAAHIAAAAAAQwAABtbnRyUkdCIFhZWiAH4AABAAEAAAAAAABhY3NwAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAQAA9tYAAQAAAADTLQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAlkZXNjAAAA8AAAACRyWFlaAAABFAAAABRnWFlaAAABKAAAABRiWFlaAAABPAAAABR3dHB0AAABUAAAABRyVFJDAAABZAAAAChnVFJDAAABZAAAAChiVFJDAAABZAAAAChjcHJ0AAABjAAAADxtbHVjAAAAAAAAAAEAAAAMZW5VUwAAAAgAAAAcAHMAUgBHAEJYWVogAAAAAAAAb6IAADj1AAADkFhZWiAAAAAAAABimQAAt4UAABjaWFlaIAAAAAAAACSgAAAPhAAAts9YWVogAAAAAAAA9tYAAQAAAADTLXBhcmEAAAAAAAQAAAACZmYAAPKnAAANWQAAE9AAAApbAAAAAAAAAABtbHVjAAAAAAAAAAEAAAAMZW5VUwAAACAAAAAcAEcAbwBvAGcAbABlACAASQBuAGMALgAgADIAMAAxADb/2wBDAAMCAgICAgMCAgIDAwMDBAYEBAQEBAgGBgUGCQgKCgkICQkKDA8MCgsOCwkJDRENDg8QEBEQCgwSExIQEw8QEBD/2wBDAQMDAwQDBAgEBAgQCwkLEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBD/wAARCAAuAGQDASIAAhEBAxEB/8QAHQABAAIDAAMBAAAAAAAAAAAAAAUGAgQHAQMICf/EAC4QAAICAQQBAwMDAwUAAAAAAAECAwQFAAYREhMHFCIVISMxMlEIF5EkQlJTcf/EABUBAQEAAAAAAAAAAAAAAAAAAAAB/8QAGhEBAQEBAAMAAAAAAAAAAAAAAAERQQISYf/aAAwDAQACEQMRAD8A/Ur3jHJHHipOVWATNP04iUluAnJ/VjwTwvPAHy69k7bOqD6gYLZNmbI5Pd2Vw9cS46GBXyaY8LRRHmY2o2sQO3dfIW4kLRcwpwq8yFqFt7YPpzZlyt3au79t59tyhcslmsm3zPeuVpbAS1+HHBZZFu2BJ5XaTpYUcAdpUleMvak3rvemuA7N/t/j7WdyWx/UzGxtuSUTz0Mda21G1iScWQmQcwVeZVkjQ3FeSSRmSN/xno8T71f0u9Lmyd3KZ+PZd7JYnJnOWknrYaRo7qPJKXmcUEeNlNrv5eRIGk5LDsxe4ruGmvn/AAuO/pys4SPE7sX0muJdaTM3LF2fCzWJ7duBbjzyxx1kgd3hgEplUcvHVjkJcAsI6DY3ohj8Rc3Rl6vp5uPa12SnYs3r426uIqxVKfta1fzR0oizCGy6wh+wWONl8sSP4pGD6R01QdjemeyttvHkcN6X7LxFg2I7fv8AF1K6yTyLXkiS12irx8yeOeaPngcJLKAerEG/agaaaaCD3Js/E7qerJk7eZhNNi0f0/M26AYkg/MV5E8g+I+z8gff+Tzlj9p4vG5KfK17OVeaxBDXdLGVszwhIwQpWGR2jRzz8nVQz/qxYjU1pq7TDTTTUFR3ht9dyXo8dYawie3Lx+z3XfxU7gdhIelUryqkwDv2J/I37eqh4vFUd44+1Try4LFJZu11e5FJv3JWWiVU8UpgWWt+QKqUyH4jJeeZj1b5TdB4B/UD+NOAPuAP41RS6mP3vSjuZSHbuNbKX4EjlrS7xyEtVSteaQCMvWIi/wBVL4i6RKzQ9ZD94o6+tIemZzk1G1uWfP0pUriWyMZv3LqhtPK00kQVGhV4hJI/V+EPRI4wiRgRr0LTU0VjOeneB3BRmx97I7nhjnuR3majufJU5VlSBYQFkgnR1j6qCYgRGz8yMpdix2G2Thno4vHG7n/FiOnt2G4L4lfo6OPPIJu9j7xqD5i/ZS6nlXcNP6aCpJ6X7ajydbLrk92mepZe1GjbvyzQs7SiUh4TZ8cidhwI3UoF5QAISurbppoGmmsS6r9mYD/06DLTWPkj/wCxf868NKoHKDyH/ipHJ/ydBnpppoNb3nOQ+nrUsHiHzNP04iXluFTsT8mPDHhQeAvy69k7bOq1l8fvEbi+r4XKNLR9klf6XPbhgrGbu5afkU5JvIFKAfl8Z4/YCCzeoYfenthRO4bI8NJKS3RZreexI4RZLkiey8azx9GeNVHidpGDoqhAskvak+rVpqqS7d3Pk4pkv7jt0xNAekdZ6c61Z+lYo8flp/cxyQzMrOCOZWJQ8xpBlHs/PJmzlm9UNzPVJdvpjVsZ7b7xdB8hTE3xb8g/J+8cHlPhqqtOmq2+08w+KrY8eom4Es11lD5AV8d7idnUqruDVMQZOeV6ooPHyDDkaxubOuyz5Kxjd7ZnEvkpZZmNOrj+YmaoldCpkquWMZjSVTIXPcBWLxARALNpqPwOHh2/iauGqzNJBTiSCEGGGIJGihVVUhREVQB9gqgD9BwOBqQ0DURndr43cVeetfs5SJLAiDmllLNNx43LL1eB0ZOSfl1I7DhW5UAal9NNwVO76Z7dv/UPPkd0L9TCLN4N0ZKHoFk7jxdJx4Dz9iY+vK/E8r9tSOM2ji8TYjtVbWWd4qUVBRZy1qyhij/axWWRlaU8/KUgyP8A7mbU3pq+1TDTTTUV/9k=&quot;);"></div></div><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><div class="lhggkp7q ln8gz9je ppled2lx t3g6t33p qnwaluaf" style="background-image: url(&quot;blob:https://web.whatsapp.com/e7e2cf80-e070-463c-bc76-8f2da3f8bb74&quot;);"></div></div></div></div></div></div></div></div><div role="button" tabindex="0" aria-label="Open Picture" class="gndfcl4n l8fojup5 paxyh2gw sfeitywo cqsf3vkf p357zi0d ac2vgrno laorhtua gfz4du6o r7fjleex g0rxnol2" style="width: 330px; height: 223.575px;"><span class=""></span><div class="g0rxnol2 ln8gz9je ppled2lx"><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><img alt="Prompt hacks are different when you have Indian values" src="data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDABsSFBcUERsXFhceHBsgKEIrKCUlKFE6PTBCYFVlZF9VXVtqeJmBanGQc1tdhbWGkJ6jq62rZ4C8ybqmx5moq6T/2wBDARweHigjKE4rK06kbl1upKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKT/wgARCAAxAEgDASIAAhEBAxEB/8QAGAABAQEBAQAAAAAAAAAAAAAAAAIDAQb/xAAUAQEAAAAAAAAAAAAAAAAAAAAA/9oADAMBAAIQAxAAAAD0yJNWQ1ZDWKB0RnuMWwxbScuRTgnlSDpzlBTpx0AAAAAf/8QAFBEBAAAAAAAAAAAAAAAAAAAAQP/aAAgBAgEBPwAn/8QAFBEBAAAAAAAAAAAAAAAAAAAAQP/aAAgBAwEBPwAn/8QAIRAAAQUAAwACAwAAAAAAAAAAAQACERNRAxIhEDAgMWH/2gAIAQEAAT8A/MvAMeppn1chhsyrhiuGK4YrhiuGJruwn55B2bCr/qpOqk6qTqpOpjC39mfo7t1d26u7dQM+rkMNkGFa1WtVrVa1WMxNLXCQFAwfDm9hCqGIcbY9CrbiqbiqbiAAED7P//4AAwD/2Q==" class="jciay5ix tvf2evcx oq44ahr5 lb5m6g5c fsmudgz7" style="width: 100%;"></div><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><img alt="Prompt hacks are different when you have Indian values" src="blob:https://web.whatsapp.com/3f515986-03c3-4c7b-99c7-8f6e9db57ce2" class="jciay5ix tvf2evcx oq44ahr5 lb5m6g5c osz0hll6 nq7eualt em5jvqoa a21kwdn3" style="width: 100%;"></div></div></div><div class="cm280p3y bkoknyjm esbfogcw jfqm35v0 bxcbqipq mhcwslh8"><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Prompt hacks are different when you have Indian values</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:41 pm</span></span></span></div></div><div class="lrw9n60e lhggkp7q fz4q5utg b9fczbqn"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:41 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 cvsrm5e1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="Reactions 😂, 😅, 👍 6 in total"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="😂" draggable="false" class="bd96tnyg p83aruvr b85 emoji apple _11JPr" tabindex="-1" style="background-position: -60px -60px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="😅" draggable="false" class="bd96tnyg p83aruvr b85 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -80px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk o2es7gts i2cterl7" style="flex-shrink: 0;"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 f8m0rgwh r15c9g6i ilf8vifs gfz4du6o r7fjleex qnz2jpws kjemk6od"><div class="dn5wc90o ilf8vifs gndfcl4n p357zi0d"><span class="f8jlpxt4 qfejxiq4 ce5kru2g">6</span></div></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3AE4D69B1C792BFFE0D1_917892563038@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe ashish +91 78925 63038" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno po56q9xr"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">A</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe ashish" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">ashish</span><span class="WJuYU" testid="author" role="button">+91 78925 63038</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[1:41 pm, 26/02/2024] +91 78925 63038: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="akr8p6hy _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d jake6gi8" role=""><span dir="auto" aria-label="Maybe Paras Chopra" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Paras Chopra</span><span class="WJuYU" testid="author">+91 98682 21372</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">What are Indian values?</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>snarky answer :)
whatever pleases the current dispensation</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:41 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:41 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB067C19F15616D56C8DF_919550164716@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Ravi Theja +91 95501 64716" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno j3zl77sr"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">R</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d gvktkc8w" role=""><span dir="auto" aria-label="Maybe Ravi Theja" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Ravi Theja</span><span class="WJuYU" testid="author" role="button">+91 95501 64716</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[1:42 pm, 26/02/2024] +91 95501 64716: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="ckklvomo _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d fyja2idx" role=""><span dir="auto" aria-label="Maybe Ritik Madan" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Ritik Madan</span><span class="WJuYU" testid="author">+91 87655 34151</span></div><div class="_37DQv" dir="ltr" role="button"><div class="l7jjieqr tvf2evcx oq44ahr5 lb5m6g5c g97rt3p3 fewfhwl7 jdwybkuq kjemk6od"><span data-icon="status-image" class=""><svg viewBox="0 0 16 20" height="20" width="16" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 16 20"><title>status-image</title><path fill="currentColor" d="M13.822,4.668H7.14l-1.068-1.09C5.922,3.425,5.624,3.3,5.409,3.3H3.531 c-0.214,0-0.51,0.128-0.656,0.285L1.276,5.296C1.13,5.453,1.01,5.756,1.01,5.971v1.06c0,0.001-0.001,0.002-0.001,0.003v6.983 c0,0.646,0.524,1.17,1.17,1.17h11.643c0.646,0,1.17-0.524,1.17-1.17v-8.18C14.992,5.191,14.468,4.668,13.822,4.668z M7.84,13.298 c-1.875,0-3.395-1.52-3.395-3.396c0-1.875,1.52-3.395,3.395-3.395s3.396,1.52,3.396,3.395C11.236,11.778,9.716,13.298,7.84,13.298z  M7.84,7.511c-1.321,0-2.392,1.071-2.392,2.392s1.071,2.392,2.392,2.392s2.392-1.071,2.392-2.392S9.161,7.511,7.84,7.511z"></path></svg></span></div><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Prompt hacks are different when you have Indian values</span></div></div></div><div class="jrEtJ"><div class="kknmh"><div class="g0rxnol2 ln8gz9je ppled2lx"><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><div class="lhggkp7q ln8gz9je ppled2lx t3g6t33p qnwaluaf" style="background-image: url(&quot;data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDABsSFBcUERsXFhceHBsgKEIrKCUlKFE6PTBCYFVlZF9VXVtqeJmBanGQc1tdhbWGkJ6jq62rZ4C8ybqmx5moq6T/2wBDARweHigjKE4rK06kbl1upKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKT/wgARCAAxAEgDASIAAhEBAxEB/8QAGAABAQEBAQAAAAAAAAAAAAAAAAIDAQb/xAAUAQEAAAAAAAAAAAAAAAAAAAAA/9oADAMBAAIQAxAAAAD0yJNWQ1ZDWKB0RnuMWwxbScuRTgnlSDpzlBTpx0AAAAAf/8QAFBEBAAAAAAAAAAAAAAAAAAAAQP/aAAgBAgEBPwAn/8QAFBEBAAAAAAAAAAAAAAAAAAAAQP/aAAgBAwEBPwAn/8QAIRAAAQUAAwACAwAAAAAAAAAAAQACERNRAxIhEDAgMWH/2gAIAQEAAT8A/MvAMeppn1chhsyrhiuGK4YrhiuGJruwn55B2bCr/qpOqk6qTqpOpjC39mfo7t1d26u7dQM+rkMNkGFa1WtVrVa1WMxNLXCQFAwfDm9hCqGIcbY9CrbiqbiqbiAAED7P//4AAwD/2Q==&quot;);"></div></div><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><div class="lhggkp7q ln8gz9je ppled2lx t3g6t33p qnwaluaf" style="background-image: url(&quot;blob:https://web.whatsapp.com/3f515986-03c3-4c7b-99c7-8f6e9db57ce2&quot;);"></div></div></div></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>still did not respond in hindi</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:42 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:42 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="reaction 😂"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="😂" draggable="false" class="bd96tnyg p83aruvr b85 emoji apple _11JPr" tabindex="-1" style="background-position: -60px -60px;"></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3AA41F8693B28E1A6FD5_918765534151@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Ritik Madan +91 87655 34151" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno ckklvomo"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">R</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d fyja2idx" role=""><span dir="auto" aria-label="Maybe Ritik Madan" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Ritik Madan</span><span class="WJuYU" testid="author" role="button">+91 87655 34151</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[1:42 pm, 26/02/2024] +91 87655 34151: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>But couldn’t say no XD</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:42 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:42 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="reaction 👍"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj _3sxvM" data-id="false_120363049558306142@g.us_3EB01E6DE65E70C9A3A966_919717617888@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _27hEJ _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Mayank Gupta +91 97176 17888" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://media-tir3-1.cdn.whatsapp.net/v/t61.24694-24/343036186_1200021077373038_2599390388356811675_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdQH5f-eBJEjhGv4eMR98rDpMMlzf6QBeyaAdo_msZlH7A&amp;oe=65EAF23E&amp;_nc_sid=e6ed6c&amp;_nc_cat=100" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div class="cm280p3y ktbp76dp ocd2b0bc folpon7g aa0kojfi snweb893 g0rxnol2 jnl3jror copyable-text" data-pre-plain-text="[1:45 pm, 26/02/2024] +91 97176 17888: "><div><div class="bxcbqipq mhcwslh8 ocd2b0bc jfqm35v0"><div class="_3IzYj _6rIWC p357zi0d a7s2a5ab" role=""><span dir="auto" aria-label="Maybe Mayank Gupta" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Mayank Gupta</span><span class="WJuYU" testid="author" role="button">+91 97176 17888</span></div></div><div role="button" tabindex="0" aria-label="Open Picture" class="gndfcl4n l8fojup5 paxyh2gw sfeitywo cqsf3vkf p357zi0d ac2vgrno laorhtua gfz4du6o r7fjleex g0rxnol2" style="width: 330px; height: 126.225px;"><span class=""></span><div class="g0rxnol2 ln8gz9je ppled2lx"><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><img alt="Ummm...I am worried" src="data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/4gHYSUNDX1BST0ZJTEUAAQEAAAHIAAAAAAQwAABtbnRyUkdCIFhZWiAH4AABAAEAAAAAAABhY3NwAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAQAA9tYAAQAAAADTLQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAlkZXNjAAAA8AAAACRyWFlaAAABFAAAABRnWFlaAAABKAAAABRiWFlaAAABPAAAABR3dHB0AAABUAAAABRyVFJDAAABZAAAAChnVFJDAAABZAAAAChiVFJDAAABZAAAAChjcHJ0AAABjAAAADxtbHVjAAAAAAAAAAEAAAAMZW5VUwAAAAgAAAAcAHMAUgBHAEJYWVogAAAAAAAAb6IAADj1AAADkFhZWiAAAAAAAABimQAAt4UAABjaWFlaIAAAAAAAACSgAAAPhAAAts9YWVogAAAAAAAA9tYAAQAAAADTLXBhcmEAAAAAAAQAAAACZmYAAPKnAAANWQAAE9AAAApbAAAAAAAAAABtbHVjAAAAAAAAAAEAAAAMZW5VUwAAACAAAAAcAEcAbwBvAGcAbABlACAASQBuAGMALgAgADIAMAAxADb/2wBDAAMCAgICAgMCAgIDAwMDBAYEBAQEBAgGBgUGCQgKCgkICQkKDA8MCgsOCwkJDRENDg8QEBEQCgwSExIQEw8QEBD/2wBDAQMDAwQDBAgEBAgQCwkLEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBD/wAARCAAmAGQDASIAAhEBAxEB/8QAHAABAAEFAQEAAAAAAAAAAAAAAAUBAwQGBwIJ/8QAKRAAAgICAgEDBAEFAAAAAAAAAQIDBAAFBhESBxMhFCIxQRYII1Fhwf/EABcBAQEBAQAAAAAAAAAAAAAAAAABAgP/xAAeEQEAAgEEAwAAAAAAAAAAAAAAARExEhNBcQIiUf/aAAwDAQACEQMRAD8A+qeMi+TV99a000HGbkdXYM8RjlkYKAgkUyDto5ACUDgEo3RIPWXNBDuq+lpwciuwW9mkKi1NBH4I7/sgf96AJ7IVQfEct2d3a0zi74zj7fOMMa/fRU4u+Ou0hjGM6tmMYwGMYwGMYwILkX8uE1M8ZGtZDZiW4twyL41/cQyPGU77cRiQKrAAs6sWAQq+t3Nl63rsKJ1/DeHSUW+28s3JriSp3M484iKBDeMKxt4MF83mZfJBCHm6DjNR5JTxF5e2vn33/vGe8ZlWNsr8Gr11rZ2ksvDTheeRa1aSxMyopYhIolaSRuh8IilmPQAJIGct2fCv6do+Nw8z2HofrLFSpT+khRfTqae9FWudyvClNahshHNyUyoI+laWcSAN7gHUdrJtIdZbl0lOrb2KQO1SC1ZavDLMFPgkkqxyNGpboFhG5A7IVvwdY1O09YJp0Xe8F4dThMMLO9TldqyyymdRKoVtdGCqwmR1bsFnVYyqKxlWwLm25Pw69FoK+449uLsXIXq3aSScVvzrDJHLDJA9oewRSkjleJx9R7bIyM32+05WEv8Aq36d8T296v8AxXm0dy+0N67NrfTne2UsyNWiCPJNXpMkkghWGM9sWX2xG3RTxE/sNj6oR0opNVw7i9i2xT3YrPJLEEaArGW6daLliGMwA8R2I4z2PMrHl8duc6tVbD8t45otbZW14V49dupr0clbsf3HeSrCUk68j7YVh8Aefz2AktXtK24qLdqRW443AIW1TlrSfIB+UlVWHwR+R8HsfkEZmZpib71VPqOuhl9OdKOGNFNIORLyRmtKyRxFEaiao+55JHUdTFQsEjMVJjR9zyBjGMBjGMBjGMBjGMChHYIB67/eY1CnYp66ClY2di9NDGqPbsLGJZiPy7CNVQE/vxVR/gDGMtil6nYtxQx19pZpNHNHKzwLGTIqsC0beasPFwCpIAbonxKnoizc1uws7nX7GDf2qtOmkwsUI4ojHdZwAhkZkLqE6YgIy9kjyJA6LGLlKSOMYyKYxjAYxjAYxjAYxjA//9k=" class="jciay5ix tvf2evcx oq44ahr5 lb5m6g5c fsmudgz7" style="width: 100%;"></div><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><img alt="Ummm...I am worried" src="blob:https://web.whatsapp.com/3ec5182d-f2c9-41e5-9473-280ed5d938af" class="jciay5ix tvf2evcx oq44ahr5 lb5m6g5c osz0hll6 nq7eualt em5jvqoa a21kwdn3" style="width: 100%;"></div></div></div><div class="cm280p3y bkoknyjm esbfogcw jfqm35v0 bxcbqipq mhcwslh8"><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Ummm...I am worried</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:45 pm</span></span></span></div></div><div class="lrw9n60e lhggkp7q fz4q5utg b9fczbqn"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:45 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 cvsrm5e1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB00E1A264ED2BA5A90ED_919717617888@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a"><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d a7s2a5ab" role=""><span dir="auto" aria-label="Maybe Mayank Gupta" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Mayank Gupta</span><span class="WJuYU" testid="author" role="button">+91 97176 17888</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[1:46 pm, 26/02/2024] +91 97176 17888: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g755h7wn _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d a7s2a5ab" role=""><span dir="auto" aria-label="Maybe Mayank Gupta" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Mayank Gupta</span><span class="WJuYU" testid="author">+91 97176 17888</span></div><div class="_37DQv" dir="ltr" role="button"><div class="l7jjieqr tvf2evcx oq44ahr5 lb5m6g5c g97rt3p3 fewfhwl7 jdwybkuq kjemk6od"><span data-icon="status-image" class=""><svg viewBox="0 0 16 20" height="20" width="16" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 16 20"><title>status-image</title><path fill="currentColor" d="M13.822,4.668H7.14l-1.068-1.09C5.922,3.425,5.624,3.3,5.409,3.3H3.531 c-0.214,0-0.51,0.128-0.656,0.285L1.276,5.296C1.13,5.453,1.01,5.756,1.01,5.971v1.06c0,0.001-0.001,0.002-0.001,0.003v6.983 c0,0.646,0.524,1.17,1.17,1.17h11.643c0.646,0,1.17-0.524,1.17-1.17v-8.18C14.992,5.191,14.468,4.668,13.822,4.668z M7.84,13.298 c-1.875,0-3.395-1.52-3.395-3.396c0-1.875,1.52-3.395,3.395-3.395s3.396,1.52,3.396,3.395C11.236,11.778,9.716,13.298,7.84,13.298z  M7.84,7.511c-1.321,0-2.392,1.071-2.392,2.392s1.071,2.392,2.392,2.392s2.392-1.071,2.392-2.392S9.161,7.511,7.84,7.511z"></path></svg></span></div><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Ummm...I am worried</span></div></div></div><div class="jrEtJ"><div class="kknmh"><div class="g0rxnol2 ln8gz9je ppled2lx"><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><div class="lhggkp7q ln8gz9je ppled2lx t3g6t33p qnwaluaf" style="background-image: url(&quot;data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/4gHYSUNDX1BST0ZJTEUAAQEAAAHIAAAAAAQwAABtbnRyUkdCIFhZWiAH4AABAAEAAAAAAABhY3NwAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAQAA9tYAAQAAAADTLQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAlkZXNjAAAA8AAAACRyWFlaAAABFAAAABRnWFlaAAABKAAAABRiWFlaAAABPAAAABR3dHB0AAABUAAAABRyVFJDAAABZAAAAChnVFJDAAABZAAAAChiVFJDAAABZAAAAChjcHJ0AAABjAAAADxtbHVjAAAAAAAAAAEAAAAMZW5VUwAAAAgAAAAcAHMAUgBHAEJYWVogAAAAAAAAb6IAADj1AAADkFhZWiAAAAAAAABimQAAt4UAABjaWFlaIAAAAAAAACSgAAAPhAAAts9YWVogAAAAAAAA9tYAAQAAAADTLXBhcmEAAAAAAAQAAAACZmYAAPKnAAANWQAAE9AAAApbAAAAAAAAAABtbHVjAAAAAAAAAAEAAAAMZW5VUwAAACAAAAAcAEcAbwBvAGcAbABlACAASQBuAGMALgAgADIAMAAxADb/2wBDAAMCAgICAgMCAgIDAwMDBAYEBAQEBAgGBgUGCQgKCgkICQkKDA8MCgsOCwkJDRENDg8QEBEQCgwSExIQEw8QEBD/2wBDAQMDAwQDBAgEBAgQCwkLEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBD/wAARCAAmAGQDASIAAhEBAxEB/8QAHAABAAEFAQEAAAAAAAAAAAAAAAUBAwQGBwIJ/8QAKRAAAgICAgEDBAEFAAAAAAAAAQIDBAAFBhESBxMhFCIxQRYII1Fhwf/EABcBAQEBAQAAAAAAAAAAAAAAAAABAgP/xAAeEQEAAgEEAwAAAAAAAAAAAAAAARExEhNBcQIiUf/aAAwDAQACEQMRAD8A+qeMi+TV99a000HGbkdXYM8RjlkYKAgkUyDto5ACUDgEo3RIPWXNBDuq+lpwciuwW9mkKi1NBH4I7/sgf96AJ7IVQfEct2d3a0zi74zj7fOMMa/fRU4u+Ou0hjGM6tmMYwGMYwGMYwILkX8uE1M8ZGtZDZiW4twyL41/cQyPGU77cRiQKrAAs6sWAQq+t3Nl63rsKJ1/DeHSUW+28s3JriSp3M484iKBDeMKxt4MF83mZfJBCHm6DjNR5JTxF5e2vn33/vGe8ZlWNsr8Gr11rZ2ksvDTheeRa1aSxMyopYhIolaSRuh8IilmPQAJIGct2fCv6do+Nw8z2HofrLFSpT+khRfTqae9FWudyvClNahshHNyUyoI+laWcSAN7gHUdrJtIdZbl0lOrb2KQO1SC1ZavDLMFPgkkqxyNGpboFhG5A7IVvwdY1O09YJp0Xe8F4dThMMLO9TldqyyymdRKoVtdGCqwmR1bsFnVYyqKxlWwLm25Pw69FoK+449uLsXIXq3aSScVvzrDJHLDJA9oewRSkjleJx9R7bIyM32+05WEv8Aq36d8T296v8AxXm0dy+0N67NrfTne2UsyNWiCPJNXpMkkghWGM9sWX2xG3RTxE/sNj6oR0opNVw7i9i2xT3YrPJLEEaArGW6daLliGMwA8R2I4z2PMrHl8duc6tVbD8t45otbZW14V49dupr0clbsf3HeSrCUk68j7YVh8Aefz2AktXtK24qLdqRW443AIW1TlrSfIB+UlVWHwR+R8HsfkEZmZpib71VPqOuhl9OdKOGNFNIORLyRmtKyRxFEaiao+55JHUdTFQsEjMVJjR9zyBjGMBjGMBjGMBjGMChHYIB67/eY1CnYp66ClY2di9NDGqPbsLGJZiPy7CNVQE/vxVR/gDGMtil6nYtxQx19pZpNHNHKzwLGTIqsC0beasPFwCpIAbonxKnoizc1uws7nX7GDf2qtOmkwsUI4ojHdZwAhkZkLqE6YgIy9kjyJA6LGLlKSOMYyKYxjAYxjAYxjAYxjA//9k=&quot;);"></div></div><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><div class="lhggkp7q ln8gz9je ppled2lx t3g6t33p qnwaluaf" style="background-image: url(&quot;blob:https://web.whatsapp.com/3ec5182d-f2c9-41e5-9473-280ed5d938af&quot;);"></div></div></div></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Though this might just be only joke that will survive all of humanity. Years later all jokes will start with "Why don't scientists trust atoms"</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5 hnx8ox4h mw4yctpw gj5xqxfh">Edited</span><span class="tvf2evcx oq44ahr5">1:46 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto"><span class="mw4yctpw gj5xqxfh">Edited</span>1:46 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="reaction 😂"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="😂" draggable="false" class="bd96tnyg p83aruvr b85 emoji apple _11JPr" tabindex="-1" style="background-position: -60px -60px;"></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3ABC471FDF334972CE76_917987544173@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Abhinash Khare +91 79875 44173" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno akr8p6hy"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">A</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d jake6gi8" role=""><span dir="auto" aria-label="Maybe Abhinash Khare" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Abhinash Khare</span><span class="WJuYU" testid="author" role="button">+91 79875 44173</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[1:47 pm, 26/02/2024] +91 79875 44173: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="akr8p6hy _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d jake6gi8" role=""><span dir="auto" aria-label="Maybe Sid" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Sid</span><span class="WJuYU" testid="author">+91 93343 72044</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">I will just say that, whatever the output quality, let's all remember  that at-least it's happening as somewhat of a grassroots level from India. Things will improve</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>My personal issue is with the hype they (&amp; media — probably paid news) generated. India’s answer to OpenAI , first billion dollar AI startup and stuff. They should have remained humble like Sarvam. Any criticism will help them improve only.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:47 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:47 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="Reactions 👍 3 in total"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk o2es7gts i2cterl7" style="flex-shrink: 0;"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 f8m0rgwh r15c9g6i ilf8vifs gfz4du6o r7fjleex qnz2jpws kjemk6od"><div class="dn5wc90o ilf8vifs gndfcl4n p357zi0d"><span class="f8jlpxt4 qfejxiq4 ce5kru2g">3</span></div></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3AFB5E10E680A61ED1B1_919868221372@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Paras Chopra +91 98682 21372" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/162690799_131241352786128_2676048871288803711_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRLabAEX9bmDVVjQ8t15GgUazuMiCZOrKVtAXDMdkl8Pw&amp;oe=65EAF085&amp;_nc_sid=e6ed6c&amp;_nc_cat=103" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d jake6gi8" role=""><span dir="auto" aria-label="Maybe Paras Chopra" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Paras Chopra</span><span class="WJuYU" testid="author" role="button">+91 98682 21372</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[1:48 pm, 26/02/2024] +91 98682 21372: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g755h7wn _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d a7s2a5ab" role=""><span dir="auto" aria-label="Maybe Mayank Gupta" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Mayank Gupta</span><span class="WJuYU" testid="author">+91 97176 17888</span></div><div class="_37DQv" dir="ltr" role="button"><div class="l7jjieqr tvf2evcx oq44ahr5 lb5m6g5c g97rt3p3 fewfhwl7 jdwybkuq kjemk6od"><span data-icon="status-image" class=""><svg viewBox="0 0 16 20" height="20" width="16" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 16 20"><title>status-image</title><path fill="currentColor" d="M13.822,4.668H7.14l-1.068-1.09C5.922,3.425,5.624,3.3,5.409,3.3H3.531 c-0.214,0-0.51,0.128-0.656,0.285L1.276,5.296C1.13,5.453,1.01,5.756,1.01,5.971v1.06c0,0.001-0.001,0.002-0.001,0.003v6.983 c0,0.646,0.524,1.17,1.17,1.17h11.643c0.646,0,1.17-0.524,1.17-1.17v-8.18C14.992,5.191,14.468,4.668,13.822,4.668z M7.84,13.298 c-1.875,0-3.395-1.52-3.395-3.396c0-1.875,1.52-3.395,3.395-3.395s3.396,1.52,3.396,3.395C11.236,11.778,9.716,13.298,7.84,13.298z  M7.84,7.511c-1.321,0-2.392,1.071-2.392,2.392s1.071,2.392,2.392,2.392s2.392-1.071,2.392-2.392S9.161,7.511,7.84,7.511z"></path></svg></span></div><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Ummm...I am worried</span></div></div></div><div class="jrEtJ"><div class="kknmh"><div class="g0rxnol2 ln8gz9je ppled2lx"><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><div class="lhggkp7q ln8gz9je ppled2lx t3g6t33p qnwaluaf" style="background-image: url(&quot;data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/4gHYSUNDX1BST0ZJTEUAAQEAAAHIAAAAAAQwAABtbnRyUkdCIFhZWiAH4AABAAEAAAAAAABhY3NwAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAQAA9tYAAQAAAADTLQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAlkZXNjAAAA8AAAACRyWFlaAAABFAAAABRnWFlaAAABKAAAABRiWFlaAAABPAAAABR3dHB0AAABUAAAABRyVFJDAAABZAAAAChnVFJDAAABZAAAAChiVFJDAAABZAAAAChjcHJ0AAABjAAAADxtbHVjAAAAAAAAAAEAAAAMZW5VUwAAAAgAAAAcAHMAUgBHAEJYWVogAAAAAAAAb6IAADj1AAADkFhZWiAAAAAAAABimQAAt4UAABjaWFlaIAAAAAAAACSgAAAPhAAAts9YWVogAAAAAAAA9tYAAQAAAADTLXBhcmEAAAAAAAQAAAACZmYAAPKnAAANWQAAE9AAAApbAAAAAAAAAABtbHVjAAAAAAAAAAEAAAAMZW5VUwAAACAAAAAcAEcAbwBvAGcAbABlACAASQBuAGMALgAgADIAMAAxADb/2wBDAAMCAgICAgMCAgIDAwMDBAYEBAQEBAgGBgUGCQgKCgkICQkKDA8MCgsOCwkJDRENDg8QEBEQCgwSExIQEw8QEBD/2wBDAQMDAwQDBAgEBAgQCwkLEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBD/wAARCAAmAGQDASIAAhEBAxEB/8QAHAABAAEFAQEAAAAAAAAAAAAAAAUBAwQGBwIJ/8QAKRAAAgICAgEDBAEFAAAAAAAAAQIDBAAFBhESBxMhFCIxQRYII1Fhwf/EABcBAQEBAQAAAAAAAAAAAAAAAAABAgP/xAAeEQEAAgEEAwAAAAAAAAAAAAAAARExEhNBcQIiUf/aAAwDAQACEQMRAD8A+qeMi+TV99a000HGbkdXYM8RjlkYKAgkUyDto5ACUDgEo3RIPWXNBDuq+lpwciuwW9mkKi1NBH4I7/sgf96AJ7IVQfEct2d3a0zi74zj7fOMMa/fRU4u+Ou0hjGM6tmMYwGMYwGMYwILkX8uE1M8ZGtZDZiW4twyL41/cQyPGU77cRiQKrAAs6sWAQq+t3Nl63rsKJ1/DeHSUW+28s3JriSp3M484iKBDeMKxt4MF83mZfJBCHm6DjNR5JTxF5e2vn33/vGe8ZlWNsr8Gr11rZ2ksvDTheeRa1aSxMyopYhIolaSRuh8IilmPQAJIGct2fCv6do+Nw8z2HofrLFSpT+khRfTqae9FWudyvClNahshHNyUyoI+laWcSAN7gHUdrJtIdZbl0lOrb2KQO1SC1ZavDLMFPgkkqxyNGpboFhG5A7IVvwdY1O09YJp0Xe8F4dThMMLO9TldqyyymdRKoVtdGCqwmR1bsFnVYyqKxlWwLm25Pw69FoK+449uLsXIXq3aSScVvzrDJHLDJA9oewRSkjleJx9R7bIyM32+05WEv8Aq36d8T296v8AxXm0dy+0N67NrfTne2UsyNWiCPJNXpMkkghWGM9sWX2xG3RTxE/sNj6oR0opNVw7i9i2xT3YrPJLEEaArGW6daLliGMwA8R2I4z2PMrHl8duc6tVbD8t45otbZW14V49dupr0clbsf3HeSrCUk68j7YVh8Aefz2AktXtK24qLdqRW443AIW1TlrSfIB+UlVWHwR+R8HsfkEZmZpib71VPqOuhl9OdKOGNFNIORLyRmtKyRxFEaiao+55JHUdTFQsEjMVJjR9zyBjGMBjGMBjGMBjGMChHYIB67/eY1CnYp66ClY2di9NDGqPbsLGJZiPy7CNVQE/vxVR/gDGMtil6nYtxQx19pZpNHNHKzwLGTIqsC0beasPFwCpIAbonxKnoizc1uws7nX7GDf2qtOmkwsUI4ojHdZwAhkZkLqE6YgIy9kjyJA6LGLlKSOMYyKYxjAYxjAYxjAYxjA//9k=&quot;);"></div></div><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><div class="lhggkp7q ln8gz9je ppled2lx t3g6t33p qnwaluaf" style="background-image: url(&quot;blob:https://web.whatsapp.com/3ec5182d-f2c9-41e5-9473-280ed5d938af&quot;);"></div></div></div></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Wow, this is signature openai</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:48 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:48 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB08FF549CD17E7F06D78_919717617888@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Mayank Gupta +91 97176 17888" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://media-tir3-1.cdn.whatsapp.net/v/t61.24694-24/343036186_1200021077373038_2599390388356811675_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdQH5f-eBJEjhGv4eMR98rDpMMlzf6QBeyaAdo_msZlH7A&amp;oe=65EAF23E&amp;_nc_sid=e6ed6c&amp;_nc_cat=100" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d a7s2a5ab" role=""><span dir="auto" aria-label="Maybe Mayank Gupta" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Mayank Gupta</span><span class="WJuYU" testid="author" role="button">+91 97176 17888</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[1:48 pm, 26/02/2024] +91 97176 17888: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="akr8p6hy _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d jake6gi8" role=""><span dir="auto" aria-label="Maybe Paras Chopra" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Paras Chopra</span><span class="WJuYU" testid="author">+91 98682 21372</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Wow, this is signature openai</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>The first line. It might be a fun nod after all</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:48 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:48 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_166387A0DFF3C43589E5D829FF34108E_919685138692@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Manoj +91 96851 38692" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno m8ez62k1"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">M</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d lr20dbn2" role=""><span dir="auto" aria-label="Maybe Manoj" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Manoj</span><span class="WJuYU" testid="author" role="button">+91 96851 38692</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[1:48 pm, 26/02/2024] +91 96851 38692: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="j3zl77sr _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d gvktkc8w" role=""><span dir="auto" aria-label="Maybe Ravi Theja" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Ravi Theja</span><span class="WJuYU" testid="author">+91 95501 64716</span></div><div class="_37DQv" role="button"><div class="l7jjieqr tvf2evcx oq44ahr5 lb5m6g5c g97rt3p3 fewfhwl7 jdwybkuq kjemk6od"><span data-icon="status-image" class=""><svg viewBox="0 0 16 20" height="20" width="16" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 16 20"><title>status-image</title><path fill="currentColor" d="M13.822,4.668H7.14l-1.068-1.09C5.922,3.425,5.624,3.3,5.409,3.3H3.531 c-0.214,0-0.51,0.128-0.656,0.285L1.276,5.296C1.13,5.453,1.01,5.756,1.01,5.971v1.06c0,0.001-0.001,0.002-0.001,0.003v6.983 c0,0.646,0.524,1.17,1.17,1.17h11.643c0.646,0,1.17-0.524,1.17-1.17v-8.18C14.992,5.191,14.468,4.668,13.822,4.668z M7.84,13.298 c-1.875,0-3.395-1.52-3.395-3.396c0-1.875,1.52-3.395,3.395-3.395s3.396,1.52,3.396,3.395C11.236,11.778,9.716,13.298,7.84,13.298z  M7.84,7.511c-1.321,0-2.392,1.071-2.392,2.392s1.071,2.392,2.392,2.392s2.392-1.071,2.392-2.392S9.161,7.511,7.84,7.511z"></path></svg></span></div><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;"><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="😅" draggable="false" class="b85 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -80px;"></span></div></div></div><div class="jrEtJ"><div class="kknmh"><div class="g0rxnol2 ln8gz9je ppled2lx"><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><div class="lhggkp7q ln8gz9je ppled2lx t3g6t33p qnwaluaf" style="background-image: url(&quot;data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/4gHYSUNDX1BST0ZJTEUAAQEAAAHIAAAAAAQwAABtbnRyUkdCIFhZWiAH4AABAAEAAAAAAABhY3NwAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAQAA9tYAAQAAAADTLQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAlkZXNjAAAA8AAAACRyWFlaAAABFAAAABRnWFlaAAABKAAAABRiWFlaAAABPAAAABR3dHB0AAABUAAAABRyVFJDAAABZAAAAChnVFJDAAABZAAAAChiVFJDAAABZAAAAChjcHJ0AAABjAAAADxtbHVjAAAAAAAAAAEAAAAMZW5VUwAAAAgAAAAcAHMAUgBHAEJYWVogAAAAAAAAb6IAADj1AAADkFhZWiAAAAAAAABimQAAt4UAABjaWFlaIAAAAAAAACSgAAAPhAAAts9YWVogAAAAAAAA9tYAAQAAAADTLXBhcmEAAAAAAAQAAAACZmYAAPKnAAANWQAAE9AAAApbAAAAAAAAAABtbHVjAAAAAAAAAAEAAAAMZW5VUwAAACAAAAAcAEcAbwBvAGcAbABlACAASQBuAGMALgAgADIAMAAxADb/2wBDAAMCAgICAgMCAgIDAwMDBAYEBAQEBAgGBgUGCQgKCgkICQkKDA8MCgsOCwkJDRENDg8QEBEQCgwSExIQEw8QEBD/2wBDAQMDAwQDBAgEBAgQCwkLEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBD/wAARCAAuAGQDASIAAhEBAxEB/8QAHQABAAIDAAMBAAAAAAAAAAAAAAUGAgQHAQMICf/EAC4QAAICAQQBAwMDAwUAAAAAAAECAwQFAAYREhMHFCIVISMxMlEIF5EkQlJTcf/EABUBAQEAAAAAAAAAAAAAAAAAAAAB/8QAGhEBAQEBAAMAAAAAAAAAAAAAAAERQQISYf/aAAwDAQACEQMRAD8A/Ur3jHJHHipOVWATNP04iUluAnJ/VjwTwvPAHy69k7bOqD6gYLZNmbI5Pd2Vw9cS46GBXyaY8LRRHmY2o2sQO3dfIW4kLRcwpwq8yFqFt7YPpzZlyt3au79t59tyhcslmsm3zPeuVpbAS1+HHBZZFu2BJ5XaTpYUcAdpUleMvak3rvemuA7N/t/j7WdyWx/UzGxtuSUTz0Mda21G1iScWQmQcwVeZVkjQ3FeSSRmSN/xno8T71f0u9Lmyd3KZ+PZd7JYnJnOWknrYaRo7qPJKXmcUEeNlNrv5eRIGk5LDsxe4ruGmvn/AAuO/pys4SPE7sX0muJdaTM3LF2fCzWJ7duBbjzyxx1kgd3hgEplUcvHVjkJcAsI6DY3ohj8Rc3Rl6vp5uPa12SnYs3r426uIqxVKfta1fzR0oizCGy6wh+wWONl8sSP4pGD6R01QdjemeyttvHkcN6X7LxFg2I7fv8AF1K6yTyLXkiS12irx8yeOeaPngcJLKAerEG/agaaaaCD3Js/E7qerJk7eZhNNi0f0/M26AYkg/MV5E8g+I+z8gff+Tzlj9p4vG5KfK17OVeaxBDXdLGVszwhIwQpWGR2jRzz8nVQz/qxYjU1pq7TDTTTUFR3ht9dyXo8dYawie3Lx+z3XfxU7gdhIelUryqkwDv2J/I37eqh4vFUd44+1Try4LFJZu11e5FJv3JWWiVU8UpgWWt+QKqUyH4jJeeZj1b5TdB4B/UD+NOAPuAP41RS6mP3vSjuZSHbuNbKX4EjlrS7xyEtVSteaQCMvWIi/wBVL4i6RKzQ9ZD94o6+tIemZzk1G1uWfP0pUriWyMZv3LqhtPK00kQVGhV4hJI/V+EPRI4wiRgRr0LTU0VjOeneB3BRmx97I7nhjnuR3majufJU5VlSBYQFkgnR1j6qCYgRGz8yMpdix2G2Thno4vHG7n/FiOnt2G4L4lfo6OPPIJu9j7xqD5i/ZS6nlXcNP6aCpJ6X7ajydbLrk92mepZe1GjbvyzQs7SiUh4TZ8cidhwI3UoF5QAISurbppoGmmsS6r9mYD/06DLTWPkj/wCxf868NKoHKDyH/ipHJ/ydBnpppoNb3nOQ+nrUsHiHzNP04iXluFTsT8mPDHhQeAvy69k7bOq1l8fvEbi+r4XKNLR9klf6XPbhgrGbu5afkU5JvIFKAfl8Z4/YCCzeoYfenthRO4bI8NJKS3RZreexI4RZLkiey8azx9GeNVHidpGDoqhAskvak+rVpqqS7d3Pk4pkv7jt0xNAekdZ6c61Z+lYo8flp/cxyQzMrOCOZWJQ8xpBlHs/PJmzlm9UNzPVJdvpjVsZ7b7xdB8hTE3xb8g/J+8cHlPhqqtOmq2+08w+KrY8eom4Es11lD5AV8d7idnUqruDVMQZOeV6ooPHyDDkaxubOuyz5Kxjd7ZnEvkpZZmNOrj+YmaoldCpkquWMZjSVTIXPcBWLxARALNpqPwOHh2/iauGqzNJBTiSCEGGGIJGihVVUhREVQB9gqgD9BwOBqQ0DURndr43cVeetfs5SJLAiDmllLNNx43LL1eB0ZOSfl1I7DhW5UAal9NNwVO76Z7dv/UPPkd0L9TCLN4N0ZKHoFk7jxdJx4Dz9iY+vK/E8r9tSOM2ji8TYjtVbWWd4qUVBRZy1qyhij/axWWRlaU8/KUgyP8A7mbU3pq+1TDTTTUV/9k=&quot;);"></div></div><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><div class="lhggkp7q ln8gz9je ppled2lx t3g6t33p qnwaluaf" style="background-image: url(&quot;blob:https://web.whatsapp.com/e7e2cf80-e070-463c-bc76-8f2da3f8bb74&quot;);"></div></div></div></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Isn't that the whole value prop? Indian language support</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:48 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:48 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A2318D8A605688175C0_919354493228@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _27hEJ _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Azhan Mohammed +91 93544 93228" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://media-tir3-1.cdn.whatsapp.net/v/t61.24694-24/415823620_588200936799230_8959200103469063155_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdR_pTfPCMFAxFn-TVmoUaWYSLvgE1ORXxiXANZ9ST0xVg&amp;oe=65EAF835&amp;_nc_sid=e6ed6c&amp;_nc_cat=100" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div class="cm280p3y ktbp76dp ocd2b0bc folpon7g aa0kojfi snweb893 g0rxnol2 jnl3jror copyable-text" data-pre-plain-text="[1:50 pm, 26/02/2024] +91 93544 93228: "><div><div class="bxcbqipq mhcwslh8 ocd2b0bc jfqm35v0"><div class="_3IzYj _6rIWC p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe Azhan Mohammed" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Azhan Mohammed</span><span class="WJuYU" testid="author" role="button">+91 93544 93228</span></div></div><div role="button" tabindex="0" aria-label="Open Picture" class="gndfcl4n l8fojup5 paxyh2gw sfeitywo cqsf3vkf p357zi0d ac2vgrno laorhtua gfz4du6o r7fjleex g0rxnol2" style="width: 330px; height: 161.287px;"><span class=""></span><div class="g0rxnol2 ln8gz9je ppled2lx"><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><img alt="My experience has been, not okay?" src="data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDABsSFBcUERsXFhceHBsgKEIrKCUlKFE6PTBCYFVlZF9VXVtqeJmBanGQc1tdhbWGkJ6jq62rZ4C8ybqmx5moq6T/2wBDARweHigjKE4rK06kbl1upKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKT/wgARCAAjAEgDASIAAhEBAxEB/8QAGAABAQEBAQAAAAAAAAAAAAAAAAIBAwb/xAAUAQEAAAAAAAAAAAAAAAAAAAAA/9oADAMBAAIQAxAAAAD03O+B0yLAN2R0YJzZNKJUFzQAAAAAB//EABQRAQAAAAAAAAAAAAAAAAAAADD/2gAIAQIBAT8Af//EABQRAQAAAAAAAAAAAAAAAAAAADD/2gAIAQMBAT8Af//EACEQAAICAQQCAwAAAAAAAAAAAAABAhNRAxEhQRIxEDBx/9oACAEBAAE/AB60Uy6JdEuiXRLolkci5YxwTbfItLftlUcFUcFUcFUcFUcEVtwTl4zT5/C5YLlguWC5YLlghqKb2S+Zw8mvZUt+xaK29sWlFdblccFccCiovhfZ//4AAwD/2Q==" class="jciay5ix tvf2evcx oq44ahr5 lb5m6g5c fsmudgz7" style="width: 100%;"></div><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><img alt="My experience has been, not okay?" src="blob:https://web.whatsapp.com/c7c382b9-c70f-4a6e-acdf-7482234a9994" class="jciay5ix tvf2evcx oq44ahr5 lb5m6g5c osz0hll6 nq7eualt em5jvqoa a21kwdn3" style="width: 100%;"></div></div></div><div class="cm280p3y bkoknyjm esbfogcw jfqm35v0 bxcbqipq mhcwslh8"><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>My experience has been, not okay?</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:50 pm</span></span></span></div></div><div class="lrw9n60e lhggkp7q fz4q5utg b9fczbqn"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:50 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 cvsrm5e1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3FF0DA1424EE9307BE35B7EB52FFEBD3_919334372044@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Sid +91 93343 72044" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno akr8p6hy"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">S</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d jake6gi8" role=""><span dir="auto" aria-label="Maybe Sid" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Sid</span><span class="WJuYU" testid="author" role="button">+91 93343 72044</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[1:50 pm, 26/02/2024] +91 93343 72044: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="akr8p6hy _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d jake6gi8" role=""><span dir="auto" aria-label="Maybe Abhinash Khare" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Abhinash Khare</span><span class="WJuYU" testid="author">+91 79875 44173</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">My personal issue is with the hype they (&amp; media — probably paid news) generated. India’s answer to OpenAI , first billion dollar AI startup and stuff. They should have remained humble like Sarvam. Any criticism will help them improve only.</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Fair</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:50 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:50 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB001DB2BB1406366EBB9_919703944446@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Pratiksha +91 97039 44446" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno fz47docv"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">P</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d swew6qod" role=""><span dir="auto" aria-label="Maybe Pratiksha" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Pratiksha</span><span class="WJuYU" testid="author" role="button">+91 97039 44446</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[1:51 pm, 26/02/2024] +91 97039 44446: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g755h7wn _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d a7s2a5ab" role=""><span dir="auto" aria-label="Maybe Mayank Gupta" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Mayank Gupta</span><span class="WJuYU" testid="author">+91 97176 17888</span></div><div class="_37DQv" dir="ltr" role="button"><div class="l7jjieqr tvf2evcx oq44ahr5 lb5m6g5c g97rt3p3 fewfhwl7 jdwybkuq kjemk6od"><span data-icon="status-image" class=""><svg viewBox="0 0 16 20" height="20" width="16" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 16 20"><title>status-image</title><path fill="currentColor" d="M13.822,4.668H7.14l-1.068-1.09C5.922,3.425,5.624,3.3,5.409,3.3H3.531 c-0.214,0-0.51,0.128-0.656,0.285L1.276,5.296C1.13,5.453,1.01,5.756,1.01,5.971v1.06c0,0.001-0.001,0.002-0.001,0.003v6.983 c0,0.646,0.524,1.17,1.17,1.17h11.643c0.646,0,1.17-0.524,1.17-1.17v-8.18C14.992,5.191,14.468,4.668,13.822,4.668z M7.84,13.298 c-1.875,0-3.395-1.52-3.395-3.396c0-1.875,1.52-3.395,3.395-3.395s3.396,1.52,3.396,3.395C11.236,11.778,9.716,13.298,7.84,13.298z  M7.84,7.511c-1.321,0-2.392,1.071-2.392,2.392s1.071,2.392,2.392,2.392s2.392-1.071,2.392-2.392S9.161,7.511,7.84,7.511z"></path></svg></span></div><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Ummm...I am worried</span></div></div></div><div class="jrEtJ"><div class="kknmh"><div class="g0rxnol2 ln8gz9je ppled2lx"><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><div class="lhggkp7q ln8gz9je ppled2lx t3g6t33p qnwaluaf" style="background-image: url(&quot;data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/4gHYSUNDX1BST0ZJTEUAAQEAAAHIAAAAAAQwAABtbnRyUkdCIFhZWiAH4AABAAEAAAAAAABhY3NwAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAQAA9tYAAQAAAADTLQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAlkZXNjAAAA8AAAACRyWFlaAAABFAAAABRnWFlaAAABKAAAABRiWFlaAAABPAAAABR3dHB0AAABUAAAABRyVFJDAAABZAAAAChnVFJDAAABZAAAAChiVFJDAAABZAAAAChjcHJ0AAABjAAAADxtbHVjAAAAAAAAAAEAAAAMZW5VUwAAAAgAAAAcAHMAUgBHAEJYWVogAAAAAAAAb6IAADj1AAADkFhZWiAAAAAAAABimQAAt4UAABjaWFlaIAAAAAAAACSgAAAPhAAAts9YWVogAAAAAAAA9tYAAQAAAADTLXBhcmEAAAAAAAQAAAACZmYAAPKnAAANWQAAE9AAAApbAAAAAAAAAABtbHVjAAAAAAAAAAEAAAAMZW5VUwAAACAAAAAcAEcAbwBvAGcAbABlACAASQBuAGMALgAgADIAMAAxADb/2wBDAAMCAgICAgMCAgIDAwMDBAYEBAQEBAgGBgUGCQgKCgkICQkKDA8MCgsOCwkJDRENDg8QEBEQCgwSExIQEw8QEBD/2wBDAQMDAwQDBAgEBAgQCwkLEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBD/wAARCAAmAGQDASIAAhEBAxEB/8QAHAABAAEFAQEAAAAAAAAAAAAAAAUBAwQGBwIJ/8QAKRAAAgICAgEDBAEFAAAAAAAAAQIDBAAFBhESBxMhFCIxQRYII1Fhwf/EABcBAQEBAQAAAAAAAAAAAAAAAAABAgP/xAAeEQEAAgEEAwAAAAAAAAAAAAAAARExEhNBcQIiUf/aAAwDAQACEQMRAD8A+qeMi+TV99a000HGbkdXYM8RjlkYKAgkUyDto5ACUDgEo3RIPWXNBDuq+lpwciuwW9mkKi1NBH4I7/sgf96AJ7IVQfEct2d3a0zi74zj7fOMMa/fRU4u+Ou0hjGM6tmMYwGMYwGMYwILkX8uE1M8ZGtZDZiW4twyL41/cQyPGU77cRiQKrAAs6sWAQq+t3Nl63rsKJ1/DeHSUW+28s3JriSp3M484iKBDeMKxt4MF83mZfJBCHm6DjNR5JTxF5e2vn33/vGe8ZlWNsr8Gr11rZ2ksvDTheeRa1aSxMyopYhIolaSRuh8IilmPQAJIGct2fCv6do+Nw8z2HofrLFSpT+khRfTqae9FWudyvClNahshHNyUyoI+laWcSAN7gHUdrJtIdZbl0lOrb2KQO1SC1ZavDLMFPgkkqxyNGpboFhG5A7IVvwdY1O09YJp0Xe8F4dThMMLO9TldqyyymdRKoVtdGCqwmR1bsFnVYyqKxlWwLm25Pw69FoK+449uLsXIXq3aSScVvzrDJHLDJA9oewRSkjleJx9R7bIyM32+05WEv8Aq36d8T296v8AxXm0dy+0N67NrfTne2UsyNWiCPJNXpMkkghWGM9sWX2xG3RTxE/sNj6oR0opNVw7i9i2xT3YrPJLEEaArGW6daLliGMwA8R2I4z2PMrHl8duc6tVbD8t45otbZW14V49dupr0clbsf3HeSrCUk68j7YVh8Aefz2AktXtK24qLdqRW443AIW1TlrSfIB+UlVWHwR+R8HsfkEZmZpib71VPqOuhl9OdKOGNFNIORLyRmtKyRxFEaiao+55JHUdTFQsEjMVJjR9zyBjGMBjGMBjGMBjGMChHYIB67/eY1CnYp66ClY2di9NDGqPbsLGJZiPy7CNVQE/vxVR/gDGMtil6nYtxQx19pZpNHNHKzwLGTIqsC0beasPFwCpIAbonxKnoizc1uws7nX7GDf2qtOmkwsUI4ojHdZwAhkZkLqE6YgIy9kjyJA6LGLlKSOMYyKYxjAYxjAYxjAYxjA//9k=&quot;);"></div></div><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><div class="lhggkp7q ln8gz9je ppled2lx t3g6t33p qnwaluaf" style="background-image: url(&quot;blob:https://web.whatsapp.com/3ec5182d-f2c9-41e5-9473-280ed5d938af&quot;);"></div></div></div></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>looks like this is a hardcoded answer.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:51 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:51 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB050254874B91AFC57E6_919717617888@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Mayank Gupta +91 97176 17888" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://media-tir3-1.cdn.whatsapp.net/v/t61.24694-24/343036186_1200021077373038_2599390388356811675_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdQH5f-eBJEjhGv4eMR98rDpMMlzf6QBeyaAdo_msZlH7A&amp;oe=65EAF23E&amp;_nc_sid=e6ed6c&amp;_nc_cat=100" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d a7s2a5ab" role=""><span dir="auto" aria-label="Maybe Mayank Gupta" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Mayank Gupta</span><span class="WJuYU" testid="author" role="button">+91 97176 17888</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[1:52 pm, 26/02/2024] +91 97176 17888: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="fz47docv _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d swew6qod" role=""><span dir="auto" aria-label="Maybe Pratiksha" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Pratiksha</span><span class="WJuYU" testid="author">+91 97039 44446</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">looks like this is a hardcoded answer.</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Yeah I guess as a reference to the OpenAI joke that was viral and became a way to detect if a system was using OAI APIs</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:52 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:52 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB0A73BA0EF9BF4E061F0_917219388991@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _27hEJ _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Atik Shaikh +91 72193 88991" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno a82mwcnm"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">A</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div class="cm280p3y ktbp76dp ocd2b0bc folpon7g aa0kojfi snweb893 g0rxnol2 jnl3jror copyable-text" data-pre-plain-text="[1:54 pm, 26/02/2024] +91 72193 88991: "><div><div class="bxcbqipq mhcwslh8 ocd2b0bc jfqm35v0"><div class="_3IzYj _6rIWC p357zi0d giysludt" role=""><span dir="auto" aria-label="Maybe Atik Shaikh" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Atik Shaikh</span><span class="WJuYU" testid="author" role="button">+91 72193 88991</span></div></div><div role="button" tabindex="0" aria-label="Open Picture" class="gndfcl4n l8fojup5 paxyh2gw sfeitywo cqsf3vkf p357zi0d ac2vgrno laorhtua gfz4du6o r7fjleex g0rxnol2" style="width: 330px; height: 171.544px;"><span class=""></span><div class="g0rxnol2 ln8gz9je ppled2lx"><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><img alt="Thats basic maths ngl" src="data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/4gHYSUNDX1BST0ZJTEUAAQEAAAHIAAAAAAQwAABtbnRyUkdCIFhZWiAH4AABAAEAAAAAAABhY3NwAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAQAA9tYAAQAAAADTLQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAlkZXNjAAAA8AAAACRyWFlaAAABFAAAABRnWFlaAAABKAAAABRiWFlaAAABPAAAABR3dHB0AAABUAAAABRyVFJDAAABZAAAAChnVFJDAAABZAAAAChiVFJDAAABZAAAAChjcHJ0AAABjAAAADxtbHVjAAAAAAAAAAEAAAAMZW5VUwAAAAgAAAAcAHMAUgBHAEJYWVogAAAAAAAAb6IAADj1AAADkFhZWiAAAAAAAABimQAAt4UAABjaWFlaIAAAAAAAACSgAAAPhAAAts9YWVogAAAAAAAA9tYAAQAAAADTLXBhcmEAAAAAAAQAAAACZmYAAPKnAAANWQAAE9AAAApbAAAAAAAAAABtbHVjAAAAAAAAAAEAAAAMZW5VUwAAACAAAAAcAEcAbwBvAGcAbABlACAASQBuAGMALgAgADIAMAAxADb/2wBDAAMCAgICAgMCAgIDAwMDBAYEBAQEBAgGBgUGCQgKCgkICQkKDA8MCgsOCwkJDRENDg8QEBEQCgwSExIQEw8QEBD/2wBDAQMDAwQDBAgEBAgQCwkLEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBD/wAARCAAzAGQDASIAAhEBAxEB/8QAGwABAAIDAQEAAAAAAAAAAAAAAAUGAQIEAwn/xAAyEAACAQQCAgECAwYHAQAAAAABAgMABAURBhITIQcUIjFSoRUXIzJBgQgWJEJDUWGR/8QAFwEBAQEBAAAAAAAAAAAAAAAAAAECA//EACARAQACAQMFAQAAAAAAAAAAAAABESECAyISMUFRYZH/2gAMAwEAAhEDEQA/APqnSoOW75aOZRWUOKs244bDyS3jS9Zluu7DoqgnsOoX0VA9k99gKZyue3uRuXUTFTWYr89x9Z06uq8dilKV0aKi4OSYa7v73FWmStJrzGiM3tulwrS2wkXsnkQHadlBI3rYGxUpXmYIid9BVivKK4/yXwGK9jxsnM8Al5NJdwx27ZOESPJaDd0gXtstCPcgA2g/m1XviOe8Pz9rPfYLk+HyVtbQQ3U01pfxTJFDLEJopGZSQqPEyyKx9MhDDYO6nPBF+Sngi/J+tXiZVd/lX44iNysnPOOIbO3S7uQ2WgHhgcRlJX+77UYTwkMfREsej9w3i9+V/jbGsy5HnvG7UqLcsJsvbpoTxvJBvbf8iRSun5ljcjYU6tPgi/JTwRfk/WnEyqmL+XfjHOZD9k4T5C4xkL4osn01rmLeWXqzogborE6LyxqDr+Z1H4sN9OX+SOCcfurqyz3McFjbixtheXUN3koYXgty6oJXVmBVC7ooY6HZlG9kVYvBF+T9aeCL8n604mXDx+WwusVFf4vLNkrO+3eW9z9QJ0kjlPdTG49NHph1IJHXXs0qRVQo0o0KVlUPya541bQWzciymOx7+Y/QXN28KvDcmN1DwmUFfIFZ9ej6LbBBIqpXnEf8PGTjkmyPGPjq7S9yN7HI81lYuJ7+Wb6G7Ukqe0zzS/TSA7Znk8bbLdTdchhbHIXtre3M2RSW1YPGtvf3EMbEMD96RuEkG9enBBGwfW6j7rjmKulFlI/IBF5Z75pYsvexFZHOynZZQ5X7z1jG0TroBdKKoj0/dJnUlvBJxHJR5aGPKNJ/pZluYbV0nS42N+RIpJY5Q52EaVWBBYGp39jYK9uxe3NpZX13C2xPLBE8qdXfoOwXYCFpFX+o+7eySaYXE2XHrBbGwbJzRvM8m72+uLyUM7dm3JO7sFGzpe2lA0oA0K7zcIFZ+sukOjqJt/2Gtn+1QVn92nFhinw8eOskglPjfri7L7rTzSubMr4ehgMc88BHXt45pCGEjNIdsz8VfF/I8kcxyH434tlL8wR2purzD288xhjJMcfd0LdFJJC70NnVT95em1sZryGyubp443dLeFAJZWAOkXuVUEkaBYge/ZA90lyMEN/DjWiujLNFJMrJayNEFRkUhpQvRW3IulLBmAcqCEcqETf/AB7wDKZVM7k+D8fvMlHPFcpeT4yGSdZohIIpBIylgyCeYK29r5ZNa7Haw+PeA4toXxnB+P2bW6hITBjIYzGogFuAvVRoCBVi0P8AYoX+UaqcMyLoESewD6jY/wBdf9f+1urBlDDeiN+xo/8Aw0AAAaA0BWaUoFKUoFKUoI2/tLm/vrdbXO5LH/RvHPNHbwRGK5Qk/wAN3lib0ep2I2VwNexsE85sLuHN2c0vLcsY/GR9CYLb6ecqhBLuIO4Y9w/VZF2Y/Q6hwdr+TMx5RobfP4e2iu7YR2NtcWTvN9SvdpHLCdRInUx/w1RSvRyXIYBPDFZk3UOUvH5Zgb22sb5oHktYuq2YiSNZoJ28zDyiQSkn7OodFKEoWe1JbvOOvGvbO8GfyCxW/n8lr44PHc+Qgr5CYu48etL0ZN7Pfv613j8Px3VTxH+e4s3j8fnOccTvTb2BlyllZ4Ka3uZ3Z3Ec8Ra9k8EXpV6skpZkfTjelttQKUpQKUpQKUpQKUpQKUpQY0N717FOq6I6jR/H1SlA0N717rNKUClKUClKUClKUClKUClKUH//2Q==" class="jciay5ix tvf2evcx oq44ahr5 lb5m6g5c fsmudgz7" style="width: 100%;"></div><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><img alt="Thats basic maths ngl" src="blob:https://web.whatsapp.com/1f14b8b6-41b6-4b9c-9178-52219ac679ae" class="jciay5ix tvf2evcx oq44ahr5 lb5m6g5c osz0hll6 nq7eualt em5jvqoa a21kwdn3" style="width: 100%;"></div></div></div><div class="cm280p3y bkoknyjm esbfogcw jfqm35v0 bxcbqipq mhcwslh8"><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Thats basic maths ngl</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:54 pm</span></span></span></div></div><div class="lrw9n60e lhggkp7q fz4q5utg b9fczbqn"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:54 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 cvsrm5e1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB078083EC9ECDF1C16_919082776009@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Harsha +91 90827 76009" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://media-tir3-1.cdn.whatsapp.net/v/t61.24694-24/322461936_856881328727760_4977520626474252607_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdT6vyZkUO3Cy5CxH8NWudJxdbjdhx0tMihkdXxCDAQixg&amp;oe=65EAE19B&amp;_nc_sid=e6ed6c&amp;_nc_cat=109" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d gvktkc8w" role=""><span dir="auto" aria-label="Maybe Harsha" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Harsha</span><span class="WJuYU" testid="author" role="button">+91 90827 76009</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[1:55 pm, 26/02/2024] +91 90827 76009: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="a82mwcnm _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d giysludt" role=""><span dir="auto" aria-label="Maybe Atik Shaikh" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Atik Shaikh</span><span class="WJuYU" testid="author">+91 72193 88991</span></div><div class="_37DQv" dir="ltr" role="button"><div class="l7jjieqr tvf2evcx oq44ahr5 lb5m6g5c g97rt3p3 fewfhwl7 jdwybkuq kjemk6od"><span data-icon="status-image" class=""><svg viewBox="0 0 16 20" height="20" width="16" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 16 20"><title>status-image</title><path fill="currentColor" d="M13.822,4.668H7.14l-1.068-1.09C5.922,3.425,5.624,3.3,5.409,3.3H3.531 c-0.214,0-0.51,0.128-0.656,0.285L1.276,5.296C1.13,5.453,1.01,5.756,1.01,5.971v1.06c0,0.001-0.001,0.002-0.001,0.003v6.983 c0,0.646,0.524,1.17,1.17,1.17h11.643c0.646,0,1.17-0.524,1.17-1.17v-8.18C14.992,5.191,14.468,4.668,13.822,4.668z M7.84,13.298 c-1.875,0-3.395-1.52-3.395-3.396c0-1.875,1.52-3.395,3.395-3.395s3.396,1.52,3.396,3.395C11.236,11.778,9.716,13.298,7.84,13.298z  M7.84,7.511c-1.321,0-2.392,1.071-2.392,2.392s1.071,2.392,2.392,2.392s2.392-1.071,2.392-2.392S9.161,7.511,7.84,7.511z"></path></svg></span></div><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Thats basic maths ngl</span></div></div></div><div class="jrEtJ"><div class="kknmh"><div class="g0rxnol2 ln8gz9je ppled2lx"><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><div class="lhggkp7q ln8gz9je ppled2lx t3g6t33p qnwaluaf" style="background-image: url(&quot;data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/4gHYSUNDX1BST0ZJTEUAAQEAAAHIAAAAAAQwAABtbnRyUkdCIFhZWiAH4AABAAEAAAAAAABhY3NwAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAQAA9tYAAQAAAADTLQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAlkZXNjAAAA8AAAACRyWFlaAAABFAAAABRnWFlaAAABKAAAABRiWFlaAAABPAAAABR3dHB0AAABUAAAABRyVFJDAAABZAAAAChnVFJDAAABZAAAAChiVFJDAAABZAAAAChjcHJ0AAABjAAAADxtbHVjAAAAAAAAAAEAAAAMZW5VUwAAAAgAAAAcAHMAUgBHAEJYWVogAAAAAAAAb6IAADj1AAADkFhZWiAAAAAAAABimQAAt4UAABjaWFlaIAAAAAAAACSgAAAPhAAAts9YWVogAAAAAAAA9tYAAQAAAADTLXBhcmEAAAAAAAQAAAACZmYAAPKnAAANWQAAE9AAAApbAAAAAAAAAABtbHVjAAAAAAAAAAEAAAAMZW5VUwAAACAAAAAcAEcAbwBvAGcAbABlACAASQBuAGMALgAgADIAMAAxADb/2wBDAAMCAgICAgMCAgIDAwMDBAYEBAQEBAgGBgUGCQgKCgkICQkKDA8MCgsOCwkJDRENDg8QEBEQCgwSExIQEw8QEBD/2wBDAQMDAwQDBAgEBAgQCwkLEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBD/wAARCAAzAGQDASIAAhEBAxEB/8QAGwABAAIDAQEAAAAAAAAAAAAAAAUGAQIEAwn/xAAyEAACAQQCAgECAwYHAQAAAAABAgMABAURBhITIQcUIjFSoRUXIzJBgQgWJEJDUWGR/8QAFwEBAQEBAAAAAAAAAAAAAAAAAAECA//EACARAQACAQMFAQAAAAAAAAAAAAABESECAyISMUFRYZH/2gAMAwEAAhEDEQA/APqnSoOW75aOZRWUOKs244bDyS3jS9Zluu7DoqgnsOoX0VA9k99gKZyue3uRuXUTFTWYr89x9Z06uq8dilKV0aKi4OSYa7v73FWmStJrzGiM3tulwrS2wkXsnkQHadlBI3rYGxUpXmYIid9BVivKK4/yXwGK9jxsnM8Al5NJdwx27ZOESPJaDd0gXtstCPcgA2g/m1XviOe8Pz9rPfYLk+HyVtbQQ3U01pfxTJFDLEJopGZSQqPEyyKx9MhDDYO6nPBF+Sngi/J+tXiZVd/lX44iNysnPOOIbO3S7uQ2WgHhgcRlJX+77UYTwkMfREsej9w3i9+V/jbGsy5HnvG7UqLcsJsvbpoTxvJBvbf8iRSun5ljcjYU6tPgi/JTwRfk/WnEyqmL+XfjHOZD9k4T5C4xkL4osn01rmLeWXqzogborE6LyxqDr+Z1H4sN9OX+SOCcfurqyz3McFjbixtheXUN3koYXgty6oJXVmBVC7ooY6HZlG9kVYvBF+T9aeCL8n604mXDx+WwusVFf4vLNkrO+3eW9z9QJ0kjlPdTG49NHph1IJHXXs0qRVQo0o0KVlUPya541bQWzciymOx7+Y/QXN28KvDcmN1DwmUFfIFZ9ej6LbBBIqpXnEf8PGTjkmyPGPjq7S9yN7HI81lYuJ7+Wb6G7Ukqe0zzS/TSA7Znk8bbLdTdchhbHIXtre3M2RSW1YPGtvf3EMbEMD96RuEkG9enBBGwfW6j7rjmKulFlI/IBF5Z75pYsvexFZHOynZZQ5X7z1jG0TroBdKKoj0/dJnUlvBJxHJR5aGPKNJ/pZluYbV0nS42N+RIpJY5Q52EaVWBBYGp39jYK9uxe3NpZX13C2xPLBE8qdXfoOwXYCFpFX+o+7eySaYXE2XHrBbGwbJzRvM8m72+uLyUM7dm3JO7sFGzpe2lA0oA0K7zcIFZ+sukOjqJt/2Gtn+1QVn92nFhinw8eOskglPjfri7L7rTzSubMr4ehgMc88BHXt45pCGEjNIdsz8VfF/I8kcxyH434tlL8wR2purzD288xhjJMcfd0LdFJJC70NnVT95em1sZryGyubp443dLeFAJZWAOkXuVUEkaBYge/ZA90lyMEN/DjWiujLNFJMrJayNEFRkUhpQvRW3IulLBmAcqCEcqETf/AB7wDKZVM7k+D8fvMlHPFcpeT4yGSdZohIIpBIylgyCeYK29r5ZNa7Haw+PeA4toXxnB+P2bW6hITBjIYzGogFuAvVRoCBVi0P8AYoX+UaqcMyLoESewD6jY/wBdf9f+1urBlDDeiN+xo/8Aw0AAAaA0BWaUoFKUoFKUoI2/tLm/vrdbXO5LH/RvHPNHbwRGK5Qk/wAN3lib0ep2I2VwNexsE85sLuHN2c0vLcsY/GR9CYLb6ecqhBLuIO4Y9w/VZF2Y/Q6hwdr+TMx5RobfP4e2iu7YR2NtcWTvN9SvdpHLCdRInUx/w1RSvRyXIYBPDFZk3UOUvH5Zgb22sb5oHktYuq2YiSNZoJ28zDyiQSkn7OodFKEoWe1JbvOOvGvbO8GfyCxW/n8lr44PHc+Qgr5CYu48etL0ZN7Pfv613j8Px3VTxH+e4s3j8fnOccTvTb2BlyllZ4Ka3uZ3Z3Ec8Ra9k8EXpV6skpZkfTjelttQKUpQKUpQKUpQKUpQKUpQY0N717FOq6I6jR/H1SlA0N717rNKUClKUClKUClKUClKUClKUH//2Q==&quot;);"></div></div><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><div class="lhggkp7q ln8gz9je ppled2lx t3g6t33p qnwaluaf" style="background-image: url(&quot;blob:https://web.whatsapp.com/1f14b8b6-41b6-4b9c-9178-52219ac679ae&quot;);"></div></div></div></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Seems at the level of GPT3.5 basically</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:55 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:55 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="reaction 😂"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="😂" draggable="false" class="bd96tnyg p83aruvr b85 emoji apple _11JPr" tabindex="-1" style="background-position: -60px -60px;"></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3AFE4A8B123087B162D7_917892563038@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _27hEJ _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe ashish +91 78925 63038" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno po56q9xr"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">A</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div class="cm280p3y ktbp76dp ocd2b0bc folpon7g aa0kojfi snweb893 g0rxnol2 jnl3jror copyable-text" data-pre-plain-text="[1:56 pm, 26/02/2024] +91 78925 63038: "><div><div class="bxcbqipq mhcwslh8 ocd2b0bc jfqm35v0"><div class="_3IzYj _6rIWC p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe ashish" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">ashish</span><span class="WJuYU" testid="author" role="button">+91 78925 63038</span></div></div><div role="button" tabindex="0" aria-label="Open Picture" class="gndfcl4n l8fojup5 paxyh2gw sfeitywo cqsf3vkf p357zi0d ac2vgrno laorhtua gfz4du6o r7fjleex g0rxnol2" style="width: 330px; height: 208.931px;"><span class=""></span><div class="g0rxnol2 ln8gz9je ppled2lx"><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><img alt="Inference speed is slow, but slashdot effect, i guess
Pretty reasonable quality of text generation for n=1 on a cancer q.


I think we will see quite a few “made in india” llms

have been hearing about Jio’s hanooman and a Marathi llm

Decent job by ola, seems like" src="data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDABsSFBcUERsXFhceHBsgKEIrKCUlKFE6PTBCYFVlZF9VXVtqeJmBanGQc1tdhbWGkJ6jq62rZ4C8ybqmx5moq6T/2wBDARweHigjKE4rK06kbl1upKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKT/wgARCAAuAEgDASIAAhEBAxEB/8QAGQABAAMBAQAAAAAAAAAAAAAAAAECAwQG/8QAFAEBAAAAAAAAAAAAAAAAAAAAAP/aAAwDAQACEAMQAAAA9LlpkWAABogObqwKLii8EzEmygvhvkVWEJESkssP/8QAFBEBAAAAAAAAAAAAAAAAAAAAMP/aAAgBAgEBPwB//8QAFBEBAAAAAAAAAAAAAAAAAAAAMP/aAAgBAwEBPwB//8QAJRAAAgEDAwQCAwAAAAAAAAAAAAECERNhAxIhMTJBURBSQmKh/9oACAEBAAE/ADU07jqmqC0Y05VWWYeizD0WYLwWIeizBeBKhLtdSGooR2qLL2C9gvYL2C9gvfqRba5VCXaxVOvPx56HkUJNVSVCOlJc0F0JcRYtWWC5LBdlgWpLBdlguyLmp6/hpyk67iXRm6P0N0X+BujXsN0fobo/Q3xrxDkvYITU0NVVCyxaOSyy0WX7Fo5I6VHViSXRH//+AAMA/9k=" class="jciay5ix tvf2evcx oq44ahr5 lb5m6g5c fsmudgz7" style="width: 100%;"></div><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><img alt="Inference speed is slow, but slashdot effect, i guess
Pretty reasonable quality of text generation for n=1 on a cancer q.


I think we will see quite a few “made in india” llms

have been hearing about Jio’s hanooman and a Marathi llm

Decent job by ola, seems like" src="blob:https://web.whatsapp.com/e8dac52e-3aba-4908-8a10-fbc1580310eb" class="jciay5ix tvf2evcx oq44ahr5 lb5m6g5c osz0hll6 nq7eualt em5jvqoa a21kwdn3" style="width: 100%;"></div></div></div><div class="cm280p3y bkoknyjm esbfogcw jfqm35v0 bxcbqipq mhcwslh8"><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Inference speed is slow, but slashdot effect, i guess
Pretty reasonable quality of text generation for n=1 on a cancer q.


I think we will see quite a few “made in india” llms

have been hearing about Jio’s hanooman and a Marathi llm

Decent job by ola, seems like</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:56 pm</span></span></span></div></div><div class="lrw9n60e lhggkp7q fz4q5utg b9fczbqn"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:56 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 cvsrm5e1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_525E614D294E5CAD58_919822381281@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Dilip Thomas Ittyera +91 98223 81281" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/309186150_817654219469995_3888201881878390184_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRh29wTc1XaHX8pEd-BMHein5ijlTG5dv5VX2PQwa0OVw&amp;oe=65EAE41B&amp;_nc_sid=e6ed6c&amp;_nc_cat=110" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d dud7p2xx" role=""><span dir="auto" aria-label="Maybe Dilip Thomas Ittyera" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Dilip Thomas Ittyera</span><span class="WJuYU" testid="author" role="button">+91 98223 81281</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:06 pm, 26/02/2024] +91 98223 81281: "><div class="M6sU5"><div class="xzT4z gndfcl4n l8fojup5 paxyh2gw sfeitywo cqsf3vkf k2bacm8l ajgl1lbb p357zi0d sap93d0t ovllcyds kvdvyush gfz4du6o r7fjleex rrq4r3yd"><img alt="" class="tvf2evcx oq44ahr5 lb5m6g5c n43pk08i fpj7ivsd jpthtbts" src="data:image/jpeg;base64,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAMCAgICAgMCAgIDAwMDBAYEBAQEBAgGBgUGCQgKCgkICQkKDA8MCgsOCwkJDRENDg8QEBEQCgwSExIQEw8QEBD/2wBDAQMDAwQDBAgEBAgQCwkLEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBD/wAARCABkAGQDASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwD5r30oao7eG4vJRDbQtI56Ko5rRTw34gdBIuk3BX1C1NzIp7z1p4k4FTSaHri43aXcj/tmagbTtUjJWTT7kEf9Mm/wpXCwu7FHmVG0F0n37eVfqhqLce4xTAteZSF+areZSTXMNvG000myKMbnbrgetA7FoOT8qjJNdBP4W+w+Fz4r1TxLoNnBv8v7LNfp9rJ/64j5sVY0q4+HMenW+t2+r3EiXB8iKaSPA81lJ+UY5I6+3FTeGPh38K1keaTxMo8xjJPJLIBNIx75ZeajnHynJ2d1pepSRxWOuWEry42r5u0/rWjPot3btsea0d842pdRlvyzmu9g+GHwq1J1WHx8I2bqrxxNj82Gas3H7LtlfQvd6frdjdKvKN9hLlvT7mSKOYLHm76VqceN1nL8w3D5c5H4UV20X7J2vXMYmjnsIlboss00J/75IFFHMFjxPWrfVry3UaPq0+n3COGEkTYP0riW8bfGDwvfP9r1vVJrUNlWikGcZ9xzXoQakkCSLskVWB6girEnY2PhZ+1DdaT/AGva65MusTanaG3iXVIQGs37SRkDGRV+P4rfEaZhMut6ZcR44zZKQR9c1wGoeD9G1FWPk+S7dSneua/4QvxBoMvneHtZnjTOfLLb0/75P9KVkB7/AGnxq8VQ7f7Q8N+Gr9VXHzWxQn3yKlX4zGSQfbvhfoEihcER3MiZPr04rwm38YeINPbyda0f7Qq9ZbXr9Sp5rd03xZoWrN5cN6iTd4pPkcfgaOVBY9fuPit4NuFTd8H4kOP3nl6hnJ9twqSz8dfBi6kEetfCvW44XXEnk3Vu35ZWvNVOQOhpyle/TvSshnt+m2/7LN1PLZ65H41s3gaK4sba3t0kjgjkjA+ZUxl8g5PoRVy40n9k1ZhC3irxVbOPvrPpLjb+T5rxi+vJbDVodYtXKsBHMGP3cgDg+o4FeuX3xy+Cfxl06x8OeKNLtPDutaZEzS6nGh8uVVXlAVGSWPY1KWg36i33gv8AZtureSTRfi9cwyjG2O606VePyNYtt4N+FazI1r8ctPt2VvlZre4TafXISsqGH4D6hNHHZeLLsZOWVmwQPoRWreeD/gnIU/s34gXMbHqJo8gfiBSv6hY3pNHs59pj/aispI0XYhfVLoYA7AHoKK4yTwf8N45GVfHyOAeGEfB/Sii/mFjzFW9KftyK9u+FP7LWveO9AHibxBrJ0G0uVSSxX7N58txGyhhLt3qFQqwwc5PPHFfRnwv/AGQfgvo/kt4nu5PEmpKyv/pkpt4FYPldsKMM8YDK7OpweMHFKWIgny3NI4epJXtofDlv4R8VXWlnXLbw7qkunBWY3aWcjQgKcE7wNuAQQeexrLKniv2Y0bwrptjp8Ol6XZW0VpBGIobeCNVjjjHAVUHAAHYV594j/ZN+A+vaxca7rfw3spLm5/1v2W5ubRS3c+XDIseT1J25JJJJpKtfdDdC2zPygmt7eb/XRK3oehH41k6h4Z03UF23FtFOO3mDDr9HHzCvov8Aa2+Buj/BXxva/wDCLyynQNeikubOGZ9z2rowEkO48sq7l2s3zYbBLEFj4P5g3VtF3V0ZOPK7M5j+y9e0SQto+sTRwjpBegzR/QOPmH417L8Mfg38SvHVi+pa5p9loNsm5VnmlZ/NZcdI1G4A5PJ7g10P7N/wvsviR4sluNdtWm0bSEEtwol2b5Wz5SHAJIyrEjjIUjPr9d6pY2unwtDaxqiRxrtRRgD5RxxXkYzNPY1vq1P4rXb7eR7eByj2+H+tVfhbsvPuz4N8W/sw/tJSfa9f8A61Y6tb28hjksLF2jcIBkEpKMMTyB615cvj7xF4Vvj4d8c6Pc6Lf2jYmS60eDOSAfmwikjHI9jX6NXWrah4A8TQ6xY4LwlRcIjfJMn8SMOM/wBDW9+0l8EvDfx3+FJ8ceD9O06fVbW1a8s5ZYkHnBFO62lcjIXluhyrBTzypvCY91PdqGeMy9Uvep7Hwj4c8afDfVtNuXuvDemajqbpi3ubZYohEfVoyhJ/OsLVNS8YKuNFk0SFV+6p0mEt+ZBB/KuF8VfDW00gpqV9pOqeFZJjmO6hbzbRz7OpwP0qLTLz4iaTGHtbi08TWS/88pB5uPbvXqJHkWOxtPiV8RtJhFnJ4g1KFlP3bfSbLZ+GIqK59fipoMI8rVLa6srleHhliIZTRRYLI/W/w7r2m+MNLtvF2liRLDWLdNQtlmUBljkG8BgONwztIHAIIFcl4i1iabUDJasRHC/7sjPUcn9elfO/7K3xumsdDk8B+IvlSB3uLFhsVnjIJMUafKGw2Sepwxx0APt0mseGpJlhj1mBFeESM1zuhUOf4FLAZYcdOD2Pevmq+KhQqOjUdpLv17NH1GHwdSvTVanHmi+2tu6f9eZ614D8a3txZxrc3TPIvy7h6j6V6fD40s9P06a88SahFDYW8ZlluZmCiJQM5J9K+S18caP4TuWjk1gPGfLdXtcSZ3EDYM/KX5IwSORg1598Svi18UPiJHLbaLpWo2mlQwwi305ZcR3BaJS0kkwMbEqxZgMYBCDGQxbkxHEeDwcfeqJy9Vb5vb9Too8PYrEyu4uMe7X5Lf8AQwv25PitonxI8YaJZ+F43nsdJtpSLkqQ0jzCNnXaeV2FdhB53K46AE/MTNIrZZWH1Fd5rnwT+KuuXDapHHZ7/ljFt9qCsg5ySxVVOWLN1JG7HOK5LxD8OviV4O3zavo91FBG4T7TG3mRElA3DqSMAHk+oIr1MqznB42lGMa8JTfRNfdby2PGzTLa2GrScaUlBaJtduvz3Psr9mPTbfwn8I9C1G+1q3lbxNdXWsQW/k7GgBmezEZYkiVidOkcYwQDjHGT6hqWsafrFmBuiW4KmSOTy2TK4xt57Hrz0zivkn9n34gaPNoZ+H/xBvy3m3Rj09724OyBJCrL5BKlYnjlEsnLHcZjheGJ+kpNDtfDNjdyW/jPWdYi+zi3kh1CS3kj3ghvMykSsrnc2QpC/OSVPy4+czHD144+rWjK1tde1tLd0/wPs8txWGlltKhON76K3e+t+zV7+Zi+ObHWLKG3v9Yt5Y47+N5YXk/jAbB59eh+jLWr+zT48vtM8U3Hge8t7u68Pa1lVVYmkitrj16YVXGQffB9axfFWsaVq3g7+yftSpc2Tedbod2UduHRW2nhgFO3jlUOeCD9CeB9Xh8D/B3T7jXltbS4tdN8yZvJVCvy5UHGMkcD3xXoZfVhibVYadzy8zozwn7mprfZ/kfHn7efgT4d+Fprnxt4Ru5raWN/+KgsLJg1urF1jWcx/dRyzqrcYbchwCGLfE9vJ4H1pvtGk3Ys7rtJYzfZpM/7SHKMfyr63+NniSxtvBOu3HiK4JvPGU8UNrC2396I54pnkcMc+WBGBkfxMnavlrUfAvhvVM408W8rjcGhGDj14r6TBVJVKXNLufLY6nGjV5Y9tfUr3Gka5dOHl17TbzaNqyX9i3nY7Alcg/WisZ/AWtWrGHT/ABdNHCPuq7HI/Wius47lbxK2uWs0PiBdenjnsyPsrKQGjbORsPVSDzkc16P4X/a58XW1nbaf4o0CLUZ4UWKO8WVkJ2gAM8e1tzeu0qD6CvM9a87ULwDkwQfKF9+5/pVNYbeMHzVJDqUBXqMjGR715eOo4bG+5WgpWPay+ricDH2lGbjf+tj9LPgf4dHizwHoni/xJozWupahbee1vPMJlXc7skgG0bGaMxkqclTnnsPS7jwzEF+VcfhXmPwV+Pvw98baFax6drEMF0ka+bZysFliOMbSvtg8jivYf7a0toIplvreQSZ2qsgLLj1A6fjX8e8RSzNZlVeIpuDTfu2fuq+iXku/U/Y8BiqNTDwcZ82m/c46/wBBaNshOPcZFc9qdr5EEtvcQieCZSksTrlXXuCDXf3+v6Oqt5lwm4DI+bivNvGXxA8JaTay3F9qNvbxgfekkAH608tniq01GMW35GletRjF3aPjn9q7w/rHwz17Rte8IaneabpFzuuIFtV2/ZrrLBl3ZH8LYXHQbu2GbmvAf7VE1tLBb/ELTbu7W2tfIgvtLuBbTiQ3DymWdSrJNlXMXRSI1QDOwZ6/4weOfDvx0mPgnw5bz3Rtg01pdfaUgh84diXQ7wVyMKVJO0Z6ivAIfhzdASyXS3FuySGKOKRPvOpw2ScEAEMOB1GPev6q4ejWxGU0o5rH96lbX4rdL+du/qz8kzCr9Wxsnl79xu9ul+tv+AfS3iz9rjwjfQ29n4b823CxH7RcXJE25y3Hlr5KFMLwQyvk8hhXOeP/ANsnxtc+H4fD/h/xabjTrllmMEa+W8DgxSD5mXKYIwApIyknUN83zrf6bJpMxjudP2gjgtn1xkHPPQ1lXCAyH92qnPbP9TXqYXKcLR1p3t67/cYYvNsVXVqlr+mq9LnvNr4pk8RWMV1q1rNBdlCrKJxJE5baTJ8ylgxxjksByec4D2m0U3Hkw/abeaRV+6GTcScYDLwQDnrjgdK43wXdao2hxNfW6iBflt5twy4HYr14459CPx3P7UhOYw6/L1Pp/SvXSSVongyb5nzbl2Twzo9y5mZp3LfxCc80Vzd5r2lCba0zIQP+WU7Rg9eykDPvRTJuye1l09bZY1jDyNHiR2kz8+cllxjHGBg57nvgD2GmzSCRUkH97Eg6+o44/WqL+C9N3A215e2+OyS5H/jwNQN4d8RWZB0/WIbhf7s6lCB9RnP6Vl9Xp9joWKqr7RtQ+G9PvJmmtby4srrPyPHJtOPqK6Gz1r4yaLN5mk/EbUnGMBZn80fk2RXAfb/FGnkfbtHd1H8UDbx+nSrVv8QPJwjyTQnpiRSP515GYZTUxUuaPLJdpxUvubTOvC46NJWldPvF2Ov13x18bL618nUvHVyEH/PGNYm/76UA1wl1NrWq3AudevL/AFFh8pM0zP0+pq9cePo5vlmuonB/z2qo3jDSdu1pkx6KMVWWZZ9Td5U4R/wpL8khYzG+3Voyk/V3/Vm7YTIqwyWNpPDJDGBIxl+8wYhSoAG0BdgwSTkE55wG6x4g8QatLFY6hdT+XCNqq0u/A69T+P5mub/4SjSUYtDqE8ef7rVX/wCEi03zPMW+k3g5B28/yr0ZYW8nJdTOGMcYKL6HaWelaLdWzrrdvLcWxXhlm8sq2ckY2ksSOByOTznpVfxJcaNrF0tx/YNnbpHEsCLt3HYpyMk9+vIHQ4Oa5eTxVbXGPO1K5f8A4D/h9KWPxFpMZ3Klw7erRlv51rSo+zWruY167qyvFWNSRpZECruWJRj5RjAqvp9pHrlwbGxupIoY/mmkVc/gCeM1HH4tjZhHZ6fcO7cAeUea67TZppLNJbi0jtpX+Z407H3961ZzjrPQ9JsYBBDYxkDks6gsx9ST3oqx5lFSBnqxqVWOKKKslD1NJNa29x8txBHIP9pQaKKAIl0fSl5XT7cf9sxTxpemjpYW/wD37FFFSMVLGxRgy2cAI/6Zr/hR9hsSxb7HBk8/6sUUUAK1nZr921hH/ABSrbwY4hQfRRRRQgFVVX7qgUuaKKQDdzepooooA//Z"><div class="k5z04t9s cm280p3y p357zi0d ggj6brxn m0h2a7mj lb5m6g5c f8m0rgwh ac2vgrno idwf4z32 onmzirdk gfz4du6o r7fjleex n1yiu2zv itegkywt eb4rp10x rppts313 ln8gz9je"><div class="aoi073rw suguakab jjtagnqy c32ccnay tvf2evcx m0h2a7mj lb5m6g5c j4enbv94 gfz4du6o r7fjleex lhj4utae ljrqcn24" title="Microsoft’s LongRoPE Breaks the Limit of Context Window of LLMs, Extents it to 2 Million Tokens"><span dir="ltr" aria-label="" class="ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae i033jvx7 cw3vfol9 _11JPr" style="min-height: 0px;">Microsoft’s LongRoPE Breaks the Limit of Context Window of LLMs, Extents it to 2 Million Tokens</span></div><div class="jjtagnqy tvf2evcx ttu8nud2 lb5m6g5c dsh4tgtl gfz4du6o r7fjleex lhj4utae"><span class="g0rxnol2 gfz4du6o r7fjleex" style="display: -webkit-box; overflow: hidden; -webkit-box-orient: vertical; -webkit-line-clamp: 2;"><span dir="ltr" title="Large Language Models (LLMs) have achieved remarkable success across various tasks. However, they often grapple with a limited context…" aria-label="" class="i033jvx7 _11JPr" style="min-height: 0px;">Large Language Models (LLMs) have achieved remarkable success across various tasks. However, they often grapple with a limited context…</span></span></div><div class="j8h4s3v0 tvf2evcx oq44ahr5 lb5m6g5c dsh4tgtl qbqilfqo"><span dir="ltr" aria-label="" class="_11JPr" style="min-height: 0px;">medium.com</span></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span><a dir="auto" href="https://medium.com/syncedreview/microsofts-longrope-breaks-the-limit-of-context-window-of-llms-extents-it-to-2-million-tokens-eb43efdbadff" title="https://medium.com/syncedreview/microsofts-longrope-breaks-the-limit-of-context-window-of-llms-extents-it-to-2-million-tokens-eb43efdbadff" target="_blank" rel="noopener noreferrer" class="_11JPr selectable-text copyable-text">https://medium.com/syncedreview/microsofts-longrope-breaks-the-limit-of-context-window-of-llms-extents-it-to-2-million-tokens-eb43efdbadff</a></span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">2:06 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:06 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="reaction 👍"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A080BB7398B53D9D416_919354493228@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Azhan Mohammed +91 93544 93228" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://media-tir3-1.cdn.whatsapp.net/v/t61.24694-24/415823620_588200936799230_8959200103469063155_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdR_pTfPCMFAxFn-TVmoUaWYSLvgE1ORXxiXANZ9ST0xVg&amp;oe=65EAF835&amp;_nc_sid=e6ed6c&amp;_nc_cat=100" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe Azhan Mohammed" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Azhan Mohammed</span><span class="WJuYU" testid="author" role="button">+91 93544 93228</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:14 pm, 26/02/2024] +91 93544 93228: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>what are some good rerankers available, either in open source, or a subscription based model</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">2:14 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:14 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="reaction 👍"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB038BD206F26CDFF5C5C_917737887058@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Nirant K +91 77378 87058" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno g9gonq0i"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">N</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author" role="button">+91 77378 87058</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:20 pm, 26/02/2024] +91 77378 87058: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="po56q9xr _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe Azhan Mohammed" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Azhan Mohammed</span><span class="WJuYU" testid="author">+91 93544 93228</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">what are some good rerankers available, either in open source, or a subscription based model</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Cohere and BGE are both decent in my experience — work from <span role="button" tabindex="0" class=""><span dir="auto" data-jid="919550164716@c.us" data-display="~Ravi Theja" aria-label="Maybe Ravi Theja" class="ajgl1lbb o0rubyzf _11JPr selectable-text select-all copyable-text" data-plain-text="@~Ravi Theja" data-app-text-template="​919550164716@c.us​">@<span dir="ltr">~Ravi Theja</span></span></span> indicates Cohere was good.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">2:20 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:20 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="Reactions 👍, 🙏 4 in total"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍🏻" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -40px -60px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="🙏" draggable="false" class="bd96tnyg p83aruvr b93 emoji apple _11JPr" tabindex="-1" style="background-position: 0px -20px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk o2es7gts i2cterl7" style="flex-shrink: 0;"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 f8m0rgwh r15c9g6i ilf8vifs gfz4du6o r7fjleex qnz2jpws kjemk6od"><div class="dn5wc90o ilf8vifs gndfcl4n p357zi0d"><span class="f8jlpxt4 qfejxiq4 ce5kru2g">4</span></div></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB00D6BD24946B8FC7EE6_919550164716@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Ravi Theja +91 95501 64716" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno j3zl77sr"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">R</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d gvktkc8w" role=""><span dir="auto" aria-label="Maybe Ravi Theja" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Ravi Theja</span><span class="WJuYU" testid="author" role="button">+91 95501 64716</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:21 pm, 26/02/2024] +91 95501 64716: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g9gonq0i _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author">+91 77378 87058</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Cohere and BGE are both decent in my experience — work from <span class=""><span dir="auto" data-jid="919550164716@c.us" data-display="~Ravi Theja" aria-label="Maybe Ravi Theja" class="_11JPr">@<span dir="ltr">~Ravi Theja</span></span></span> indicates Cohere was good.</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span><span role="button" tabindex="0" class=""><span dir="auto" data-jid="919632834013@c.us" data-display="~Kartik Mandaville" aria-label="Maybe Kartik Mandaville" class="ajgl1lbb o0rubyzf _11JPr selectable-text select-all copyable-text" data-plain-text="@~Kartik Mandaville" data-app-text-template="​919632834013@c.us​">@<span dir="ltr">~Kartik Mandaville</span></span></span> seems using cohere reranker in their product Albus.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">2:21 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:21 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_1FF23353F4D9413FD44BAD1C2B1B5DB4_447505487797@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Ganaraj +44 7505 487797" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/357029048_327201076325969_1848146647852128456_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRSyKF7yeee8lWajqAEgBA6aceUAZCQopKkjl7QAXDNmQ&amp;oe=65EAD16A&amp;_nc_sid=e6ed6c&amp;_nc_cat=108" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d aqebjpab" role=""><span dir="auto" aria-label="Maybe Ganaraj" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Ganaraj</span><span class="WJuYU" testid="author" role="button">+44 7505 487797</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:22 pm, 26/02/2024] +44 7505 487797: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g9gonq0i _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author">+91 77378 87058</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Cohere and BGE are both decent in my experience — work from <span class=""><span dir="auto" data-jid="919550164716@c.us" data-display="~Ravi Theja" aria-label="Maybe Ravi Theja" class="_11JPr">@<span dir="ltr">~Ravi Theja</span></span></span> indicates Cohere was good.</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>I tried launching bge in hugging face last week ( paid inference ). While it deployed it on a GPU machine, it doesn't use GPU at all. So, it's quite slow and unusable</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">2:22 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:22 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB0EADAF0542E01699E3C_917737887058@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Nirant K +91 77378 87058" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno g9gonq0i"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">N</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author" role="button">+91 77378 87058</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:25 pm, 26/02/2024] +91 77378 87058: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="awij7w2s _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d aqebjpab" role=""><span dir="auto" aria-label="Maybe Ganaraj" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Ganaraj</span><span class="WJuYU" testid="author">+44 7505 487797</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">I tried launching bge in hugging face last week ( paid inference ). While it deployed it on a GPU machine, it doesn't use GPU at all. So, it's quite slow and unusable</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>For other readers: Huggingface Infra is not a reflection of the model itself</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">2:25 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:25 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="Reactions 😂, 👍 4 in total"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="😂" draggable="false" class="bd96tnyg p83aruvr b85 emoji apple _11JPr" tabindex="-1" style="background-position: -60px -60px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk o2es7gts i2cterl7" style="flex-shrink: 0;"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 f8m0rgwh r15c9g6i ilf8vifs gfz4du6o r7fjleex qnz2jpws kjemk6od"><div class="dn5wc90o ilf8vifs gndfcl4n p357zi0d"><span class="f8jlpxt4 qfejxiq4 ce5kru2g">4</span></div></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_F3BA79001E6C219C9EB6A16A07F92B18_447505487797@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Ganaraj +44 7505 487797" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/357029048_327201076325969_1848146647852128456_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRSyKF7yeee8lWajqAEgBA6aceUAZCQopKkjl7QAXDNmQ&amp;oe=65EAD16A&amp;_nc_sid=e6ed6c&amp;_nc_cat=108" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d aqebjpab" role=""><span dir="auto" aria-label="Maybe Ganaraj" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Ganaraj</span><span class="WJuYU" testid="author" role="button">+44 7505 487797</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:26 pm, 26/02/2024] +44 7505 487797: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Anyone from hugging face here who can look into this? <img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="😅" draggable="false" class="b85 emoji apple _11JPr selectable-text copyable-text" data-plain-text="😅" tabindex="-1" style="background-position: -20px -80px;"></span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">2:26 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:26 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A65E9AEDC095BB5090D_919008639111@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Ambika +91 90086 39111" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno exofpctk"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">A</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d a1q56urb" role=""><span dir="auto" aria-label="Maybe Ambika" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Ambika</span><span class="WJuYU" testid="author" role="button">+91 90086 39111</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:27 pm, 26/02/2024] +91 90086 39111: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="awij7w2s _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d aqebjpab" role=""><span dir="auto" aria-label="Maybe Ganaraj" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Ganaraj</span><span class="WJuYU" testid="author">+44 7505 487797</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">I tried launching bge in hugging face last week ( paid inference ). While it deployed it on a GPU machine, it doesn't use GPU at all. So, it's quite slow and unusable</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>oh - did you make your own build or was it directly through the inference api stuff</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">2:27 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:27 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB0C00D97A6A8C5977215_917737887058@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Nirant K +91 77378 87058" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno g9gonq0i"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">N</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author" role="button">+91 77378 87058</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:27 pm, 26/02/2024] +91 77378 87058: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="awij7w2s _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d aqebjpab" role=""><span dir="auto" aria-label="Maybe Ganaraj" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Ganaraj</span><span class="WJuYU" testid="author">+44 7505 487797</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Anyone from hugging face here who can look into this? <img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="😅" draggable="false" class="b85 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -80px;"></span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Huggingface folks are pretty active on Twitter/X, probably you can tweet to them directly. Wouldn't want this forum to be another support channel for them</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">2:27 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:27 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="Reactions 😂, 👍 2 in total"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="😂" draggable="false" class="bd96tnyg p83aruvr b85 emoji apple _11JPr" tabindex="-1" style="background-position: -60px -60px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk o2es7gts i2cterl7" style="flex-shrink: 0;"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 f8m0rgwh r15c9g6i ilf8vifs gfz4du6o r7fjleex qnz2jpws kjemk6od"><div class="dn5wc90o ilf8vifs gndfcl4n p357zi0d"><span class="f8jlpxt4 qfejxiq4 ce5kru2g">2</span></div></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_157B770FA1C9D2F11B3D99915579E044_918368770456@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Ankur Goel +91 83687 70456" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno po56q9xr"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">A</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe Ankur Goel" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Ankur Goel</span><span class="WJuYU" testid="author" role="button">+91 83687 70456</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:28 pm, 26/02/2024] +91 83687 70456: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span><strong class="_11JPr selectable-text copyable-text" data-app-text-template="*${appText}*">Hackathon style question</strong>
I've been working with Game designers. one of the challenge I see is that they don't know how to separate theme/setting of the game from the mechanics. I want to make a simple bot where they can input a whole theme of the game with the mechanics they're thinking. I should be able to isolate the mechanics part from it and show them similar game made on those mechanics.

For POC: I am thinking of creating a small database of games(name, link and description of mechanics) and create a chatbot where anyone can enter their prompt and I can show them game whose mechanics matches most of what they described.
If you were to hack this together in half a day, how will you do it?</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">2:28 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:28 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="reaction 👍"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_5F114B59F63D978773A1D915E059A03D_447505487797@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Ganaraj +44 7505 487797" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/357029048_327201076325969_1848146647852128456_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRSyKF7yeee8lWajqAEgBA6aceUAZCQopKkjl7QAXDNmQ&amp;oe=65EAD16A&amp;_nc_sid=e6ed6c&amp;_nc_cat=108" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d aqebjpab" role=""><span dir="auto" aria-label="Maybe Ganaraj" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Ganaraj</span><span class="WJuYU" testid="author" role="button">+44 7505 487797</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:28 pm, 26/02/2024] +44 7505 487797: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="exofpctk _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d a1q56urb" role=""><span dir="auto" aria-label="Maybe Ambika" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Ambika</span><span class="WJuYU" testid="author">+91 90086 39111</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">oh - did you make your own build or was it directly through the inference api stuff</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Directly, click, click, click launch</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">2:28 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:28 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A11D61C10B30D47A04C_919008639111@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Ambika +91 90086 39111" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno exofpctk"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">A</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d a1q56urb" role=""><span dir="auto" aria-label="Maybe Ambika" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Ambika</span><span class="WJuYU" testid="author" role="button">+91 90086 39111</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:29 pm, 26/02/2024] +91 90086 39111: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Accha, I think you can check more on the blogs instead of the docs  - occasionally they will highlight issues that people have faced</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">2:29 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:29 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_BB3CDA2A236EAB5877_919739978074@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Shashank Shekhar +91 97399 78074" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://media-tir3-1.cdn.whatsapp.net/v/t61.24694-24/187690209_1786246634880189_5326339996351315025_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdR1TtRpzvtA8RdENGAPHWRIVX8S-_Y9rPxLtNk-x1d2ow&amp;oe=65EAD50F&amp;_nc_sid=e6ed6c&amp;_nc_cat=103" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d c7ayxzn8" role=""><span dir="auto" aria-label="Maybe Shashank Shekhar" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Shashank Shekhar</span><span class="WJuYU" testid="author" role="button">+91 97399 78074</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:30 pm, 26/02/2024] +91 97399 78074: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Cohere performs slightly better than BGE with Open AI embeddings. For open source LLMs, Cohere performs significantly better (tried for Zephyr and instructor set of embeddings</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">2:30 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:30 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="reaction 🙏"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="🙏" draggable="false" class="bd96tnyg p83aruvr b93 emoji apple _11JPr" tabindex="-1" style="background-position: 0px -20px;"></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_0BBCDD9C2C0A31AB44007B268E0F9D3B_919665543333@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Palash +91 96655 43333" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno j3zl77sr"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">P</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d gvktkc8w" role=""><span dir="auto" aria-label="Maybe Palash" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Palash</span><span class="WJuYU" testid="author" role="button">+91 96655 43333</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:30 pm, 26/02/2024] +91 96655 43333: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="po56q9xr _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe Ankur Goel" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Ankur Goel</span><span class="WJuYU" testid="author">+91 83687 70456</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;"><strong class="_11JPr">Hackathon style question</strong>
I've been working with Game designers. one of the challenge I see is that they don't know how to separate theme/setting of the game from the mechanics. I want to make a simple bot where they can input a whole theme of the game with the mechanics they're thinking. I should be able to isolate the mechanics part from it and show them similar game made on those mechanics.

For POC: I am thinking of creating a small database of games(name, link and description of mechanics) and create a chatbot where anyone can enter their prompt and I can show them game whose mechanics matches most of what they described.
If you were to hack this together in half a day, how will you do it?</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Create a custom GPT and open it up for usage?
The generation of the list also can happen with GPT 4 itself</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">2:30 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:30 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_59AC2821D38E1413C6C8F57CE8D28D67_919334372044@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Sid +91 93343 72044" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno akr8p6hy"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">S</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d jake6gi8" role=""><span dir="auto" aria-label="Maybe Sid" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Sid</span><span class="WJuYU" testid="author" role="button">+91 93343 72044</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:30 pm, 26/02/2024] +91 93343 72044: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="d6kbw4o5 _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d c7ayxzn8" role=""><span dir="auto" aria-label="Maybe Shashank Shekhar" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Shashank Shekhar</span><span class="WJuYU" testid="author">+91 97399 78074</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Cohere performs slightly better than BGE with Open AI embeddings. For open source LLMs, Cohere performs significantly better (tried for Zephyr and instructor set of embeddings</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Any place to play around with this ? Seems promising</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">2:30 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:30 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB0E610111769BA77E45C_917737887058@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Nirant K +91 77378 87058" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno g9gonq0i"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">N</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author" role="button">+91 77378 87058</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:32 pm, 26/02/2024] +91 77378 87058: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Speaking of re-ranking and OSS embeddings:

Most OSS embeddings fine-tuned on in-domain data will beat most things off the shelf. I've seen 200 triplet fine-tuning samples do the trick in a case of good luck too.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5 hnx8ox4h mw4yctpw gj5xqxfh">Edited</span><span class="tvf2evcx oq44ahr5">2:32 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto"><span class="mw4yctpw gj5xqxfh">Edited</span>2:32 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_8F1A46C9CAEEE1386494766F1DE1F523_447505487797@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Ganaraj +44 7505 487797" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/357029048_327201076325969_1848146647852128456_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRSyKF7yeee8lWajqAEgBA6aceUAZCQopKkjl7QAXDNmQ&amp;oe=65EAD16A&amp;_nc_sid=e6ed6c&amp;_nc_cat=108" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d aqebjpab" role=""><span dir="auto" aria-label="Maybe Ganaraj" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Ganaraj</span><span class="WJuYU" testid="author" role="button">+44 7505 487797</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:33 pm, 26/02/2024] +44 7505 487797: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="d6kbw4o5 _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d c7ayxzn8" role=""><span dir="auto" aria-label="Maybe Shashank Shekhar" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Shashank Shekhar</span><span class="WJuYU" testid="author">+91 97399 78074</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Cohere performs slightly better than BGE with Open AI embeddings. For open source LLMs, Cohere performs significantly better (tried for Zephyr and instructor set of embeddings</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Are you talking about reranking ?</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">2:33 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:33 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_750B871B6BFD1ECF627BC96B75E44351_918368770456@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Ankur Goel +91 83687 70456" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno po56q9xr"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">A</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe Ankur Goel" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Ankur Goel</span><span class="WJuYU" testid="author" role="button">+91 83687 70456</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:34 pm, 26/02/2024] +91 83687 70456: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="j3zl77sr _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d gvktkc8w" role=""><span dir="auto" aria-label="Maybe Palash" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Palash</span><span class="WJuYU" testid="author">+91 96655 43333</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Create a custom GPT and open it up for usage?
The generation of the list also can happen with GPT 4 itself</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Thanks. Let me look into this.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">2:34 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:34 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB0D3955E1A86EE1893AC_917737887058@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Nirant K +91 77378 87058" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno g9gonq0i"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">N</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author" role="button">+91 77378 87058</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:34 pm, 26/02/2024] +91 77378 87058: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="po56q9xr _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe Ankur Goel" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Ankur Goel</span><span class="WJuYU" testid="author">+91 83687 70456</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;"><strong class="_11JPr">Hackathon style question</strong>
I've been working with Game designers. one of the challenge I see is that they don't know how to separate theme/setting of the game from the mechanics. I want to make a simple bot where they can input a whole theme of the game with the mechanics they're thinking. I should be able to isolate the mechanics part from it and show them similar game made on those mechanics.

For POC: I am thinking of creating a small database of games(name, link and description of mechanics) and create a chatbot where anyone can enter their prompt and I can show them game whose mechanics matches most of what they described.
If you were to hack this together in half a day, how will you do it?</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>If you can actually model this as a classical retrieval and retrieval augmented generation or a rag system whether retrieval or on the gameplay is built on a specific embedding which is aware that this is a game, a mechanic description or something else and you use that to retrieve and rank your results and then an LM which verifies these and connects the dot to the actual question and conversation so far.

Since I'm pressed for time I would probably use the perplexity online chat API and not even build my own database or use the open AIGPT 4 models with web to do this so that the verification part of this is quite fast in terms of dev time.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">2:34 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:34 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="Reactions 👍 3 in total"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk o2es7gts i2cterl7" style="flex-shrink: 0;"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 f8m0rgwh r15c9g6i ilf8vifs gfz4du6o r7fjleex qnz2jpws kjemk6od"><div class="dn5wc90o ilf8vifs gndfcl4n p357zi0d"><span class="f8jlpxt4 qfejxiq4 ce5kru2g">3</span></div></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_FD1A1A37F9DCB7E438EC48450AA8AC89_447505487797@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Ganaraj +44 7505 487797" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/357029048_327201076325969_1848146647852128456_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRSyKF7yeee8lWajqAEgBA6aceUAZCQopKkjl7QAXDNmQ&amp;oe=65EAD16A&amp;_nc_sid=e6ed6c&amp;_nc_cat=108" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d aqebjpab" role=""><span dir="auto" aria-label="Maybe Ganaraj" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Ganaraj</span><span class="WJuYU" testid="author" role="button">+44 7505 487797</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:36 pm, 26/02/2024] +44 7505 487797: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g9gonq0i _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author">+91 77378 87058</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Speaking of re-ranking and OSS embeddings:

Most OSS embeddings fine-tuned on in-domain data will beat most things off the shelf. I've seen 200 triplet fine-tuning samples do the trick in a case of good luck too.</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Is there any work on using matryoshka embeddings for these OSS? So that I can still use 256 embeddings instead of the huge ones that come with OSS?</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">2:36 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:36 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj _3sxvM" data-id="false_120363049558306142@g.us_3EB020436D0614A5BBBAE9_917737887058@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Nirant K +91 77378 87058" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno g9gonq0i"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">N</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author" role="button">+91 77378 87058</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:38 pm, 26/02/2024] +91 77378 87058: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="awij7w2s _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d aqebjpab" role=""><span dir="auto" aria-label="Maybe Ganaraj" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Ganaraj</span><span class="WJuYU" testid="author">+44 7505 487797</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Is there any work on using matryoshka embeddings for these OSS? So that I can still use 256 embeddings instead of the huge ones that come with OSS?</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>I believe Sentence Transformers and Jina are working on MRL models.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">2:38 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:38 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="Reactions 👍 2 in total"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍🏻" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -40px -60px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk o2es7gts i2cterl7" style="flex-shrink: 0;"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 f8m0rgwh r15c9g6i ilf8vifs gfz4du6o r7fjleex qnz2jpws kjemk6od"><div class="dn5wc90o ilf8vifs gndfcl4n p357zi0d"><span class="f8jlpxt4 qfejxiq4 ce5kru2g">2</span></div></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj _3sxvM" data-id="false_120363049558306142@g.us_3EB08F75A968144E307DA1_917737887058@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a"><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="copyable-text" data-pre-plain-text="[2:39 pm, 26/02/2024] +91 77378 87058: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Nomic has one too already? v1.5 english</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">2:39 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:39 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="reaction 👍"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB04C5AA47BA6F5BA5663_917737887058@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _27hEJ"><div class="_1BOF7 _2AOIt"><div class="cm280p3y ktbp76dp ocd2b0bc folpon7g aa0kojfi snweb893 g0rxnol2 jnl3jror copyable-text" data-pre-plain-text="[2:40 pm, 26/02/2024] +91 77378 87058: "><div><div class="bxcbqipq mhcwslh8 ocd2b0bc jfqm35v0"><div class="_3IzYj _6rIWC p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author" role="button">+91 77378 87058</span></div></div><div role="button" tabindex="0" aria-label="Open Picture" class="gndfcl4n l8fojup5 paxyh2gw sfeitywo cqsf3vkf p357zi0d ac2vgrno laorhtua gfz4du6o r7fjleex g0rxnol2" style="width: 330px; height: 187.917px;"><span class=""></span><div class="g0rxnol2 ln8gz9je ppled2lx"><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><img alt="https://huggingface.co/nomic-ai/nomic-embed-text-v1.5" src="data:image/jpeg;base64,/9j/2wCEAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDIBCQkJDAsMGA0NGDIhHCEyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMv/AABEIACkASAMBIgACEQEDEQH/xAGiAAABBQEBAQEBAQAAAAAAAAAAAQIDBAUGBwgJCgsQAAIBAwMCBAMFBQQEAAABfQECAwAEEQUSITFBBhNRYQcicRQygZGhCCNCscEVUtHwJDNicoIJChYXGBkaJSYnKCkqNDU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6g4SFhoeIiYqSk5SVlpeYmZqio6Slpqeoqaqys7S1tre4ubrCw8TFxsfIycrS09TV1tfY2drh4uPk5ebn6Onq8fLz9PX29/j5+gEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoLEQACAQIEBAMEBwUEBAABAncAAQIDEQQFITEGEkFRB2FxEyIygQgUQpGhscEJIzNS8BVictEKFiQ04SXxFxgZGiYnKCkqNTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqCg4SFhoeIiYqSk5SVlpeYmZqio6Slpqeoqaqys7S1tre4ubrCw8TFxsfIycrS09TV1tfY2dri4+Tl5ufo6ery8/T19vf4+fr/2gAMAwEAAhEDEQA/APHIItOazZ7i6u0uvMwI47VXQpgc7zIDnOeNv41JDBpLSgT3t+kewkslkjnfkYGDKOMZ5z2HBzxVjfbGYyX2MwYqGwDjPb15PP1q1DqVxA4eO5u0eJAluyTlTEA+7H0zk4GOTn2OhAiw6Vh915fAiVwgFmvMe07WP7zhi2AV5ABJ3HGCyeLTltUa3uruS4KpvjktVRAcHcAwkJIBxg4GcngY5lh1FgAtxLeSI20TKlzt3qgAjHIONuOM54wABinPqIRESza+hjw6urXW4MhLYUYUY+V2B653McDOKLAZpHAxnOOeKm8u32k+dLnAwPK77Tnv2bA9xk8dDoWurvbbXE2orPGXkjkivNmyVwQzj5SeQEB5yQp55GKDyhZpTa+bFExYBWfLbD2JAGffgZ9KAK+D6GjB9DU63FwsnmLNIH2GPcGOdpXaV+m3jHpxU13qmo36xreX9zcLGNqCaVnCjjgZPA4H5CgClg+hpcH0NXW1bU3RY21C6ZFjEQUzMQECsoXr0Cu4x6Mw7mqnzUlfqBJBdNbyb0SMnj/WRq44IPRgR1H4jI6E0txePdOHkSJSM/6qJIxyxbooA6k/QYA4AFTreWf9ifYpLNjcidpVuFdF4KqNrDYWIGCfvAcnjvVrRtZt9N1b7fc2bzPHGotzBIkLROpXa/KMrHapB3Kc7iTzzTuBl+edqrtXCsWHyjPbqccjjoff1NOa6doyhSPB25IjUHgYHOM/X16nJro7Hxm2nX+q3lrZrby3bK1uIYrZVg2nKAgwHIBVD8uzJXPXBFGDxdrFpIJLaSzgIuEucRWECr5iKVVsBMcAkgdATu680XAyzeO0YQpFgFTxEgPAwOcZ6dR3PJ55pjzGQgsqjAA+VQvQY7D2/HrWr/wlOof8++kf+Ce0/wDjVQ3viLUtQsLeyupYZILdAkQ+zRBlAVE+8F3E7Y0GSc4UUAZu/wBqN/tWpa+JNTsZIZLSS3geFERWitIlJCOkiliF+Yho0OWyeOvJy6+8TajqQl+1LYO8z+ZJKNOt1kZt24kuEDcnrzzyDwaLgZO/2o3+360s0rTzyTOEDSMWIRAigk54UAAD2AwKZRcDUsJkENrFL9pZDcP8sd+kIyQmDhlOz5ghLnghccbciWFyTDKzzmdL2OMSDU41IRRhQMjIIAwJc7QMDFY1FAGreSTRWSW1xPLIjOZGRL9JUMu0/PtXODhkGTydrDPZZH1SSzj+xx3GoJHE7mJItRDRphspjauDhgGJGN3BGODWNRQBaa9kaYP5tzgOZFzNlg5xubOOpx1+nXFSx6nJsitZ572TT0Qp9mW5KjBIZgMggAuobG09B1IzVCigC7Fql1C80iXV4ssjLJvS4IJkDBt7f3jkZHQg4OeOXSX0E8EMUy30gjl3kNdgrggeZtBT5WZhndzxgEHGTQooAlnaBnzbxyRpzxJIHPU45AHbA6dQTxnAioooA//Z" class="jciay5ix tvf2evcx oq44ahr5 lb5m6g5c fsmudgz7" style="height: 100%;"></div><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><img alt="https://huggingface.co/nomic-ai/nomic-embed-text-v1.5" src="blob:https://web.whatsapp.com/c8ad0564-d86b-45f5-806f-6ab1564cfb67" class="jciay5ix tvf2evcx oq44ahr5 lb5m6g5c osz0hll6 nq7eualt em5jvqoa a21kwdn3" style="height: 100%;"></div></div></div><div class="cm280p3y bkoknyjm esbfogcw jfqm35v0 bxcbqipq mhcwslh8"><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span><a dir="auto" href="https://huggingface.co/nomic-ai/nomic-embed-text-v1.5" title="https://huggingface.co/nomic-ai/nomic-embed-text-v1.5" target="_blank" rel="noopener noreferrer" class="_11JPr selectable-text copyable-text">https://huggingface.co/nomic-ai/nomic-embed-text-v1.5</a></span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">2:40 pm</span></span></span></div></div><div class="lrw9n60e lhggkp7q fz4q5utg b9fczbqn"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:40 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 cvsrm5e1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="reaction 👍"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A1D160F92B0234EEDF9_919739978074@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Shashank Shekhar +91 97399 78074" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://media-tir3-1.cdn.whatsapp.net/v/t61.24694-24/187690209_1786246634880189_5326339996351315025_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdR1TtRpzvtA8RdENGAPHWRIVX8S-_Y9rPxLtNk-x1d2ow&amp;oe=65EAD50F&amp;_nc_sid=e6ed6c&amp;_nc_cat=103" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d c7ayxzn8" role=""><span dir="auto" aria-label="Maybe Shashank Shekhar" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Shashank Shekhar</span><span class="WJuYU" testid="author" role="button">+91 97399 78074</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:44 pm, 26/02/2024] +91 97399 78074: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="awij7w2s _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d aqebjpab" role=""><span dir="auto" aria-label="Maybe Ganaraj" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Ganaraj</span><span class="WJuYU" testid="author">+44 7505 487797</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Are you talking about reranking ?</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Yes. On both MRR and Hit rate Cohere reranker outperforms BGE</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">2:44 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:44 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_243F0A71A604885B10B54A55F51B2B9E_447505487797@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Ganaraj +44 7505 487797" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/357029048_327201076325969_1848146647852128456_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRSyKF7yeee8lWajqAEgBA6aceUAZCQopKkjl7QAXDNmQ&amp;oe=65EAD16A&amp;_nc_sid=e6ed6c&amp;_nc_cat=108" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d aqebjpab" role=""><span dir="auto" aria-label="Maybe Ganaraj" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Ganaraj</span><span class="WJuYU" testid="author" role="button">+44 7505 487797</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:47 pm, 26/02/2024] +44 7505 487797: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="d6kbw4o5 _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d c7ayxzn8" role=""><span dir="auto" aria-label="Maybe Shashank Shekhar" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Shashank Shekhar</span><span class="WJuYU" testid="author">+91 97399 78074</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Yes. On both MRR and Hit rate Cohere reranker outperforms BGE</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>I agree on this. What I'm confused is about why/how open ai embedding comes into picture here ? I was under the impression that rerankers work on raw text ?</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">2:47 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:47 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_04A3BA442586911953_919739978074@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Shashank Shekhar +91 97399 78074" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://media-tir3-1.cdn.whatsapp.net/v/t61.24694-24/187690209_1786246634880189_5326339996351315025_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdR1TtRpzvtA8RdENGAPHWRIVX8S-_Y9rPxLtNk-x1d2ow&amp;oe=65EAD50F&amp;_nc_sid=e6ed6c&amp;_nc_cat=103" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d c7ayxzn8" role=""><span dir="auto" aria-label="Maybe Shashank Shekhar" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Shashank Shekhar</span><span class="WJuYU" testid="author" role="button">+91 97399 78074</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:48 pm, 26/02/2024] +91 97399 78074: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span><a dir="auto" href="https://platform.openai.com/docs/guides/embeddings" title="https://platform.openai.com/docs/guides/embeddings" target="_blank" rel="noopener noreferrer" class="_11JPr selectable-text copyable-text">https://platform.openai.com/docs/guides/embeddings</a></span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">2:48 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:48 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB07A7B62B838CDFC4835_918368770456@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Ankur Goel +91 83687 70456" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno po56q9xr"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">A</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe Ankur Goel" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Ankur Goel</span><span class="WJuYU" testid="author" role="button">+91 83687 70456</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:49 pm, 26/02/2024] +91 83687 70456: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g9gonq0i _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author">+91 77378 87058</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">If you can actually model this as a classical retrieval and retrieval augmented generation or a rag system whether retrieval or on the gameplay is built on a specific embedding which is aware that this is a game, a mechanic description or something else and you use that to retrieve and rank your results and then an LM which verifies these and connects the dot to the actual question and conversation so far.

Since I'm pressed for time I would probably use the perplexity online chat API and not even build my own database or use the open AIGPT 4 models with web to do this so that the verification part of this is quite fast in terms of dev time.</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Perplexity looks like it'll be a faster solution for a basic POC. Thanks a bunch.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">2:49 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:49 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A352BA87332548C2903_919354493228@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Azhan Mohammed +91 93544 93228" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://media-tir3-1.cdn.whatsapp.net/v/t61.24694-24/415823620_588200936799230_8959200103469063155_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdR_pTfPCMFAxFn-TVmoUaWYSLvgE1ORXxiXANZ9ST0xVg&amp;oe=65EAF835&amp;_nc_sid=e6ed6c&amp;_nc_cat=100" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe Azhan Mohammed" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Azhan Mohammed</span><span class="WJuYU" testid="author" role="button">+91 93544 93228</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[3:04 pm, 26/02/2024] +91 93544 93228: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g9gonq0i _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author">+91 77378 87058</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Cohere and BGE are both decent in my experience — work from <span class=""><span dir="auto" data-jid="919550164716@c.us" data-display="~Ravi Theja" aria-label="Maybe Ravi Theja" class="_11JPr">@<span dir="ltr">~Ravi Theja</span></span></span> indicates Cohere was good.</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>What about RankGPT</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">3:04 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">3:04 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A7F5A9CEB8590BAD288_919785225224@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Ritz +91 97852 25224" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno ro7tsnc9"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">R</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d pfvzdcj5" role=""><span dir="auto" aria-label="Maybe Ritz" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Ritz</span><span class="WJuYU" testid="author" role="button">+91 97852 25224</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[3:25 pm, 26/02/2024] +91 97852 25224: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>LLM from Ola Founder is out.
<a dir="auto" href="https://chat.olakrutrim.com/home" title="https://chat.olakrutrim.com/home" target="_blank" rel="noopener noreferrer" class="_11JPr selectable-text copyable-text">https://chat.olakrutrim.com/home</a></span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">3:25 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">3:25 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB01819486AD929C13D98_919616406460@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe AHM - TokenBender +91 96164 06460" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://media-tir3-1.cdn.whatsapp.net/v/t61.24694-24/371309693_821316576353465_3869448251820597856_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdSpPJ_gWW16OQ7OQi2--mfrgwkSUE74Bv9kWbSYiqldWw&amp;oe=65EADA7D&amp;_nc_sid=e6ed6c&amp;_nc_cat=103" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d oyzge2kb" role=""><span dir="auto" aria-label="Maybe AHM - TokenBender" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">AHM - TokenBender</span><span class="WJuYU" testid="author" role="button">+91 96164 06460</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[3:34 pm, 26/02/2024] +91 96164 06460: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g9gonq0i _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author">+91 77378 87058</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">I believe Sentence Transformers and Jina are working on MRL models.</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="M6sU5"><div class="xzT4z gndfcl4n l8fojup5 paxyh2gw sfeitywo cqsf3vkf k2bacm8l ajgl1lbb p357zi0d sap93d0t ovllcyds kvdvyush gfz4du6o r7fjleex rrq4r3yd"><div class="k5z04t9s cm280p3y p357zi0d ggj6brxn m0h2a7mj lb5m6g5c f8m0rgwh ac2vgrno idwf4z32 onmzirdk gfz4du6o r7fjleex n1yiu2zv itegkywt eb4rp10x rppts313 ln8gz9je"><div class="aoi073rw suguakab jjtagnqy c32ccnay tvf2evcx m0h2a7mj lb5m6g5c j4enbv94 gfz4du6o r7fjleex lhj4utae ljrqcn24" title="x.com"><span dir="ltr" aria-label="" class="ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae i033jvx7 cw3vfol9 _11JPr" style="min-height: 0px;">x.com</span></div><div class="jjtagnqy tvf2evcx ttu8nud2 lb5m6g5c dsh4tgtl gfz4du6o r7fjleex lhj4utae"><span class="g0rxnol2 gfz4du6o r7fjleex" style="display: -webkit-box; overflow: hidden; -webkit-box-orient: vertical; -webkit-line-clamp: 2;"><span dir="ltr" title="https://x.com/tomaarsen/status/1761020783336235186?s=20" aria-label="" class="i033jvx7 _11JPr" style="min-height: 0px;">https://x.com/tomaarsen/status/1761020783336235186?s=20</span></span></div><div class="j8h4s3v0 tvf2evcx oq44ahr5 lb5m6g5c dsh4tgtl qbqilfqo"><span dir="ltr" aria-label="" class="_11JPr" style="min-height: 0px;">x.com</span></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>yes, i saw this from sentence transformers for MRL
<a dir="auto" href="https://x.com/tomaarsen/status/1761020783336235186?s=20" title="https://x.com/tomaarsen/status/1761020783336235186?s=20" target="_blank" rel="noopener noreferrer" class="_11JPr selectable-text copyable-text">https://x.com/tomaarsen/status/1761020783336235186?s=20</a></span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">3:34 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">3:34 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_15B0A0661525EB2CE7491BF433F6189C_919555579357@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Deepak +91 95555 79357" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://media-tir3-1.cdn.whatsapp.net/v/t61.24694-24/390069960_349915594273961_1178973845831250855_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRHhywQpofpquX0pIrHN_Y12ndtS2bMnZa-MtQI4H79zg&amp;oe=65EAE91E&amp;_nc_sid=e6ed6c&amp;_nc_cat=105" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d lr20dbn2" role=""><span dir="auto" aria-label="Maybe Deepak" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Deepak</span><span class="WJuYU" testid="author" role="button">+91 95555 79357</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[3:37 pm, 26/02/2024] +91 95555 79357: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>For people who have built copilots, how are you measuring user experience and copilot efficacy in production?</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5 hnx8ox4h mw4yctpw gj5xqxfh">Edited</span><span class="tvf2evcx oq44ahr5">3:37 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto"><span class="mw4yctpw gj5xqxfh">Edited</span>3:37 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_2F8A578CFF882DC6AD237E36634DE794_919999503196@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _27hEJ _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe undefined +91 99995 03196" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="lhggkp7q qq0sjtgm tkdu00h0 p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><span data-icon="default-user-color" class="aqebjpab"><svg viewBox="0 0 212 212" height="212" width="212" preserveAspectRatio="xMidYMid meet" class="ln8gz9je ppled2lx" version="1.1" x="0px" y="0px" enable-background="new 0 0 212 212"><title>default-user-color</title><path fill="currentColor" class="background" d="M106.251,0.5C164.653,0.5,212,47.846,212,106.25S164.653,212,106.25,212C47.846,212,0.5,164.654,0.5,106.25 S47.846,0.5,106.251,0.5z"></path><g><path fill="#FFFFFF" class="primary" d="M173.561,171.615c-0.601-0.915-1.287-1.907-2.065-2.955c-0.777-1.049-1.645-2.155-2.608-3.299 c-0.964-1.144-2.024-2.326-3.184-3.527c-1.741-1.802-3.71-3.646-5.924-5.47c-2.952-2.431-6.339-4.824-10.204-7.026 c-1.877-1.07-3.873-2.092-5.98-3.055c-0.062-0.028-0.118-0.059-0.18-0.087c-9.792-4.44-22.106-7.529-37.416-7.529 s-27.624,3.089-37.416,7.529c-0.338,0.153-0.653,0.318-0.985,0.474c-1.431,0.674-2.806,1.376-4.128,2.101 c-0.716,0.393-1.417,0.792-2.101,1.197c-3.421,2.027-6.475,4.191-9.15,6.395c-2.213,1.823-4.182,3.668-5.924,5.47 c-1.161,1.201-2.22,2.384-3.184,3.527c-0.964,1.144-1.832,2.25-2.609,3.299c-0.778,1.049-1.464,2.04-2.065,2.955 c-0.557,0.848-1.033,1.622-1.447,2.324c-0.033,0.056-0.073,0.119-0.104,0.174c-0.435,0.744-0.79,1.392-1.07,1.926 c-0.559,1.068-0.818,1.678-0.818,1.678v0.398c18.285,17.927,43.322,28.985,70.945,28.985c27.678,0,52.761-11.103,71.055-29.095 v-0.289c0,0-0.619-1.45-1.992-3.778C174.594,173.238,174.117,172.463,173.561,171.615z"></path><path fill="#FFFFFF" class="primary" d="M106.002,125.5c2.645,0,5.212-0.253,7.68-0.737c1.234-0.242,2.443-0.542,3.624-0.896 c1.772-0.532,3.482-1.188,5.12-1.958c2.184-1.027,4.242-2.258,6.15-3.67c2.863-2.119,5.39-4.646,7.509-7.509 c0.706-0.954,1.367-1.945,1.98-2.971c0.919-1.539,1.729-3.155,2.422-4.84c0.462-1.123,0.872-2.277,1.226-3.458 c0.177-0.591,0.341-1.188,0.49-1.792c0.299-1.208,0.542-2.443,0.725-3.701c0.275-1.887,0.417-3.827,0.417-5.811 c0-1.984-0.142-3.925-0.417-5.811c-0.184-1.258-0.426-2.493-0.725-3.701c-0.15-0.604-0.313-1.202-0.49-1.793 c-0.354-1.181-0.764-2.335-1.226-3.458c-0.693-1.685-1.504-3.301-2.422-4.84c-0.613-1.026-1.274-2.017-1.98-2.971 c-2.119-2.863-4.646-5.39-7.509-7.509c-1.909-1.412-3.966-2.643-6.15-3.67c-1.638-0.77-3.348-1.426-5.12-1.958 c-1.181-0.355-2.39-0.655-3.624-0.896c-2.468-0.484-5.035-0.737-7.68-0.737c-21.162,0-37.345,16.183-37.345,37.345 C68.657,109.317,84.84,125.5,106.002,125.5z"></path></g></svg></span></div></div><div class="_1BOF7 _2AOIt"><div class="cm280p3y eu4mztcy ocd2b0bc folpon7g aa0kojfi snweb893 g0rxnol2 jnl3jror copyable-text" data-pre-plain-text="[3:38 pm, 26/02/2024] +91 99995 03196: "><div><div class="bxcbqipq mhcwslh8 ocd2b0bc jfqm35v0"><div class="_3IzYj _6rIWC p357zi0d aqebjpab" role=""><span class="WJuYU" testid="author" role="button">+91 99995 03196</span></div></div><div role="button" tabindex="0" aria-label="Open Picture" class="gndfcl4n l8fojup5 paxyh2gw sfeitywo cqsf3vkf p357zi0d ac2vgrno laorhtua gfz4du6o r7fjleex g0rxnol2" style="width: 240px; height: 338.028px;"><span class=""></span><div class="g0rxnol2 ln8gz9je ppled2lx"><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><img alt="I remember character consistency discussion on this channel earlier with Dall-E.. This genid thingy seem to work for me to a large extent.  Not sure if it's noob finding for this grp 🙈, thought of sharing regardless" src="data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEABsbGxscGx4hIR4qLSgtKj04MzM4PV1CR0JHQl2NWGdYWGdYjX2Xe3N7l33gsJycsOD/2c7Z//////////////8BGxsbGxwbHiEhHiotKC0qPTgzMzg9XUJHQkdCXY1YZ1hYZ1iNfZd7c3uXfeCwnJyw4P/Zztn////////////////CABEIAEgADgMBIgACEQEDEQH/xAAsAAADAQEBAAAAAAAAAAAAAAAAAgMEAQUBAQEAAAAAAAAAAAAAAAAAAAAB/9oADAMBAAIQAxAAAADxk1yHIcW09MZEXQVLgH//xAAfEAACAgICAwEAAAAAAAAAAAABAgADETESISJBQlH/2gAIAQEAAT8AVOtTj1KlITmdZ1LchiAuBFdSiqTqXOjKuNzg2AYVIGYEHAGOhXcRFFYfPctcEieQA6h3qVAFQfyXOpCr9ezEsCLiWOHM/8QAFhEBAQEAAAAAAAAAAAAAAAAAERAA/9oACAECAQE/AGMN/8QAFxEBAQEBAAAAAAAAAAAAAAAAEQABEP/aAAgBAwEBPwCdCNHjf//Z" class="jciay5ix tvf2evcx oq44ahr5 lb5m6g5c fsmudgz7" style="width: 100%;"></div><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><img alt="I remember character consistency discussion on this channel earlier with Dall-E.. This genid thingy seem to work for me to a large extent.  Not sure if it's noob finding for this grp 🙈, thought of sharing regardless" src="blob:https://web.whatsapp.com/972db10a-b89f-4b89-8fcf-fe53f9cd4333" class="jciay5ix tvf2evcx oq44ahr5 lb5m6g5c osz0hll6 nq7eualt em5jvqoa a21kwdn3" style="width: 100%;"></div></div><span data-icon="ic-hd-label-small" class="lhggkp7q s7ynmu90 hydhjofj qtm54r9o"><svg viewBox="0 0 20 20" height="20" width="20" preserveAspectRatio="xMidYMid meet" class="kvstyow8" fill="none"><title>ic-hd-label-small</title><g opacity="0.9"><path fill-rule="evenodd" clip-rule="evenodd" d="M5.19997 3.6001C3.43267 3.6001 2 5.03277 2 6.80007V13.2C2 14.9673 3.43267 16.4 5.19997 16.4H14.7999C16.5672 16.4 17.9998 14.9673 17.9998 13.2V6.80007C17.9998 5.03277 16.5672 3.6001 14.7999 3.6001H5.19997ZM9.70171 13.2H8.40983V10.4616H6.49626V13.2H5.19997V6.80007H6.49626V9.38905H8.40983V6.80007H9.70171V13.2ZM11.0288 13.2V6.80007H12.7263C13.4759 6.80007 14.0726 7.03743 14.5164 7.51215C14.9632 7.98687 15.191 8.63741 15.1999 9.46377V10.5011C15.1999 11.3421 14.9765 12.0029 14.5297 12.4835C14.0858 12.9612 13.4729 13.2 12.6911 13.2H11.0288ZM12.3999 7.87698V12.1275H12.7131C13.1452 12.1275 13.4494 12.0147 13.6258 11.789C13.8022 11.5605 13.8948 11.1678 13.9036 10.611V9.49894C13.9036 8.90114 13.8198 8.48503 13.6523 8.2506C13.4847 8.01324 13.1996 7.8887 12.7969 7.87698H12.3999Z" fill="currentColor"></path></g></svg></span></div><div class="cm280p3y czh1irh3 esbfogcw jfqm35v0 bxcbqipq mhcwslh8"><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>I remember character consistency discussion on this channel earlier with Dall-E.. This genid thingy seem to work for me to a large extent.  Not sure if it's noob finding for this grp <img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="🙈" draggable="false" class="b90 emoji apple _11JPr selectable-text copyable-text" data-plain-text="🙈" tabindex="-1" style="background-position: -40px -60px;">, thought of sharing regardless</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">3:38 pm</span></span></span></div></div><div class="lrw9n60e lhggkp7q fz4q5utg b9fczbqn"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">3:38 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 cvsrm5e1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="Reactions ❤, 👍 7 in total"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="❤️" draggable="false" class="bd96tnyg eipqoq80 b8 emoji apple _11JPr" tabindex="-1" style="background-position: -20px 0px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk o2es7gts i2cterl7" style="flex-shrink: 0;"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 f8m0rgwh r15c9g6i ilf8vifs gfz4du6o r7fjleex qnz2jpws kjemk6od"><div class="dn5wc90o ilf8vifs gndfcl4n p357zi0d"><span class="f8jlpxt4 qfejxiq4 ce5kru2g">7</span></div></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB0A7C3184DF984DE2EE4_919667857738@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Anagh +91 96678 57738" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://media-tir3-1.cdn.whatsapp.net/v/t61.24694-24/425178224_917925546602882_1166822420051696924_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdT_Zishe9A61aqHRVHdwfoGCj_dCe95elk0_3Gzx38flg&amp;oe=65EAF206&amp;_nc_sid=e6ed6c&amp;_nc_cat=103" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d dud7p2xx" role=""><span dir="auto" aria-label="Maybe Anagh" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Anagh</span><span class="WJuYU" testid="author" role="button">+91 96678 57738</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[3:43 pm, 26/02/2024] +91 96678 57738: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="awij7w2s _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d aqebjpab" role=""><span class="WJuYU" testid="author">+91 99995 03196</span></div><div class="_37DQv" dir="ltr" role="button"><div class="l7jjieqr tvf2evcx oq44ahr5 lb5m6g5c g97rt3p3 fewfhwl7 jdwybkuq kjemk6od"><span data-icon="status-image" class=""><svg viewBox="0 0 16 20" height="20" width="16" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 16 20"><title>status-image</title><path fill="currentColor" d="M13.822,4.668H7.14l-1.068-1.09C5.922,3.425,5.624,3.3,5.409,3.3H3.531 c-0.214,0-0.51,0.128-0.656,0.285L1.276,5.296C1.13,5.453,1.01,5.756,1.01,5.971v1.06c0,0.001-0.001,0.002-0.001,0.003v6.983 c0,0.646,0.524,1.17,1.17,1.17h11.643c0.646,0,1.17-0.524,1.17-1.17v-8.18C14.992,5.191,14.468,4.668,13.822,4.668z M7.84,13.298 c-1.875,0-3.395-1.52-3.395-3.396c0-1.875,1.52-3.395,3.395-3.395s3.396,1.52,3.396,3.395C11.236,11.778,9.716,13.298,7.84,13.298z  M7.84,7.511c-1.321,0-2.392,1.071-2.392,2.392s1.071,2.392,2.392,2.392s2.392-1.071,2.392-2.392S9.161,7.511,7.84,7.511z"></path></svg></span></div><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">I remember character consistency discussion on this channel earlier with Dall-E.. This genid thingy seem to work for me to a large extent.  Not sure if it's noob finding for this grp <img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="🙈" draggable="false" class="b90 emoji apple _11JPr" tabindex="-1" style="background-position: -40px -60px;">, thought of sharing regardless</span></div></div></div><div class="jrEtJ"><div class="kknmh"><div class="g0rxnol2 ln8gz9je ppled2lx"><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><div class="lhggkp7q ln8gz9je ppled2lx t3g6t33p qnwaluaf" style="background-image: url(&quot;data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEABsbGxscGx4hIR4qLSgtKj04MzM4PV1CR0JHQl2NWGdYWGdYjX2Xe3N7l33gsJycsOD/2c7Z//////////////8BGxsbGxwbHiEhHiotKC0qPTgzMzg9XUJHQkdCXY1YZ1hYZ1iNfZd7c3uXfeCwnJyw4P/Zztn////////////////CABEIAEgADgMBIgACEQEDEQH/xAAsAAADAQEBAAAAAAAAAAAAAAAAAgMEAQUBAQEAAAAAAAAAAAAAAAAAAAAB/9oADAMBAAIQAxAAAADxk1yHIcW09MZEXQVLgH//xAAfEAACAgICAwEAAAAAAAAAAAABAgADETESISJBQlH/2gAIAQEAAT8AVOtTj1KlITmdZ1LchiAuBFdSiqTqXOjKuNzg2AYVIGYEHAGOhXcRFFYfPctcEieQA6h3qVAFQfyXOpCr9ezEsCLiWOHM/8QAFhEBAQEAAAAAAAAAAAAAAAAAERAA/9oACAECAQE/AGMN/8QAFxEBAQEBAAAAAAAAAAAAAAAAEQABEP/aAAgBAwEBPwCdCNHjf//Z&quot;);"></div></div><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><div class="lhggkp7q ln8gz9je ppled2lx t3g6t33p qnwaluaf" style="background-image: url(&quot;blob:https://web.whatsapp.com/972db10a-b89f-4b89-8fcf-fe53f9cd4333&quot;);"></div></div></div></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Interesting</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">3:43 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">3:43 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A323E0923855E4F0EDC_6590690114@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Nitin Mahajan +65 9069 0114" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno fz47docv"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">N</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d swew6qod" role=""><span dir="auto" aria-label="Maybe Nitin Mahajan" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Nitin Mahajan</span><span class="WJuYU" testid="author" role="button">+65 9069 0114</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[3:43 pm, 26/02/2024] +65 9069 0114: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="awij7w2s _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d aqebjpab" role=""><span class="WJuYU" testid="author">+91 99995 03196</span></div><div class="_37DQv" dir="ltr" role="button"><div class="l7jjieqr tvf2evcx oq44ahr5 lb5m6g5c g97rt3p3 fewfhwl7 jdwybkuq kjemk6od"><span data-icon="status-image" class=""><svg viewBox="0 0 16 20" height="20" width="16" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 16 20"><title>status-image</title><path fill="currentColor" d="M13.822,4.668H7.14l-1.068-1.09C5.922,3.425,5.624,3.3,5.409,3.3H3.531 c-0.214,0-0.51,0.128-0.656,0.285L1.276,5.296C1.13,5.453,1.01,5.756,1.01,5.971v1.06c0,0.001-0.001,0.002-0.001,0.003v6.983 c0,0.646,0.524,1.17,1.17,1.17h11.643c0.646,0,1.17-0.524,1.17-1.17v-8.18C14.992,5.191,14.468,4.668,13.822,4.668z M7.84,13.298 c-1.875,0-3.395-1.52-3.395-3.396c0-1.875,1.52-3.395,3.395-3.395s3.396,1.52,3.396,3.395C11.236,11.778,9.716,13.298,7.84,13.298z  M7.84,7.511c-1.321,0-2.392,1.071-2.392,2.392s1.071,2.392,2.392,2.392s2.392-1.071,2.392-2.392S9.161,7.511,7.84,7.511z"></path></svg></span></div><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">I remember character consistency discussion on this channel earlier with Dall-E.. This genid thingy seem to work for me to a large extent.  Not sure if it's noob finding for this grp <img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="🙈" draggable="false" class="b90 emoji apple _11JPr" tabindex="-1" style="background-position: -40px -60px;">, thought of sharing regardless</span></div></div></div><div class="jrEtJ"><div class="kknmh"><div class="g0rxnol2 ln8gz9je ppled2lx"><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><div class="lhggkp7q ln8gz9je ppled2lx t3g6t33p qnwaluaf" style="background-image: url(&quot;data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEABsbGxscGx4hIR4qLSgtKj04MzM4PV1CR0JHQl2NWGdYWGdYjX2Xe3N7l33gsJycsOD/2c7Z//////////////8BGxsbGxwbHiEhHiotKC0qPTgzMzg9XUJHQkdCXY1YZ1hYZ1iNfZd7c3uXfeCwnJyw4P/Zztn////////////////CABEIAEgADgMBIgACEQEDEQH/xAAsAAADAQEBAAAAAAAAAAAAAAAAAgMEAQUBAQEAAAAAAAAAAAAAAAAAAAAB/9oADAMBAAIQAxAAAADxk1yHIcW09MZEXQVLgH//xAAfEAACAgICAwEAAAAAAAAAAAABAgADETESISJBQlH/2gAIAQEAAT8AVOtTj1KlITmdZ1LchiAuBFdSiqTqXOjKuNzg2AYVIGYEHAGOhXcRFFYfPctcEieQA6h3qVAFQfyXOpCr9ezEsCLiWOHM/8QAFhEBAQEAAAAAAAAAAAAAAAAAERAA/9oACAECAQE/AGMN/8QAFxEBAQEBAAAAAAAAAAAAAAAAEQABEP/aAAgBAwEBPwCdCNHjf//Z&quot;);"></div></div><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><div class="lhggkp7q ln8gz9je ppled2lx t3g6t33p qnwaluaf" style="background-image: url(&quot;blob:https://web.whatsapp.com/972db10a-b89f-4b89-8fcf-fe53f9cd4333&quot;);"></div></div></div></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Has anyone found a way to do this on stable diffusion btw? Dalle is expensive and animated pictures only.

Apart from making Lora or dreambooths with images of a person. Is there any other way?</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">3:43 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">3:43 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB0D753B97697C1CA4340_447505487797@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Ganaraj +44 7505 487797" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/357029048_327201076325969_1848146647852128456_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRSyKF7yeee8lWajqAEgBA6aceUAZCQopKkjl7QAXDNmQ&amp;oe=65EAD16A&amp;_nc_sid=e6ed6c&amp;_nc_cat=108" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d aqebjpab" role=""><span dir="auto" aria-label="Maybe Ganaraj" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Ganaraj</span><span class="WJuYU" testid="author" role="button">+44 7505 487797</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[3:43 pm, 26/02/2024] +44 7505 487797: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="awij7w2s _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d aqebjpab" role=""><span class="WJuYU" testid="author">+91 99995 03196</span></div><div class="_37DQv" dir="ltr" role="button"><div class="l7jjieqr tvf2evcx oq44ahr5 lb5m6g5c g97rt3p3 fewfhwl7 jdwybkuq kjemk6od"><span data-icon="status-image" class=""><svg viewBox="0 0 16 20" height="20" width="16" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 16 20"><title>status-image</title><path fill="currentColor" d="M13.822,4.668H7.14l-1.068-1.09C5.922,3.425,5.624,3.3,5.409,3.3H3.531 c-0.214,0-0.51,0.128-0.656,0.285L1.276,5.296C1.13,5.453,1.01,5.756,1.01,5.971v1.06c0,0.001-0.001,0.002-0.001,0.003v6.983 c0,0.646,0.524,1.17,1.17,1.17h11.643c0.646,0,1.17-0.524,1.17-1.17v-8.18C14.992,5.191,14.468,4.668,13.822,4.668z M7.84,13.298 c-1.875,0-3.395-1.52-3.395-3.396c0-1.875,1.52-3.395,3.395-3.395s3.396,1.52,3.396,3.395C11.236,11.778,9.716,13.298,7.84,13.298z  M7.84,7.511c-1.321,0-2.392,1.071-2.392,2.392s1.071,2.392,2.392,2.392s2.392-1.071,2.392-2.392S9.161,7.511,7.84,7.511z"></path></svg></span></div><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">I remember character consistency discussion on this channel earlier with Dall-E.. This genid thingy seem to work for me to a large extent.  Not sure if it's noob finding for this grp <img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="🙈" draggable="false" class="b90 emoji apple _11JPr" tabindex="-1" style="background-position: -40px -60px;">, thought of sharing regardless</span></div></div></div><div class="jrEtJ"><div class="kknmh"><div class="g0rxnol2 ln8gz9je ppled2lx"><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><div class="lhggkp7q ln8gz9je ppled2lx t3g6t33p qnwaluaf" style="background-image: url(&quot;data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEABsbGxscGx4hIR4qLSgtKj04MzM4PV1CR0JHQl2NWGdYWGdYjX2Xe3N7l33gsJycsOD/2c7Z//////////////8BGxsbGxwbHiEhHiotKC0qPTgzMzg9XUJHQkdCXY1YZ1hYZ1iNfZd7c3uXfeCwnJyw4P/Zztn////////////////CABEIAEgADgMBIgACEQEDEQH/xAAsAAADAQEBAAAAAAAAAAAAAAAAAgMEAQUBAQEAAAAAAAAAAAAAAAAAAAAB/9oADAMBAAIQAxAAAADxk1yHIcW09MZEXQVLgH//xAAfEAACAgICAwEAAAAAAAAAAAABAgADETESISJBQlH/2gAIAQEAAT8AVOtTj1KlITmdZ1LchiAuBFdSiqTqXOjKuNzg2AYVIGYEHAGOhXcRFFYfPctcEieQA6h3qVAFQfyXOpCr9ezEsCLiWOHM/8QAFhEBAQEAAAAAAAAAAAAAAAAAERAA/9oACAECAQE/AGMN/8QAFxEBAQEBAAAAAAAAAAAAAAAAEQABEP/aAAgBAwEBPwCdCNHjf//Z&quot;);"></div></div><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><div class="lhggkp7q ln8gz9je ppled2lx t3g6t33p qnwaluaf" style="background-image: url(&quot;blob:https://web.whatsapp.com/972db10a-b89f-4b89-8fcf-fe53f9cd4333&quot;);"></div></div></div></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Not at all. I for one atleast definitely did not know that you could do this.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">3:43 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">3:43 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="Reactions ➕, 👍 2 in total"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="➕" draggable="false" class="bd96tnyg p83aruvr b8 emoji apple _11JPr" tabindex="-1" style="background-position: -80px 0px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍🏼" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -60px -60px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk o2es7gts i2cterl7" style="flex-shrink: 0;"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 f8m0rgwh r15c9g6i ilf8vifs gfz4du6o r7fjleex qnz2jpws kjemk6od"><div class="dn5wc90o ilf8vifs gndfcl4n p357zi0d"><span class="f8jlpxt4 qfejxiq4 ce5kru2g">2</span></div></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_A8E7EFC9A1450B84C2_919953076613@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Divya Tak +91 99530 76613" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://media-tir3-1.cdn.whatsapp.net/v/t61.24694-24/410492006_324390497176271_5353027943618772658_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdSQ9ZtWws8-83EAQRI2uJioh6FA3x_B170fWcmooYVo_g&amp;oe=65EAF979&amp;_nc_sid=e6ed6c&amp;_nc_cat=103" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d mglqvgej" role=""><span dir="auto" aria-label="Maybe Divya Tak" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Divya Tak</span><span class="WJuYU" testid="author" role="button">+91 99530 76613</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[3:56 pm, 26/02/2024] +91 99530 76613: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="fz47docv _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d swew6qod" role=""><span dir="auto" aria-label="Maybe Nitin Mahajan" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Nitin Mahajan</span><span class="WJuYU" testid="author">+65 9069 0114</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Has anyone found a way to do this on stable diffusion btw? Dalle is expensive and animated pictures only.

Apart from making Lora or dreambooths with images of a person. Is there any other way?</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span><span role="button" tabindex="0" class=""><span dir="auto" data-jid="917407651462@c.us" data-display="~Soumyadeep Mukherjee" aria-label="Maybe Soumyadeep Mukherjee" class="ajgl1lbb o0rubyzf _11JPr selectable-text select-all copyable-text" data-plain-text="@~Soumyadeep Mukherjee" data-app-text-template="​917407651462@c.us​">@<span dir="ltr">~Soumyadeep Mukherjee</span></span></span> or other dashtoon folks might have insight</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">3:56 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">3:56 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_0552D09151AF9B02ED_919384207320@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _27hEJ _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Abhinand +91 93842 07320" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://media-tir3-1.cdn.whatsapp.net/v/t61.24694-24/370513816_130017406865075_2138891260654861262_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRoJrgxFGZ4v5B8oUsG5p-Tlf_3R0hfg3rO79jWpsXG8A&amp;oe=65EB00B2&amp;_nc_sid=e6ed6c&amp;_nc_cat=100" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div class="cm280p3y eu4mztcy ocd2b0bc folpon7g aa0kojfi snweb893 g0rxnol2 jnl3jror copyable-text" data-pre-plain-text="[4:11 pm, 26/02/2024] +91 93842 07320: "><div><div class="bxcbqipq mhcwslh8 ocd2b0bc jfqm35v0"><div class="_3IzYj _6rIWC p357zi0d sthcsby1" role=""><span dir="auto" aria-label="Maybe Abhinand" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Abhinand</span><span class="WJuYU" testid="author" role="button">+91 93842 07320</span></div></div><div role="button" tabindex="0" aria-label="Open Picture" class="gndfcl4n l8fojup5 paxyh2gw sfeitywo cqsf3vkf p357zi0d ac2vgrno laorhtua gfz4du6o r7fjleex g0rxnol2" style="width: 240px; height: 320.277px;"><span class=""></span><div class="g0rxnol2 ln8gz9je ppled2lx"><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><img alt="Guess I just broke the UI with this prompt, it also renders the HTML in model's response, clicking that button causes the UI to freeze and also the colors got dull. Its fine after refreshing though." src="data:image/jpeg;base64,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAMCAgICAgMCAgIDAwMDBAYEBAQEBAgGBgUGCQgKCgkICQkKDA8MCgsOCwkJDRENDg8QEBEQCgwSExIQEw8QEBD/2wBDAQMDAwQDBAgEBAgQCwkLEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBD/wAARCABkAEoDASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwD6M/4Ub8E/+iPeCP8Awn7T/wCN15xf+AvhhN4wvvD+j/DX4ZhtMkVm09/DOmS3d8ogimaCHF2jRylXbDTRKo3KSCoDP7frl9fabpNxfabpbahcQqGW2V9hcZG7BAJyFycAEnGAMmprPzbizguL2z+zXEsavND5gfynIBZdw4bByMjrilGquf2fW19n+e3y3IunLl6ni2q/CfwbLsl8OfBbwLLG8ZVlPhLTpzBMJCjK5+1xgkb0cqueIZRu3OiixD8EPC/2WWCb4W/Dc3aXLItwvgm1WF4jHhWEf2ktgSOrElgSsMihQWR69m8mP+7+tHkx/wB39a15x2PGLj4PeD2k32vwR8EJGJLdNj+EdPdiJDCHcMLkDbEBcF+MncmwSeWRKap8I/BcaXv9l/Afwc7RXUsdvnwrpr+ZELPckgzcx5U3B2/NsY4KlFX9/Xs/kx/3f1o8mP8Au/rRzhY8IufhVoAtVuLb9n3wh5klmWEB8IaUzxXAkH3j9uVSGTICq2MkMXGNhuwfBnwu2tWkM3wd8ALp8aRW13J/whliPOmCl5LqNxdFo4CF8pYmjeRZJEYlkDGvavJj/u/rR5Mf939aOcLHD2/wO+CrKd3wf8Enn/oX7T/43Uv/AAo34J/9Ee8Ef+E/af8Axuu2VVUbVGBS1DY7Fe+uIbaFZJ1uCpmijHkRyO25pFVciMEhckbiflC7ixCgmuOv9H8I6mba1uJvHKfZ/J06M299rkGfvorO8brv+4xaZyeqsz/MpPaXLMkLMpwRj+dYWm3XiU3Wqf2w1itsLsf2X9mZy5tfJiyZ9wAEnnefwuV2eXzndQhla90fw7Dp1jJPN4rMKCNofJvtVeb91HLKPNEbmQ5DOCJPvt5aNuYRqIY9T8MLLp9gsfjDctpFbwM1prBXy9qyqZZCuDJhAGeU+ZksjHLMp3vtVx/z0NH2q4/56GnYDB1TU/DF9cXFrdR+MFexmEjm1tNYiRmSRx8jxKFlXdI3yqWVlCkgqikalvZ6Rr8a3Eba5EIGkIEk99ZkmeIOcqxQuAs3GQREwKrsePC2vtVx/wA9DR9quP8AnoaVgJNQ0Wz1KBLe4mvkSOVplNvfzwMWYMCC0bqxXDnCk7QQpABVcNn0GxuL59Rkn1ESvAbYrHqVwkWwlzkRq4QP87fOF3cLz8q4b9quP+eho+1XH/PQ0WA0VUIoRc4UY5JJ/M8mnUUUgIbz/j3f8P5is2tK8/493/D+YrNqkAUUUUwCiiigAooooA2KKKKgCG8/493/AA/mKw7ePUll3XV3bSR/P8sduyNy2U5LnouQeOTyNv3a3Lz/AI93/D+YrNqkAUUUUwCiiigAooooA2KKKKgCK4RpIWVRknH86pfY7j/nn+oq5dNdrGrWUMMsnmRhlllMYEZcB2BCtlgm4hcYYgAlQdwz4bnxU1xKs+i6UkAtt0bpqcjO1xx+7ZTAAqdfnDE8D5OeHcCT7Hcf88/1FH2O4/55/qKbJceJgl4YdI0xnSMm0DajIolfzHAWQiA+WNgibID/ADO64IUM7prjxIutW8FvpOmvpLK32i6fUJEuIzt+UJAISrgngkyrgc89KLgH2O4/55/qKPsdx/zz/UVpUUXAzfsdx/zz/UUfY7j/AJ5/qK0qKLgFFFFICG8/493/AA/mK56OTXEaaNrO0dEuEWCRrtt0kBWMvI4EQCOGMoCAsCEQl1LkJ0U2SFjGMucZK5A4J6fhUccUMnKhx0PzRbev1FNAZ9LVyEW87FUWUEf34Cn/AKEoqX7JFnPf/dX/AAp3AzsNjdg4oq+NPtlma4WNRK6qjPsXLKpJUE46As2P94+tP+yx/wCVX/Ci4jNorS+yx/5Vf8KZNAkcZZeSOnyr/hRcLlmiqVpcXDTeVMv3txXgAjBxzgkc9RV2k1YE7jWVXG1hmmfZ1/56S/8Afxv8aKKVwaE+zL/z0l/7+N/jTWgVWQeZL8zY/wBY3oT60UUXYkkO+zL/AM9Jf+/jf40fZl/56S/9/G/xoop3YWD7Mv8Az0l/7+N/jQbWNhhnlIP/AE0b/Giii7CyFhtobfJiTBPGepqWiile40rH/9k=" class="jciay5ix tvf2evcx oq44ahr5 lb5m6g5c fsmudgz7" style="width: 100%;"></div><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><img alt="Guess I just broke the UI with this prompt, it also renders the HTML in model's response, clicking that button causes the UI to freeze and also the colors got dull. Its fine after refreshing though." src="blob:https://web.whatsapp.com/6f5003a0-bb65-4471-9a9e-49def60ef25a" class="jciay5ix tvf2evcx oq44ahr5 lb5m6g5c osz0hll6 nq7eualt em5jvqoa a21kwdn3" style="width: 100%;"></div></div></div><div class="cm280p3y czh1irh3 esbfogcw jfqm35v0 bxcbqipq mhcwslh8"><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Guess I just broke the UI with this prompt, it also renders the HTML in model's response, clicking that button causes the UI to freeze and also the colors got dull. Its fine after refreshing though.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">4:11 pm</span></span></span></div></div><div class="lrw9n60e lhggkp7q fz4q5utg b9fczbqn"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">4:11 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 cvsrm5e1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="Reactions 😂 4 in total"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="😂" draggable="false" class="bd96tnyg p83aruvr b85 emoji apple _11JPr" tabindex="-1" style="background-position: -60px -60px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk o2es7gts i2cterl7" style="flex-shrink: 0;"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 f8m0rgwh r15c9g6i ilf8vifs gfz4du6o r7fjleex qnz2jpws kjemk6od"><div class="dn5wc90o ilf8vifs gndfcl4n p357zi0d"><span class="f8jlpxt4 qfejxiq4 ce5kru2g">4</span></div></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj _3sxvM" data-id="false_120363049558306142@g.us_A6A9B6B74F2022A4F0446F38813A1B2A_919685138692@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Manoj +91 96851 38692" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno m8ez62k1"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">M</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d lr20dbn2" role=""><span dir="auto" aria-label="Maybe Manoj" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Manoj</span><span class="WJuYU" testid="author" role="button">+91 96851 38692</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[4:35 pm, 26/02/2024] +91 96851 38692: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="fz47docv _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d swew6qod" role=""><span dir="auto" aria-label="Maybe Nitin Mahajan" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Nitin Mahajan</span><span class="WJuYU" testid="author">+65 9069 0114</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Has anyone found a way to do this on stable diffusion btw? Dalle is expensive and animated pictures only.

Apart from making Lora or dreambooths with images of a person. Is there any other way?</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Checkout scenario_gg</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">4:35 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">4:35 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="reaction 👍"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_C1A4691AD8D5D3289202037A6CABBC4D_919685138692@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a"><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d lr20dbn2" role=""><span dir="auto" aria-label="Maybe Manoj" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Manoj</span><span class="WJuYU" testid="author" role="button">+91 96851 38692</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[4:36 pm, 26/02/2024] +91 96851 38692: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="m8ez62k1 _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d lr20dbn2" role=""><span dir="auto" aria-label="Maybe Manoj" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Manoj</span><span class="WJuYU" testid="author">+91 96851 38692</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Checkout scenario_gg</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>They have figured it out</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">4:36 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">4:36 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A751785891EE957FB65_6590690114@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Nitin Mahajan +65 9069 0114" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno fz47docv"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">N</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d swew6qod" role=""><span dir="auto" aria-label="Maybe Nitin Mahajan" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Nitin Mahajan</span><span class="WJuYU" testid="author" role="button">+65 9069 0114</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[4:38 pm, 26/02/2024] +65 9069 0114: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="m8ez62k1 _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d lr20dbn2" role=""><span dir="auto" aria-label="Maybe Manoj" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Manoj</span><span class="WJuYU" testid="author">+91 96851 38692</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Checkout scenario_gg</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="M6sU5"><div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span><a dir="auto" href="https://www.scenario.com/" title="https://www.scenario.com/" target="_blank" rel="noopener noreferrer" class="_11JPr selectable-text copyable-text">https://www.scenario.com/</a>

This one? Those are animated game characters?</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">4:38 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">4:38 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB0F572607A7DE8D1B5B6_917977314565@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Dhruv Anand +91 79773 14565" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno m8ez62k1"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">D</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d lr20dbn2" role=""><span dir="auto" aria-label="Maybe Dhruv Anand" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Dhruv Anand</span><span class="WJuYU" testid="author" role="button">+91 79773 14565</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[5:20 pm, 26/02/2024] +91 79773 14565: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Does anyone know of efficient models/libraries to do OCR on videos? Looking for something more sophisticated than keyframe extraction+tesseract. Something that detects animated text (text getting written out character-by-character/word-by-word), and processes the minimal number of frames to get the actual text.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">5:20 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">5:20 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3AFEE5C37AE10126E042_917407651462@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Soumyadeep Mukherjee +91 74076 51462" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/424242886_353057184272898_3175861170545036892_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdSAbzAZ6xOnG7Ln3kMzxPYrvQh8tij45kPm-x7s3Cl8ig&amp;oe=65EAFA5F&amp;_nc_sid=e6ed6c&amp;_nc_cat=103" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d a7s2a5ab" role=""><span dir="auto" aria-label="Maybe Soumyadeep Mukherjee" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Soumyadeep Mukherjee</span><span class="WJuYU" testid="author" role="button">+91 74076 51462</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[6:53 pm, 26/02/2024] +91 74076 51462: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="fz47docv _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d swew6qod" role=""><span dir="auto" aria-label="Maybe Nitin Mahajan" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Nitin Mahajan</span><span class="WJuYU" testid="author">+65 9069 0114</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Has anyone found a way to do this on stable diffusion btw? Dalle is expensive and animated pictures only.

Apart from making Lora or dreambooths with images of a person. Is there any other way?</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Scenario and Dashtoon both allow you to do this with a mix of Lora’s and IP adapters. Feel free to try on dashtoon, you can train with one image with dashtoon.

This genid thing is actually seed and prompt sharing. We also started with this over stable diffusion but it doesn’t work on as many characters we need <img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="😅" draggable="false" class="b85 emoji apple _11JPr selectable-text copyable-text" data-plain-text="😅" tabindex="-1" style="background-position: -20px -80px;"></span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5 hnx8ox4h mw4yctpw gj5xqxfh">Edited</span><span class="tvf2evcx oq44ahr5">6:53 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto"><span class="mw4yctpw gj5xqxfh">Edited</span>6:53 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="Reactions 👍, ❤ 2 in total"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍🏼" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -60px -60px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="❤️" draggable="false" class="bd96tnyg eipqoq80 b8 emoji apple _11JPr" tabindex="-1" style="background-position: -20px 0px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk o2es7gts i2cterl7" style="flex-shrink: 0;"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 f8m0rgwh r15c9g6i ilf8vifs gfz4du6o r7fjleex qnz2jpws kjemk6od"><div class="dn5wc90o ilf8vifs gndfcl4n p357zi0d"><span class="f8jlpxt4 qfejxiq4 ce5kru2g">2</span></div></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj _3sxvM" data-id="false_120363049558306142@g.us_7E75BAA323DE6527149B9A71BC363B61_918072173935@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Anubhav Mishra +91 80721 73935" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/422474244_2207779056257205_6380017147338436444_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdQcBWFoU7N5fP-tIl9Il6cyVmnu6_LWDTkkQQWClawzfg&amp;oe=65EAE6EE&amp;_nc_sid=e6ed6c&amp;_nc_cat=104" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d gvktkc8w" role=""><span dir="auto" aria-label="Maybe Anubhav Mishra" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Anubhav Mishra</span><span class="WJuYU" testid="author" role="button">+91 80721 73935</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[7:54 pm, 26/02/2024] +91 80721 73935: "><div class="M6sU5"><div class="xzT4z gndfcl4n l8fojup5 paxyh2gw sfeitywo cqsf3vkf k2bacm8l ajgl1lbb p357zi0d sap93d0t ovllcyds kvdvyush gfz4du6o r7fjleex rrq4r3yd"><img alt="" class="tvf2evcx oq44ahr5 lb5m6g5c n43pk08i fpj7ivsd jpthtbts" src="data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/4gIoSUNDX1BST0ZJTEUAAQEAAAIYAAAAAAQwAABtbnRyUkdCIFhZWiAAAAAAAAAAAAAAAABhY3NwAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAQAA9tYAAQAAAADTLQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAlkZXNjAAAA8AAAAHRyWFlaAAABZAAAABRnWFlaAAABeAAAABRiWFlaAAABjAAAABRyVFJDAAABoAAAAChnVFJDAAABoAAAAChiVFJDAAABoAAAACh3dHB0AAAByAAAABRjcHJ0AAAB3AAAADxtbHVjAAAAAAAAAAEAAAAMZW5VUwAAAFgAAAAcAHMAUgBHAEIAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAFhZWiAAAAAAAABvogAAOPUAAAOQWFlaIAAAAAAAAGKZAAC3hQAAGNpYWVogAAAAAAAAJKAAAA+EAAC2z3BhcmEAAAAAAAQAAAACZmYAAPKnAAANWQAAE9AAAApbAAAAAAAAAABYWVogAAAAAAAA9tYAAQAAAADTLW1sdWMAAAAAAAAAAQAAAAxlblVTAAAAIAAAABwARwBvAG8AZwBsAGUAIABJAG4AYwAuACAAMgAwADEANv/bAEMABgQFBgUEBgYFBgcHBggKEAoKCQkKFA4PDBAXFBgYFxQWFhodJR8aGyMcFhYgLCAjJicpKikZHy0wLSgwJSgpKP/bAEMBBwcHCggKEwoKEygaFhooKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKP/AABEIAIwAjAMBIgACEQEDEQH/xAAcAAACAwEBAQEAAAAAAAAAAAAFBgMEBwIIAQD/xAA8EAACAQMDAgMFBQUIAwEAAAABAgMABBEFEiEGMRNBUQciYXGBFDKRobEjJEJSwQgVM0NicoLRFpKy8P/EABoBAAIDAQEAAAAAAAAAAAAAAAIDAAEEBQb/xAAmEQADAAICAQQCAgMAAAAAAAAAAQIDESExEgQTQVEicRQyQlKB/9oADAMBAAIRAxEAPwBreBGGcY5718Rp4f8ACkyB5Gl3V9Wv4LlWsoykSj3lcZBrmHqqNhtvIGiZv44/+qx47dynS0x+SVFNS9jWupArtu4sj1IyK4ksrC8X9kQhPkO1DrfUba7DGCaOQY4UnDZqcKmRwUbGaPx+gfP7I20y6szutJHXH8h4/Cvo1a+iUrcRiVe2RwamSWeP7j7l+NQXl3uhbxYuccHFVr7L2vgVdb1u+imd4Ytqjtkc1S0zqDU78yM07KqeQGKH6j1f/d8xivbfxE3EYI5xV7TdV0fUIjJbS/Z2bghuBmr8UWqbD9rf3cY8TxQX780x9N6tqV8k7QhW8H7wz3rPdQjlhhP2e7hfzHvimD2U6jJa6hdwXhAjuE91ieM1E2uitb+DRLLWlR/DvEeGQ+bDANGYLoS4ZGBSvglsr+ww3huQNpHBINLM0FxZzE2kjbPJT5Ufua7B8PoabqOC7XbKgJ+Ipav+ncSbraQxtnPwNc2fUUUd0bW9Ki5AyQDzijsFzHcjKOCvzouK6B5XYnTpqNi5WWIlf5loTrMsdxp+SAJohlT51oNxKsQZZgHiI8/KlvW9MtTpDXUB2y4OF9TS6nkYm1IB02L7RBFc2smVZcsh7inOynjjtkUqcgUlQ22o2NrGZkCAJ3QVZi1WTwx7/wCNTTQO0XZLY7FLAENkANwTQy90i1uCfEj2N64psC5MQYo2O5cc/So3tl8BtytGWfufezQTStKpLpOH412Z5d9MSIN1pJn8jUcMur6d7pdnTsVfkU/z2WDKUH3ewQ8/UUOu7dkD7wGCruIx5UXJQs2evMLpnvVaJNu0KnKg+tHLe+troDwpo5Bt9cHPyqjdWMU0YbYV3DI4pc1a1TToHuGk2xp555J9BU2TRW9p2p2GnRLDLBC88ik++udvxrJTr0sjYVBx2A8sVc6iuRf3j3Nw5YZ43Hk+gzVSFVWNSyhR3BWrWn2Gk10XbfUhMo+0O0ZPmDzRC2146fIJ7d3YebKSMfQ5oUqKz7pEYhT8/rVuxaRHYRKGTucAZH0qaQfJo3TXXTxr4vi5595u3Hx8qeIurDdwllCSEDvGf1FYHJdNHdgPAEIHDYwD9KJ2+pi2mE+Gt5PKSPt8sVTko07RbiA69c6hqcf7dvcQnyWnK3mhlG+xnAbvtzil72fQWXU2nTPfOxkUjDbcD9KYLjotoVL6dcn4AGh/RP2Q3uqX0X7NrSR8nGdwwasaRY3VyElvpF8FeUiB7fOhOsSatpCx7rd7iEcsaWk129tr9pYZHVSc7G5Aq1XPJTn8WbEY4549kiAp25oDd6BbmdjHgLQGw6xuJ8JcsiL6qOaY7bXdMEQyzsfMkUboUp2H7rQi/KsGI7bh2qhLp9zBnbv9cnkU8mMHyqNoAfKg1roJtMQZIzh96Y3YyUOGNVtRVXt5VDe86YCMP60+3FhFIDuQfOhV5o4MbeCcHHAPap12TQoGx2wjcrDEQG7utZV7YHMEun20ZAUo0rEevYf1rcv7uuY4gWX3tuDtORWJ+21WOvWUTjBFqeMd/eNRsuFpmYWWltqs/bKL+Zotc9MXaxr4KsPTHFN3s90pVsfEcctxT4NNjkiC7R9az3marRux4VU7MHbSru1UrKWYfLNRx2N4yJNAjq+7b7pzmtfvNDSS7KhM5zn/ALqxadNIivHEoKbiGHmvbkfl+FF7pH6czOx0q5vlHirtbPPH61cvtAn0/EpXw9wA93hT863XRun4Y8B41298Y7H/APfnXHW3Tcd507c+EuHUbgP6VFTbKrHKRlvQGoX+kXskds/31ztYZD9q0GHqy73B7mxZFB5aE4pI6Is53vkyGLQ8kEckeYH5Vseg6FcG0kFw6xpJyE28j61KmnzLBWSJ/G52jMNZ6iv76+kU3DiAn3U+FXNIhjmI8VFbPfIoP1Pbix6mvLdWLBHxn86MaC4yorFldL5O1gmHPC4CgfSbaWRVtPEkU4OF86sJrUCqAum8fIUzydJ2t9bwzozRyMoLYPDVHH0uYF2LggeZ5rT45muKS/4ch36eW9w2/wBmhBa/ABhkEGsS6n9oGo6lM8enO1paqcAr95vr5VqvQxdultPeUs0jRhmZjkknzp+PPOSnMnNT2FylRPGPSrZFRuKeWUnhB8qwL+0FCg6h09QoDG1PP/I16GIrzf7ab+XVeoLaURqIYC9vGUHfnz+PIpdNLsbjmq5XwW+l7QWmj2pPdxuxTJbc4I5FBbgzWtjapFCZCqDOPLiqM3VVxZFUGmSsOxZmCisLl09nUilKHRLQM+7HOKu21sqSH3e/cUK0HX4L5EDwSQuRja486N3d9baZEJrrcFPI2qWJ+gq1pBNbDFuu1BxzVi9h8XTZ1QZJQ4pNHXdg8vhRQXIP80kZUfjTT0/qa6ihGxlOOcjginy0ZrT7EPoQQWfUcrtGWURsxPfsR+m6nTX+srHTYNtuRNcEYCDy+dJNlZ3kMvUE1q+wWysdzDjaNxKj6Ck95WkcsxJYnJzVZMnitIv0/pvcflXRdusX+oTXdwAZZW3NV+yVY8bRihkDdqJWx7Vjp77OxM6WkaD051GI4o7e7HuLwH9Kb1ZZFDowKnsQayS2J470fs9TubeAJG52+hpuP1DnijDn9D5vyjsy7TYftt5Bbo6jxXCDJ75Nem9NtRaWEFun3Y0Cj8K8XWl7eQSw3sLuhhcMD6GtIsfar1G8sc011F4AAAjCAbvnV+luce9nn09HpHcNxXIz6VywrKNM6y0266n0i8tblo5LtDFdwu3Cny/OtYyGXIOQexrdGRXvXwEnshPesK1G2gub02c8RdvGaYt22sGOK3ViM1k/VOkvYdRPMrKqu3iDIPvI3cfMGleoXTN/oqX5S/kE6jE89m0MLbHPGR5Un3HRtw+oQ3c2oStFHgvGWOHIOewIAz2p7ChZDnz5qcxq0eWHFZpbnlG1x5IT9BhktdXTa37LOcBcCn7qm1mv9HhFuSBjDbRnNK8Bj+3sRgbTWjaayfYIg7DJ4Ao5nyb2Ve5WzKNF6Ju5deW+S8lii4zCjtGMgY7du/Patg6ft7i0gWK6mWZ1/jC7fyqnGqRzkxgDnyo1aKcZ70crfYnItLgX+qxbWnTOrx22UkUM0n+ouMH/AOqxVO9bF7SHW26evJJmCyXWyGNQe4DZz+ANY6g97vScvZr9N/XgvW47UUtvKhtvRO2xkUijoIL2o7UTjHu0NtaKR/dpZDMNckt9Rmgt9PhKRIAo93BJ+NFOudD00yWUekKEkFsDMq+T8c0yaL04EujPOQ20+6oFEIOkxPrP2uQ7YQdzY8/hRzitY3K7PK5MCX9WZT0npF7qOtQ2lqjvKGwSB93nua9U2ssdtHBZvKDOIxx5nHel7o/R7K31C8vrV0K3BC4TsNtWNalDdU6VbxDEiq7sR5LjFbcGP2p8m9tsTrxYakkx51FNbW96oS5hSUDtuHaoL19rZr7aSFmHNbNJ8BJuXtGd3kIgvZoj/lyMn4HFR3dwkEJB7kdqIdTx41q6K9i2fyFBNSjlW2eWBFlfGdjHGeO2a5t8Vo7OK24RQsbiEXcavbZAblgc55rQbO8jECiERsN3O4dvgKy4avPbt7ulzu4PZdufzNM3T+vSTusSaPfAZ7yCPH/1TIWmMuLpbGlSq3aqx4c+6fj6UecvFZSNDsEm07d/bPln4UD8O5uvDEtukLblICtkjDA5/CjGpQtPYSxI2C+Bn4UcdtGTLWpTYtap0Ze9SzQz6xq8IEa4SOBfdXPfue9d23sq0lQPF1CdvlgUYgCWtsiSj7vG41S1W9jto2uV3OiKSVQ8mme0vlCF6utaT0SRezPQ07z3B+birC+z3RE7Szf+9UNA1v7fYLcRRMqt/DKMNRmPVFX/ABbYY9RU9qPon8vJ/syH/wAI0hPuzyj/AJCvo6Q07yvXH1FEIL+xk7qFPxFXY/AdcoEK/Kp7OP6J/Myr/JmdWblguOPWmXS4hOREw91uDQHTrdRgg802aHHtcsfIYqRO3pi7ekC47eLpK0v5riYC1Zt8QB5DegoD0fr0mqdTNcXCB2lQojdtgHlRbrHp646pu0WG8SK3twVKYySx86k0/pyy6YsftIlJuI1O6QnGR6YpFRkWRKVqEZdk3VGopYwmRxn5UBg6knQ5jRMH1qjquotrOk3e5SGVjg+o8qVtMmfwlDscrxVZM7aVQ+Dd6fHFrlDJe3j3E8skpG9zuqssxYYJ4qvncBz5VAjGOT38kZpTry7Nynx6DKQwP95QZPI4pl0SCOFQQqg/AUv2Bj3Bsgg0wafNGAQzjNFDW+WS7rx0GFIL7s9q/XMzRxKygZJwM1FHmQgKML+tVepOLeBQeN39K1Rw9mHK/wAdFlv3hCsqLj51QuNKgfJG5T8DQhZZU+7Kw+tSrqFyv+ZkfEU/yMYL13TLqNQLS8eLHwrrSrbVUgDNfGQejCrl1etMuJUB+IqO11JY8RBSB8am0yFK/wBd1HSJ4Rd2njwytt3RDkfOtF0VhJp8bgEBucUtzRLcRIWUEHnkU06SgFkg9KikvYDsND1K2AE1vnH8pzRq3tLtVBC4TzTHNME08UEZknlSOMd2c4A+tI/VPtY6Q6dhYz6rFdzjgQWf7VifmOB9SKYoS5QNW2fuodQfStZ0yZIpUjlcxTe6SMeRNDeveo7KFYbd4vtUTglgpxg+VZ9rv9pSLw2TR+nd0n8Ml1NkfVQP61jPU3X+u9RX0t1e3SQ7/wDLt0CKo9B5/nSsuOrlzL7A5PQl7qGh6V08Lme/jt1uOQkzjcT6Adz9Kyq465trjW7TT9H+5NKFkuHXsM87QfP4n8Kya6vWmPvM7k9yxzVWOZ4p0liYpIjBgfQiqXpoU+I6Mrjo9UWLb+aJfYllXOMGkX2c9W2evW6RSSJDfoMPExxu+K+taZZxhgBWCoaemdabVLaBsViV4TPem7T7GEQq0Y94jvVKOHbIOOKM2XAAoscLfIGS3rgtRKUGD+tQ6lAlxGFkHxB9KsD3noJ1lr1l05ok99qEqoEUhEz70jeSqPM1qS3wjLTXyIF31dDpPUtzpOvwx2hXDQzxy+IsiHsSMAqaZ7aa3uoRJazRzRns0bBh+VeWtf1e51jVrnUbpy0krbseg8gPkKl6d1/UNHkWewuZI5gfI5DD0YdiK0+z+K+zE6W+Oj09MowaohR4wx6VnOke1oyAJq1ipbHLwNg/gf8AsU46P1NpGrOhtL2LeR/hSHa4PyNL8GmEmP8AEv7vF/tpj0xf3RaAQD93i/2imHTeLVfnR6Ins8i+172i3PV/UDtBJJHpMBK2sB44/nYfzH9OKzeSZ3O5mJNczkl6iIxTUgD6zedcZz3PNfcZBrnFXoh+JOOAKjIPepDX5RmpohHE7xSK8bMjqchlOCD86ddB9p/U2jlALxbyIfwXK7+PnwfzpNwMHivgUZ+lDUKuwppz0bhp3t5nVVF/okTnzMUxX9QaIy/2gI1BFtoTZ/13H/S1gK9vyrtOBn40HsxvoP3r12bHfe3fqG4RhZWdjabuzFWkYfLJx+VIGvdRar1DdC51i9mupB93eeFHoB2FL8fLcnyJqccKuPSnxEz0KqnXZ2SzH3TjHlnmviSTKeGyOxr5jEqkeYNdNwTiiB6Ponfzxk+ddeO2QeQfXNcHsa5UAnFTQTY4dN+0LqDRNiQX8sluvAhmO9QPQA9vpWq6N7eXisFW80qJ5gTlo5CoP0Oa8+Se6eK6ViBwapwmUf/Z"><div class="k5z04t9s cm280p3y p357zi0d ggj6brxn m0h2a7mj lb5m6g5c f8m0rgwh ac2vgrno idwf4z32 onmzirdk gfz4du6o r7fjleex n1yiu2zv itegkywt eb4rp10x rppts313 ln8gz9je"><div class="aoi073rw suguakab jjtagnqy c32ccnay tvf2evcx m0h2a7mj lb5m6g5c j4enbv94 gfz4du6o r7fjleex lhj4utae ljrqcn24" title="Arthur Mensch (@arthurmensch) on X"><span dir="ltr" aria-label="" class="ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae i033jvx7 cw3vfol9 _11JPr" style="min-height: 0px;">Arthur Mensch (@arthurmensch) on X</span></div><div class="jjtagnqy tvf2evcx ttu8nud2 lb5m6g5c dsh4tgtl gfz4du6o r7fjleex lhj4utae"><span class="g0rxnol2 gfz4du6o r7fjleex" style="display: -webkit-box; overflow: hidden; -webkit-box-orient: vertical; -webkit-line-clamp: 2;"><span dir="ltr" title="We’re announcing a new optimised model today! Mistral Large has top-tier reasoning capacities, is multi-lingual by design, has native function calling capacities and a 32k model. The pre-trained model has 81.2% accuracy on MMLU. Learn more on https://t.co/SOUILj8RVt. Mistral…" aria-label="" class="i033jvx7 _11JPr" style="min-height: 0px;">We’re announcing a new optimised model today! Mistral Large has top-tier reasoning capacities, is multi-lingual by design, has native function calling capacities and a 32k model. The pre-trained model has 81.2% accuracy on MMLU. Learn more on https://t.co/SOUILj8RVt. Mistral…</span></span></div><div class="j8h4s3v0 tvf2evcx oq44ahr5 lb5m6g5c dsh4tgtl qbqilfqo"><span dir="ltr" aria-label="" class="_11JPr" style="min-height: 0px;">x.com</span></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span><a dir="auto" href="https://x.com/arthurmensch/status/1762121295330725965?t=BLYLr2dlQC0cx-uFOTs93A&amp;s=08" title="https://x.com/arthurmensch/status/1762121295330725965?t=BLYLr2dlQC0cx-uFOTs93A&amp;s=08" target="_blank" rel="noopener noreferrer" class="_11JPr selectable-text copyable-text">https://x.com/arthurmensch/status/1762121295330725965?t=BLYLr2dlQC0cx-uFOTs93A&amp;s=08</a></span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">7:54 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">7:54 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="reaction 👍"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj _3sxvM" data-id="false_120363049558306142@g.us_2EFB2D0A0932D23AA510C1E71B604AE1_918072173935@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a"><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="copyable-text" data-pre-plain-text="[7:54 pm, 26/02/2024] +91 80721 73935: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Noice!</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">7:54 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">7:54 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj _3sxvM" data-id="false_120363049558306142@g.us_CFF33B86917472912C12D426815AC065_918072173935@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a"><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d gvktkc8w" role=""><span dir="auto" aria-label="Maybe Anubhav Mishra" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Anubhav Mishra</span><span class="WJuYU" testid="author" role="button">+91 80721 73935</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[8:00 pm, 26/02/2024] +91 80721 73935: "><div class="M6sU5"><div class="xzT4z gndfcl4n l8fojup5 paxyh2gw sfeitywo cqsf3vkf k2bacm8l ajgl1lbb p357zi0d sap93d0t ovllcyds kvdvyush gfz4du6o r7fjleex rrq4r3yd"><div class="k5z04t9s cm280p3y p357zi0d ggj6brxn m0h2a7mj lb5m6g5c f8m0rgwh ac2vgrno idwf4z32 onmzirdk gfz4du6o r7fjleex n1yiu2zv itegkywt eb4rp10x rppts313 ln8gz9je"><div class="aoi073rw suguakab jjtagnqy c32ccnay tvf2evcx m0h2a7mj lb5m6g5c j4enbv94 gfz4du6o r7fjleex lhj4utae ljrqcn24" title="Le Chat"><span dir="ltr" aria-label="" class="ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae i033jvx7 cw3vfol9 _11JPr" style="min-height: 0px;">Le Chat</span></div><div class="jjtagnqy tvf2evcx ttu8nud2 lb5m6g5c dsh4tgtl gfz4du6o r7fjleex lhj4utae"><span class="g0rxnol2 gfz4du6o r7fjleex" style="display: -webkit-box; overflow: hidden; -webkit-box-orient: vertical; -webkit-line-clamp: 2;"><span dir="ltr" title="Our assistant is now in beta access, demonstrating what can be built with our technology." aria-label="" class="i033jvx7 _11JPr" style="min-height: 0px;">Our assistant is now in beta access, demonstrating what can be built with our technology.</span></span></div><div class="j8h4s3v0 tvf2evcx oq44ahr5 lb5m6g5c dsh4tgtl qbqilfqo"><span dir="ltr" aria-label="" class="_11JPr" style="min-height: 0px;">mistral.ai</span></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span><a dir="auto" href="https://mistral.ai/news/le-chat-mistral/" title="https://mistral.ai/news/le-chat-mistral/" target="_blank" rel="noopener noreferrer" class="_11JPr selectable-text copyable-text">https://mistral.ai/news/le-chat-mistral/</a></span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">8:00 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">8:00 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_679B1280D560637D744CEE1CB69E2E13_918072173935@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a"><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="copyable-text" data-pre-plain-text="[8:01 pm, 26/02/2024] +91 80721 73935: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Ohh they'll directly complete with chatGPT it seems.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">8:01 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">8:01 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_99A2ED9F24071A54017930EDA3777F6D_919405887831@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _27hEJ _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe undefined +91 94058 87831" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/399291232_322590160516040_4962715274003434715_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdTGsUQy94NyrgiROgCWqc8B6_UCziwtTCaJxkQye9zvOQ&amp;oe=65EAEEE1&amp;_nc_sid=e6ed6c&amp;_nc_cat=110" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div class="cm280p3y eu4mztcy ocd2b0bc folpon7g aa0kojfi snweb893 g0rxnol2 jnl3jror copyable-text" data-pre-plain-text="[8:13 pm, 26/02/2024] +91 94058 87831: "><div><div class="bxcbqipq mhcwslh8 ocd2b0bc jfqm35v0"><div class="_3IzYj _6rIWC p357zi0d giysludt" role=""><span class="WJuYU" testid="author" role="button">+91 94058 87831</span></div></div><div class="cm280p3y czh1irh3 kzyzudjh"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="j3zl77sr _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d gvktkc8w" role=""><span dir="auto" aria-label="Maybe Anubhav Mishra" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Anubhav Mishra</span><span class="WJuYU" testid="author">+91 80721 73935</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">https://mistral.ai/news/le-chat-mistral/</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div role="button" tabindex="0" aria-label="Open Picture" class="gndfcl4n l8fojup5 paxyh2gw sfeitywo cqsf3vkf p357zi0d ac2vgrno laorhtua gfz4du6o r7fjleex g0rxnol2" style="width: 240px; height: 338.028px;"><span class=""></span><div class="g0rxnol2 ln8gz9je ppled2lx"><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><img alt="Not good at Hindi given the claim of being multilingual" src="data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEABsbGxscGx4hIR4qLSgtKj04MzM4PV1CR0JHQl2NWGdYWGdYjX2Xe3N7l33gsJycsOD/2c7Z//////////////8BGxsbGxwbHiEhHiotKC0qPTgzMzg9XUJHQkdCXY1YZ1hYZ1iNfZd7c3uXfeCwnJyw4P/Zztn////////////////CABEIAEYAIQMBIgACEQEDEQH/xAArAAEBAQEBAQAAAAAAAAAAAAAAAwIBBAYBAQAAAAAAAAAAAAAAAAAAAAD/2gAMAwEAAhADEAAAAPpY04bToAY3HZt0Aef0RoaABGsqnXOgGch0HQf/xAAhEAABAwMFAQEAAAAAAAAAAAABAAIREiAhAyIxMkEEEP/aAAgBAQABPwBxkQqJjcgIEItoBNRQ+jTA9RZk70W47EKmI3lYKoCDw09ZtoNRwUytp9skzCHFji30FabQ7dkWFhk8wmSG2GZmE10iw59VQZzlN1GkxCcKo/P/xAAWEQADAAAAAAAAAAAAAAAAAAAQETD/2gAIAQIBAT8Aqh//xAAUEQEAAAAAAAAAAAAAAAAAAAAw/9oACAEDAQE/AH//2Q==" class="jciay5ix tvf2evcx oq44ahr5 lb5m6g5c fsmudgz7" style="width: 100%;"></div><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><img alt="Not good at Hindi given the claim of being multilingual" src="blob:https://web.whatsapp.com/fcccfd1f-6c0d-4714-9dc8-d2f77116fb95" class="jciay5ix tvf2evcx oq44ahr5 lb5m6g5c osz0hll6 nq7eualt em5jvqoa a21kwdn3" style="width: 100%;"></div></div></div><div class="cm280p3y czh1irh3 esbfogcw jfqm35v0 bxcbqipq mhcwslh8"><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Not good at Hindi given the claim of being multilingual</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">8:13 pm</span></span></span></div></div><div class="lrw9n60e lhggkp7q fz4q5utg b9fczbqn"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">8:13 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 cvsrm5e1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj _3sxvM" data-id="false_120363049558306142@g.us_F7D51A77FF85E93693_918742023550@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Shubham +91 87420 23550" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno k4182h19"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">S</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d oyzge2kb" role=""><span dir="auto" aria-label="Maybe Shubham" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Shubham</span><span class="WJuYU" testid="author" role="button">+91 87420 23550</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[8:20 pm, 26/02/2024] +91 87420 23550: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g755h7wn _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d a7s2a5ab" role=""><span dir="auto" aria-label="Maybe Soumyadeep Mukherjee" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Soumyadeep Mukherjee</span><span class="WJuYU" testid="author">+91 74076 51462</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Scenario and Dashtoon both allow you to do this with a mix of Lora’s and IP adapters. Feel free to try on dashtoon, you can train with one image with dashtoon.

This genid thing is actually seed and prompt sharing. We also started with this over stable diffusion but it doesn’t work on as many characters we need <img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="😅" draggable="false" class="b85 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -80px;"></span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Why so?</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">8:20 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">8:20 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_01ACBC4373A347A39F_918742023550@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a"><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="copyable-text" data-pre-plain-text="[8:21 pm, 26/02/2024] +91 87420 23550: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>What number of characters does it not work for?</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">8:21 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">8:21 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB09872EAC7742E9CB3DF_919550164716@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Ravi Theja +91 95501 64716" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno j3zl77sr"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">R</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d gvktkc8w" role=""><span dir="auto" aria-label="Maybe Ravi Theja" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Ravi Theja</span><span class="WJuYU" testid="author" role="button">+91 95501 64716</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[8:23 pm, 26/02/2024] +91 95501 64716: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="a82mwcnm _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d giysludt" role=""><span class="WJuYU" testid="author">+91 94058 87831</span></div><div class="_37DQv" dir="ltr" role="button"><div class="l7jjieqr tvf2evcx oq44ahr5 lb5m6g5c g97rt3p3 fewfhwl7 jdwybkuq kjemk6od"><span data-icon="status-image" class=""><svg viewBox="0 0 16 20" height="20" width="16" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 16 20"><title>status-image</title><path fill="currentColor" d="M13.822,4.668H7.14l-1.068-1.09C5.922,3.425,5.624,3.3,5.409,3.3H3.531 c-0.214,0-0.51,0.128-0.656,0.285L1.276,5.296C1.13,5.453,1.01,5.756,1.01,5.971v1.06c0,0.001-0.001,0.002-0.001,0.003v6.983 c0,0.646,0.524,1.17,1.17,1.17h11.643c0.646,0,1.17-0.524,1.17-1.17v-8.18C14.992,5.191,14.468,4.668,13.822,4.668z M7.84,13.298 c-1.875,0-3.395-1.52-3.395-3.396c0-1.875,1.52-3.395,3.395-3.395s3.396,1.52,3.396,3.395C11.236,11.778,9.716,13.298,7.84,13.298z  M7.84,7.511c-1.321,0-2.392,1.071-2.392,2.392s1.071,2.392,2.392,2.392s2.392-1.071,2.392-2.392S9.161,7.511,7.84,7.511z"></path></svg></span></div><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Not good at Hindi given the claim of being multilingual</span></div></div></div><div class="jrEtJ"><div class="kknmh"><div class="g0rxnol2 ln8gz9je ppled2lx"><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><div class="lhggkp7q ln8gz9je ppled2lx t3g6t33p qnwaluaf" style="background-image: url(&quot;data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEABsbGxscGx4hIR4qLSgtKj04MzM4PV1CR0JHQl2NWGdYWGdYjX2Xe3N7l33gsJycsOD/2c7Z//////////////8BGxsbGxwbHiEhHiotKC0qPTgzMzg9XUJHQkdCXY1YZ1hYZ1iNfZd7c3uXfeCwnJyw4P/Zztn////////////////CABEIAEYAIQMBIgACEQEDEQH/xAArAAEBAQEBAQAAAAAAAAAAAAAAAwIBBAYBAQAAAAAAAAAAAAAAAAAAAAD/2gAMAwEAAhADEAAAAPpY04bToAY3HZt0Aef0RoaABGsqnXOgGch0HQf/xAAhEAABAwMFAQEAAAAAAAAAAAABAAIREiAhAyIxMkEEEP/aAAgBAQABPwBxkQqJjcgIEItoBNRQ+jTA9RZk70W47EKmI3lYKoCDw09ZtoNRwUytp9skzCHFji30FabQ7dkWFhk8wmSG2GZmE10iw59VQZzlN1GkxCcKo/P/xAAWEQADAAAAAAAAAAAAAAAAAAAQETD/2gAIAQIBAT8Aqh//xAAUEQEAAAAAAAAAAAAAAAAAAAAw/9oACAEDAQE/AH//2Q==&quot;);"></div></div><div class="lhggkp7q jnl3jror p357zi0d gndfcl4n ac2vgrno ln8gz9je ppled2lx"><div class="lhggkp7q ln8gz9je ppled2lx t3g6t33p qnwaluaf" style="background-image: url(&quot;blob:https://web.whatsapp.com/fcccfd1f-6c0d-4714-9dc8-d2f77116fb95&quot;);"></div></div></div></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>They did not mention about Hindi right. 'English, French, Spanish, German, and Italian' is what the blog says</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">8:23 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">8:23 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="reaction 👍"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_8C4A7270116F196FFA96217B2E14272B_919405887831@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe undefined +91 94058 87831" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/399291232_322590160516040_4962715274003434715_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdTGsUQy94NyrgiROgCWqc8B6_UCziwtTCaJxkQye9zvOQ&amp;oe=65EAEEE1&amp;_nc_sid=e6ed6c&amp;_nc_cat=110" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d giysludt" role=""><span class="WJuYU" testid="author" role="button">+91 94058 87831</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[8:26 pm, 26/02/2024] +91 94058 87831: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="j3zl77sr _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d gvktkc8w" role=""><span dir="auto" aria-label="Maybe Ravi Theja" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Ravi Theja</span><span class="WJuYU" testid="author">+91 95501 64716</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">They did not mention about Hindi right. 'English, French, Spanish, German, and Italian' is what the blog says</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Ah, makes sense targeting the European languages for them. Then this is a surprising behaviour to be able to generate coherent outputs in other languages</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">8:26 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">8:26 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="reaction 👍"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj _3sxvM" data-id="false_120363049558306142@g.us_3ABCE2A026801E9E9AB4_919967280880@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Siddharth Bulia +91 99672 80880" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/409282256_337200822467759_9122011097835893379_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdTMW2ySRsfzYMrK2OWzAWaAh7s6lO4LDFc3QZOHshQGmQ&amp;oe=65EADE70&amp;_nc_sid=e6ed6c&amp;_nc_cat=101" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d dud7p2xx" role=""><span dir="auto" aria-label="Maybe Siddharth Bulia" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Siddharth Bulia</span><span class="WJuYU" testid="author" role="button">+91 99672 80880</span></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t r15c9g6i"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l mhp1pqu9 nu7pwgvd dnb887gk gjuq5ydh i2cterl7 l7jjieqr a6r0u4sv bx0vhl82 ntr8esoy"><span data-icon="recalled" class=""><svg viewBox="0 0 24 24" height="24" width="24" preserveAspectRatio="xMidYMid meet" class="" fill="none"><title>recalled</title><path fill-rule="evenodd" clip-rule="evenodd" d="M7.75897 6.43054C8.93584 5.533 10.4057 5 12 5C15.866 5 19 8.13401 19 12C19 13.5943 18.467 15.0642 17.5695 16.241L7.75897 6.43054ZM6.35707 7.85707C5.50399 9.01706 5 10.4497 5 12C5 15.866 8.13401 19 12 19C13.5503 19 14.9829 18.496 16.1429 17.6429L6.35707 7.85707ZM12 3C7.02944 3 3 7.02944 3 12C3 16.9706 7.02944 21 12 21C16.9706 21 21 16.9706 21 12C21 7.02944 16.9706 3 12 3Z" fill="currentColor"></path></svg></span></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 kiiy14zj j4enbv94"><div class="_21Ahp c4mcV">This message was deleted<span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">8:37 pm</span></span></span></div></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">8:37 pm</span></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"></div></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj _3sxvM" data-id="false_120363049558306142@g.us_3ABC53841B16A83BA62E_919967280880@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a"><div class="_1BOF7 _2AOIt"><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d dud7p2xx" role=""><span dir="auto" aria-label="Maybe Siddharth Bulia" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Siddharth Bulia</span><span class="WJuYU" testid="author" role="button">+91 99672 80880</span></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t r15c9g6i"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l mhp1pqu9 nu7pwgvd dnb887gk gjuq5ydh i2cterl7 l7jjieqr a6r0u4sv bx0vhl82 ntr8esoy"><span data-icon="recalled" class=""><svg viewBox="0 0 24 24" height="24" width="24" preserveAspectRatio="xMidYMid meet" class="" fill="none"><title>recalled</title><path fill-rule="evenodd" clip-rule="evenodd" d="M7.75897 6.43054C8.93584 5.533 10.4057 5 12 5C15.866 5 19 8.13401 19 12C19 13.5943 18.467 15.0642 17.5695 16.241L7.75897 6.43054ZM6.35707 7.85707C5.50399 9.01706 5 10.4497 5 12C5 15.866 8.13401 19 12 19C13.5503 19 14.9829 18.496 16.1429 17.6429L6.35707 7.85707ZM12 3C7.02944 3 3 7.02944 3 12C3 16.9706 7.02944 21 12 21C16.9706 21 21 16.9706 21 12C21 7.02944 16.9706 3 12 3Z" fill="currentColor"></path></svg></span></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 kiiy14zj j4enbv94"><div class="_21Ahp c4mcV">This message was deleted<span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">8:37 pm</span></span></span></div></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">8:37 pm</span></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"></div></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A4E4054A8BD5662B654_919967280880@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a"><div class="_1BOF7 _2AOIt"><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d dud7p2xx" role=""><span dir="auto" aria-label="Maybe Siddharth Bulia" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Siddharth Bulia</span><span class="WJuYU" testid="author" role="button">+91 99672 80880</span></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t r15c9g6i"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l mhp1pqu9 nu7pwgvd dnb887gk gjuq5ydh i2cterl7 l7jjieqr a6r0u4sv bx0vhl82 ntr8esoy"><span data-icon="recalled" class=""><svg viewBox="0 0 24 24" height="24" width="24" preserveAspectRatio="xMidYMid meet" class="" fill="none"><title>recalled</title><path fill-rule="evenodd" clip-rule="evenodd" d="M7.75897 6.43054C8.93584 5.533 10.4057 5 12 5C15.866 5 19 8.13401 19 12C19 13.5943 18.467 15.0642 17.5695 16.241L7.75897 6.43054ZM6.35707 7.85707C5.50399 9.01706 5 10.4497 5 12C5 15.866 8.13401 19 12 19C13.5503 19 14.9829 18.496 16.1429 17.6429L6.35707 7.85707ZM12 3C7.02944 3 3 7.02944 3 12C3 16.9706 7.02944 21 12 21C16.9706 21 21 16.9706 21 12C21 7.02944 16.9706 3 12 3Z" fill="currentColor"></path></svg></span></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 kiiy14zj j4enbv94"><div class="_21Ahp c4mcV">This message was deleted<span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">8:38 pm</span></span></span></div></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">8:38 pm</span></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"></div></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj _3sxvM" data-id="false_120363049558306142@g.us_3AB988FEC3A447A4CA19_917737887058@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Nirant K +91 77378 87058" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno g9gonq0i"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">N</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author" role="button">+91 77378 87058</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[8:40 pm, 26/02/2024] +91 77378 87058: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="oj03u76w _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d dud7p2xx" role=""><span dir="auto" aria-label="Maybe Siddharth Bulia" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Siddharth Bulia</span><span class="WJuYU" testid="author">+91 99672 80880</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">I am trying to receive SSE stream on my frontend ReactJS App.
I want to send the payload as part of POST request, but not able to find support for it in any library (EventSource/Axios/etc). Existing libraries support only GET request.

Has anyone able to create a frontend app supporting SSE?</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Watercooler perhaps? Not a good forum for web dev support?</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">8:40 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">8:40 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3ADA5690C5C7A3ACF702_917737887058@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a"><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author" role="button">+91 77378 87058</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[8:43 pm, 26/02/2024] +91 77378 87058: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="a82mwcnm _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d giysludt" role=""><span class="WJuYU" testid="author">+91 94058 87831</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Ah, makes sense targeting the European languages for them. Then this is a surprising behaviour to be able to generate coherent outputs in other languages</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Related question: what's a good embedding model for these European languages?</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">8:43 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">8:43 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3AC5A76F5FEF30B746DD_917407651462@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Soumyadeep Mukherjee +91 74076 51462" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/424242886_353057184272898_3175861170545036892_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdSAbzAZ6xOnG7Ln3kMzxPYrvQh8tij45kPm-x7s3Cl8ig&amp;oe=65EAFA5F&amp;_nc_sid=e6ed6c&amp;_nc_cat=103" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d a7s2a5ab" role=""><span dir="auto" aria-label="Maybe Soumyadeep Mukherjee" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Soumyadeep Mukherjee</span><span class="WJuYU" testid="author" role="button">+91 74076 51462</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[8:53 pm, 26/02/2024] +91 74076 51462: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="k4182h19 _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d oyzge2kb" role=""><span dir="auto" aria-label="Maybe Shubham" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Shubham</span><span class="WJuYU" testid="author">+91 87420 23550</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">What number of characters does it not work for?</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Try it yourself. Multiple reasons from my experience
<ul class="cw3vfol9 hmy10g0s g0rxnol2 qt60bha0 inww9tbj fj62nrjn" dir="ltr"><li dir="auto" class="nqtxkp62 nirfx0j2 s8gevp1y qxgpk8xi l6n90272 kyf3mdpm c9j2cm7k"><span class="_11JPr selectable-text copyable-text" data-pre-plain-text="- ">you can only describe things uniquely to a limit using prompts </span></li>
<li dir="auto" class="nqtxkp62 nirfx0j2 s8gevp1y qxgpk8xi l6n90272 kyf3mdpm c9j2cm7k"><span class="_11JPr selectable-text copyable-text" data-pre-plain-text="- ">⁠even if you describe heavily, consistency will come if the prompt alignment is super high. Despite midj level alignment, humans expect way way more when it comes to characters. </span></li>
<li dir="auto" class="nqtxkp62 nirfx0j2 s8gevp1y qxgpk8xi l6n90272 kyf3mdpm c9j2cm7k"><span class="_11JPr selectable-text copyable-text" data-pre-plain-text="- ">⁠seed is just a make shift proxy of some level of consistency as you traverse the latent space</span></li></ul></span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">8:53 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">8:53 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A90A38F28DCCBA47CB7_919967280880@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Siddharth Bulia +91 99672 80880" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/409282256_337200822467759_9122011097835893379_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdTMW2ySRsfzYMrK2OWzAWaAh7s6lO4LDFc3QZOHshQGmQ&amp;oe=65EADE70&amp;_nc_sid=e6ed6c&amp;_nc_cat=101" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d dud7p2xx" role=""><span dir="auto" aria-label="Maybe Siddharth Bulia" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Siddharth Bulia</span><span class="WJuYU" testid="author" role="button">+91 99672 80880</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[9:01 pm, 26/02/2024] +91 99672 80880: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g9gonq0i _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author">+91 77378 87058</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Watercooler perhaps? Not a good forum for web dev support?</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Apologies!</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">9:01 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">9:01 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB0E26662376BE7B90F7A_919420377689@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Sachin +91 94203 77689" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/288255936_175268804896333_254729857614157612_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdSDscQpC-6RYSYdPTmlunURMp_nKZOzVAknk7bZr3HCwg&amp;oe=65EB0784&amp;_nc_sid=e6ed6c&amp;_nc_cat=109" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo hbm07cg7 n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d pfvzdcj5" role=""><span dir="auto" aria-label="Maybe Sachin" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Sachin</span><span class="WJuYU" testid="author" role="button">+91 94203 77689</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[9:11 pm, 26/02/2024] +91 94203 77689: "><div class="M6sU5"><div class="xzT4z gndfcl4n l8fojup5 paxyh2gw sfeitywo cqsf3vkf k2bacm8l ajgl1lbb p357zi0d f8m0rgwh ovllcyds kvdvyush gfz4du6o r7fjleex rrq4r3yd"><div class="gndfcl4n p357zi0d ac2vgrno gfz4du6o r7fjleex g0rxnol2 ln8gz9je" style="height: 82.5px;"><img alt="" class="tvf2evcx oq44ahr5 lb5m6g5c ppled2lx ln8gz9je jpthtbts" src="..."></div><div class="k5z04t9s cm280p3y p357zi0d ggj6brxn m0h2a7mj lb5m6g5c f8m0rgwh ac2vgrno idwf4z32 onmzirdk gfz4du6o r7fjleex n1yiu2zv itegkywt eb4rp10x rppts313 ln8gz9je"><div class="aoi073rw suguakab jjtagnqy c32ccnay tvf2evcx m0h2a7mj lb5m6g5c j4enbv94 gfz4du6o r7fjleex lhj4utae ljrqcn24" title="🧞 Genie: Generative Interactive Environments"><span dir="ltr" aria-label="" class="ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae i033jvx7 cw3vfol9 _11JPr" style="min-height: 0px;"><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="🧞" draggable="false" class="b137 emoji apple _11JPr" tabindex="-1" style="background-position: -80px -20px;"> Genie: Generative Interactive Environments</span></div><div class="jjtagnqy tvf2evcx ttu8nud2 lb5m6g5c dsh4tgtl gfz4du6o r7fjleex lhj4utae"><span class="g0rxnol2 gfz4du6o r7fjleex" style="display: -webkit-box; overflow: hidden; -webkit-box-orient: vertical; -webkit-line-clamp: 2;"><span dir="ltr" title="A Foundation Model for Playable Worlds" aria-label="" class="i033jvx7 _11JPr" style="min-height: 0px;">A Foundation Model for Playable Worlds</span></span></div><div class="j8h4s3v0 tvf2evcx oq44ahr5 lb5m6g5c dsh4tgtl qbqilfqo"><span dir="ltr" aria-label="" class="_11JPr" style="min-height: 0px;">sites.google.com</span></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span><a dir="auto" href="https://sites.google.com/view/genie-2024/home" title="https://sites.google.com/view/genie-2024/home" target="_blank" rel="noopener noreferrer" class="_11JPr selectable-text copyable-text">https://sites.google.com/view/genie-2024/home</a>.   Deepmind released a detailed paper about training foundational interactive video generation model. They train 11B autoregressive next frame prediction model, spatio-temporal aware tokenizer, latent action model to condition the next frame generation (This is just training artifact. Don't need it for inference, except knowledge of available user actions). Their examples look quite cool and they claim to be able to generate 16 seconds worth of video at 10 fps. (No dataset or model weights; but paper is nice; not just marketing)</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5 hnx8ox4h mw4yctpw gj5xqxfh">Edited</span><span class="tvf2evcx oq44ahr5">9:11 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto"><span class="mw4yctpw gj5xqxfh">Edited</span>9:11 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="Reactions ❤ 2 in total"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="❤️" draggable="false" class="bd96tnyg eipqoq80 b8 emoji apple _11JPr" tabindex="-1" style="background-position: -20px 0px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk o2es7gts i2cterl7" style="flex-shrink: 0;"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 f8m0rgwh r15c9g6i ilf8vifs gfz4du6o r7fjleex qnz2jpws kjemk6od"><div class="dn5wc90o ilf8vifs gndfcl4n p357zi0d"><span class="f8jlpxt4 qfejxiq4 ce5kru2g">2</span></div></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj _3sxvM" data-id="false_120363049558306142@g.us_B6791DB85130A4F974504107971FB7B7_16503346780@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe chetanya rastogi +1 (650) 334-6780" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/415952272_775600611140651_2177361123082181361_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdSVywBByELINJJZtEX8APHCldzFuQvLZHIw4P-1m6Xd3A&amp;oe=65EAD03D&amp;_nc_sid=e6ed6c&amp;_nc_cat=111" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d mglqvgej" role=""><span dir="auto" aria-label="Maybe chetanya rastogi" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">chetanya rastogi</span><span class="WJuYU" testid="author" role="button">+1 (650) 334-6780</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[9:20 pm, 26/02/2024] +1 (650) 334-6780: "><div class="M6sU5"><div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span><a dir="auto" href="https://www.ft.com/content/cd6eb51a-3276-450f-87fd-97e8410db9eb" title="https://www.ft.com/content/cd6eb51a-3276-450f-87fd-97e8410db9eb" target="_blank" rel="noopener noreferrer" class="_11JPr selectable-text copyable-text">https://www.ft.com/content/cd6eb51a-3276-450f-87fd-97e8410db9eb</a></span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">9:20 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">9:20 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_5D6B264435D1FC1243910310F70BEA21_16503346780@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a"><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d mglqvgej" role=""><span dir="auto" aria-label="Maybe chetanya rastogi" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">chetanya rastogi</span><span class="WJuYU" testid="author" role="button">+1 (650) 334-6780</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[9:31 pm, 26/02/2024] +1 (650) 334-6780: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="b2boiwr0 _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d mglqvgej" role=""><span dir="auto" aria-label="Maybe chetanya rastogi" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">chetanya rastogi</span><span class="WJuYU" testid="author">+1 (650) 334-6780</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">https://www.ft.com/content/cd6eb51a-3276-450f-87fd-97e8410db9eb</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="M6sU5"><div class="xzT4z gndfcl4n l8fojup5 paxyh2gw sfeitywo cqsf3vkf k2bacm8l ajgl1lbb p357zi0d sap93d0t ovllcyds kvdvyush gfz4du6o r7fjleex rrq4r3yd"><div class="k5z04t9s cm280p3y p357zi0d ggj6brxn m0h2a7mj lb5m6g5c f8m0rgwh ac2vgrno idwf4z32 onmzirdk gfz4du6o r7fjleex n1yiu2zv itegkywt eb4rp10x rppts313 ln8gz9je"><div class="aoi073rw suguakab jjtagnqy c32ccnay tvf2evcx m0h2a7mj lb5m6g5c j4enbv94 gfz4du6o r7fjleex lhj4utae ljrqcn24" title="Microsoft Strikes Deal With Mistral in Push Beyond OpenAI - Slashdot"><span dir="ltr" aria-label="" class="ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae i033jvx7 cw3vfol9 _11JPr" style="min-height: 0px;">Microsoft Strikes Deal With Mistral in Push Beyond OpenAI - Slashdot</span></div><div class="jjtagnqy tvf2evcx ttu8nud2 lb5m6g5c dsh4tgtl gfz4du6o r7fjleex lhj4utae"><span class="g0rxnol2 gfz4du6o r7fjleex" style="display: -webkit-box; overflow: hidden; -webkit-box-orient: vertical; -webkit-line-clamp: 2;"><span dir="ltr" title="Microsoft has struck a deal with French AI startup Mistral as it seeks to broaden its involvement in the fast-growing industry beyond OpenAI. From a report: The US tech giant will provide the 10-month-old Paris-based company with help in bringing its AI models to market. Microsoft will also take a m..." aria-label="" class="i033jvx7 _11JPr" style="min-height: 0px;">Microsoft has struck a deal with French AI startup Mistral as it seeks to broaden its involvement in the fast-growing industry beyond OpenAI. From a report: The US tech giant will provide the 10-month-old Paris-based company with help in bringing its AI models to market. Microsoft will also take a m...</span></span></div><div class="j8h4s3v0 tvf2evcx oq44ahr5 lb5m6g5c dsh4tgtl qbqilfqo"><span dir="ltr" aria-label="" class="_11JPr" style="min-height: 0px;">slashdot.org</span></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Unpaywalled version <a dir="auto" href="https://slashdot.org/story/24/02/26/1431255/microsoft-strikes-deal-with-mistral-in-push-beyond-openai" title="https://slashdot.org/story/24/02/26/1431255/microsoft-strikes-deal-with-mistral-in-push-beyond-openai" target="_blank" rel="noopener noreferrer" class="_11JPr selectable-text copyable-text">https://slashdot.org/story/24/02/26/1431255/microsoft-strikes-deal-with-mistral-in-push-beyond-openai</a> credits to <span role="button" tabindex="0" class=""><span dir="auto" data-jid="917977314565@c.us" data-display="~Dhruv Anand" aria-label="Maybe Dhruv Anand" class="ajgl1lbb o0rubyzf _11JPr selectable-text select-all copyable-text" data-plain-text="@~Dhruv Anand" data-app-text-template="​917977314565@c.us​">@<span dir="ltr">~Dhruv Anand</span></span></span></span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">9:31 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">9:31 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="Reactions ❤, 👍 5 in total"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="❤️" draggable="false" class="bd96tnyg eipqoq80 b8 emoji apple _11JPr" tabindex="-1" style="background-position: -20px 0px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk o2es7gts i2cterl7" style="flex-shrink: 0;"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 f8m0rgwh r15c9g6i ilf8vifs gfz4du6o r7fjleex qnz2jpws kjemk6od"><div class="dn5wc90o ilf8vifs gndfcl4n p357zi0d"><span class="f8jlpxt4 qfejxiq4 ce5kru2g">5</span></div></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB0B41FEEF7BD8743B9DC_919819739552@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Rahul Deora +91 98197 39552" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/424643744_950879756689408_5500617842927232288_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdTfGuI1ycSPTskdKddpc_-NRgesc1W-1A31K6kHxI7DTA&amp;oe=65EAF644&amp;_nc_sid=e6ed6c&amp;_nc_cat=101" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo hbm07cg7 n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d a7s2a5ab" role=""><span dir="auto" aria-label="Maybe Rahul Deora" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Rahul Deora</span><span class="WJuYU" testid="author" role="button">+91 98197 39552</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[11:35 pm, 26/02/2024] +91 98197 39552: "><div class="M6sU5"><div class="xzT4z gndfcl4n l8fojup5 paxyh2gw sfeitywo cqsf3vkf k2bacm8l ajgl1lbb p357zi0d f8m0rgwh ovllcyds kvdvyush gfz4du6o r7fjleex rrq4r3yd"><div class="gndfcl4n p357zi0d ac2vgrno gfz4du6o r7fjleex g0rxnol2 ln8gz9je" style="height: 185.625px;"><img alt="" class="tvf2evcx oq44ahr5 lb5m6g5c ppled2lx ln8gz9je jpthtbts" src="..."></div><div class="k5z04t9s cm280p3y p357zi0d ggj6brxn m0h2a7mj lb5m6g5c f8m0rgwh ac2vgrno idwf4z32 o3plsq22 gfz4du6o r7fjleex n1yiu2zv itegkywt eb4rp10x rppts313 ln8gz9je"><div class="aoi073rw suguakab jjtagnqy c32ccnay tvf2evcx m0h2a7mj lb5m6g5c j4enbv94 gfz4du6o r7fjleex lhj4utae ljrqcn24" title="Hila Chefer (@hila_chefer) on X"><span dir="ltr" aria-label="" class="ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae i033jvx7 cw3vfol9 _11JPr" style="min-height: 0px;">Hila Chefer (@hila_chefer) on X</span></div><div class="jjtagnqy tvf2evcx ttu8nud2 lb5m6g5c dsh4tgtl gfz4du6o r7fjleex lhj4utae aoi073rw psqdn66s c32ccnay"><span class="g0rxnol2 gfz4du6o r7fjleex" style="display: -webkit-box; overflow: hidden; -webkit-box-orient: vertical; -webkit-line-clamp: 2;"><span dir="ltr" title="2024 is the year of text-to-video models 🎥

Join me tomorrow as we dive into ✨Lumiere✨our T2V model from @GoogleAI!
We'll discuss Lumiere's architecture, applications, and more🤩

⏰ tomorrow at 20:00 IDT (10:00 PDT)
🚀 sign up now: https://t.co/zVT2quQqu5" aria-label="" class="i033jvx7 _11JPr" style="min-height: 0px;">2024 is the year of text-to-video models <img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="🎥" draggable="false" class="b27 emoji apple _11JPr" tabindex="-1" style="background-position: -20px 0px;">

Join me tomorrow as we dive into <img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="✨" draggable="false" class="b7 emoji apple _11JPr" tabindex="-1" style="background-position: -80px -40px;">Lumiere<img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="✨" draggable="false" class="b7 emoji apple _11JPr" tabindex="-1" style="background-position: -80px -40px;">our T2V model from @GoogleAI!
We'll discuss Lumiere's architecture, applications, and more<img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="🤩" draggable="false" class="b105 emoji apple _11JPr" tabindex="-1" style="background-position: -40px 0px;">

<img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="⏰" draggable="false" class="b1 emoji apple _11JPr" tabindex="-1" style="background-position: -40px -40px;"> tomorrow at 20:00 IDT (10:00 PDT)
<img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="🚀" draggable="false" class="b93 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -40px;"> sign up now: https://t.co/zVT2quQqu5</span></span></div><div class="j8h4s3v0 tvf2evcx oq44ahr5 lb5m6g5c dsh4tgtl qbqilfqo"><span dir="ltr" aria-label="" class="_11JPr" style="min-height: 0px;">twitter.com</span></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Happing right now:
Excited to share insights about our text-to-video model<img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="✨" draggable="false" class="b7 emoji apple _11JPr selectable-text copyable-text" data-plain-text="✨" tabindex="-1" style="background-position: -80px -40px;">Google's Lumiere<img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="✨" draggable="false" class="b7 emoji apple _11JPr selectable-text copyable-text" data-plain-text="✨" tabindex="-1" style="background-position: -80px -40px;">
Join us at the Vision-Language Club meetup <img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="📅" draggable="false" class="b76 emoji apple _11JPr selectable-text copyable-text" data-plain-text="📅" tabindex="-1" style="background-position: 0px -20px;">
We'll explore Lumiere's architecture, applications, and results <img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="💡" draggable="false" class="b74 emoji apple _11JPr selectable-text copyable-text" data-plain-text="💡" tabindex="-1" style="background-position: -80px -40px;">
<a dir="auto" href="https://twitter.com/hila_chefer/status/1761792814868750745" title="https://twitter.com/hila_chefer/status/1761792814868750745" target="_blank" rel="noopener noreferrer" class="_11JPr selectable-text copyable-text">https://twitter.com/hila_chefer/status/1761792814868750745</a>

Join at <a dir="auto" href="https://technion.zoom.us/j/98830698694" title="https://technion.zoom.us/j/98830698694" target="_blank" rel="noopener noreferrer" class="_11JPr selectable-text copyable-text">https://technion.zoom.us/j/98830698694</a></span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">11:35 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">11:35 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="_1jHIY _2OvAm _2zFLj focusable-list-item" tabindex="-1"><div class="UzMP7 n6BPp"><span dir="auto" aria-label="" class="_11JPr" style="min-height: 0px;">TODAY</span></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A95789A7AD2ECFF725D_917219388991@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Atik Shaikh +91 72193 88991" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno a82mwcnm"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">A</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d giysludt" role=""><span dir="auto" aria-label="Maybe Atik Shaikh" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Atik Shaikh</span><span class="WJuYU" testid="author" role="button">+91 72193 88991</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[0:01 am, 27/02/2024] +91 72193 88991: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="b2boiwr0 _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d mglqvgej" role=""><span dir="auto" aria-label="Maybe chetanya rastogi" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">chetanya rastogi</span><span class="WJuYU" testid="author">+1 (650) 334-6780</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">https://www.ft.com/content/cd6eb51a-3276-450f-87fd-97e8410db9eb</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Microsoft seems to be the ultimate winner in this AI war <img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="🫡" draggable="false" class="b142 emoji apple _11JPr selectable-text copyable-text" data-plain-text="🫡" tabindex="-1" style="background-position: -20px -40px;"></span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">12:01 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">12:01 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_8C974B85805BE08BB512AB1F108E056E_447505487797@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Ganaraj +44 7505 487797" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/357029048_327201076325969_1848146647852128456_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRSyKF7yeee8lWajqAEgBA6aceUAZCQopKkjl7QAXDNmQ&amp;oe=65EAD16A&amp;_nc_sid=e6ed6c&amp;_nc_cat=108" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d aqebjpab" role=""><span dir="auto" aria-label="Maybe Ganaraj" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Ganaraj</span><span class="WJuYU" testid="author" role="button">+44 7505 487797</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[0:02 am, 27/02/2024] +44 7505 487797: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="a82mwcnm _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d giysludt" role=""><span dir="auto" aria-label="Maybe Atik Shaikh" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Atik Shaikh</span><span class="WJuYU" testid="author">+91 72193 88991</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Microsoft seems to be the ultimate winner in this AI war <img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="🫡" draggable="false" class="b142 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -40px;"></span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>When everyone is out digging for gold, the winner is the guy who is selling shovels <img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="😂" draggable="false" class="b85 emoji apple _11JPr selectable-text copyable-text" data-plain-text="😂" tabindex="-1" style="background-position: -60px -60px;"></span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">12:02 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">12:02 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="Reactions ❤, 😂 3 in total"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="❤️" draggable="false" class="bd96tnyg eipqoq80 b8 emoji apple _11JPr" tabindex="-1" style="background-position: -20px 0px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="😂" draggable="false" class="bd96tnyg p83aruvr b85 emoji apple _11JPr" tabindex="-1" style="background-position: -60px -60px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk o2es7gts i2cterl7" style="flex-shrink: 0;"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 f8m0rgwh r15c9g6i ilf8vifs gfz4du6o r7fjleex qnz2jpws kjemk6od"><div class="dn5wc90o ilf8vifs gndfcl4n p357zi0d"><span class="f8jlpxt4 qfejxiq4 ce5kru2g">3</span></div></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3AD9D00B85D4C0C2F99B_919822381281@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Dilip Thomas Ittyera +91 98223 81281" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/309186150_817654219469995_3888201881878390184_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRh29wTc1XaHX8pEd-BMHein5ijlTG5dv5VX2PQwa0OVw&amp;oe=65EAE41B&amp;_nc_sid=e6ed6c&amp;_nc_cat=110" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d dud7p2xx" role=""><span dir="auto" aria-label="Maybe Dilip Thomas Ittyera" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Dilip Thomas Ittyera</span><span class="WJuYU" testid="author" role="button">+91 98223 81281</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[0:03 am, 27/02/2024] +91 98223 81281: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="a82mwcnm _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d giysludt" role=""><span dir="auto" aria-label="Maybe Atik Shaikh" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Atik Shaikh</span><span class="WJuYU" testid="author">+91 72193 88991</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Microsoft seems to be the ultimate winner in this AI war <img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="🫡" draggable="false" class="b142 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -40px;"></span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Winning is at a point in time. And it never ends in one war</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">12:03 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">12:03 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="Reactions ❤, 💯, 👌 6 in total"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="❤️" draggable="false" class="bd96tnyg eipqoq80 b8 emoji apple _11JPr" tabindex="-1" style="background-position: -20px 0px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="💯" draggable="false" class="bd96tnyg p83aruvr b75 emoji apple _11JPr" tabindex="-1" style="background-position: -60px -20px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👌" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: 0px -40px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk o2es7gts i2cterl7" style="flex-shrink: 0;"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 f8m0rgwh r15c9g6i ilf8vifs gfz4du6o r7fjleex qnz2jpws kjemk6od"><div class="dn5wc90o ilf8vifs gndfcl4n p357zi0d"><span class="f8jlpxt4 qfejxiq4 ce5kru2g">6</span></div></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3AE616783A3DA9F3A888_917219388991@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Atik Shaikh +91 72193 88991" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno a82mwcnm"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">A</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d giysludt" role=""><span dir="auto" aria-label="Maybe Atik Shaikh" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Atik Shaikh</span><span class="WJuYU" testid="author" role="button">+91 72193 88991</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[0:04 am, 27/02/2024] +91 72193 88991: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="awij7w2s _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d aqebjpab" role=""><span dir="auto" aria-label="Maybe Ganaraj" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Ganaraj</span><span class="WJuYU" testid="author">+44 7505 487797</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">When everyone is out digging for gold, the winner is the guy who is selling shovels <img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="😂" draggable="false" class="b85 emoji apple _11JPr" tabindex="-1" style="background-position: -60px -60px;"></span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><div class="g0rxnol2 pglj95m3 p357zi0d ac2vgrno n1yiu2zv tt8xd2xn aumn88wd k0lnf8n4 htq1q0kf n3nyt3io _11JPr selectable-text copyable-text"><span class="Ov-s3"><img crossorigin="anonymous" src="/img/c5a15be93e425dcb8a26b06645ad4574_a_2144-64.png" alt="😂" draggable="false" class="URwQL _2KRZE _11JPr selectable-text copyable-text" data-plain-text="😂" tabindex="-1" style="visibility: visible;"></span><span class="Ov-s3"><img crossorigin="anonymous" src="/img/c5a15be93e425dcb8a26b06645ad4574_a_1884-64.png" alt="💯" draggable="false" class="URwQL _2KRZE _11JPr selectable-text copyable-text" data-plain-text="💯" tabindex="-1" style="visibility: visible;"></span></div></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">12:04 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A671C4EC200AB29D22E_919822381281@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Dilip Thomas Ittyera +91 98223 81281" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/309186150_817654219469995_3888201881878390184_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRh29wTc1XaHX8pEd-BMHein5ijlTG5dv5VX2PQwa0OVw&amp;oe=65EAE41B&amp;_nc_sid=e6ed6c&amp;_nc_cat=110" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d dud7p2xx" role=""><span dir="auto" aria-label="Maybe Dilip Thomas Ittyera" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Dilip Thomas Ittyera</span><span class="WJuYU" testid="author" role="button">+91 98223 81281</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[0:06 am, 27/02/2024] +91 98223 81281: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="awij7w2s _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d aqebjpab" role=""><span dir="auto" aria-label="Maybe Ganaraj" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Ganaraj</span><span class="WJuYU" testid="author">+44 7505 487797</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">When everyone is out digging for gold, the winner is the guy who is selling shovels <img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="😂" draggable="false" class="b85 emoji apple _11JPr" tabindex="-1" style="background-position: -60px -60px;"></span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>I guess however much shovels as many could sell when the treasure is found and the digging ends for the time being, there will only be a few winners who struck gold</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">12:06 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">12:06 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_AED94E648E9EABD2DACB8228C4E82CCB_918072173935@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Anubhav Mishra +91 80721 73935" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/422474244_2207779056257205_6380017147338436444_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdQcBWFoU7N5fP-tIl9Il6cyVmnu6_LWDTkkQQWClawzfg&amp;oe=65EAE6EE&amp;_nc_sid=e6ed6c&amp;_nc_cat=104" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo hbm07cg7 n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d gvktkc8w" role=""><span dir="auto" aria-label="Maybe Anubhav Mishra" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Anubhav Mishra</span><span class="WJuYU" testid="author" role="button">+91 80721 73935</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[0:20 am, 27/02/2024] +91 80721 73935: "><div class="M6sU5"><div class="xzT4z gndfcl4n l8fojup5 paxyh2gw sfeitywo cqsf3vkf k2bacm8l ajgl1lbb p357zi0d f8m0rgwh ovllcyds kvdvyush gfz4du6o r7fjleex rrq4r3yd"><div class="gndfcl4n p357zi0d ac2vgrno gfz4du6o r7fjleex g0rxnol2 ln8gz9je" style="height: 173.057px;"><img alt="" class="tvf2evcx oq44ahr5 lb5m6g5c ppled2lx ln8gz9je jpthtbts" src="..."></div><div class="k5z04t9s cm280p3y p357zi0d ggj6brxn m0h2a7mj lb5m6g5c f8m0rgwh ac2vgrno idwf4z32 onmzirdk gfz4du6o r7fjleex n1yiu2zv itegkywt eb4rp10x rppts313 ln8gz9je"><div class="aoi073rw suguakab jjtagnqy c32ccnay tvf2evcx m0h2a7mj lb5m6g5c j4enbv94 gfz4du6o r7fjleex lhj4utae ljrqcn24" title="PRESS RELEASE: Future Software Should Be Memory Safe | ONCD | The White House"><span dir="ltr" aria-label="" class="ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae i033jvx7 cw3vfol9 _11JPr" style="min-height: 0px;">PRESS RELEASE: Future Software Should Be Memory Safe | ONCD | The White House</span></div><div class="jjtagnqy tvf2evcx ttu8nud2 lb5m6g5c dsh4tgtl gfz4du6o r7fjleex lhj4utae"><span class="g0rxnol2 gfz4du6o r7fjleex" style="display: -webkit-box; overflow: hidden; -webkit-box-orient: vertical; -webkit-line-clamp: 2;"><span dir="ltr" title="Leaders in Industry Support White House Call to Address Root Cause of Many of the Worst Cyber Attacks Read the full report here WASHINGTON – Today, the White House Office of the National Cyber Director (ONCD) released a report calling on the technical community to proactively reduce the attack surface in cyberspace. ONCD makes the…" aria-label="" class="i033jvx7 _11JPr" style="min-height: 0px;">Leaders in Industry Support White House Call to Address Root Cause of Many of the Worst Cyber Attacks Read the full report here WASHINGTON – Today, the White House Office of the National Cyber Director (ONCD) released a report calling on the technical community to proactively reduce the attack surface in cyberspace. ONCD makes the…</span></span></div><div class="j8h4s3v0 tvf2evcx oq44ahr5 lb5m6g5c dsh4tgtl qbqilfqo"><span dir="ltr" aria-label="" class="_11JPr" style="min-height: 0px;">www.whitehouse.gov</span></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span><a dir="auto" href="https://www.whitehouse.gov/oncd/briefing-room/2024/02/26/press-release-technical-report/" title="https://www.whitehouse.gov/oncd/briefing-room/2024/02/26/press-release-technical-report/" target="_blank" rel="noopener noreferrer" class="_11JPr selectable-text copyable-text">https://www.whitehouse.gov/oncd/briefing-room/2024/02/26/press-release-technical-report/</a>

RIP C++</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">12:20 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">12:20 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="Reactions 😮, 😢 2 in total"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="😮" draggable="false" class="bd96tnyg p83aruvr b87 emoji apple _11JPr" tabindex="-1" style="background-position: -40px -40px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="😢" draggable="false" class="bd96tnyg p83aruvr b87 emoji apple _11JPr" tabindex="-1" style="background-position: 0px 0px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk o2es7gts i2cterl7" style="flex-shrink: 0;"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 f8m0rgwh r15c9g6i ilf8vifs gfz4du6o r7fjleex qnz2jpws kjemk6od"><div class="dn5wc90o ilf8vifs gndfcl4n p357zi0d"><span class="f8jlpxt4 qfejxiq4 ce5kru2g">2</span></div></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_E345CB4F3766D6202BB21A24D9435774_447901983685@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Abhishek Shivkumar +44 7901 983685" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno oj03u76w"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">A</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d dud7p2xx" role=""><span dir="auto" aria-label="Maybe Abhishek Shivkumar" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Abhishek Shivkumar</span><span class="WJuYU" testid="author" role="button">+44 7901 983685</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[0:23 am, 27/02/2024] +44 7901 983685: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="j3zl77sr _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d gvktkc8w" role=""><span dir="auto" aria-label="Maybe Anubhav Mishra" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Anubhav Mishra</span><span class="WJuYU" testid="author">+91 80721 73935</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">https://www.whitehouse.gov/oncd/briefing-room/2024/02/26/press-release-technical-report/

RIP C++</span></div></div></div><div class="jrEtJ"><div class="kknmh"><div class="ln8gz9je ppled2lx t3g6t33p qnwaluaf" style="background-image: url(&quot;data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/4gIoSUNDX1BST0ZJTEUAAQEAAAIYAAAAAAQwAABtbnRyUkdCIFhZWiAAAAAAAAAAAAAAAABhY3NwAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAQAA9tYAAQAAAADTLQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAlkZXNjAAAA8AAAAHRyWFlaAAABZAAAABRnWFlaAAABeAAAABRiWFlaAAABjAAAABRyVFJDAAABoAAAAChnVFJDAAABoAAAAChiVFJDAAABoAAAACh3dHB0AAAByAAAABRjcHJ0AAAB3AAAADxtbHVjAAAAAAAAAAEAAAAMZW5VUwAAAFgAAAAcAHMAUgBHAEIAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAFhZWiAAAAAAAABvogAAOPUAAAOQWFlaIAAAAAAAAGKZAAC3hQAAGNpYWVogAAAAAAAAJKAAAA+EAAC2z3BhcmEAAAAAAAQAAAACZmYAAPKnAAANWQAAE9AAAApbAAAAAAAAAABYWVogAAAAAAAA9tYAAQAAAADTLW1sdWMAAAAAAAAAAQAAAAxlblVTAAAAIAAAABwARwBvAG8AZwBsAGUAIABJAG4AYwAuACAAMgAwADEANv/bAEMABgQFBgUEBgYFBgcHBggKEAoKCQkKFA4PDBAXFBgYFxQWFhodJR8aGyMcFhYgLCAjJicpKikZHy0wLSgwJSgpKP/bAEMBBwcHCggKEwoKEygaFhooKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKP/AABEIAE4ATgMBIgACEQEDEQH/xAAcAAACAgMBAQAAAAAAAAAAAAABAgYHAwQFCAD/xAAwEAACAQMEAQQBAQcFAAAAAAABAgMEBREABhIhEwcUIjFBIxUWJDJCUWEXM3GBlP/EABoBAAMBAQEBAAAAAAAAAAAAAAIDBAEABQb/xAAmEQACAgECBQQDAAAAAAAAAAAAAQIRAyExBBJBcfBRYZHhIoGh/9oADAMBAAIRAxEAPwCgwNEDTY0wGvq1E8lsQLpsabGjjRqILYuNEDTY0QNGogtigaONNjRA0aiA2KBo402NHGjSBbMIGjjTY1fM98tlqp6upunp7O8MdLFDNPHbaA01PI8UbLKHWnIVyHVuLuwBkxx+gJMk3CqVlEVzbsoUDUg2xs+97nprhUWWkjmp7eiyVUklRFCsKkMQxMjKMYRjn8Y71aMu47LRSs1ZsO5otq8fvkmtVAo+bFk8/wDCApyBCjBTIAx3kmZbd3+LXXX+quG2L/JuQ0sNRHHX0lPFUVIMq06qGjgjc5aRAMh+lIGMYKcnEZFH8I69w444t6sqrafpJuKq3MlLcaCjCUcyGrppazJI7fxv4PI8XNFbi7KAesEkqDqXP0y3FVX25rarTTQ0yTqUgNwiBgSZv0EbyssgZgVAEiq5yMqCcavqD1OvFHtaGppdgyxSRVy2qO0RTus6EU/mGI/ACFCfjH0M/Xek3D6qVFsSxXO4bMFXQyUcFyaspqrzNQJOCvYMQ4sVLAElQ3YB+8TriuJ5rUV8/YbxYq38+Dy1uGx3Dbl4qLVeaf21fT8fJFzV+PJQw7UkHpgejrn416S3Lu6jTcG4IKDa97q7YaoWqoWCgo2ppZ4wI417p3ZmVuLIrOe+PxweOuBPuex0aBav0+r4jaXVq9pbXRKWEjAotR/CgICOl48CQfsnB1XDisjSuGvcnliinpIo3GjjVr+oVxpZ7BdIqfaU1km95B5DVUFJE8CuJXWJOMKSANwJ5EscRgE/I5qwLqvFkc1bVE+RcrpMwAau3fUQNj3LJFVRLMKGkEkBpmZmi8dr78nkAU8uGBwPQfJ7XFKgatu5bh9PLn5RW3D1DdJUWN41akRGVVjUAouF+oYvx341J7Gpc6fNFpbfRTBqmit5aeyhq7w3C4sqBTSF6FF8xx8hJiY+PB6GOef8fWrw3VRRCe+y3etpZ6FbfSpJKtHJGixNeGL5QSs7DKufi6krgdHvUDH+kwPR32P/ACane27xs26WXcNHQ3zc8FZVrBiuvNfSJLC6StMjRFpUJ/UyzAHPf2pOdJzzbqVPT290HjSVrQj99Wkj9D5YkmM9Mb9D4Z5YDE4xbkZfgrsAxxxOSw+ROB1jW9XI6CsbaTzNLS1D7YoPa01PB5IiSZPiWaQMoHWDhyfz/mSVNh23L6dQbXG6ovItxFwkrJKuhfkREYgip7ocQF4/1H6P4IA2L7Z9r3e77Xq5txUwp7LRU1C8Bq6E+5SFiT8hVDjyBI+jjS4ZYxknb3fn7OlFtV2ILuGK0Nvy/NdasRyHcNQojlpHkiEfuU5uWjlVs8eXx49gNhgxXEc3XBQrcKjlW1MkqUdF7PFGqCVDTx/7n6p8ZC4+uYJz/KNWNuq5enEO77lWCr3jBWTT+5ke1VVK8PkYrJlHDN/WFb7OGX8YGI/LJ6VzOGmffsjBVQF2pSQqgBR/wAAB/YAaoxTaSdPb0E5EnatHJqorbHsK6LaKmepg/aFBzeemED8/HW9FRI4+sdg/9ZzmHhdT+/1+yBtart+3Zt2moeSKWOOv9sYOSFvsr8hhZZcAdZbvUEA1Vhbabd79eyJ8jSpGtjWWmiWaoijkmjgR3CtLIGKoCf5jxBOB99An+wOlxoga4Nsldr2RNc6Y1NJebS1OkjpJIxnAjVY55OZHizxKU0hGAT2oIByBhpdm1Ulka6VVdRUMMVXJSTxVAm8tOyAFmdFjYhOTxpnvDSIDjOo3jXTs1TbKYs1ztstewlidFWq8KcFbMiMApJ5L0CCpX77+tLamtmbzR9DbqLBQwTxxfvTZJAz8WeNKsrGOJPInwdjIC9ZOWHWMkbK7OnF2oKCW6WuKSrty3PyF5GWGMxGXjJxQlXCDljBGCO+9a9tuVlhq2e4bfWqpzSrAI0rJImEvx5Tcu/kcPhccRyHRxg/Wm5WaBqX9pbfWsVHJm8dZJCZVxFxAPfE5STJwciZgOJCleufS/wCA3HyzoWzYlZc3gW33K21CyyCMOhlCAmeCJcsYwO/cxNgZZVJ5BW+OubW2EUVvSpnuVD5JIY54adRKzTI4HasI/H8SWVssMNG6/Y1z694J7hUy0dOaWlklZooOZfxISSq8j22BgZP3jWIDRxUt2xcpR6IUDTgaIXTgaaKs0wNHGmA0caTZQxcaONNjRA1tgMXGiBpsaYDW2AwAaYDRA04GtsFgA04GiBpwNbYLZ//Z&quot;);"></div></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>RUST !</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">12:23 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">12:23 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_23373792AA2C40B4ED550CD26AE03844_919334372044@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Sid +91 93343 72044" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno akr8p6hy"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">S</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d jake6gi8" role=""><span dir="auto" aria-label="Maybe Sid" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Sid</span><span class="WJuYU" testid="author" role="button">+91 93343 72044</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[0:25 am, 27/02/2024] +91 93343 72044: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="oj03u76w _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d dud7p2xx" role=""><span dir="auto" aria-label="Maybe Abhishek Shivkumar" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Abhishek Shivkumar</span><span class="WJuYU" testid="author">+44 7901 983685</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">RUST !</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>COBOL</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">12:25 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">12:25 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB0DF9645FD5A6BD4423C_918250673368@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Anmol +91 82506 73368" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno b2boiwr0"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">A</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d mglqvgej" role=""><span dir="auto" aria-label="Maybe Anmol" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Anmol</span><span class="WJuYU" testid="author" role="button">+91 82506 73368</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[0:26 am, 27/02/2024] +91 82506 73368: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="j3zl77sr _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d gvktkc8w" role=""><span dir="auto" aria-label="Maybe Anubhav Mishra" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Anubhav Mishra</span><span class="WJuYU" testid="author">+91 80721 73935</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">https://www.whitehouse.gov/oncd/briefing-room/2024/02/26/press-release-technical-report/

RIP C++</span></div></div></div><div class="jrEtJ"><div class="kknmh"><div class="ln8gz9je ppled2lx t3g6t33p qnwaluaf" style="background-image: url(&quot;data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/4gIoSUNDX1BST0ZJTEUAAQEAAAIYAAAAAAQwAABtbnRyUkdCIFhZWiAAAAAAAAAAAAAAAABhY3NwAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAQAA9tYAAQAAAADTLQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAlkZXNjAAAA8AAAAHRyWFlaAAABZAAAABRnWFlaAAABeAAAABRiWFlaAAABjAAAABRyVFJDAAABoAAAAChnVFJDAAABoAAAAChiVFJDAAABoAAAACh3dHB0AAAByAAAABRjcHJ0AAAB3AAAADxtbHVjAAAAAAAAAAEAAAAMZW5VUwAAAFgAAAAcAHMAUgBHAEIAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAFhZWiAAAAAAAABvogAAOPUAAAOQWFlaIAAAAAAAAGKZAAC3hQAAGNpYWVogAAAAAAAAJKAAAA+EAAC2z3BhcmEAAAAAAAQAAAACZmYAAPKnAAANWQAAE9AAAApbAAAAAAAAAABYWVogAAAAAAAA9tYAAQAAAADTLW1sdWMAAAAAAAAAAQAAAAxlblVTAAAAIAAAABwARwBvAG8AZwBsAGUAIABJAG4AYwAuACAAMgAwADEANv/bAEMABgQFBgUEBgYFBgcHBggKEAoKCQkKFA4PDBAXFBgYFxQWFhodJR8aGyMcFhYgLCAjJicpKikZHy0wLSgwJSgpKP/bAEMBBwcHCggKEwoKEygaFhooKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKP/AABEIAE4ATgMBIgACEQEDEQH/xAAcAAACAgMBAQAAAAAAAAAAAAABAgYHAwQFCAD/xAAwEAACAQMEAQQBAQcFAAAAAAABAgMEBREABhIhEwcUIjFBIxUWJDJCUWEXM3GBlP/EABoBAAMBAQEBAAAAAAAAAAAAAAIDBAEABQb/xAAmEQACAgECBQQDAAAAAAAAAAAAAQIRAyExBBJBcfBRYZHhIoGh/9oADAMBAAIRAxEAPwCgwNEDTY0wGvq1E8lsQLpsabGjjRqILYuNEDTY0QNGogtigaONNjRA0aiA2KBo402NHGjSBbMIGjjTY1fM98tlqp6upunp7O8MdLFDNPHbaA01PI8UbLKHWnIVyHVuLuwBkxx+gJMk3CqVlEVzbsoUDUg2xs+97nprhUWWkjmp7eiyVUklRFCsKkMQxMjKMYRjn8Y71aMu47LRSs1ZsO5otq8fvkmtVAo+bFk8/wDCApyBCjBTIAx3kmZbd3+LXXX+quG2L/JuQ0sNRHHX0lPFUVIMq06qGjgjc5aRAMh+lIGMYKcnEZFH8I69w444t6sqrafpJuKq3MlLcaCjCUcyGrppazJI7fxv4PI8XNFbi7KAesEkqDqXP0y3FVX25rarTTQ0yTqUgNwiBgSZv0EbyssgZgVAEiq5yMqCcavqD1OvFHtaGppdgyxSRVy2qO0RTus6EU/mGI/ACFCfjH0M/Xek3D6qVFsSxXO4bMFXQyUcFyaspqrzNQJOCvYMQ4sVLAElQ3YB+8TriuJ5rUV8/YbxYq38+Dy1uGx3Dbl4qLVeaf21fT8fJFzV+PJQw7UkHpgejrn416S3Lu6jTcG4IKDa97q7YaoWqoWCgo2ppZ4wI417p3ZmVuLIrOe+PxweOuBPuex0aBav0+r4jaXVq9pbXRKWEjAotR/CgICOl48CQfsnB1XDisjSuGvcnliinpIo3GjjVr+oVxpZ7BdIqfaU1km95B5DVUFJE8CuJXWJOMKSANwJ5EscRgE/I5qwLqvFkc1bVE+RcrpMwAau3fUQNj3LJFVRLMKGkEkBpmZmi8dr78nkAU8uGBwPQfJ7XFKgatu5bh9PLn5RW3D1DdJUWN41akRGVVjUAouF+oYvx341J7Gpc6fNFpbfRTBqmit5aeyhq7w3C4sqBTSF6FF8xx8hJiY+PB6GOef8fWrw3VRRCe+y3etpZ6FbfSpJKtHJGixNeGL5QSs7DKufi6krgdHvUDH+kwPR32P/ACane27xs26WXcNHQ3zc8FZVrBiuvNfSJLC6StMjRFpUJ/UyzAHPf2pOdJzzbqVPT290HjSVrQj99Wkj9D5YkmM9Mb9D4Z5YDE4xbkZfgrsAxxxOSw+ROB1jW9XI6CsbaTzNLS1D7YoPa01PB5IiSZPiWaQMoHWDhyfz/mSVNh23L6dQbXG6ovItxFwkrJKuhfkREYgip7ocQF4/1H6P4IA2L7Z9r3e77Xq5txUwp7LRU1C8Bq6E+5SFiT8hVDjyBI+jjS4ZYxknb3fn7OlFtV2ILuGK0Nvy/NdasRyHcNQojlpHkiEfuU5uWjlVs8eXx49gNhgxXEc3XBQrcKjlW1MkqUdF7PFGqCVDTx/7n6p8ZC4+uYJz/KNWNuq5enEO77lWCr3jBWTT+5ke1VVK8PkYrJlHDN/WFb7OGX8YGI/LJ6VzOGmffsjBVQF2pSQqgBR/wAAB/YAaoxTaSdPb0E5EnatHJqorbHsK6LaKmepg/aFBzeemED8/HW9FRI4+sdg/9ZzmHhdT+/1+yBtart+3Zt2moeSKWOOv9sYOSFvsr8hhZZcAdZbvUEA1Vhbabd79eyJ8jSpGtjWWmiWaoijkmjgR3CtLIGKoCf5jxBOB99An+wOlxoga4Nsldr2RNc6Y1NJebS1OkjpJIxnAjVY55OZHizxKU0hGAT2oIByBhpdm1Ulka6VVdRUMMVXJSTxVAm8tOyAFmdFjYhOTxpnvDSIDjOo3jXTs1TbKYs1ztstewlidFWq8KcFbMiMApJ5L0CCpX77+tLamtmbzR9DbqLBQwTxxfvTZJAz8WeNKsrGOJPInwdjIC9ZOWHWMkbK7OnF2oKCW6WuKSrty3PyF5GWGMxGXjJxQlXCDljBGCO+9a9tuVlhq2e4bfWqpzSrAI0rJImEvx5Tcu/kcPhccRyHRxg/Wm5WaBqX9pbfWsVHJm8dZJCZVxFxAPfE5STJwciZgOJCleufS/wCA3HyzoWzYlZc3gW33K21CyyCMOhlCAmeCJcsYwO/cxNgZZVJ5BW+OubW2EUVvSpnuVD5JIY54adRKzTI4HasI/H8SWVssMNG6/Y1z694J7hUy0dOaWlklZooOZfxISSq8j22BgZP3jWIDRxUt2xcpR6IUDTgaIXTgaaKs0wNHGmA0caTZQxcaONNjRA1tgMXGiBpsaYDW2AwAaYDRA04GtsFgA04GiBpwNbYLZ//Z&quot;);"></div></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>omg!!</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">12:26 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">12:26 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A0059492D0F0AD09130_19377081307@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Dr. Pratik Desai +1 (937) 708-1307" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/407052186_368942458989574_5837140328940210822_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRT7N2iCE5tWNPEgrn8zUMppMyQOh2yI-vDe0xjKk_mmA&amp;oe=65EAD8D6&amp;_nc_sid=e6ed6c&amp;_nc_cat=110" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe Dr. Pratik Desai" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Dr. Pratik Desai</span><span class="WJuYU" testid="author" role="button">+1 (937) 708-1307</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[0:28 am, 27/02/2024] +1 (937) 708-1307: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Anything that helps php to die is welcomed.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">12:28 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">12:28 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="Reactions 😂, 👍 8 in total"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="😂" draggable="false" class="bd96tnyg p83aruvr b85 emoji apple _11JPr" tabindex="-1" style="background-position: -60px -60px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk o2es7gts i2cterl7" style="flex-shrink: 0;"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 f8m0rgwh r15c9g6i ilf8vifs gfz4du6o r7fjleex qnz2jpws kjemk6od"><div class="dn5wc90o ilf8vifs gndfcl4n p357zi0d"><span class="f8jlpxt4 qfejxiq4 ce5kru2g">8</span></div></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_7571419151708972564_919971016948@c.us"><div class="_2OvAm focusable-list-item _1jHIY"><span class=""></span><div class="UzMP7 n6BPp"><div class="_1BOF7"><div class="_1sykI"><div class="Mk0Bp _30scZ"><span dir="auto" aria-label="" class="l7jjieqr ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">+91 99710 16948</span></div> joined from the community</div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"></div></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj _3sxvM" data-id="false_120363049558306142@g.us_3A45F2AC3B2289FA1D1E_919404434273@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Tushar | Billion Gradient +91 94044 34273" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/426383821_414642844458538_3319204348665974805_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdSiMOkY_kflpsSI0o69aMyu7NEXW8MCifPSolz5oUMudg&amp;oe=65EAEA22&amp;_nc_sid=e6ed6c&amp;_nc_cat=100" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d oyzge2kb" role=""><span dir="auto" aria-label="Maybe Tushar | Billion Gradient" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Tushar | Billion Gradient</span><span class="WJuYU" testid="author" role="button">+91 94044 34273</span></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t r15c9g6i"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l mhp1pqu9 nu7pwgvd dnb887gk gjuq5ydh i2cterl7 l7jjieqr a6r0u4sv bx0vhl82 ntr8esoy"><span data-icon="recalled" class=""><svg viewBox="0 0 24 24" height="24" width="24" preserveAspectRatio="xMidYMid meet" class="" fill="none"><title>recalled</title><path fill-rule="evenodd" clip-rule="evenodd" d="M7.75897 6.43054C8.93584 5.533 10.4057 5 12 5C15.866 5 19 8.13401 19 12C19 13.5943 18.467 15.0642 17.5695 16.241L7.75897 6.43054ZM6.35707 7.85707C5.50399 9.01706 5 10.4497 5 12C5 15.866 8.13401 19 12 19C13.5503 19 14.9829 18.496 16.1429 17.6429L6.35707 7.85707ZM12 3C7.02944 3 3 7.02944 3 12C3 16.9706 7.02944 21 12 21C16.9706 21 21 16.9706 21 12C21 7.02944 16.9706 3 12 3Z" fill="currentColor"></path></svg></span></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 kiiy14zj j4enbv94"><div class="_21Ahp c4mcV">This message was deleted<span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:32 am</span></span></span></div></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:32 am</span></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"></div></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A96B02160AB6508FD92_919404434273@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a"><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d oyzge2kb" role=""><span dir="auto" aria-label="Maybe Tushar | Billion Gradient" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Tushar | Billion Gradient</span><span class="WJuYU" testid="author" role="button">+91 94044 34273</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[1:34 am, 27/02/2024] +91 94044 34273: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="awij7w2s _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d aqebjpab" role=""><span dir="auto" aria-label="Maybe Ganaraj" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Ganaraj</span><span class="WJuYU" testid="author">+44 7505 487797</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">When everyone is out digging for gold, the winner is the guy who is selling shovels <img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="😂" draggable="false" class="b85 emoji apple _11JPr" tabindex="-1" style="background-position: -60px -60px;"></span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="M6sU5"><div class="xzT4z gndfcl4n l8fojup5 paxyh2gw sfeitywo cqsf3vkf k2bacm8l ajgl1lbb p357zi0d sap93d0t ovllcyds kvdvyush gfz4du6o r7fjleex rrq4r3yd"><img alt="" class="tvf2evcx oq44ahr5 lb5m6g5c n43pk08i fpj7ivsd jpthtbts" src="data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAASABIAAD/4QBYRXhpZgAATU0AKgAAAAgAAgESAAMAAAABAAEAAIdpAAQAAAABAAAAJgAAAAAAA6ABAAMAAAABAAEAAKACAAQAAAABAAAAjKADAAQAAAABAAAAjAAAAAD/7QA4UGhvdG9zaG9wIDMuMAA4QklNBAQAAAAAAAA4QklNBCUAAAAAABDUHYzZjwCyBOmACZjs+EJ+/8AAEQgAjACMAwEiAAIRAQMRAf/EAB8AAAEFAQEBAQEBAAAAAAAAAAABAgMEBQYHCAkKC//EALUQAAIBAwMCBAMFBQQEAAABfQECAwAEEQUSITFBBhNRYQcicRQygZGhCCNCscEVUtHwJDNicoIJChYXGBkaJSYnKCkqNDU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6g4SFhoeIiYqSk5SVlpeYmZqio6Slpqeoqaqys7S1tre4ubrCw8TFxsfIycrS09TV1tfY2drh4uPk5ebn6Onq8fLz9PX29/j5+v/EAB8BAAMBAQEBAQEBAQEAAAAAAAABAgMEBQYHCAkKC//EALURAAIBAgQEAwQHBQQEAAECdwABAgMRBAUhMQYSQVEHYXETIjKBCBRCkaGxwQkjM1LwFWJy0QoWJDThJfEXGBkaJicoKSo1Njc4OTpDREVGR0hJSlNUVVZXWFlaY2RlZmdoaWpzdHV2d3h5eoKDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uLj5OXm5+jp6vLz9PX29/j5+v/bAEMAAgICAgICAwICAwQDAwMEBQQEBAQFBwUFBQUFBwgHBwcHBwcICAgICAgICAoKCgoKCgsLCwsLDQ0NDQ0NDQ0NDf/bAEMBAgICAwMDBgMDBg0JBwkNDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDf/dAAQACf/aAAwDAQACEQMRAD8A/EVQKlUdqYAM1YUCtogCg4q0iCo0XnNWlHrRIACjNejfDTwsPE3iiytrhGa0WVTMVO049M9RnuR2ri9PsZ9Qu47S2QySysEVQO7HAr9NPhH8BJPCtrZT69GlvcXCiVmDb2wcHkDPP5YrKpK0bmlKHM7HZD4Ew6L4g0+HSpCseqWxZ44wAsUKjkDPYZxn17mvbNH+HmkaBbeRpVkgUgbiRksfc9a9F8HWdvdkXmC7KnkIW6iMMSB/Imu21CKC3g+dQXGT0r5HMqs6tRrm0P0DJaVOlRU+XVnx5450kxRSNCAqxZ3J2PsK/P8A+KlraaNHJevb7jJKXZUzhkzgjHt61+kfxEmjgacRr8rYY9P48gCvhX4p29q2nXBfDAR7VB64PJ/M0YGvUpu19C82wlKrBytqfL2p2AtTFPHkwXMayxn/AGW5wfcVlBaWDV3u4V0ybP8Aogwg/wBnt+VTbccAcV9fCXNFM/PKkOSTiQFRUJQZq2fQioyBmqMyqU59KidPxq43WmMueO1UpAUSvFRFRV1kwKgIGarRjsf/0PxJQcirSLk1Aoq3EK32AlVfSrUSb2CnjPH51Cq1etlzLHk7fmGT6DufwqW7gfbH7OXw5tEuLXxBqeltcLcMjQzyKSqHPYdOPevtuW+1ZNag0eBIba2YjZIW+dgeXGMjGOnAr5w+HOqNomgjTrWd5oGjj8hjyxQ4IdVH3cg8k8Y96+qPCPheLV9QsfE8jidAk8AV+iOjlQy+55U+2K8jHV+RXPoMnwsakuV9j27TLeHRNNhitQpDKXZvUtzz9Olcf4s+Jvh7w+gGoSbY1yGmfOwHp+A9+ldlq1gmoaZJayTNApA/1Z2k+oBr4c+NXhjwrdxW2laVp8zXnnMdlo8xnnZ+PnwTuI7DpXzUJRqVG5n2cqU4U0qZq+NfFml6uVeymSW3Znmd42BzBCocn6Zbb+NfIvxXu4pF8m2+YxhtwXJBK4wP55r6S8afCE/DH9nbXde1SSaDX7iAKnmMHaKIgHyvQcrk1+a13r/iW40lNV1DUDcrMu7ar7SrNj+HGDgGvQwlHmV0ebmOKcLQktWcvbyD+13ccbyR9MnpXTY71zmmW6Oy3DnLudwz2rpfpX1NCPuWZ8JXknNtEeKhapz1qE4qjEZTDT6QjNAELc8VEU5qdl5qLJ9KqNylc//R/E9BmraKBVeFWYhQCSxAAAyST0H1Jr6g+HX7JXxv+IsMN9YaGdK06fBW91ZvsqFD/EsZBlcf7qc9q6VFvYTaW584otWVG3ljgdj0r9TvCf7Ang7SI43+IPiK71S7bDG300La24PpvkV5G+uFz6DoPr3wJ+y/8C/ByW91pHhewu53Qjz75Tfyqw95zIAT24x6VvHDS3ZLqJHwR+x34f8AHPxhmfwYiJD4d0QLNfamUO+LeCI4lPG6RwCADkBRk+/2B8GdH13wwPEugeJpnkudL8Q39oit0EUZURso7CRMP/wLivs74ajTNMNxoFvZQ2Dt+8URRrGHC4XouBkAjj0rzD46aFL4Xuj4+slxZXvk2+plRxFOuEhuG/2XXETnsVT1OPKznAXw/NT3Tuz2shx3JiuWezVkZNy1tMCbltsUQJYg4/WvB28erN4uNn4H0rzbe2cNfam0ZdIkVvn8tRzK4GcAcA034l+JL0eEruPSXP2m8iRYFQ5JdztC8e/WsnT7jXfh14cm1fUr3TNQnkhQtCrCy8raMeXHu3IQM8crz1NfH0acXoz9BnXcfh1OI/bC8dWGp/BSWzt9QRmuyCVKmOT5Dx8jAMN3QgivyGglso9DihnDyyycRoGwoz3I74r2n9oPxa/i3xCJb5boXbyMx3sDH5ZOERfLdl+X165rxuHTNlzCkgwLdMsvX5yeB/jXu4SgklHzPlM0xrnUcrbKxJp1o0MQD5yGzz/Ktg+tJkCmk969xKysfJvdsazelQkjNDHFQ7hQIloqLPpS0AKevFNwKWmZxwKav0Gj/9L2r9mj9i/wR8D9Eg8QeLLe21zxlIm+5vpVEtvZt/zztFdfl2g4aTG5j0wMCvpXxZG+nxxXcmBaSsEZ06IzfdLHrz611V5qsVx4ubwrnIjsDcOvrvOB+QrnreS31VJPDmqEOl4r2rA+/Ct+FezFe7ZHM1qeKXeqrZavJHfIk9tOduTzscccemRXoGmLMkKzaSDNBEwkMJOSAMZ2n6djXgmhXcksniDw3rB/0zTbl1Yt1xGdu79M12nw48bx23iW00VyZBcKyOR90LglWP1xURk+pdj1TXrl1v3vdHIF1HEt3b4PUgdCB2OMH616npepaJ468Jrc30UdxpuqWrJdQTYKbHU71fPTbzn0I9q8Z1uUad4tjhi5/cbpP9xn4rzS+8Z33gKPxZ8NLDdNqWuo0nh6BSFJ/tHKSFS2FxAxllwT91dvUitLq1iGndNHwj42fxlot43iP4Yi48U+A4b6eOxumGJvs0EhBEkZ+cxqVISYA7lALYJxS/Ez4r+GtU8NWi3mjRQfblQ+bcDeVUfe27T1B7191/Fzwf4E/Z+/Zx1TVBLLcWmiaQ1jYec2ZJLm7AhhVl+6WMjjJ7DJr+fXVfE2s3WmxfbpmuHjUpECThB7CvlcfltOFVcmh9Zl+aVp0pc+tit4o1vTJPEN1PZLmNJSYI1O7aM8ZJzj6UadqsF1mJx5c5+Zsn75PUj/AAr1f4bfB6V9D1Dxhq1t9sXTtNuNT+znIDtGhdVY+5696+e728udS1B71wolmfcFiXaq+gUDoBXdRo+yV+p4+Iruq3c75mFQs3FVbWLUhGRcRMQoyXA6DHekeUYrdnK0Odx3quZOeDUTyZNQb+aLiLYkNShx3qkHqUMcVSsBYLDtTdwqLcaNxp2A/9P6/wBW8SPY/Hmcv8gktY7Yg9CF5yPrmu817zNNuLfXIMpHLNsOOxHIP44xXzT4hvby68dotwNmp21vbxOcnD3CBmdhnnY46H8Oxr6UW7XxH4N2LxPGvmqrHkPGMkH6Hg17kXbY5mfJnjuY6f8AHLXxECsGr2iSjHfzogSfz5qTwLHjUrO4jLGZ50Ln2Q4A+lVPivNBZ+ObfVNRKrI9nBb7R1LCJRx+NX/h9fRyeI1tlULtCuoHQYwKxtaRV9D3rxdLb2fjazLjm+tbiLd3JG1xn6Zryf8AaH0O8uNI0f4g6FldT8NXCzAr94wMRvA+gAb6iut+NF89hrvhq5i4ZmkQ491XP8q05b2K+0WS0nAdZFwQeRyK01egLTc+WP2jPE/iP4y/s/3fhuKNJL15oL+2jh3fvza5YpznDMjNtHdsV+RHhvS31i9h0pIGkurmaO2ijx83mOwUDHXOTX7NeFYh4W8Q3Ph+6HmabKxe3VudoJzt/wCA9BXzT4/8I+E/g18c9d+LmqWqnQ9F0yHWbSzT5FvNYvy8EFsnpueOSZyPuopPYV52OwjqWnfqejl+NjSUoM7z4lap4G/Z1+DU2n6msd5ruvaZPp2n2IwHneWIxSTuOqwx7uuOWwBz0/KTwPoTaprcVvjO1Q3Sux8T6z4z+LfjdvEni6eS4vNTzIobPlxQ7iFiiXokSDhVAAx75r27wb4CXR/H1vp2zHn2EEynHXLMD/KifvSSjsjlvy6sveMfAsFj4RS7iURzX89tZZx/fcE/+Og14n8TPCS+F10W7Vdn9pWjMw7b4W2E/iMGvvL4t6QtppfhDSFTMl1q6yBezGOGTbx9SK8z/aH8DwSXHhLTpDiJQNLUgf8ALSXDs+fZgAKudNJMhTuz8/3lANQiUZqbVrG50nVLvSrsYms5ngcf7UbFT+eKzNxFYtpaFmmripg4xWWr4qyknrSQF8MKfuFVA/FSgjFO4H//1PTPiVcMdej1i0/4/rBtwHTzYyeU9yByvofrXpng/wAcW19NYppsoMV8djoow26TjOOobPBHrXzx478RBNQu7r73lRyYHq3rWB8Cr6/1Px1Y3SyAwWyzXtwo4xLEMRkezOyk/T3Ney5JOxzpXVz0X47Wr3vj6xit4zJNPckAj7qovFd34P8AC66Rr8NzOcMsYA9CDXS6x4WtNZ8TadrGoTYitg0rAd5Cen0rU1YLNeRXNt+7SNlwR3UdaFCzbYGb8emjhsND1A4zFdJH/wB9qf61xtnrLPpolxyOMfyrqPj9Ks/gWwuV6rfWx+mTj+teX6PJ9q1C301R+7McRb/eHWj7QP4RJdH1bWtRjlhjO5SHDAfdr4r/AGqr3VfGfxP0f4bySBrPQoEmuFXq81yAw3c9Ui2hfZj71+iXxP1yTwnpNrp2jJi7vmWMMB91e5z9K/Pa9thq/wAc/Ehu+Z3uAiSMPmPkIEUfTAA+lRX+GwQjrc5bWvB1nouoaDqUSARiM2snschl/rXt/iHQo7H4n+BvIAX+0tLeLPc+TKrD/wAdJrQsfDsHirVLjw3NbSzszoIYoRmUvnqoAJyMZ6HivbtX+E9/rt94Y8U2Ml0tz4UaW3hthAHN2ZQFZGfI2upX065yBURpdS5VF1PC/j9rdpo/xM8A2RUzfZku7oQRjLSyAIsSD3ZjjJ4Aya5nx/HqHiTxH4S8GmMSamt0usanIvIjklAWKNT2CJnj8e9ZPxC13T9S/aI0OXxNpGqaffaQTZPBOFMUQI37n2jIJYgqc4I/T3fw/p9pD8QP+ErmjDrNK1w7t0WKOPjHoAop2crkvSx+Ufx6tY7H4w+KraL7sd/ICf8AaAG7/wAezXkO6vavGlheeOda8V+KraMtNNfXGoxL1LwM7E4/4B834V4duHbmvOqfE2bosq1Tq9UsipFaiMrAaSvxVjNZqtVkNxWjVykz/9XgPiD4x8G2UczT6gsk7hjLs5AZjyoHXFVP2bPHej3F34gvbFJlQNHbwNKuzdsLGTbk525wM98V8o3+l2ViZZbq5Fz5QLTPuzgLyTjNeyfs7TeC9dihkvLt7dSx3K+YF3E5IyODyfXmvTU3KeqM+VKOjP0WsvEK3VrG7sBg92B6/jWwuoQSx/61Rg4XkdRWf4f+HXhDUIk2gyRnBGJ2IPv96tPXfhz4Vs08uC1kMhQYfznABz2+b0rqMU9TmfiXBJqfgMWyzCedtQtnIU5Korrk8egrC0vQ5LFobmLmQKCT3rW13w5pmkeHt+jWxN1cXcEDAyOxKksx7+wrDstF+I8rBLaQIvRQVJIHas29blWdjs9f0qfxLDEk0RJVco+OVcDg18QePPDl74M+J+q+JLqPDW88d5IgPzSQyIGk2jv8hJHvxX3LHofiDQNPl1zxv4hSw021jMkzNtUbVBJ5NfCXiLx5B8T9fOvabcMNMBksxMy7pZPIJAHsGVlb6YqJvQpJns3wr8fTeEvijfeMNBt4L9Y7ULEJsmM+eMhlK9GH8uKw/F3xe+P3gPTLe20/xx4b0qG61u61aCe90uaSW4e5keZ7aQQmfbEpkIBCKxwACSDXwdofxw8U/C6yXTbWO3vHUvC4njZiFgcqvzBlx8uBiuz8B+LIPjE0h8bJZp9smuooTcHzPJRgjK6DcHjjSRQuOSwyAT82OXF4qNOlexvhsM6tTlbt5n2ZovxD+L/g/wAO3mm/FDRPDHjm98azJ4gsru31i202S902UKmY7S+S2mdIxEVixzu4PrXbX3ibRPHx8Q6dN4b1bwdZW2leSt3cQxp9radG3mF4mkjOwYQbT83ODXwx8XfDHhXVvFXjPw9qUxu7rTdD07T/AA4VkIi05YY45Cil85WWYuoUfdVuD6L8L/2ZviBfqT4QW/voUsozIDNDaRQNK+9/K86VNxLRLuKEEjAJweePBZj7ej7WCdrfmb4nAOlUUJteprWvhrQ/BHgq78f67G/7uJLd7ODEhimfOyOWQZj3EAZXJx0Nfnnr0McGr3Ihg+zxSP5scWdwRJRvVc4GcA19KeJvEPxMv9cu/hTd2U95pljqAtjYXMxWFJ03IkihXZY2PPzbmXnPoR4x4y0pbnxTc6YVt9MurNxaypLMZdwRQAwMcZBVQPdiD0450nNSajaxm6TgnLdHnnWnhueKrg7eDUoNZmZaVjirAbiqINWQQRWkZaAf/9b8r/iPaT6X4hOkWGVubxVjmEbGQSQk/KUbHIOCD9K+8v2ffAQ03SrYSRrtZVLBu+fY16P8Sv2MvB6aHceM/Bmu3tvrXh+BnUXsiy2ksatvaIll3IcE7SG4J5zXN+EvjDZ6NZ2uh6Z4dl1XVAi7yjKsCn/akGQK9eNJxleRzufNHQ+rrLwddCLd4fuTZyMOY1fCE+wzx+Fes6b4d8TxaYLbVGF5Ft+/kF4yB19wK+ddD1j4yatGJdMTQ9HDnIUo90659TlQTWzqfhf4463bPBqXj2a3gkHzJp9pFb8DqN2GbHPrW0pLZIizPYdPmsrGJftMsImSfLNIRwmCCee9Sa34/wDBmm2zQNqSRlCGzDMEkJU5xkHOD6V8lan+z5C0cT6vrurX0km8uZLtwDjpwpArDb4EeGVt2kgWUTQsw3NK7bhngnJ9KzvLsXbuzxj9pXxf41+I3iaXT4dQuF8L26xCG3D5WRiPmeT156DpXkXwrsNUs1PhyMyPeX96PslsqgszsArHGO5A56ce1fUc3wluOWtJpFjUkSGUFo4wozlj0C+9ee/2C3hHxLDY2c6HxF4qia20xlYYsNJiVjc3bOeIzOdyIxxtQO+eleHmmMWFp86fvPRdr935JXb8j1MLh6laSTXurXb8vN7fM+X/AIv6DeQeJtQ0bwyJtU07To0N3eQW+YmnkbEzZVSShkxHGSR0OBzz0Hgfw94y8KWK6p/ZjXM9vAbIwzQLFCloiO6Ts0bs8hDOx24ViRz1zXoXjvW9J1O6tvAngS5t5tMhlWa7u0I8u6uYM5mdu1vCgOwHgDLcsa4W68Qwv8RdEsTcSxLqEqado2nK3lkRXoCXGoXCjnEn3YIj1+8flHzePHFS9kqdTVvvu13fa+/ktD0adCDqqUdEn8vl/WrJfid418KxeKYryMs9l4l06CUl4WT9yYTErFQSQ+7cNpO4EHdsNVfBXxh8c+HVvJvB2ranpCyORcJBcEHyQcpvPygquSc4qDWvCN/cXOrafcwOfsN9JHarcRABoSAco5XcAWJOQcZNeZ6fr3iv4e69JcXWmR3Wn3itbX2nSxh7S7tJcB43ZemQMqwIZGAYc1dGqqCcaS5rLbvY0q4R1YqVR2fc4/4h2vi3wtr9n45h1I6lbeJWbUbbU7csYriRifPjfPSWGQlXU9Dg9CK4DUb0XhjuHYi5lkLNK4zkt2Y9ce9fUP8AZ/hbSdHfTGnlufhj4puN9tO/7y68MawR8vnAc7lHyvjCzw/MPmHHzx4o8J6z4P8AEF1oOuxKs9o4BZDuimjb5o5Y2HDJIpDKw4INZ5dj/bS5Jv3le3S663XddV/mZYvDeyhaK919Oz/4O5SsdEsr+JbV7r7BqJJEf2vCWk47BZukbZ/vjaf7y1kX2naho94+napbyW1zH96OVSp56EZ6g9iMg9jXbQT3Vnpiz2gSe0lYLJazbXQn1G/gH/I5rq9N1zwpqNimk60ix2EZ2JBeNI0EDvxiC5VXuLE56B1lhJ5Ix06JYmtSn7yuv6/q34nNHDUqkdHZni681MM4ruPF3guPQI4tS0q7+26fcBmXcFM0QH94xl4pUPRZYnKk9Qp4rgRcRY+8v516dGrGpBThscFWnKEuWR//1/QNV8P6v8cfGGseBLK+uNM8DeF5Fj1E25KXGqXxAYhm/hjTgYFT+Iv2cNM8E+F7sfDPTzLreo7EH2iTfHFAvLHLHAY469R2r3L4S6fa6Zq/jCC0XakupGVs92ZRn+Ve7xQxPAGdQSRk/hX0cknuecqjWx+XvhjxJ488HXw0vxfp02mFDtEk4xA46ZWTlCOPWvf4/il4Tt7dW1bxLpltlQdpuU3HPsDX1dqnh/R9Rt3gvrWOeKQYeORQyMCO4IINfIPjP9nD4N6Tr9l4x0rw7BZ39vc+YFgZkt2cDhmgB8skHkfKOea58RNUqbn2OrCU3XqKltc6e6v21K3W5tZS0BTIkfgEEk5GecYridW1/wAPaVbSSXGpJuXO47gFz6D1rM+Il3dDTVtI5WiSYkPsO0kAZ61+afxp1bV5/DkmopfTwNBfC0SOJgsezaTk8Fixx1z+FfGTzLE4qXKpWR99DKcLg6bqSjzNdz798QfFrRtV8Fa1pHhxl3TRpbyzDJxJcHy44o8ffnlPCKOnLHgV5d8RdUb4S/C2/wDFOvvb33i7WrMWEMrKHW0sxhEtl7Haow+PvHjoOfn74Z6rfx6Lp+npM2yyvNCghfP7wS69M8dzc7u9wkSeVC/SJSSo3HcOr/b0v7iyutI0K1xHZ2+wRoB90IFwK8KjSlWxk6dV3UXb8mej9c/2NziraXX4nynH4vstK0qXUbsxy6lcBRJaKNhmAP7qFwMbYcjzJz95wEjGAWNePWPiPV4/Gdv4yvbh7vVYryK9yOu+JwwHHAUbdoUYAHA4rk7u+ubm4dpXJaRtzMOCSa2RCkJggiG0TDLsPvH8a+yo4KEakqm7l+p8DVxU5JR2SP1cufGvgHxRpFprhvrWKO8j37LhgskTkc4IPBHcdM18kfFL4j+HHtP7M0cRXdwrlTInzJsHfI4zXzvDGsls8DZ8uMblXsD61zskjB9qgDnHA964ZZdGEtXdHqPNZShZLU7vwb45vfCWo3DS28ep6RqafZ9U0ufPkXcBOfqsifejcfMjAEdwfd9S8OaT4y8K2Ok6RetfQFWHhLU5yBMcfM+jXp/gmQ/6gngn7vDYHh/wt8P6Z4q8dafomso0tm8d5cSxqxTzPsdrLcKjEc7HaMB9pDbScEHBGz8KPEmoap48XQ7lYV0vxPfrb31jDGIbdQ5LK8KJtEUkTcxsmNvTpxXDmWC5YfXKTtKP6L/g/NaDweJ5pfV6iun/AF/X4HKJbyxeGtUs72No3hzmNxhkdGxgg8gg8YrjrfUbqNCsiCb5SgZuWAPYnPzL/stkV7H8Z5JpoNI1lpGFzqf22K9K4UXDWNw0CSuAOZHRRvPAZucA14nB9wfhXs4arDEUo1JI8nEQeHqShFj45ngYG3doiMgYyMZ6/n3qIiPo459qu7Q/ysM8VUPBxXR8KsjlTvuf/9k="><div class="k5z04t9s cm280p3y p357zi0d ggj6brxn m0h2a7mj lb5m6g5c f8m0rgwh ac2vgrno idwf4z32 onmzirdk gfz4du6o r7fjleex n1yiu2zv itegkywt eb4rp10x rppts313 ln8gz9je"><div class="aoi073rw suguakab jjtagnqy c32ccnay tvf2evcx m0h2a7mj lb5m6g5c j4enbv94 gfz4du6o r7fjleex lhj4utae ljrqcn24" title="Rodney Brooks (@rodneyabrooks) on X"><span dir="ltr" aria-label="" class="ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae i033jvx7 cw3vfol9 _11JPr" style="min-height: 0px;">Rodney Brooks (@rodneyabrooks) on X</span></div><div class="jjtagnqy tvf2evcx ttu8nud2 lb5m6g5c dsh4tgtl gfz4du6o r7fjleex lhj4utae"><span class="g0rxnol2 gfz4du6o r7fjleex" style="display: -webkit-box; overflow: hidden; -webkit-box-orient: vertical; -webkit-line-clamp: 2;"><span dir="ltr" title="This is a really good explanation of and reminder about ChatGPT. The deep want of people (even my smart friends) to believe it is doing more is so disconcerting. There is no miracle. A new, rather tricky, tool, yes, but we're still going to have to get through life on our own." aria-label="" class="i033jvx7 _11JPr" style="min-height: 0px;">This is a really good explanation of and reminder about ChatGPT. The deep want of people (even my smart friends) to believe it is doing more is so disconcerting. There is no miracle. A new, rather tricky, tool, yes, but we're still going to have to get through life on our own.</span></span></div><div class="j8h4s3v0 tvf2evcx oq44ahr5 lb5m6g5c dsh4tgtl qbqilfqo"><span dir="ltr" aria-label="" class="_11JPr" style="min-height: 0px;">x.com</span></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>There is no gold :)

<a dir="auto" href="https://x.com/rodneyabrooks/status/1645284719494569986?s=20" title="https://x.com/rodneyabrooks/status/1645284719494569986?s=20" target="_blank" rel="noopener noreferrer" class="_11JPr selectable-text copyable-text">https://x.com/rodneyabrooks/status/1645284719494569986?s=20</a></span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:34 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:34 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_0A8EB2D93ED55D565738260BEF11887C_919334372044@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Sid +91 93343 72044" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno akr8p6hy"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">S</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d jake6gi8" role=""><span dir="auto" aria-label="Maybe Sid" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Sid</span><span class="WJuYU" testid="author" role="button">+91 93343 72044</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[1:36 am, 27/02/2024] +91 93343 72044: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="k4182h19 _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d oyzge2kb" role=""><span dir="auto" aria-label="Maybe Tushar | Billion Gradient" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Tushar | Billion Gradient</span><span class="WJuYU" testid="author">+91 94044 34273</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">There is no gold :)

https://x.com/rodneyabrooks/status/1645284719494569986?s=20</span></div></div></div><div class="jrEtJ"><div class="kknmh"><div class="ln8gz9je ppled2lx t3g6t33p qnwaluaf" style="background-image: url(&quot;data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAASABIAAD/4QBYRXhpZgAATU0AKgAAAAgAAgESAAMAAAABAAEAAIdpAAQAAAABAAAAJgAAAAAAA6ABAAMAAAABAAEAAKACAAQAAAABAAAAjKADAAQAAAABAAAAjAAAAAD/7QA4UGhvdG9zaG9wIDMuMAA4QklNBAQAAAAAAAA4QklNBCUAAAAAABDUHYzZjwCyBOmACZjs+EJ+/8AAEQgAjACMAwEiAAIRAQMRAf/EAB8AAAEFAQEBAQEBAAAAAAAAAAABAgMEBQYHCAkKC//EALUQAAIBAwMCBAMFBQQEAAABfQECAwAEEQUSITFBBhNRYQcicRQygZGhCCNCscEVUtHwJDNicoIJChYXGBkaJSYnKCkqNDU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6g4SFhoeIiYqSk5SVlpeYmZqio6Slpqeoqaqys7S1tre4ubrCw8TFxsfIycrS09TV1tfY2drh4uPk5ebn6Onq8fLz9PX29/j5+v/EAB8BAAMBAQEBAQEBAQEAAAAAAAABAgMEBQYHCAkKC//EALURAAIBAgQEAwQHBQQEAAECdwABAgMRBAUhMQYSQVEHYXETIjKBCBRCkaGxwQkjM1LwFWJy0QoWJDThJfEXGBkaJicoKSo1Njc4OTpDREVGR0hJSlNUVVZXWFlaY2RlZmdoaWpzdHV2d3h5eoKDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uLj5OXm5+jp6vLz9PX29/j5+v/bAEMAAgICAgICAwICAwQDAwMEBQQEBAQFBwUFBQUFBwgHBwcHBwcICAgICAgICAoKCgoKCgsLCwsLDQ0NDQ0NDQ0NDf/bAEMBAgICAwMDBgMDBg0JBwkNDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDf/dAAQACf/aAAwDAQACEQMRAD8A/EVQKlUdqYAM1YUCtogCg4q0iCo0XnNWlHrRIACjNejfDTwsPE3iiytrhGa0WVTMVO049M9RnuR2ri9PsZ9Qu47S2QySysEVQO7HAr9NPhH8BJPCtrZT69GlvcXCiVmDb2wcHkDPP5YrKpK0bmlKHM7HZD4Ew6L4g0+HSpCseqWxZ44wAsUKjkDPYZxn17mvbNH+HmkaBbeRpVkgUgbiRksfc9a9F8HWdvdkXmC7KnkIW6iMMSB/Imu21CKC3g+dQXGT0r5HMqs6tRrm0P0DJaVOlRU+XVnx5450kxRSNCAqxZ3J2PsK/P8A+KlraaNHJevb7jJKXZUzhkzgjHt61+kfxEmjgacRr8rYY9P48gCvhX4p29q2nXBfDAR7VB64PJ/M0YGvUpu19C82wlKrBytqfL2p2AtTFPHkwXMayxn/AGW5wfcVlBaWDV3u4V0ybP8Aogwg/wBnt+VTbccAcV9fCXNFM/PKkOSTiQFRUJQZq2fQioyBmqMyqU59KidPxq43WmMueO1UpAUSvFRFRV1kwKgIGarRjsf/0PxJQcirSLk1Aoq3EK32AlVfSrUSb2CnjPH51Cq1etlzLHk7fmGT6DufwqW7gfbH7OXw5tEuLXxBqeltcLcMjQzyKSqHPYdOPevtuW+1ZNag0eBIba2YjZIW+dgeXGMjGOnAr5w+HOqNomgjTrWd5oGjj8hjyxQ4IdVH3cg8k8Y96+qPCPheLV9QsfE8jidAk8AV+iOjlQy+55U+2K8jHV+RXPoMnwsakuV9j27TLeHRNNhitQpDKXZvUtzz9Olcf4s+Jvh7w+gGoSbY1yGmfOwHp+A9+ldlq1gmoaZJayTNApA/1Z2k+oBr4c+NXhjwrdxW2laVp8zXnnMdlo8xnnZ+PnwTuI7DpXzUJRqVG5n2cqU4U0qZq+NfFml6uVeymSW3Znmd42BzBCocn6Zbb+NfIvxXu4pF8m2+YxhtwXJBK4wP55r6S8afCE/DH9nbXde1SSaDX7iAKnmMHaKIgHyvQcrk1+a13r/iW40lNV1DUDcrMu7ar7SrNj+HGDgGvQwlHmV0ebmOKcLQktWcvbyD+13ccbyR9MnpXTY71zmmW6Oy3DnLudwz2rpfpX1NCPuWZ8JXknNtEeKhapz1qE4qjEZTDT6QjNAELc8VEU5qdl5qLJ9KqNylc//R/E9BmraKBVeFWYhQCSxAAAyST0H1Jr6g+HX7JXxv+IsMN9YaGdK06fBW91ZvsqFD/EsZBlcf7qc9q6VFvYTaW584otWVG3ljgdj0r9TvCf7Ang7SI43+IPiK71S7bDG300La24PpvkV5G+uFz6DoPr3wJ+y/8C/ByW91pHhewu53Qjz75Tfyqw95zIAT24x6VvHDS3ZLqJHwR+x34f8AHPxhmfwYiJD4d0QLNfamUO+LeCI4lPG6RwCADkBRk+/2B8GdH13wwPEugeJpnkudL8Q39oit0EUZURso7CRMP/wLivs74ajTNMNxoFvZQ2Dt+8URRrGHC4XouBkAjj0rzD46aFL4Xuj4+slxZXvk2+plRxFOuEhuG/2XXETnsVT1OPKznAXw/NT3Tuz2shx3JiuWezVkZNy1tMCbltsUQJYg4/WvB28erN4uNn4H0rzbe2cNfam0ZdIkVvn8tRzK4GcAcA034l+JL0eEruPSXP2m8iRYFQ5JdztC8e/WsnT7jXfh14cm1fUr3TNQnkhQtCrCy8raMeXHu3IQM8crz1NfH0acXoz9BnXcfh1OI/bC8dWGp/BSWzt9QRmuyCVKmOT5Dx8jAMN3QgivyGglso9DihnDyyycRoGwoz3I74r2n9oPxa/i3xCJb5boXbyMx3sDH5ZOERfLdl+X165rxuHTNlzCkgwLdMsvX5yeB/jXu4SgklHzPlM0xrnUcrbKxJp1o0MQD5yGzz/Ktg+tJkCmk969xKysfJvdsazelQkjNDHFQ7hQIloqLPpS0AKevFNwKWmZxwKav0Gj/9L2r9mj9i/wR8D9Eg8QeLLe21zxlIm+5vpVEtvZt/zztFdfl2g4aTG5j0wMCvpXxZG+nxxXcmBaSsEZ06IzfdLHrz611V5qsVx4ubwrnIjsDcOvrvOB+QrnreS31VJPDmqEOl4r2rA+/Ct+FezFe7ZHM1qeKXeqrZavJHfIk9tOduTzscccemRXoGmLMkKzaSDNBEwkMJOSAMZ2n6djXgmhXcksniDw3rB/0zTbl1Yt1xGdu79M12nw48bx23iW00VyZBcKyOR90LglWP1xURk+pdj1TXrl1v3vdHIF1HEt3b4PUgdCB2OMH616npepaJ468Jrc30UdxpuqWrJdQTYKbHU71fPTbzn0I9q8Z1uUad4tjhi5/cbpP9xn4rzS+8Z33gKPxZ8NLDdNqWuo0nh6BSFJ/tHKSFS2FxAxllwT91dvUitLq1iGndNHwj42fxlot43iP4Yi48U+A4b6eOxumGJvs0EhBEkZ+cxqVISYA7lALYJxS/Ez4r+GtU8NWi3mjRQfblQ+bcDeVUfe27T1B7191/Fzwf4E/Z+/Zx1TVBLLcWmiaQ1jYec2ZJLm7AhhVl+6WMjjJ7DJr+fXVfE2s3WmxfbpmuHjUpECThB7CvlcfltOFVcmh9Zl+aVp0pc+tit4o1vTJPEN1PZLmNJSYI1O7aM8ZJzj6UadqsF1mJx5c5+Zsn75PUj/AAr1f4bfB6V9D1Dxhq1t9sXTtNuNT+znIDtGhdVY+5696+e728udS1B71wolmfcFiXaq+gUDoBXdRo+yV+p4+Iruq3c75mFQs3FVbWLUhGRcRMQoyXA6DHekeUYrdnK0Odx3quZOeDUTyZNQb+aLiLYkNShx3qkHqUMcVSsBYLDtTdwqLcaNxp2A/9P6/wBW8SPY/Hmcv8gktY7Yg9CF5yPrmu817zNNuLfXIMpHLNsOOxHIP44xXzT4hvby68dotwNmp21vbxOcnD3CBmdhnnY46H8Oxr6UW7XxH4N2LxPGvmqrHkPGMkH6Hg17kXbY5mfJnjuY6f8AHLXxECsGr2iSjHfzogSfz5qTwLHjUrO4jLGZ50Ln2Q4A+lVPivNBZ+ObfVNRKrI9nBb7R1LCJRx+NX/h9fRyeI1tlULtCuoHQYwKxtaRV9D3rxdLb2fjazLjm+tbiLd3JG1xn6Zryf8AaH0O8uNI0f4g6FldT8NXCzAr94wMRvA+gAb6iut+NF89hrvhq5i4ZmkQ491XP8q05b2K+0WS0nAdZFwQeRyK01egLTc+WP2jPE/iP4y/s/3fhuKNJL15oL+2jh3fvza5YpznDMjNtHdsV+RHhvS31i9h0pIGkurmaO2ijx83mOwUDHXOTX7NeFYh4W8Q3Ph+6HmabKxe3VudoJzt/wCA9BXzT4/8I+E/g18c9d+LmqWqnQ9F0yHWbSzT5FvNYvy8EFsnpueOSZyPuopPYV52OwjqWnfqejl+NjSUoM7z4lap4G/Z1+DU2n6msd5ruvaZPp2n2IwHneWIxSTuOqwx7uuOWwBz0/KTwPoTaprcVvjO1Q3Sux8T6z4z+LfjdvEni6eS4vNTzIobPlxQ7iFiiXokSDhVAAx75r27wb4CXR/H1vp2zHn2EEynHXLMD/KifvSSjsjlvy6sveMfAsFj4RS7iURzX89tZZx/fcE/+Og14n8TPCS+F10W7Vdn9pWjMw7b4W2E/iMGvvL4t6QtppfhDSFTMl1q6yBezGOGTbx9SK8z/aH8DwSXHhLTpDiJQNLUgf8ALSXDs+fZgAKudNJMhTuz8/3lANQiUZqbVrG50nVLvSrsYms5ngcf7UbFT+eKzNxFYtpaFmmripg4xWWr4qyknrSQF8MKfuFVA/FSgjFO4H//1PTPiVcMdej1i0/4/rBtwHTzYyeU9yByvofrXpng/wAcW19NYppsoMV8djoow26TjOOobPBHrXzx478RBNQu7r73lRyYHq3rWB8Cr6/1Px1Y3SyAwWyzXtwo4xLEMRkezOyk/T3Ney5JOxzpXVz0X47Wr3vj6xit4zJNPckAj7qovFd34P8AC66Rr8NzOcMsYA9CDXS6x4WtNZ8TadrGoTYitg0rAd5Cen0rU1YLNeRXNt+7SNlwR3UdaFCzbYGb8emjhsND1A4zFdJH/wB9qf61xtnrLPpolxyOMfyrqPj9Ks/gWwuV6rfWx+mTj+teX6PJ9q1C301R+7McRb/eHWj7QP4RJdH1bWtRjlhjO5SHDAfdr4r/AGqr3VfGfxP0f4bySBrPQoEmuFXq81yAw3c9Ui2hfZj71+iXxP1yTwnpNrp2jJi7vmWMMB91e5z9K/Pa9thq/wAc/Ehu+Z3uAiSMPmPkIEUfTAA+lRX+GwQjrc5bWvB1nouoaDqUSARiM2snschl/rXt/iHQo7H4n+BvIAX+0tLeLPc+TKrD/wAdJrQsfDsHirVLjw3NbSzszoIYoRmUvnqoAJyMZ6HivbtX+E9/rt94Y8U2Ml0tz4UaW3hthAHN2ZQFZGfI2upX065yBURpdS5VF1PC/j9rdpo/xM8A2RUzfZku7oQRjLSyAIsSD3ZjjJ4Aya5nx/HqHiTxH4S8GmMSamt0usanIvIjklAWKNT2CJnj8e9ZPxC13T9S/aI0OXxNpGqaffaQTZPBOFMUQI37n2jIJYgqc4I/T3fw/p9pD8QP+ErmjDrNK1w7t0WKOPjHoAop2crkvSx+Ufx6tY7H4w+KraL7sd/ICf8AaAG7/wAezXkO6vavGlheeOda8V+KraMtNNfXGoxL1LwM7E4/4B834V4duHbmvOqfE2bosq1Tq9UsipFaiMrAaSvxVjNZqtVkNxWjVykz/9XgPiD4x8G2UczT6gsk7hjLs5AZjyoHXFVP2bPHej3F34gvbFJlQNHbwNKuzdsLGTbk525wM98V8o3+l2ViZZbq5Fz5QLTPuzgLyTjNeyfs7TeC9dihkvLt7dSx3K+YF3E5IyODyfXmvTU3KeqM+VKOjP0WsvEK3VrG7sBg92B6/jWwuoQSx/61Rg4XkdRWf4f+HXhDUIk2gyRnBGJ2IPv96tPXfhz4Vs08uC1kMhQYfznABz2+b0rqMU9TmfiXBJqfgMWyzCedtQtnIU5Korrk8egrC0vQ5LFobmLmQKCT3rW13w5pmkeHt+jWxN1cXcEDAyOxKksx7+wrDstF+I8rBLaQIvRQVJIHas29blWdjs9f0qfxLDEk0RJVco+OVcDg18QePPDl74M+J+q+JLqPDW88d5IgPzSQyIGk2jv8hJHvxX3LHofiDQNPl1zxv4hSw021jMkzNtUbVBJ5NfCXiLx5B8T9fOvabcMNMBksxMy7pZPIJAHsGVlb6YqJvQpJns3wr8fTeEvijfeMNBt4L9Y7ULEJsmM+eMhlK9GH8uKw/F3xe+P3gPTLe20/xx4b0qG61u61aCe90uaSW4e5keZ7aQQmfbEpkIBCKxwACSDXwdofxw8U/C6yXTbWO3vHUvC4njZiFgcqvzBlx8uBiuz8B+LIPjE0h8bJZp9smuooTcHzPJRgjK6DcHjjSRQuOSwyAT82OXF4qNOlexvhsM6tTlbt5n2ZovxD+L/g/wAO3mm/FDRPDHjm98azJ4gsru31i202S902UKmY7S+S2mdIxEVixzu4PrXbX3ibRPHx8Q6dN4b1bwdZW2leSt3cQxp9radG3mF4mkjOwYQbT83ODXwx8XfDHhXVvFXjPw9qUxu7rTdD07T/AA4VkIi05YY45Cil85WWYuoUfdVuD6L8L/2ZviBfqT4QW/voUsozIDNDaRQNK+9/K86VNxLRLuKEEjAJweePBZj7ej7WCdrfmb4nAOlUUJteprWvhrQ/BHgq78f67G/7uJLd7ODEhimfOyOWQZj3EAZXJx0Nfnnr0McGr3Ihg+zxSP5scWdwRJRvVc4GcA19KeJvEPxMv9cu/hTd2U95pljqAtjYXMxWFJ03IkihXZY2PPzbmXnPoR4x4y0pbnxTc6YVt9MurNxaypLMZdwRQAwMcZBVQPdiD0450nNSajaxm6TgnLdHnnWnhueKrg7eDUoNZmZaVjirAbiqINWQQRWkZaAf/9b8r/iPaT6X4hOkWGVubxVjmEbGQSQk/KUbHIOCD9K+8v2ffAQ03SrYSRrtZVLBu+fY16P8Sv2MvB6aHceM/Bmu3tvrXh+BnUXsiy2ksatvaIll3IcE7SG4J5zXN+EvjDZ6NZ2uh6Z4dl1XVAi7yjKsCn/akGQK9eNJxleRzufNHQ+rrLwddCLd4fuTZyMOY1fCE+wzx+Fes6b4d8TxaYLbVGF5Ft+/kF4yB19wK+ddD1j4yatGJdMTQ9HDnIUo90659TlQTWzqfhf4463bPBqXj2a3gkHzJp9pFb8DqN2GbHPrW0pLZIizPYdPmsrGJftMsImSfLNIRwmCCee9Sa34/wDBmm2zQNqSRlCGzDMEkJU5xkHOD6V8lan+z5C0cT6vrurX0km8uZLtwDjpwpArDb4EeGVt2kgWUTQsw3NK7bhngnJ9KzvLsXbuzxj9pXxf41+I3iaXT4dQuF8L26xCG3D5WRiPmeT156DpXkXwrsNUs1PhyMyPeX96PslsqgszsArHGO5A56ce1fUc3wluOWtJpFjUkSGUFo4wozlj0C+9ee/2C3hHxLDY2c6HxF4qia20xlYYsNJiVjc3bOeIzOdyIxxtQO+eleHmmMWFp86fvPRdr935JXb8j1MLh6laSTXurXb8vN7fM+X/AIv6DeQeJtQ0bwyJtU07To0N3eQW+YmnkbEzZVSShkxHGSR0OBzz0Hgfw94y8KWK6p/ZjXM9vAbIwzQLFCloiO6Ts0bs8hDOx24ViRz1zXoXjvW9J1O6tvAngS5t5tMhlWa7u0I8u6uYM5mdu1vCgOwHgDLcsa4W68Qwv8RdEsTcSxLqEqado2nK3lkRXoCXGoXCjnEn3YIj1+8flHzePHFS9kqdTVvvu13fa+/ktD0adCDqqUdEn8vl/WrJfid418KxeKYryMs9l4l06CUl4WT9yYTErFQSQ+7cNpO4EHdsNVfBXxh8c+HVvJvB2ranpCyORcJBcEHyQcpvPygquSc4qDWvCN/cXOrafcwOfsN9JHarcRABoSAco5XcAWJOQcZNeZ6fr3iv4e69JcXWmR3Wn3itbX2nSxh7S7tJcB43ZemQMqwIZGAYc1dGqqCcaS5rLbvY0q4R1YqVR2fc4/4h2vi3wtr9n45h1I6lbeJWbUbbU7csYriRifPjfPSWGQlXU9Dg9CK4DUb0XhjuHYi5lkLNK4zkt2Y9ce9fUP8AZ/hbSdHfTGnlufhj4puN9tO/7y68MawR8vnAc7lHyvjCzw/MPmHHzx4o8J6z4P8AEF1oOuxKs9o4BZDuimjb5o5Y2HDJIpDKw4INZ5dj/bS5Jv3le3S663XddV/mZYvDeyhaK919Oz/4O5SsdEsr+JbV7r7BqJJEf2vCWk47BZukbZ/vjaf7y1kX2naho94+napbyW1zH96OVSp56EZ6g9iMg9jXbQT3Vnpiz2gSe0lYLJazbXQn1G/gH/I5rq9N1zwpqNimk60ix2EZ2JBeNI0EDvxiC5VXuLE56B1lhJ5Ix06JYmtSn7yuv6/q34nNHDUqkdHZni681MM4ruPF3guPQI4tS0q7+26fcBmXcFM0QH94xl4pUPRZYnKk9Qp4rgRcRY+8v516dGrGpBThscFWnKEuWR//1/QNV8P6v8cfGGseBLK+uNM8DeF5Fj1E25KXGqXxAYhm/hjTgYFT+Iv2cNM8E+F7sfDPTzLreo7EH2iTfHFAvLHLHAY469R2r3L4S6fa6Zq/jCC0XakupGVs92ZRn+Ve7xQxPAGdQSRk/hX0cknuecqjWx+XvhjxJ488HXw0vxfp02mFDtEk4xA46ZWTlCOPWvf4/il4Tt7dW1bxLpltlQdpuU3HPsDX1dqnh/R9Rt3gvrWOeKQYeORQyMCO4IINfIPjP9nD4N6Tr9l4x0rw7BZ39vc+YFgZkt2cDhmgB8skHkfKOea58RNUqbn2OrCU3XqKltc6e6v21K3W5tZS0BTIkfgEEk5GecYridW1/wAPaVbSSXGpJuXO47gFz6D1rM+Il3dDTVtI5WiSYkPsO0kAZ61+afxp1bV5/DkmopfTwNBfC0SOJgsezaTk8Fixx1z+FfGTzLE4qXKpWR99DKcLg6bqSjzNdz798QfFrRtV8Fa1pHhxl3TRpbyzDJxJcHy44o8ffnlPCKOnLHgV5d8RdUb4S/C2/wDFOvvb33i7WrMWEMrKHW0sxhEtl7Haow+PvHjoOfn74Z6rfx6Lp+npM2yyvNCghfP7wS69M8dzc7u9wkSeVC/SJSSo3HcOr/b0v7iyutI0K1xHZ2+wRoB90IFwK8KjSlWxk6dV3UXb8mej9c/2NziraXX4nynH4vstK0qXUbsxy6lcBRJaKNhmAP7qFwMbYcjzJz95wEjGAWNePWPiPV4/Gdv4yvbh7vVYryK9yOu+JwwHHAUbdoUYAHA4rk7u+ubm4dpXJaRtzMOCSa2RCkJggiG0TDLsPvH8a+yo4KEakqm7l+p8DVxU5JR2SP1cufGvgHxRpFprhvrWKO8j37LhgskTkc4IPBHcdM18kfFL4j+HHtP7M0cRXdwrlTInzJsHfI4zXzvDGsls8DZ8uMblXsD61zskjB9qgDnHA964ZZdGEtXdHqPNZShZLU7vwb45vfCWo3DS28ep6RqafZ9U0ufPkXcBOfqsifejcfMjAEdwfd9S8OaT4y8K2Ok6RetfQFWHhLU5yBMcfM+jXp/gmQ/6gngn7vDYHh/wt8P6Z4q8dafomso0tm8d5cSxqxTzPsdrLcKjEc7HaMB9pDbScEHBGz8KPEmoap48XQ7lYV0vxPfrb31jDGIbdQ5LK8KJtEUkTcxsmNvTpxXDmWC5YfXKTtKP6L/g/NaDweJ5pfV6iun/AF/X4HKJbyxeGtUs72No3hzmNxhkdGxgg8gg8YrjrfUbqNCsiCb5SgZuWAPYnPzL/stkV7H8Z5JpoNI1lpGFzqf22K9K4UXDWNw0CSuAOZHRRvPAZucA14nB9wfhXs4arDEUo1JI8nEQeHqShFj45ngYG3doiMgYyMZ6/n3qIiPo459qu7Q/ysM8VUPBxXR8KsjlTvuf/9k=&quot;);"></div></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Based</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:36 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:36 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_31D0F679137B904F5B_919916576150@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Bharat Shetty" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/410251715_3586044468316077_4057854922213032016_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRpFpmLshf1oE7y-iIbgpuzBA-R8WZgzsR5fyq7Haharw&amp;oe=65EAD974&amp;_nc_sid=e6ed6c&amp;_nc_cat=104" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d lr20dbn2" role=""><span dir="auto" aria-label="" class="_3FuDI ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Bharat Shetty</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[7:33 am, 27/02/2024] Bharat Shetty: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="oj03u76w _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d dud7p2xx" role=""><span dir="auto" aria-label="Maybe Abhishek Shivkumar" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Abhishek Shivkumar</span><span class="WJuYU" testid="author">+44 7901 983685</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">RUST !</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Hopefully AI code co-pilots can teach and make Rust syntax simpler for folks to learn eventually :)</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">7:33 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">7:33 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="reaction 👍"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB011AA87A9AC28D2DF52_919000844590@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Chetan +91 90008 44590" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno b2boiwr0"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">C</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d mglqvgej" role=""><span dir="auto" aria-label="Maybe Chetan" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Chetan</span><span class="WJuYU" testid="author" role="button">+91 90008 44590</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[8:51 am, 27/02/2024] +91 90008 44590: "><div class="M6sU5"><div class="xzT4z gndfcl4n l8fojup5 paxyh2gw sfeitywo cqsf3vkf k2bacm8l ajgl1lbb p357zi0d sap93d0t ovllcyds kvdvyush gfz4du6o r7fjleex rrq4r3yd"><img alt="" class="tvf2evcx oq44ahr5 lb5m6g5c n43pk08i fpj7ivsd jpthtbts" src="data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/4gHYSUNDX1BST0ZJTEUAAQEAAAHIAAAAAAQwAABtbnRyUkdCIFhZWiAH4AABAAEAAAAAAABhY3NwAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAQAA9tYAAQAAAADTLQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAlkZXNjAAAA8AAAACRyWFlaAAABFAAAABRnWFlaAAABKAAAABRiWFlaAAABPAAAABR3dHB0AAABUAAAABRyVFJDAAABZAAAAChnVFJDAAABZAAAAChiVFJDAAABZAAAAChjcHJ0AAABjAAAADxtbHVjAAAAAAAAAAEAAAAMZW5VUwAAAAgAAAAcAHMAUgBHAEJYWVogAAAAAAAAb6IAADj1AAADkFhZWiAAAAAAAABimQAAt4UAABjaWFlaIAAAAAAAACSgAAAPhAAAts9YWVogAAAAAAAA9tYAAQAAAADTLXBhcmEAAAAAAAQAAAACZmYAAPKnAAANWQAAE9AAAApbAAAAAAAAAABtbHVjAAAAAAAAAAEAAAAMZW5VUwAAACAAAAAcAEcAbwBvAGcAbABlACAASQBuAGMALgAgADIAMAAxADb/2wBDAAYEBQYFBAYGBQYHBwYIChAKCgkJChQODwwQFxQYGBcUFhYaHSUfGhsjHBYWICwgIyYnKSopGR8tMC0oMCUoKSj/2wBDAQcHBwoIChMKChMoGhYaKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCj/wAARCACMAIwDASIAAhEBAxEB/8QAHAABAAICAwEAAAAAAAAAAAAAAAYHBAgBAgMF/8QANRAAAQMDAwIDBgQGAwAAAAAAAAECAwQFEQYHEgghEzFxFCJBUWGRGCNCgTI3UnOz02SUof/EABQBAQAAAAAAAAAAAAAAAAAAAAD/xAAUEQEAAAAAAAAAAAAAAAAAAAAA/9oADAMBAAIRAxEAPwDakAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAADAv9c612Svr2ReM6lp5J0jzjmrWqvH98YAzwavfiiuPfGhJ+3/Kd/rPGfqsqqdUSfRTo1Xy51qpn7xgbTlb70bq0G2VrpZJ6Z1dcaxXJT0rX8EVG4y5zsLhEynwyuSpvxR3FfLQc/wD2nf6ypt8typdxai0VNRp+W0yUKPYjnyq9siOVFRO7Ux5L9wLVod7t2qyNlwp9AQz2p6c2rHRVGXN+aP5qi+vEnW3fUHYNS3FLVfqWXT12VeCR1T8xvf8A081ROK/RyJ6kCt/U7cIaCnjboaWRrImtR7KlyNdhE7p+X5KV9u1uLRbiUSuqdBS0N4YqLFcIplV/b9L08NObfXunwVAN6GO5Jk5NLdsOoi/aa0+2zXG0S391OuIZnTKyRkflxd7q5x8F/YlcnVZVRStjk0U5kjvJrq1UVf28MDacGrc3VRWwM5z6IkjZnHJ9W5E+6xlg7LbyT7lXmro5NPutkUNP47Z/HWRH+8jcJ7qfNfsBcYAAAAAAABw45OHAap3jcncXcXci6ac21qYrfQ0TpE8RWsRVYxyNWR73IqplyphGpnun1IJ1B0Wq7XT6Sh15cI7heGJUu8WNWqnh84+KZRrc+S91TPcs/VWyWs7Dryu1LtdeIKX2573vhfIsbo+buTmd0VrmZ7pny7fLJTO9mldW6fvdmq9dXhl0ulwa5ycJHSJExrkRG5VET9S9kTH3AtWLqP1PZ6m3Tag0YlHYKlESJ2JGyvYiJ7zXuw1y4wuMJ6ofT6xq6nuW22lq6jeklNU1jZon/wBTXQuVF+ynfqk07qnVdNpS32GyT1dvjw59TFh3GV+Gojmp3a1E7q5e3f6GH1Z2pLJtLoy1RuWRtFOyn5Y/i4QK3P8A4BOK+s1nQ7DaKk28p4qi6vt9BG9jo0e5GOganJvJUamFxlV7YK9rtfbv7Z1tBcdxIYLjYqmTw5Gxthy3PwR0aJh2MqmcouC5NPVF5pNhbDUaYp4Km8RWCkfTwz54PckDO3b44zhO2VwVJpzZfVu4Vwjv+791qmxp70duY9EejfPiqJ7sSL2yiJn0Aj/R1M2o3N1POzPCWhe9uU74Wdikf3r13Yr3vtYr7bKiSa22t1NHPKkaplYqhznq1F7qmF7fMkXR7EyDdDVEMbeMcdE9rU+iTsLDv/TfZrjuFFqClrUp7c+pbU1NtdAj2PVHcnNauUw1y5yiovmoED6hd49J662/S0WGasfWJVxzYlp1Y3i1HZ75+pdfTWmdkdL/ANqb/PIVx1S6I0zp/bFKyx2C3UFX7dEzxaeBrHcVR2UynoWR01fyR0v/AGpf88gFmAAAAAAAAAAAvkaX7zuop+oWqZubNcodNxxt9kWlTP5fhorePn2V3Lkqd85N0D5d809Zr/AyG+WqguMTFy1tXTtlRq/NOSLgCMbXbi6f19S1yacfUKy3OZE9J2cHK1U91yJnyXCp379lJpU0dPVNRtVBFM1FyiSMRyIvz7nhaLNbLLS+zWe3UdBT5z4VLC2JufRqIhnAdI42xsayNEaxqYRrUwiJ8kO+OwAGNT0NLTSOfT00ET3dlcyNGqvqqGSAB5VVNDVReHUxRysznjI1HJn0U7QQx08TYoI2Rxt8msaiIn7IdwAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAf//Z"><div class="k5z04t9s cm280p3y p357zi0d ggj6brxn m0h2a7mj lb5m6g5c f8m0rgwh ac2vgrno idwf4z32 onmzirdk gfz4du6o r7fjleex n1yiu2zv itegkywt eb4rp10x rppts313 ln8gz9je"><div class="aoi073rw suguakab jjtagnqy c32ccnay tvf2evcx m0h2a7mj lb5m6g5c j4enbv94 gfz4du6o r7fjleex lhj4utae ljrqcn24" title="कृत्रिम (@Krutrim) on X"><span dir="ltr" aria-label="" class="ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae i033jvx7 cw3vfol9 _11JPr" style="min-height: 0px;">कृत्रिम (@Krutrim) on X</span></div><div class="jjtagnqy tvf2evcx ttu8nud2 lb5m6g5c dsh4tgtl gfz4du6o r7fjleex lhj4utae"><span class="g0rxnol2 gfz4du6o r7fjleex" style="display: -webkit-box; overflow: hidden; -webkit-box-orient: vertical; -webkit-line-clamp: 2;"><span dir="ltr" title="@raghavar0 @KrutrimAI Hey, thanks for bringing this to our notice. We were able to identify the root cause of a data leakage issue from one of the open source datasets used in our LLM fine-tuning. As a result, some users saw a message &quot;I am created by OpenAI&quot;. The dataset was immediately removed.(1/2)" aria-label="" class="i033jvx7 _11JPr" style="min-height: 0px;">@raghavar0 @KrutrimAI Hey, thanks for bringing this to our notice. We were able to identify the root cause of a data leakage issue from one of the open source datasets used in our LLM fine-tuning. As a result, some users saw a message "I am created by OpenAI". The dataset was immediately removed.(1/2)</span></span></div><div class="j8h4s3v0 tvf2evcx oq44ahr5 lb5m6g5c dsh4tgtl qbqilfqo"><span dir="ltr" aria-label="" class="_11JPr" style="min-height: 0px;">x.com</span></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span><a dir="auto" href="https://x.com/Krutrim/status/1762165852194189400?s=20" title="https://x.com/Krutrim/status/1762165852194189400?s=20" target="_blank" rel="noopener noreferrer" class="_11JPr selectable-text copyable-text">https://x.com/Krutrim/status/1762165852194189400?s=20</a>
If the dataset was immediately removed, is it not responding now with the same response?
If yes, after removing the dataset did you fine tune again? So fast?
Or is the response being generated with a RAG pipeline so when you remove dataset that response does not come?</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">8:51 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">8:51 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB0E99683EE6A1226CC1A_917737887058@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Nirant K +91 77378 87058" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno g9gonq0i"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">N</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author" role="button">+91 77378 87058</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[8:54 am, 27/02/2024] +91 77378 87058: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="b2boiwr0 _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d mglqvgej" role=""><span dir="auto" aria-label="Maybe Chetan" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Chetan</span><span class="WJuYU" testid="author">+91 90008 44590</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">https://x.com/Krutrim/status/1762165852194189400?s=20
If the dataset was immediately removed, is it not responding now with the same response?
If yes, after removing the dataset did you fine tune again? So fast?
Or is the response being generated with a RAG pipeline so when you remove dataset that response does not come?</span></div></div></div><div class="jrEtJ"><div class="kknmh"><div class="ln8gz9je ppled2lx t3g6t33p qnwaluaf" style="background-image: url(&quot;data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/4gHYSUNDX1BST0ZJTEUAAQEAAAHIAAAAAAQwAABtbnRyUkdCIFhZWiAH4AABAAEAAAAAAABhY3NwAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAQAA9tYAAQAAAADTLQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAlkZXNjAAAA8AAAACRyWFlaAAABFAAAABRnWFlaAAABKAAAABRiWFlaAAABPAAAABR3dHB0AAABUAAAABRyVFJDAAABZAAAAChnVFJDAAABZAAAAChiVFJDAAABZAAAAChjcHJ0AAABjAAAADxtbHVjAAAAAAAAAAEAAAAMZW5VUwAAAAgAAAAcAHMAUgBHAEJYWVogAAAAAAAAb6IAADj1AAADkFhZWiAAAAAAAABimQAAt4UAABjaWFlaIAAAAAAAACSgAAAPhAAAts9YWVogAAAAAAAA9tYAAQAAAADTLXBhcmEAAAAAAAQAAAACZmYAAPKnAAANWQAAE9AAAApbAAAAAAAAAABtbHVjAAAAAAAAAAEAAAAMZW5VUwAAACAAAAAcAEcAbwBvAGcAbABlACAASQBuAGMALgAgADIAMAAxADb/2wBDAAYEBQYFBAYGBQYHBwYIChAKCgkJChQODwwQFxQYGBcUFhYaHSUfGhsjHBYWICwgIyYnKSopGR8tMC0oMCUoKSj/2wBDAQcHBwoIChMKChMoGhYaKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCj/wAARCACMAIwDASIAAhEBAxEB/8QAHAABAAICAwEAAAAAAAAAAAAAAAYHBAgBAgMF/8QANRAAAQMDAwIDBgQGAwAAAAAAAAECAwQFEQYHEgghEzFxFCJBUWGRGCNCgTI3UnOz02SUof/EABQBAQAAAAAAAAAAAAAAAAAAAAD/xAAUEQEAAAAAAAAAAAAAAAAAAAAA/9oADAMBAAIRAxEAPwDakAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAADAv9c612Svr2ReM6lp5J0jzjmrWqvH98YAzwavfiiuPfGhJ+3/Kd/rPGfqsqqdUSfRTo1Xy51qpn7xgbTlb70bq0G2VrpZJ6Z1dcaxXJT0rX8EVG4y5zsLhEynwyuSpvxR3FfLQc/wD2nf6ypt8typdxai0VNRp+W0yUKPYjnyq9siOVFRO7Ux5L9wLVod7t2qyNlwp9AQz2p6c2rHRVGXN+aP5qi+vEnW3fUHYNS3FLVfqWXT12VeCR1T8xvf8A081ROK/RyJ6kCt/U7cIaCnjboaWRrImtR7KlyNdhE7p+X5KV9u1uLRbiUSuqdBS0N4YqLFcIplV/b9L08NObfXunwVAN6GO5Jk5NLdsOoi/aa0+2zXG0S391OuIZnTKyRkflxd7q5x8F/YlcnVZVRStjk0U5kjvJrq1UVf28MDacGrc3VRWwM5z6IkjZnHJ9W5E+6xlg7LbyT7lXmro5NPutkUNP47Z/HWRH+8jcJ7qfNfsBcYAAAAAAABw45OHAap3jcncXcXci6ac21qYrfQ0TpE8RWsRVYxyNWR73IqplyphGpnun1IJ1B0Wq7XT6Sh15cI7heGJUu8WNWqnh84+KZRrc+S91TPcs/VWyWs7Dryu1LtdeIKX2573vhfIsbo+buTmd0VrmZ7pny7fLJTO9mldW6fvdmq9dXhl0ulwa5ycJHSJExrkRG5VET9S9kTH3AtWLqP1PZ6m3Tag0YlHYKlESJ2JGyvYiJ7zXuw1y4wuMJ6ofT6xq6nuW22lq6jeklNU1jZon/wBTXQuVF+ynfqk07qnVdNpS32GyT1dvjw59TFh3GV+Gojmp3a1E7q5e3f6GH1Z2pLJtLoy1RuWRtFOyn5Y/i4QK3P8A4BOK+s1nQ7DaKk28p4qi6vt9BG9jo0e5GOganJvJUamFxlV7YK9rtfbv7Z1tBcdxIYLjYqmTw5Gxthy3PwR0aJh2MqmcouC5NPVF5pNhbDUaYp4Km8RWCkfTwz54PckDO3b44zhO2VwVJpzZfVu4Vwjv+791qmxp70duY9EejfPiqJ7sSL2yiJn0Aj/R1M2o3N1POzPCWhe9uU74Wdikf3r13Yr3vtYr7bKiSa22t1NHPKkaplYqhznq1F7qmF7fMkXR7EyDdDVEMbeMcdE9rU+iTsLDv/TfZrjuFFqClrUp7c+pbU1NtdAj2PVHcnNauUw1y5yiovmoED6hd49J662/S0WGasfWJVxzYlp1Y3i1HZ75+pdfTWmdkdL/ANqb/PIVx1S6I0zp/bFKyx2C3UFX7dEzxaeBrHcVR2UynoWR01fyR0v/AGpf88gFmAAAAAAAAAAAvkaX7zuop+oWqZubNcodNxxt9kWlTP5fhorePn2V3Lkqd85N0D5d809Zr/AyG+WqguMTFy1tXTtlRq/NOSLgCMbXbi6f19S1yacfUKy3OZE9J2cHK1U91yJnyXCp379lJpU0dPVNRtVBFM1FyiSMRyIvz7nhaLNbLLS+zWe3UdBT5z4VLC2JufRqIhnAdI42xsayNEaxqYRrUwiJ8kO+OwAGNT0NLTSOfT00ET3dlcyNGqvqqGSAB5VVNDVReHUxRysznjI1HJn0U7QQx08TYoI2Rxt8msaiIn7IdwAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAf//Z&quot;);"></div></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Yes, it's most likely to be RAG then</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">8:54 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">8:54 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="Reactions 👍 2 in total"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk o2es7gts i2cterl7" style="flex-shrink: 0;"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 f8m0rgwh r15c9g6i ilf8vifs gfz4du6o r7fjleex qnz2jpws kjemk6od"><div class="dn5wc90o ilf8vifs gndfcl4n p357zi0d"><span class="f8jlpxt4 qfejxiq4 ce5kru2g">2</span></div></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB0C0F8766C8576793CB2_919740361920@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Amit Sharma +91 97403 61920" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/328148264_595173765849116_1053863308596557803_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRucpHopl86ZVtS8zy9l6UOfvFIAa3iFvH1nJoqT59FhQ&amp;oe=65EB0630&amp;_nc_sid=e6ed6c&amp;_nc_cat=103" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Amit Sharma" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Amit Sharma</span><span class="WJuYU" testid="author" role="button">+91 97403 61920</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[8:57 am, 27/02/2024] +91 97403 61920: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="m8ez62k1 _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d lr20dbn2" role=""><span dir="auto" aria-label="Maybe Deepak" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Deepak</span><span class="WJuYU" testid="author">+91 95555 79357</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">For people who have built copilots, how are you measuring user experience and copilot efficacy in production?</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>This is a million $ question. I doubt if the whole concept is settled, so effectiveness measurement is still sometime away. Copilot UX = ChatGPT style Q&amp;A text box, some canned CTA's and a left hand / floating bar of history of your queries / chats. Perplexity is trying to innovate here but not too different. Would love to hear from others.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">8:57 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">8:57 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="reaction 💯"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="💯" draggable="false" class="bd96tnyg p83aruvr b75 emoji apple _11JPr" tabindex="-1" style="background-position: -60px -20px;"></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB0F719E5B65FF84F9B64_919000844590@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Chetan +91 90008 44590" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno b2boiwr0"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">C</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d mglqvgej" role=""><span dir="auto" aria-label="Maybe Chetan" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Chetan</span><span class="WJuYU" testid="author" role="button">+91 90008 44590</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[9:00 am, 27/02/2024] +91 90008 44590: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g9gonq0i _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author">+91 77378 87058</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Yes, it's most likely to be RAG then</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>I don't know why create all these games. Clearly explain what was built and charge a service for it and people will use it if it is useful.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">9:00 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">9:00 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="reaction 👍"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍🏽" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -80px -60px;"></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB0401A200961410E5BBE_19377081307@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Dr. Pratik Desai +1 (937) 708-1307" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/407052186_368942458989574_5837140328940210822_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRT7N2iCE5tWNPEgrn8zUMppMyQOh2yI-vDe0xjKk_mmA&amp;oe=65EAD8D6&amp;_nc_sid=e6ed6c&amp;_nc_cat=110" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe Dr. Pratik Desai" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Dr. Pratik Desai</span><span class="WJuYU" testid="author" role="button">+1 (937) 708-1307</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[9:08 am, 27/02/2024] +1 (937) 708-1307: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g9gonq0i _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author">+91 77378 87058</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Yes, it's most likely to be RAG then</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Mostly, extended prompt engineering.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">9:08 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">9:08 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_8DB67FB91EB15F89437487E79A6E43E9_918087880289@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Priyank Agrawal +91 80878 80289" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/198695200_1208919106219826_3698732522610190367_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdQWzr5q6ImL3dG1wQxR6o_efpRQapMh6u_lHs4E03hEGg&amp;oe=65EAFDE2&amp;_nc_sid=e6ed6c&amp;_nc_cat=105" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d aqebjpab" role=""><span dir="auto" aria-label="Maybe Priyank Agrawal" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Priyank Agrawal</span><span class="WJuYU" testid="author" role="button">+91 80878 80289</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[9:18 am, 27/02/2024] +91 80878 80289: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Krutrim clearly mentioned fine tuning here <a dir="auto" href="https://twitter.com/Krutrim/status/1762173810328146283?t=sO0JXYsqsVgJREhnyBpLGw&amp;s=19" title="https://twitter.com/Krutrim/status/1762173810328146283?t=sO0JXYsqsVgJREhnyBpLGw&amp;s=19" target="_blank" rel="noopener noreferrer" class="_11JPr selectable-text copyable-text">https://twitter.com/Krutrim/status/1762173810328146283?t=sO0JXYsqsVgJREhnyBpLGw&amp;s=19</a></span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">9:18 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">9:18 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB0738C4CA8DF6D805E10_19377081307@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Dr. Pratik Desai +1 (937) 708-1307" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/407052186_368942458989574_5837140328940210822_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRT7N2iCE5tWNPEgrn8zUMppMyQOh2yI-vDe0xjKk_mmA&amp;oe=65EAD8D6&amp;_nc_sid=e6ed6c&amp;_nc_cat=110" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe Dr. Pratik Desai" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Dr. Pratik Desai</span><span class="WJuYU" testid="author" role="button">+1 (937) 708-1307</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[9:23 am, 27/02/2024] +1 (937) 708-1307: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="awij7w2s _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d aqebjpab" role=""><span dir="auto" aria-label="Maybe Priyank Agrawal" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Priyank Agrawal</span><span class="WJuYU" testid="author">+91 80878 80289</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Krutrim clearly mentioned fine tuning here https://twitter.com/Krutrim/status/1762173810328146283?t=sO0JXYsqsVgJREhnyBpLGw&amp;s=19</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Someone argued for a long on twitter threads that they may have been talking about finetuning after pretraining. and, I have to activate Occam's razor.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">9:23 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">9:23 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="Reactions 👍 5 in total"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk o2es7gts i2cterl7" style="flex-shrink: 0;"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 f8m0rgwh r15c9g6i ilf8vifs gfz4du6o r7fjleex qnz2jpws kjemk6od"><div class="dn5wc90o ilf8vifs gndfcl4n p357zi0d"><span class="f8jlpxt4 qfejxiq4 ce5kru2g">5</span></div></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_A3DAFA2BC0EE5F75F05D337843F987CD_919342533525@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Luci | Manish 😎 +91 93425 33525" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/426392606_1110983826773793_5231113528370352030_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRVQKONAGRP_-qbwQhpVfrf39ogbELFJkopkK_AI2xMXA&amp;oe=65EADC8B&amp;_nc_sid=e6ed6c&amp;_nc_cat=109" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo soyzglys n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d a1q56urb" role=""><span dir="auto" aria-label="Maybe Luci | Manish 😎" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Luci | Manish <img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="😎" draggable="false" class="b86 emoji apple _11JPr" tabindex="-1" style="background-position: 0px -20px;"></span><span class="WJuYU" testid="author" role="button">+91 93425 33525</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[10:46 am, 27/02/2024] +91 93425 33525: "><div class="M6sU5"><div class="xzT4z gndfcl4n l8fojup5 paxyh2gw sfeitywo cqsf3vkf k2bacm8l ajgl1lbb p357zi0d f8m0rgwh ovllcyds kvdvyush gfz4du6o r7fjleex rrq4r3yd"><div class="gndfcl4n p357zi0d ac2vgrno gfz4du6o r7fjleex g0rxnol2 ln8gz9je" style="height: 240px;"><img alt="" class="tvf2evcx oq44ahr5 lb5m6g5c ppled2lx ln8gz9je jpthtbts" src="..."></div><div class="k5z04t9s cm280p3y p357zi0d ggj6brxn m0h2a7mj lb5m6g5c f8m0rgwh ac2vgrno idwf4z32 o3plsq22 gfz4du6o r7fjleex n1yiu2zv itegkywt eb4rp10x rppts313 ln8gz9je"><div class="aoi073rw suguakab jjtagnqy c32ccnay tvf2evcx m0h2a7mj lb5m6g5c j4enbv94 gfz4du6o r7fjleex lhj4utae ljrqcn24" title="Mckay Wrigley (@mckaywrigley) on X"><span dir="ltr" aria-label="" class="ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae i033jvx7 cw3vfol9 _11JPr" style="min-height: 0px;">Mckay Wrigley (@mckaywrigley) on X</span></div><div class="jjtagnqy tvf2evcx ttu8nud2 lb5m6g5c dsh4tgtl gfz4du6o r7fjleex lhj4utae aoi073rw psqdn66s c32ccnay"><span class="g0rxnol2 gfz4du6o r7fjleex" style="display: -webkit-box; overflow: hidden; -webkit-box-orient: vertical; -webkit-line-clamp: 2;"><span dir="ltr" title="The future of fixing bugs? Just record them. I filmed 3 separate bugs in an app and gave the videos to Gemini 1.5 Pro with my entire codebase. It correctly identified &amp; fixed each one. AI is improving insanely fast." aria-label="" class="i033jvx7 _11JPr" style="min-height: 0px;">The future of fixing bugs? Just record them. I filmed 3 separate bugs in an app and gave the videos to Gemini 1.5 Pro with my entire codebase. It correctly identified &amp; fixed each one. AI is improving insanely fast.</span></span></div><div class="j8h4s3v0 tvf2evcx oq44ahr5 lb5m6g5c dsh4tgtl qbqilfqo"><span dir="ltr" aria-label="" class="_11JPr" style="min-height: 0px;">twitter.com</span></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span><a dir="auto" href="https://twitter.com/mckaywrigley/status/1762175776726663267?t=4XpMkJltHFB3yC-vHsf8Zg&amp;s=19" title="https://twitter.com/mckaywrigley/status/1762175776726663267?t=4XpMkJltHFB3yC-vHsf8Zg&amp;s=19" target="_blank" rel="noopener noreferrer" class="_11JPr selectable-text copyable-text">https://twitter.com/mckaywrigley/status/1762175776726663267?t=4XpMkJltHFB3yC-vHsf8Zg&amp;s=19</a>

The future of fixing bugs?

Just record them.

I filmed 3 separate bugs in an app and gave the videos to Gemini 1.5 Pro with my entire codebase.

It correctly identified &amp; fixed each one.

AI is improving insanely fast.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">10:46 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">10:46 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="Reactions ❤, 😮, 👍 8 in total"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="❤️" draggable="false" class="bd96tnyg eipqoq80 b8 emoji apple _11JPr" tabindex="-1" style="background-position: -20px 0px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="😮" draggable="false" class="bd96tnyg p83aruvr b87 emoji apple _11JPr" tabindex="-1" style="background-position: -40px -40px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍🏻" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -40px -60px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk o2es7gts i2cterl7" style="flex-shrink: 0;"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 f8m0rgwh r15c9g6i ilf8vifs gfz4du6o r7fjleex qnz2jpws kjemk6od"><div class="dn5wc90o ilf8vifs gndfcl4n p357zi0d"><span class="f8jlpxt4 qfejxiq4 ce5kru2g">8</span></div></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB0BFC3E1905D1C0CC92D_917737887058@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Nirant K +91 77378 87058" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno g9gonq0i"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">N</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author" role="button">+91 77378 87058</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[10:50 am, 27/02/2024] +91 77378 87058: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="exofpctk _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d a1q56urb" role=""><span dir="auto" aria-label="Maybe Luci | Manish 😎" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Luci | Manish <img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="😎" draggable="false" class="b86 emoji apple _11JPr" tabindex="-1" style="background-position: 0px -20px;"></span><span class="WJuYU" testid="author">+91 93425 33525</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">https://twitter.com/mckaywrigley/status/1762175776726663267?t=4XpMkJltHFB3yC-vHsf8Zg&amp;s=19

The future of fixing bugs?

Just record them.

I filmed 3 separate bugs in an app and gave the videos to Gemini 1.5 Pro with my entire codebase.

It correctly identified &amp; fixed each one.

AI is improving insanely fast.</span></div></div></div><div class="jrEtJ"><div class="kknmh"><div class="ln8gz9je ppled2lx t3g6t33p qnwaluaf" style="background-image: url(&quot;data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/4gHYSUNDX1BST0ZJTEUAAQEAAAHIAAAAAAQwAABtbnRyUkdCIFhZWiAH4AABAAEAAAAAAABhY3NwAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAQAA9tYAAQAAAADTLQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAlkZXNjAAAA8AAAACRyWFlaAAABFAAAABRnWFlaAAABKAAAABRiWFlaAAABPAAAABR3dHB0AAABUAAAABRyVFJDAAABZAAAAChnVFJDAAABZAAAAChiVFJDAAABZAAAAChjcHJ0AAABjAAAADxtbHVjAAAAAAAAAAEAAAAMZW5VUwAAAAgAAAAcAHMAUgBHAEJYWVogAAAAAAAAb6IAADj1AAADkFhZWiAAAAAAAABimQAAt4UAABjaWFlaIAAAAAAAACSgAAAPhAAAts9YWVogAAAAAAAA9tYAAQAAAADTLXBhcmEAAAAAAAQAAAACZmYAAPKnAAANWQAAE9AAAApbAAAAAAAAAABtbHVjAAAAAAAAAAEAAAAMZW5VUwAAACAAAAAcAEcAbwBvAGcAbABlACAASQBuAGMALgAgADIAMAAxADb/2wBDAAYEBQYFBAYGBQYHBwYIChAKCgkJChQODwwQFxQYGBcUFhYaHSUfGhsjHBYWICwgIyYnKSopGR8tMC0oMCUoKSj/2wBDAQcHBwoIChMKChMoGhYaKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCj/wAARCABVAFUDASIAAhEBAxEB/8QAHAAAAgIDAQEAAAAAAAAAAAAABAYABQEDBwII/8QAOxAAAgEDAgMFBAcHBQEAAAAAAQIDAAQRBSESMVEGE0FhcQcigaEUFVORscHRFiMyM0JSkhdydKKy4//EABkBAAMBAQEAAAAAAAAAAAAAAAECAwAEBf/EACcRAAICAQIEBgMAAAAAAAAAAAABAhEDEiEEEyJRFDEyQWHRQoGR/9oADAMBAAIRAxEAPwDl/ZGVIdetu92jclGyccwR+OKZYvZn2hv1N1YWAktJCWjcSpuuTjxpKsyY543HNWBr6FTvJfZMohmBks7gYaJuQLf/AEoSbTFRzFvZV2pHPTH+DqfzrWfZj2mHPSpvhg/nV2bzVAqmOfU1OME90GB9DtWPrfVHUq2oXq9eGIKR8Qam8tOgWigbsHr2m3FvcXOl3KxJKpZu7yOfoa83X847Y8uldc05rrUfZZci5leScXOOKRt8Ar19a5TexEXDjoadO2aapAF0cWsv+0ih9KLQaTqk6Eq/DHGrA4wS4P4Ci76MrYy+eB8xROiz2un6JJLf2iXUU1wF4Gz/AEqTnYjxNO/SSQPoeq23dX7a1NcySiHNviR8s/gpweXj8KI7C3l7Pr7RrO/cMjNKOeV8Bk78yKLku9KOlm9+oIBAZDErYcZbGefFjpTH2OlsLvT2k0+1a3KMVdcHhyccssc7AdKjJ7eQ7LRl3qVvZN96lRF3OGQxHPKvov2J37Xek3NtOQ3AkbKCB5g/gK5Ydat52ZobK8kVSVLJCMZBwfHqKbfZ72mt9J1WSa6tL6OCSIp/JzvkEcj5fOvUnwmRK3W3yiWPioyklT3+Gdv7iH7KP/EVj6PB9jH/AIilhe3ujsMhL3H/ABzXv9utHzjF4D527VHQ+x2al3Cu2cFv+zdwjju4iyk8AH9wrjbaRb3EtzM08kUKN7rGEsCPUcq6B2i7daBqGlzWkctz3jlQB3DcwRSEdWcL3SWr8KseHitQxOepxQjilLeJHLkSlpYv61axx2hEbFwXA3XHn1NXWhdj/r3s5alpZIVSSRvdQNnOB4kdPnVfqV2126ibTroKpy3DBwgj4etF2uspaWvdW/1vbqP4UUvgH4LVHhnVUTUkgm49mjiJY01C4MfFxBTACAeuz+VMOjaCmjWJtoEkKli5LDckgD8qULTV9avJJ0ttRuswqHZZC6nBzjGRvyNaJNY18HLXUxI233NL4XLJbIzyxTpsf2t2zupHwqUgjXNeCgm4mweRIA/KpSeFydg82HcH7FXEqaA8dwS1xFO4Yo2Bljnp1Pzpg0+d5HAaUKQebyYB5+GD0pa7AyCeHVVcjIfjO4G3Avl5GryGULK7qSoLeJHPn5dan6n1Hf6Y7F6bieP3hLbBRzbLEH02rwdVuuIFJQNv6MigEvy6BHIdAdgyZx86ysikHCIN+Yz+tSqmMna3LIare/bv95qHVLwjeZvvoBSOorJx1FEWjdLqN19qaFk1O7XcTMMdDWJMdRQc2N9xTIzRQa3qvaBtcefSb5YJFt0Vy0YfILNjmD0P30vmTtQqlmv7cg/3JjP/AFprtZ1S+1AK4MgVBw53wAT+dCwwT6rerY28xjLnG52+NOuIyY3UXRGWKEnbQt/tJ2itiYz9AlI5/ueLHyqVp13Rr3TL57e8iYSKSNieE+Y++pVFxOR/kxORDsW/syuu9vNREqd2vdqwBbc8+nwpwtZMXXucADZHvkYGw6g9KSvZpiHtGY+J5GeBs8jkgjln1p3e3nfUpUt0LOuDu3hv4rUk/dHS17MPZuCMcMpDNuOC5TGPQYxQgkLOSzEnqTmstaXQfHHGvmsjH8aButMvbfjmhYODlmXi3+HWkm/c0ILysslaslqprbUg64bZuW9FLM7D+IH0qfMRblSC3ahZjsaz3hxuPuqLEkmO9kaEEZ43X3c+G9Vi01aJtNOmKGp6baX8t7JJhbpJwqyA4ZVEaHGR5k1o0CPXtK1JbnTpYrowgtwzENty8SCefWvRt4Z9Vvm4Y5sTFeLAPIAePpRN7pVtLb4WJA2PAYqM8lSaCoWi7h9p72ssw1PRg8zEE91LgZxg7MMjYCpXML20S2nZGRx0walG4i8tl12RvrfT+0NrPczRwxAOrPJ/CMjP5U7DthpEF9JPJeWkisvAU4jjHwB/CuQ6txR3b278J7psHByM0FJJErDMSHPpXXgjFR6kSyuTfS9jtg9oXZpQymGzXIwCIyx/81o/1A0ZvcsSHkzyWBjt8BXHe9iAOIosjHx+VHWeotZl1iCBOMcTKOY5Z9P1qsuWltBf1/ZKMJt9U2v0vobNS10S6vJOA4t5j/UhQq3ofCruy1AFRvvSdB2ht5gFuY+XVc9N/nRn0+F344nUDyNedkjbuqPThJaUk7HiG+Ujc0DqOtsZBCX4ox4HwpVl1Qrsj1WPes8pZs70MUGnYmae1Fz9EYXctxazsJJHL5UbjJJwR4irS01ZkPc6lEIidlmX+W/6HyNLUV8VwQa3PeCSNlcn94Dk/nVHBN2Q1tIstY+jLcAysqZ5ZIGalUf0WN407uTiIGC0q8RPT0A6VKXQHWLLMd2O5570PAGlmCcWM5GcZqVK7URDY4HeO1Jl2m4kxwj3QCP1rQc93G5Y++eEjyHDUqVmA1GXO78Z8DhsZG2B8q8rcOHyzOeRPvc9qlSswhUF04I5nbG5O9F96xAPXwqVKlJDJ2j2khCmvZlbv2XwUbVKlAx6tJ37rnUqVKAp/9k=&quot;);"></div></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>The demo to ship iteration speed has never been faster</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">10:50 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">10:50 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3AC1DFCE7ABCF35D0508_919868221372@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Paras Chopra +91 98682 21372" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/162690799_131241352786128_2676048871288803711_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRLabAEX9bmDVVjQ8t15GgUazuMiCZOrKVtAXDMdkl8Pw&amp;oe=65EAF085&amp;_nc_sid=e6ed6c&amp;_nc_cat=103" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d jake6gi8" role=""><span dir="auto" aria-label="Maybe Paras Chopra" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Paras Chopra</span><span class="WJuYU" testid="author" role="button">+91 98682 21372</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[10:52 am, 27/02/2024] +91 98682 21372: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="exofpctk _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d a1q56urb" role=""><span dir="auto" aria-label="Maybe Luci | Manish 😎" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Luci | Manish <img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="😎" draggable="false" class="b86 emoji apple _11JPr" tabindex="-1" style="background-position: 0px -20px;"></span><span class="WJuYU" testid="author">+91 93425 33525</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">https://twitter.com/mckaywrigley/status/1762175776726663267?t=4XpMkJltHFB3yC-vHsf8Zg&amp;s=19

The future of fixing bugs?

Just record them.

I filmed 3 separate bugs in an app and gave the videos to Gemini 1.5 Pro with my entire codebase.

It correctly identified &amp; fixed each one.

AI is improving insanely fast.</span></div></div></div><div class="jrEtJ"><div class="kknmh"><div class="ln8gz9je ppled2lx t3g6t33p qnwaluaf" style="background-image: url(&quot;data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/4gHYSUNDX1BST0ZJTEUAAQEAAAHIAAAAAAQwAABtbnRyUkdCIFhZWiAH4AABAAEAAAAAAABhY3NwAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAQAA9tYAAQAAAADTLQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAlkZXNjAAAA8AAAACRyWFlaAAABFAAAABRnWFlaAAABKAAAABRiWFlaAAABPAAAABR3dHB0AAABUAAAABRyVFJDAAABZAAAAChnVFJDAAABZAAAAChiVFJDAAABZAAAAChjcHJ0AAABjAAAADxtbHVjAAAAAAAAAAEAAAAMZW5VUwAAAAgAAAAcAHMAUgBHAEJYWVogAAAAAAAAb6IAADj1AAADkFhZWiAAAAAAAABimQAAt4UAABjaWFlaIAAAAAAAACSgAAAPhAAAts9YWVogAAAAAAAA9tYAAQAAAADTLXBhcmEAAAAAAAQAAAACZmYAAPKnAAANWQAAE9AAAApbAAAAAAAAAABtbHVjAAAAAAAAAAEAAAAMZW5VUwAAACAAAAAcAEcAbwBvAGcAbABlACAASQBuAGMALgAgADIAMAAxADb/2wBDAAYEBQYFBAYGBQYHBwYIChAKCgkJChQODwwQFxQYGBcUFhYaHSUfGhsjHBYWICwgIyYnKSopGR8tMC0oMCUoKSj/2wBDAQcHBwoIChMKChMoGhYaKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCj/wAARCABVAFUDASIAAhEBAxEB/8QAHAAAAgIDAQEAAAAAAAAAAAAABAYABQEDBwII/8QAOxAAAgEDAgMFBAcHBQEAAAAAAQIDAAQRBSESMVEGE0FhcQcigaEUFVORscHRFiMyM0JSkhdydKKy4//EABkBAAMBAQEAAAAAAAAAAAAAAAECAwAEBf/EACcRAAICAQIEBgMAAAAAAAAAAAABAhEDEiEEEyJRFDEyQWHRQoGR/9oADAMBAAIRAxEAPwDl/ZGVIdetu92jclGyccwR+OKZYvZn2hv1N1YWAktJCWjcSpuuTjxpKsyY543HNWBr6FTvJfZMohmBks7gYaJuQLf/AEoSbTFRzFvZV2pHPTH+DqfzrWfZj2mHPSpvhg/nV2bzVAqmOfU1OME90GB9DtWPrfVHUq2oXq9eGIKR8Qam8tOgWigbsHr2m3FvcXOl3KxJKpZu7yOfoa83X847Y8uldc05rrUfZZci5leScXOOKRt8Ar19a5TexEXDjoadO2aapAF0cWsv+0ih9KLQaTqk6Eq/DHGrA4wS4P4Ci76MrYy+eB8xROiz2un6JJLf2iXUU1wF4Gz/AEqTnYjxNO/SSQPoeq23dX7a1NcySiHNviR8s/gpweXj8KI7C3l7Pr7RrO/cMjNKOeV8Bk78yKLku9KOlm9+oIBAZDErYcZbGefFjpTH2OlsLvT2k0+1a3KMVdcHhyccssc7AdKjJ7eQ7LRl3qVvZN96lRF3OGQxHPKvov2J37Xek3NtOQ3AkbKCB5g/gK5Ydat52ZobK8kVSVLJCMZBwfHqKbfZ72mt9J1WSa6tL6OCSIp/JzvkEcj5fOvUnwmRK3W3yiWPioyklT3+Gdv7iH7KP/EVj6PB9jH/AIilhe3ujsMhL3H/ABzXv9utHzjF4D527VHQ+x2al3Cu2cFv+zdwjju4iyk8AH9wrjbaRb3EtzM08kUKN7rGEsCPUcq6B2i7daBqGlzWkctz3jlQB3DcwRSEdWcL3SWr8KseHitQxOepxQjilLeJHLkSlpYv61axx2hEbFwXA3XHn1NXWhdj/r3s5alpZIVSSRvdQNnOB4kdPnVfqV2126ibTroKpy3DBwgj4etF2uspaWvdW/1vbqP4UUvgH4LVHhnVUTUkgm49mjiJY01C4MfFxBTACAeuz+VMOjaCmjWJtoEkKli5LDckgD8qULTV9avJJ0ttRuswqHZZC6nBzjGRvyNaJNY18HLXUxI233NL4XLJbIzyxTpsf2t2zupHwqUgjXNeCgm4mweRIA/KpSeFydg82HcH7FXEqaA8dwS1xFO4Yo2Bljnp1Pzpg0+d5HAaUKQebyYB5+GD0pa7AyCeHVVcjIfjO4G3Avl5GryGULK7qSoLeJHPn5dan6n1Hf6Y7F6bieP3hLbBRzbLEH02rwdVuuIFJQNv6MigEvy6BHIdAdgyZx86ysikHCIN+Yz+tSqmMna3LIare/bv95qHVLwjeZvvoBSOorJx1FEWjdLqN19qaFk1O7XcTMMdDWJMdRQc2N9xTIzRQa3qvaBtcefSb5YJFt0Vy0YfILNjmD0P30vmTtQqlmv7cg/3JjP/AFprtZ1S+1AK4MgVBw53wAT+dCwwT6rerY28xjLnG52+NOuIyY3UXRGWKEnbQt/tJ2itiYz9AlI5/ueLHyqVp13Rr3TL57e8iYSKSNieE+Y++pVFxOR/kxORDsW/syuu9vNREqd2vdqwBbc8+nwpwtZMXXucADZHvkYGw6g9KSvZpiHtGY+J5GeBs8jkgjln1p3e3nfUpUt0LOuDu3hv4rUk/dHS17MPZuCMcMpDNuOC5TGPQYxQgkLOSzEnqTmstaXQfHHGvmsjH8aButMvbfjmhYODlmXi3+HWkm/c0ILysslaslqprbUg64bZuW9FLM7D+IH0qfMRblSC3ahZjsaz3hxuPuqLEkmO9kaEEZ43X3c+G9Vi01aJtNOmKGp6baX8t7JJhbpJwqyA4ZVEaHGR5k1o0CPXtK1JbnTpYrowgtwzENty8SCefWvRt4Z9Vvm4Y5sTFeLAPIAePpRN7pVtLb4WJA2PAYqM8lSaCoWi7h9p72ssw1PRg8zEE91LgZxg7MMjYCpXML20S2nZGRx0walG4i8tl12RvrfT+0NrPczRwxAOrPJ/CMjP5U7DthpEF9JPJeWkisvAU4jjHwB/CuQ6txR3b278J7psHByM0FJJErDMSHPpXXgjFR6kSyuTfS9jtg9oXZpQymGzXIwCIyx/81o/1A0ZvcsSHkzyWBjt8BXHe9iAOIosjHx+VHWeotZl1iCBOMcTKOY5Z9P1qsuWltBf1/ZKMJt9U2v0vobNS10S6vJOA4t5j/UhQq3ofCruy1AFRvvSdB2ht5gFuY+XVc9N/nRn0+F344nUDyNedkjbuqPThJaUk7HiG+Ujc0DqOtsZBCX4ox4HwpVl1Qrsj1WPes8pZs70MUGnYmae1Fz9EYXctxazsJJHL5UbjJJwR4irS01ZkPc6lEIidlmX+W/6HyNLUV8VwQa3PeCSNlcn94Dk/nVHBN2Q1tIstY+jLcAysqZ5ZIGalUf0WN407uTiIGC0q8RPT0A6VKXQHWLLMd2O5570PAGlmCcWM5GcZqVK7URDY4HeO1Jl2m4kxwj3QCP1rQc93G5Y++eEjyHDUqVmA1GXO78Z8DhsZG2B8q8rcOHyzOeRPvc9qlSswhUF04I5nbG5O9F96xAPXwqVKlJDJ2j2khCmvZlbv2XwUbVKlAx6tJ37rnUqVKAp/9k=&quot;);"></div></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Gemini is going to eat everyone’s lunch</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">10:52 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">10:52 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A19C6B8003968D4CEAB_19377081307@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Dr. Pratik Desai +1 (937) 708-1307" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/407052186_368942458989574_5837140328940210822_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRT7N2iCE5tWNPEgrn8zUMppMyQOh2yI-vDe0xjKk_mmA&amp;oe=65EAD8D6&amp;_nc_sid=e6ed6c&amp;_nc_cat=110" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe Dr. Pratik Desai" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Dr. Pratik Desai</span><span class="WJuYU" testid="author" role="button">+1 (937) 708-1307</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[10:55 am, 27/02/2024] +1 (937) 708-1307: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Do we have anyone here successful at tuning openTTS, Coqui or similar model for Indic voices and deploying efficiently for production?</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5 hnx8ox4h mw4yctpw gj5xqxfh">Edited</span><span class="tvf2evcx oq44ahr5">10:55 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto"><span class="mw4yctpw gj5xqxfh">Edited</span>10:55 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB07746F01754BACBE8E2_917737887058@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Nirant K +91 77378 87058" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno g9gonq0i"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">N</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author" role="button">+91 77378 87058</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[10:57 am, 27/02/2024] +91 77378 87058: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="po56q9xr _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe Dr. Pratik Desai" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Dr. Pratik Desai</span><span class="WJuYU" testid="author">+1 (937) 708-1307</span></div><div class="_37DQv" role="button"></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>cc <span role="button" tabindex="0" class=""><span dir="auto" data-jid="919952465050@c.us" data-display="~Aashay" aria-label="Maybe Aashay" class="ajgl1lbb o0rubyzf _11JPr selectable-text select-all copyable-text" data-plain-text="@~Aashay" data-app-text-template="​919952465050@c.us​">@<span dir="ltr">~Aashay</span></span></span> <span role="button" tabindex="0" class=""><span dir="auto" data-jid="919915123897@c.us" data-display="~Harveen" aria-label="Maybe Harveen" class="ajgl1lbb o0rubyzf _11JPr selectable-text select-all copyable-text" data-plain-text="@~Harveen" data-app-text-template="​919915123897@c.us​">@<span dir="ltr">~Harveen</span></span></span> would you happen to know someone better than Pratik dada at Indic TTS?</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">10:57 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">10:57 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="reaction ➕"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="➕" draggable="false" class="bd96tnyg p83aruvr b8 emoji apple _11JPr" tabindex="-1" style="background-position: -80px 0px;"></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_05123E61E4C5F6F9F61EA13C6CA098EB_918898020185@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Sourasis Roy +91 88980 20185" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno m8ez62k1"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">S</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d lr20dbn2" role=""><span dir="auto" aria-label="Maybe Sourasis Roy" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Sourasis Roy</span><span class="WJuYU" testid="author" role="button">+91 88980 20185</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[10:59 am, 27/02/2024] +91 88980 20185: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="exofpctk _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d a1q56urb" role=""><span dir="auto" aria-label="Maybe Luci | Manish 😎" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Luci | Manish <img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="😎" draggable="false" class="b86 emoji apple _11JPr" tabindex="-1" style="background-position: 0px -20px;"></span><span class="WJuYU" testid="author">+91 93425 33525</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">https://twitter.com/mckaywrigley/status/1762175776726663267?t=4XpMkJltHFB3yC-vHsf8Zg&amp;s=19

The future of fixing bugs?

Just record them.

I filmed 3 separate bugs in an app and gave the videos to Gemini 1.5 Pro with my entire codebase.

It correctly identified &amp; fixed each one.

AI is improving insanely fast.</span></div></div></div><div class="jrEtJ"><div class="kknmh"><div class="ln8gz9je ppled2lx t3g6t33p qnwaluaf" style="background-image: url(&quot;data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/4gHYSUNDX1BST0ZJTEUAAQEAAAHIAAAAAAQwAABtbnRyUkdCIFhZWiAH4AABAAEAAAAAAABhY3NwAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAQAA9tYAAQAAAADTLQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAlkZXNjAAAA8AAAACRyWFlaAAABFAAAABRnWFlaAAABKAAAABRiWFlaAAABPAAAABR3dHB0AAABUAAAABRyVFJDAAABZAAAAChnVFJDAAABZAAAAChiVFJDAAABZAAAAChjcHJ0AAABjAAAADxtbHVjAAAAAAAAAAEAAAAMZW5VUwAAAAgAAAAcAHMAUgBHAEJYWVogAAAAAAAAb6IAADj1AAADkFhZWiAAAAAAAABimQAAt4UAABjaWFlaIAAAAAAAACSgAAAPhAAAts9YWVogAAAAAAAA9tYAAQAAAADTLXBhcmEAAAAAAAQAAAACZmYAAPKnAAANWQAAE9AAAApbAAAAAAAAAABtbHVjAAAAAAAAAAEAAAAMZW5VUwAAACAAAAAcAEcAbwBvAGcAbABlACAASQBuAGMALgAgADIAMAAxADb/2wBDAAYEBQYFBAYGBQYHBwYIChAKCgkJChQODwwQFxQYGBcUFhYaHSUfGhsjHBYWICwgIyYnKSopGR8tMC0oMCUoKSj/2wBDAQcHBwoIChMKChMoGhYaKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCj/wAARCABVAFUDASIAAhEBAxEB/8QAHAAAAgIDAQEAAAAAAAAAAAAABAYABQEDBwII/8QAOxAAAgEDAgMFBAcHBQEAAAAAAQIDAAQRBSESMVEGE0FhcQcigaEUFVORscHRFiMyM0JSkhdydKKy4//EABkBAAMBAQEAAAAAAAAAAAAAAAECAwAEBf/EACcRAAICAQIEBgMAAAAAAAAAAAABAhEDEiEEEyJRFDEyQWHRQoGR/9oADAMBAAIRAxEAPwDl/ZGVIdetu92jclGyccwR+OKZYvZn2hv1N1YWAktJCWjcSpuuTjxpKsyY543HNWBr6FTvJfZMohmBks7gYaJuQLf/AEoSbTFRzFvZV2pHPTH+DqfzrWfZj2mHPSpvhg/nV2bzVAqmOfU1OME90GB9DtWPrfVHUq2oXq9eGIKR8Qam8tOgWigbsHr2m3FvcXOl3KxJKpZu7yOfoa83X847Y8uldc05rrUfZZci5leScXOOKRt8Ar19a5TexEXDjoadO2aapAF0cWsv+0ih9KLQaTqk6Eq/DHGrA4wS4P4Ci76MrYy+eB8xROiz2un6JJLf2iXUU1wF4Gz/AEqTnYjxNO/SSQPoeq23dX7a1NcySiHNviR8s/gpweXj8KI7C3l7Pr7RrO/cMjNKOeV8Bk78yKLku9KOlm9+oIBAZDErYcZbGefFjpTH2OlsLvT2k0+1a3KMVdcHhyccssc7AdKjJ7eQ7LRl3qVvZN96lRF3OGQxHPKvov2J37Xek3NtOQ3AkbKCB5g/gK5Ydat52ZobK8kVSVLJCMZBwfHqKbfZ72mt9J1WSa6tL6OCSIp/JzvkEcj5fOvUnwmRK3W3yiWPioyklT3+Gdv7iH7KP/EVj6PB9jH/AIilhe3ujsMhL3H/ABzXv9utHzjF4D527VHQ+x2al3Cu2cFv+zdwjju4iyk8AH9wrjbaRb3EtzM08kUKN7rGEsCPUcq6B2i7daBqGlzWkctz3jlQB3DcwRSEdWcL3SWr8KseHitQxOepxQjilLeJHLkSlpYv61axx2hEbFwXA3XHn1NXWhdj/r3s5alpZIVSSRvdQNnOB4kdPnVfqV2126ibTroKpy3DBwgj4etF2uspaWvdW/1vbqP4UUvgH4LVHhnVUTUkgm49mjiJY01C4MfFxBTACAeuz+VMOjaCmjWJtoEkKli5LDckgD8qULTV9avJJ0ttRuswqHZZC6nBzjGRvyNaJNY18HLXUxI233NL4XLJbIzyxTpsf2t2zupHwqUgjXNeCgm4mweRIA/KpSeFydg82HcH7FXEqaA8dwS1xFO4Yo2Bljnp1Pzpg0+d5HAaUKQebyYB5+GD0pa7AyCeHVVcjIfjO4G3Avl5GryGULK7qSoLeJHPn5dan6n1Hf6Y7F6bieP3hLbBRzbLEH02rwdVuuIFJQNv6MigEvy6BHIdAdgyZx86ysikHCIN+Yz+tSqmMna3LIare/bv95qHVLwjeZvvoBSOorJx1FEWjdLqN19qaFk1O7XcTMMdDWJMdRQc2N9xTIzRQa3qvaBtcefSb5YJFt0Vy0YfILNjmD0P30vmTtQqlmv7cg/3JjP/AFprtZ1S+1AK4MgVBw53wAT+dCwwT6rerY28xjLnG52+NOuIyY3UXRGWKEnbQt/tJ2itiYz9AlI5/ueLHyqVp13Rr3TL57e8iYSKSNieE+Y++pVFxOR/kxORDsW/syuu9vNREqd2vdqwBbc8+nwpwtZMXXucADZHvkYGw6g9KSvZpiHtGY+J5GeBs8jkgjln1p3e3nfUpUt0LOuDu3hv4rUk/dHS17MPZuCMcMpDNuOC5TGPQYxQgkLOSzEnqTmstaXQfHHGvmsjH8aButMvbfjmhYODlmXi3+HWkm/c0ILysslaslqprbUg64bZuW9FLM7D+IH0qfMRblSC3ahZjsaz3hxuPuqLEkmO9kaEEZ43X3c+G9Vi01aJtNOmKGp6baX8t7JJhbpJwqyA4ZVEaHGR5k1o0CPXtK1JbnTpYrowgtwzENty8SCefWvRt4Z9Vvm4Y5sTFeLAPIAePpRN7pVtLb4WJA2PAYqM8lSaCoWi7h9p72ssw1PRg8zEE91LgZxg7MMjYCpXML20S2nZGRx0walG4i8tl12RvrfT+0NrPczRwxAOrPJ/CMjP5U7DthpEF9JPJeWkisvAU4jjHwB/CuQ6txR3b278J7psHByM0FJJErDMSHPpXXgjFR6kSyuTfS9jtg9oXZpQymGzXIwCIyx/81o/1A0ZvcsSHkzyWBjt8BXHe9iAOIosjHx+VHWeotZl1iCBOMcTKOY5Z9P1qsuWltBf1/ZKMJt9U2v0vobNS10S6vJOA4t5j/UhQq3ofCruy1AFRvvSdB2ht5gFuY+XVc9N/nRn0+F344nUDyNedkjbuqPThJaUk7HiG+Ujc0DqOtsZBCX4ox4HwpVl1Qrsj1WPes8pZs70MUGnYmae1Fz9EYXctxazsJJHL5UbjJJwR4irS01ZkPc6lEIidlmX+W/6HyNLUV8VwQa3PeCSNlcn94Dk/nVHBN2Q1tIstY+jLcAysqZ5ZIGalUf0WN407uTiIGC0q8RPT0A6VKXQHWLLMd2O5570PAGlmCcWM5GcZqVK7URDY4HeO1Jl2m4kxwj3QCP1rQc93G5Y++eEjyHDUqVmA1GXO78Z8DhsZG2B8q8rcOHyzOeRPvc9qlSswhUF04I5nbG5O9F96xAPXwqVKlJDJ2j2khCmvZlbv2XwUbVKlAx6tJ37rnUqVKAp/9k=&quot;);"></div></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>can't wait to get access to it. This can be automated right? write code and let the AI fix everything up after that</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">10:59 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">10:59 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A91CC8D0FA60C4C0FDC_19377081307@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Dr. Pratik Desai +1 (937) 708-1307" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/407052186_368942458989574_5837140328940210822_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRT7N2iCE5tWNPEgrn8zUMppMyQOh2yI-vDe0xjKk_mmA&amp;oe=65EAD8D6&amp;_nc_sid=e6ed6c&amp;_nc_cat=110" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe Dr. Pratik Desai" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Dr. Pratik Desai</span><span class="WJuYU" testid="author" role="button">+1 (937) 708-1307</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[11:00 am, 27/02/2024] +1 (937) 708-1307: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g9gonq0i _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author">+91 77378 87058</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">cc <span class=""><span dir="auto" data-jid="919952465050@c.us" data-display="~Aashay" aria-label="Maybe Aashay" class="_11JPr">@<span dir="ltr">~Aashay</span></span></span> <span class=""><span dir="auto" data-jid="919915123897@c.us" data-display="~Harveen" aria-label="Maybe Harveen" class="_11JPr">@<span dir="ltr">~Harveen</span></span></span> would you happen to know someone better than Pratik dada at Indic TTS?</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Getting inbounds for Haryanwi, Bhojpuri and other dialects. It can be a game changer if done right.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">11:00 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">11:00 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="Reactions 👍 2 in total"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk o2es7gts i2cterl7" style="flex-shrink: 0;"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 f8m0rgwh r15c9g6i ilf8vifs gfz4du6o r7fjleex qnz2jpws kjemk6od"><div class="dn5wc90o ilf8vifs gndfcl4n p357zi0d"><span class="f8jlpxt4 qfejxiq4 ce5kru2g">2</span></div></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB0C9934F6FEBAF42CFDD_917737887058@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Nirant K +91 77378 87058" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno g9gonq0i"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">N</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author" role="button">+91 77378 87058</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[11:15 am, 27/02/2024] +91 77378 87058: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Lot of these projects are uncertainty bound than talent, money or data to be honest e.g. I have the skills to make a bhojpuri dataset + fine-tune TTS, but I don't know how to make $10K from it  — so I'll skip it completely</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">11:15 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">11:15 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_C71BDDD6B98DEC709C7F01C41FC4739F_919353281004@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe shyam +91 93532 81004" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/425200829_1105715980752903_689203865832244905_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdSsS-4Nua-VDdlNbMT4_d2trURh4c23a9tJrosquwO5IQ&amp;oe=65EAFAF4&amp;_nc_sid=e6ed6c&amp;_nc_cat=109" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d dud7p2xx" role=""><span dir="auto" aria-label="Maybe shyam" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">shyam</span><span class="WJuYU" testid="author" role="button">+91 93532 81004</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[11:16 am, 27/02/2024] +91 93532 81004: "><div class="M6sU5"><div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span><a dir="auto" href="https://llmtimecapsule.octo.ai/" title="https://llmtimecapsule.octo.ai/" target="_blank" rel="noopener noreferrer" class="_11JPr selectable-text copyable-text">https://llmtimecapsule.octo.ai/</a>

LLM Time capsule - tracking state of the art LLMs.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">11:16 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">11:16 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB0E93FD904CC03E4EC85_919000844590@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Chetan +91 90008 44590" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno b2boiwr0"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">C</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d mglqvgej" role=""><span dir="auto" aria-label="Maybe Chetan" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Chetan</span><span class="WJuYU" testid="author" role="button">+91 90008 44590</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[11:27 am, 27/02/2024] +91 90008 44590: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g9gonq0i _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author">+91 77378 87058</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Lot of these projects are uncertainty bound than talent, money or data to be honest e.g. I have the skills to make a bhojpuri dataset + fine-tune TTS, but I don't know how to make $10K from it  — so I'll skip it completely</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>+1 for this. I have a TTS team sitting in my company, but its cheaper to just use Google or Azure. But I think Eleven labs pricing and the hype of AI, now we see some commercial interest for Indian language TTS. But even there, we see demand only for Hindi and maybe Tamil and Telugu.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">11:27 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">11:27 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="reaction 👍"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB05FBEF4101BA6371DE1_917737887058@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Nirant K +91 77378 87058" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno g9gonq0i"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">N</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author" role="button">+91 77378 87058</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[11:29 am, 27/02/2024] +91 77378 87058: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="b2boiwr0 _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d mglqvgej" role=""><span dir="auto" aria-label="Maybe Chetan" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Chetan</span><span class="WJuYU" testid="author">+91 90008 44590</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">+1 for this. I have a TTS team sitting in my company, but its cheaper to just use Google or Azure. But I think Eleven labs pricing and the hype of AI, now we see some commercial interest for Indian language TTS. But even there, we see demand only for Hindi and maybe Tamil and Telugu.</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Very bullish on Tamil &amp; Telugu — has applications in higher purchasing power states within India and US as well</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">11:29 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">11:29 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB0E42F286B43062A40FD_919000844590@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Chetan +91 90008 44590" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno b2boiwr0"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">C</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d mglqvgej" role=""><span dir="auto" aria-label="Maybe Chetan" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Chetan</span><span class="WJuYU" testid="author" role="button">+91 90008 44590</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[11:31 am, 27/02/2024] +91 90008 44590: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g9gonq0i _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author">+91 77378 87058</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Very bullish on Tamil &amp; Telugu — has applications in higher purchasing power states within India and US as well</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Yes. Seriously thinking of creating a quality TTS and maybe an ASR, open source model. Discussing with couple of teams. Will update and ask for help here :)</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">11:31 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">11:31 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="reaction ❤"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="❤️" draggable="false" class="bd96tnyg eipqoq80 b8 emoji apple _11JPr" tabindex="-1" style="background-position: -20px 0px;"></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj _3sxvM" data-id="false_120363049558306142@g.us_3EB0158AFB463DDF7F197D_19377081307@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Dr. Pratik Desai +1 (937) 708-1307" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/407052186_368942458989574_5837140328940210822_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRT7N2iCE5tWNPEgrn8zUMppMyQOh2yI-vDe0xjKk_mmA&amp;oe=65EAD8D6&amp;_nc_sid=e6ed6c&amp;_nc_cat=110" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe Dr. Pratik Desai" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Dr. Pratik Desai</span><span class="WJuYU" testid="author" role="button">+1 (937) 708-1307</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[11:32 am, 27/02/2024] +1 (937) 708-1307: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g9gonq0i _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author">+91 77378 87058</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Lot of these projects are uncertainty bound than talent, money or data to be honest e.g. I have the skills to make a bhojpuri dataset + fine-tune TTS, but I don't know how to make $10K from it  — so I'll skip it completely</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Yeah, my use case is grounded to this, so if someone who has already figured out and want to score some social impact brownie points, please let me know.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">11:32 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">11:32 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB0BF68F7CE5E5EEB5F4A_19377081307@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a"><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe Dr. Pratik Desai" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Dr. Pratik Desai</span><span class="WJuYU" testid="author" role="button">+1 (937) 708-1307</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[11:40 am, 27/02/2024] +1 (937) 708-1307: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="b2boiwr0 _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d mglqvgej" role=""><span dir="auto" aria-label="Maybe Chetan" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Chetan</span><span class="WJuYU" testid="author">+91 90008 44590</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Yes. Seriously thinking of creating a quality TTS and maybe an ASR, open source model. Discussing with couple of teams. Will update and ask for help here :)</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>If you have tried it before, what size of dataset one will need for decent tuning?</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">11:40 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">11:40 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3AA4698EBFB3E55AF8D7_919915123897@c.us"><div class="_3EyT- message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Harveen +91 99151 23897" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/55964039_402243816992965_6637152673499971584_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdQtkZ88AbZaodP4QnfDlPwkgUdBcxY7iFDH1wr_oFccKQ&amp;oe=65EAFA23&amp;_nc_sid=e6ed6c&amp;_nc_cat=102" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d a7s2a5ab" role=""><span dir="auto" aria-label="Maybe Harveen" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Harveen</span><span class="WJuYU" testid="author" role="button">+91 99151 23897</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[11:40 am, 27/02/2024] +91 99151 23897: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g9gonq0i _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author">+91 77378 87058</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">cc <span class=""><span dir="auto" data-jid="919952465050@c.us" data-display="~Aashay" aria-label="Maybe Aashay" class="_11JPr">@<span dir="ltr">~Aashay</span></span></span> <span class=""><span dir="auto" data-jid="919915123897@c.us" data-display="~Harveen" aria-label="Maybe Harveen" class="_11JPr">@<span dir="ltr">~Harveen</span></span></span> would you happen to know someone better than Pratik dada at Indic TTS?</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="M6sU5"><div class="xzT4z gndfcl4n l8fojup5 paxyh2gw sfeitywo cqsf3vkf k2bacm8l ajgl1lbb p357zi0d sap93d0t ovllcyds kvdvyush gfz4du6o r7fjleex rrq4r3yd"><div class="k5z04t9s cm280p3y p357zi0d ggj6brxn m0h2a7mj lb5m6g5c f8m0rgwh ac2vgrno idwf4z32 onmzirdk gfz4du6o r7fjleex n1yiu2zv itegkywt eb4rp10x rppts313 ln8gz9je"><div class="aoi073rw suguakab jjtagnqy c32ccnay tvf2evcx m0h2a7mj lb5m6g5c j4enbv94 gfz4du6o r7fjleex lhj4utae ljrqcn24" title="Research to Production - NeoDub"><span dir="ltr" aria-label="" class="ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae i033jvx7 cw3vfol9 _11JPr" style="min-height: 0px;">Research to Production - NeoDub</span></div><div class="jjtagnqy tvf2evcx ttu8nud2 lb5m6g5c dsh4tgtl gfz4du6o r7fjleex lhj4utae"><span class="g0rxnol2 gfz4du6o r7fjleex" style="display: -webkit-box; overflow: hidden; -webkit-box-orient: vertical; -webkit-line-clamp: 2;"><span dir="ltr" title="scaling in-house tech to a million users!" aria-label="" class="i033jvx7 _11JPr" style="min-height: 0px;">scaling in-house tech to a million users!</span></span></div><div class="j8h4s3v0 tvf2evcx oq44ahr5 lb5m6g5c dsh4tgtl qbqilfqo"><span dir="ltr" aria-label="" class="_11JPr" style="min-height: 0px;">black.dubverse.ai</span></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>dubverse is one company that is trying to make this happen. Although they are not perfect but I believe they will reach there.

<a dir="auto" href="https://black.dubverse.ai/p/research-to-production-neodub?subscribe_prompt=free" title="https://black.dubverse.ai/p/research-to-production-neodub?subscribe_prompt=free" target="_blank" rel="noopener noreferrer" class="_11JPr selectable-text copyable-text">https://black.dubverse.ai/p/research-to-production-neodub?subscribe_prompt=free</a></span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">11:40 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">11:40 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div role="button" tabindex="0" aria-label="Forward media" class="_1Gsa4 p357zi0d gndfcl4n ac2vgrno cxkis295 rqm6ogl5 i5tg98hk f9ovudaz przvwfww gx1rr48f f8jlpxt4 hnx8ox4h k17s6i4e ofejerhi ilbp7ui4 g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb a27i2aag rtx6r8la e3b81npk oa9ii99z fb82vjfj"><span data-icon="forward-chat" class="r83rrh3w"><svg viewBox="0 0 25 25" height="25" width="25" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 25 25"><title>forward-chat</title><path fill-rule="evenodd" clip-rule="evenodd" fill="currentColor" d="M14.248,6.973c0-0.614,0.741-0.921,1.174-0.488l5.131,5.136 c0.269,0.269,0.269,0.704,0,0.973l-5.131,5.136c-0.433,0.433-1.174,0.126-1.174-0.488v-2.319c-4.326,0-7.495,1.235-9.85,3.914 c-0.209,0.237-0.596,0.036-0.511-0.268c1.215-4.391,4.181-8.492,10.361-9.376V6.973z"></path></svg></span></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_1AABC2E79EA196BAC713EB58CF9B45D8_919342533525@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Luci | Manish 😎 +91 93425 33525" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/426392606_1110983826773793_5231113528370352030_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRVQKONAGRP_-qbwQhpVfrf39ogbELFJkopkK_AI2xMXA&amp;oe=65EADC8B&amp;_nc_sid=e6ed6c&amp;_nc_cat=109" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d a1q56urb" role=""><span dir="auto" aria-label="Maybe Luci | Manish 😎" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Luci | Manish <img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="😎" draggable="false" class="b86 emoji apple _11JPr" tabindex="-1" style="background-position: 0px -20px;"></span><span class="WJuYU" testid="author" role="button">+91 93425 33525</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[11:40 am, 27/02/2024] +91 93425 33525: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="m8ez62k1 _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d lr20dbn2" role=""><span dir="auto" aria-label="Maybe Sourasis Roy" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Sourasis Roy</span><span class="WJuYU" testid="author">+91 88980 20185</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">can't wait to get access to it. This can be automated right? write code and let the AI fix everything up after that</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>OBS integrated will be hella fun</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">11:40 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">11:40 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="reaction 💯"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="💯" draggable="false" class="bd96tnyg p83aruvr b75 emoji apple _11JPr" tabindex="-1" style="background-position: -60px -20px;"></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB0DC1C35BFA54D274627_919000844590@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Chetan +91 90008 44590" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno b2boiwr0"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">C</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d mglqvgej" role=""><span dir="auto" aria-label="Maybe Chetan" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Chetan</span><span class="WJuYU" testid="author" role="button">+91 90008 44590</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[11:43 am, 27/02/2024] +91 90008 44590: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="po56q9xr _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe Dr. Pratik Desai" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Dr. Pratik Desai</span><span class="WJuYU" testid="author">+1 (937) 708-1307</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">If you have tried it before, what size of dataset one will need for decent tuning?</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Havent tried with the new approaches. The TTS we had built was old school stuff and needed lesser data but more single person studio quality recording of fixed utterances.
We will be doing a training run with around 50 hours of data. Will update here of the status. Unfortunately have to do all this with volunteer effort and time. Ozonetel fortunately/unfortunately has a different business :)</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">11:43 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">11:43 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB0C1421872B30DA06C3A_19377081307@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Dr. Pratik Desai +1 (937) 708-1307" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/407052186_368942458989574_5837140328940210822_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRT7N2iCE5tWNPEgrn8zUMppMyQOh2yI-vDe0xjKk_mmA&amp;oe=65EAD8D6&amp;_nc_sid=e6ed6c&amp;_nc_cat=110" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe Dr. Pratik Desai" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Dr. Pratik Desai</span><span class="WJuYU" testid="author" role="button">+1 (937) 708-1307</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[11:43 am, 27/02/2024] +1 (937) 708-1307: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g755h7wn _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d a7s2a5ab" role=""><span dir="auto" aria-label="Maybe Harveen" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Harveen</span><span class="WJuYU" testid="author">+91 99151 23897</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">dubverse is one company that is trying to make this happen. Although they are not perfect but I believe they will reach there.

https://black.dubverse.ai/p/research-to-production-neodub?subscribe_prompt=free</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Do we have DubVerse or NeoDub folks in the group?</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">11:43 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">11:43 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A7FF203649342AE5D94_919915123897@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Harveen +91 99151 23897" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/55964039_402243816992965_6637152673499971584_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdQtkZ88AbZaodP4QnfDlPwkgUdBcxY7iFDH1wr_oFccKQ&amp;oe=65EAFA23&amp;_nc_sid=e6ed6c&amp;_nc_cat=102" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d a7s2a5ab" role=""><span dir="auto" aria-label="Maybe Harveen" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Harveen</span><span class="WJuYU" testid="author" role="button">+91 99151 23897</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[11:47 am, 27/02/2024] +91 99151 23897: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="b2boiwr0 _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d mglqvgej" role=""><span dir="auto" aria-label="Maybe Chetan" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Chetan</span><span class="WJuYU" testid="author">+91 90008 44590</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Yes. Seriously thinking of creating a quality TTS and maybe an ASR, open source model. Discussing with couple of teams. Will update and ask for help here :)</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>On ASR: if you can sponsor compute, I will finetune conformer and whisper models and open source them. No TnC</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">11:47 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">11:47 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="reaction 👍"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A6EC75C2A2087BD2D5E_918764022384@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Dev Aggarwal +91 87640 22384" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/415516569_6903510163080437_5920119927943060812_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdShVYDB__CgvgPewP-5DvgPChk5ggs5iqevVmhMMQt5Ug&amp;oe=65EAF20D&amp;_nc_sid=e6ed6c&amp;_nc_cat=108" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Dev Aggarwal" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Dev Aggarwal</span><span class="WJuYU" testid="author" role="button">+91 87640 22384</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[11:50 am, 27/02/2024] +91 87640 22384: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g755h7wn _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d a7s2a5ab" role=""><span dir="auto" aria-label="Maybe Harveen" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Harveen</span><span class="WJuYU" testid="author">+91 99151 23897</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">On ASR: if you can sponsor compute, I will finetune conformer and whisper models and open source them. No TnC</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Better than the whisper telugu stuff on hugginface?</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">11:50 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">11:50 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB0289ECCEEE857659744_919000844590@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Chetan +91 90008 44590" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno b2boiwr0"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">C</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d mglqvgej" role=""><span dir="auto" aria-label="Maybe Chetan" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Chetan</span><span class="WJuYU" testid="author" role="button">+91 90008 44590</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[11:51 am, 27/02/2024] +91 90008 44590: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g755h7wn _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d a7s2a5ab" role=""><span dir="auto" aria-label="Maybe Harveen" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Harveen</span><span class="WJuYU" testid="author">+91 99151 23897</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">On ASR: if you can sponsor compute, I will finetune conformer and whisper models and open source them. No TnC</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Awesome, getting a volunteer driven compute cluster going(Ozonetel has a couple of GPUs, other orgs have also said they will donate compute). Will connect with you. BTW, do you have the dataset or need a dataset also?</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">11:51 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">11:51 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A8E6ABFDB2CF11A3888_919550164716@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Ravi Theja +91 95501 64716" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno j3zl77sr"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">R</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d gvktkc8w" role=""><span dir="auto" aria-label="Maybe Ravi Theja" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Ravi Theja</span><span class="WJuYU" testid="author" role="button">+91 95501 64716</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[11:55 am, 27/02/2024] +91 95501 64716: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="b2boiwr0 _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d mglqvgej" role=""><span dir="auto" aria-label="Maybe Chetan" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Chetan</span><span class="WJuYU" testid="author">+91 90008 44590</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Havent tried with the new approaches. The TTS we had built was old school stuff and needed lesser data but more single person studio quality recording of fixed utterances.
We will be doing a training run with around 50 hours of data. Will update here of the status. Unfortunately have to do all this with volunteer effort and time. Ozonetel fortunately/unfortunately has a different business :)</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Is this effort with swecha organisation?</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">11:55 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">11:55 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj _3sxvM" data-id="false_120363049558306142@g.us_3EB0F16A226DA1FE63D25C_919000844590@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Chetan +91 90008 44590" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno b2boiwr0"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">C</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d mglqvgej" role=""><span dir="auto" aria-label="Maybe Chetan" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Chetan</span><span class="WJuYU" testid="author" role="button">+91 90008 44590</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[11:56 am, 27/02/2024] +91 90008 44590: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Yes. Along with other small efforts I am doing. As long as it is open source, I am in :)</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">11:56 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">11:56 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="reaction 👍"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB0289F2538EBE763D9E5_919000844590@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a"><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="copyable-text" data-pre-plain-text="[11:56 am, 27/02/2024] +91 90008 44590: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>trying to get a college consortium going too for compute. But that might take longer.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">11:56 am</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">11:56 am</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="reaction 👍"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3ABD11DB7405FCDE606D_14086858455@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Chiradeep Vittal +1 (408) 685-8455" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/69919829_527563724698635_5849884592617553920_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRY_z9osEmj-oOOmutEyQ1822rDRf50EKaCXN4-6fx8EA&amp;oe=65EAD1CB&amp;_nc_sid=e6ed6c&amp;_nc_cat=100" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d a1q56urb" role=""><span dir="auto" aria-label="Maybe Chiradeep Vittal" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Chiradeep Vittal</span><span class="WJuYU" testid="author" role="button">+1 (408) 685-8455</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[1:02 pm, 27/02/2024] +1 (408) 685-8455: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Is there any qualitative difference between different providers (anyscale, together, etc) apart from speed/latency? That is, for the same prompt, Llama-7b on Anyscale will give a very similar response to Llama-7b on Together?</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:02 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:02 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A50A0C77E7E472522C1_919899951010@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Rohit Agarwal +91 98999 51010" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/394041856_650775203840503_1946601418234660028_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdSigm2WDEf73uMEF0SmpqyXgxclFvRFMewswXma0MfA9Q&amp;oe=65EAE034&amp;_nc_sid=e6ed6c&amp;_nc_cat=105" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d sthcsby1" role=""><span dir="auto" aria-label="Maybe Rohit Agarwal" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Rohit Agarwal</span><span class="WJuYU" testid="author" role="button">+91 98999 51010</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[1:04 pm, 27/02/2024] +91 98999 51010: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="exofpctk _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d a1q56urb" role=""><span dir="auto" aria-label="Maybe Chiradeep Vittal" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Chiradeep Vittal</span><span class="WJuYU" testid="author">+1 (408) 685-8455</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Is there any qualitative difference between different providers (anyscale, together, etc) apart from speed/latency? That is, for the same prompt, Llama-7b on Anyscale will give a very similar response to Llama-7b on Together?</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>yes, it should. The underlying model is the same but we’ve observed minor differences at times. Although everyone claims to have the same non-quantized model behind the API</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:04 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:04 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj _3sxvM" data-id="false_120363049558306142@g.us_3EB0666BD5C72E69AFA9B1_917737887058@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Nirant K +91 77378 87058" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno g9gonq0i"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">N</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author" role="button">+91 77378 87058</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[1:04 pm, 27/02/2024] +91 77378 87058: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="exofpctk _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d a1q56urb" role=""><span dir="auto" aria-label="Maybe Chiradeep Vittal" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Chiradeep Vittal</span><span class="WJuYU" testid="author">+1 (408) 685-8455</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Is there any qualitative difference between different providers (anyscale, together, etc) apart from speed/latency? That is, for the same prompt, Llama-7b on Anyscale will give a very similar response to Llama-7b on Together?</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Yes, that is the expectation. There might be some quantization on their inferencing as well — which can lead to minor differences. I've not compared hosting providers, so hard to say</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:04 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:04 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="reaction 👍"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB0DB543ED4AF1A038B3D_917737887058@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a"><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author" role="button">+91 77378 87058</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[1:05 pm, 27/02/2024] +91 77378 87058: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g755h7wn _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d a7s2a5ab" role=""><span dir="auto" aria-label="Maybe Harveen" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Harveen</span><span class="WJuYU" testid="author">+91 99151 23897</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">On ASR: if you can sponsor compute, I will finetune conformer and whisper models and open source them. No TnC</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>What amount do you need? Can do in personal capacity and ask friends too if it's small enough</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:05 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:05 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="reaction ➕"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="➕" draggable="false" class="bd96tnyg p83aruvr b8 emoji apple _11JPr" tabindex="-1" style="background-position: -80px 0px;"></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj _3sxvM" data-id="false_120363049558306142@g.us_3A9DF2A744F057B349B9_919564191888@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Arko C | xylem.ai +91 95641 91888" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno oj03u76w"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">A</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d dud7p2xx" role=""><span dir="auto" aria-label="Maybe Arko C | xylem.ai" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Arko C | xylem.ai</span><span class="WJuYU" testid="author" role="button">+91 95641 91888</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[1:51 pm, 27/02/2024] +91 95641 91888: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="jc7tlmys _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d sthcsby1" role=""><span dir="auto" aria-label="Maybe Rohit Agarwal" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Rohit Agarwal</span><span class="WJuYU" testid="author">+91 98999 51010</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">yes, it should. The underlying model is the same but we’ve observed minor differences at times. Although everyone claims to have the same non-quantized model behind the API</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>That’s cause the LLM config is not the same for most. One of our customers has seen a lot more difference for structured data use cases (sql queries n all)</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:51 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:51 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3AB9417A7BA6452FDE6D_919564191888@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a"><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="copyable-text" data-pre-plain-text="[1:53 pm, 27/02/2024] +91 95641 91888: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Everyone claims to have fp16, so can’t comment on whether they degrade the model or anything. Cause for something like fp8, you barely see a performance drop in general. Hard to notice. But these will be nothing but conspiracy theories.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">1:53 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">1:53 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A13117537CC5366280B_919915123897@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Harveen +91 99151 23897" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/55964039_402243816992965_6637152673499971584_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdQtkZ88AbZaodP4QnfDlPwkgUdBcxY7iFDH1wr_oFccKQ&amp;oe=65EAFA23&amp;_nc_sid=e6ed6c&amp;_nc_cat=102" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d a7s2a5ab" role=""><span dir="auto" aria-label="Maybe Harveen" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Harveen</span><span class="WJuYU" testid="author" role="button">+91 99151 23897</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:01 pm, 27/02/2024] +91 99151 23897: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g9gonq0i _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author">+91 77378 87058</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">What amount do you need? Can do in personal capacity and ask friends too if it's small enough</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>8, 32G V100</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">2:01 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:01 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_31706052071709023556_917042355293@c.us"><div class="_2OvAm focusable-list-item _1jHIY"><span class=""></span><div class="UzMP7 n6BPp"><div class="_1BOF7"><div class="_1sykI"><div class="Mk0Bp _30scZ"><span dir="auto" aria-label="" class="l7jjieqr ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">+91 77378 87058</span></div> added <div class="Mk0Bp _30scZ"><span dir="auto" aria-label="" class="l7jjieqr ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">+91 70423 55293</span></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"></div></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB0046E448E7A6454E08C_917737887058@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Nirant K +91 77378 87058" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno g9gonq0i"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">N</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author" role="button">+91 77378 87058</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:19 pm, 27/02/2024] +91 77378 87058: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g755h7wn _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d a7s2a5ab" role=""><span dir="auto" aria-label="Maybe Harveen" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Harveen</span><span class="WJuYU" testid="author">+91 99151 23897</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">8, 32G V100</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>cc <span role="button" tabindex="0" class=""><span dir="auto" data-jid="917356725027@c.us" data-display="~Jibin Sabu" aria-label="Maybe Jibin Sabu" class="ajgl1lbb o0rubyzf _11JPr selectable-text select-all copyable-text" data-plain-text="@~Jibin Sabu" data-app-text-template="​917356725027@c.us​">@<span dir="ltr">~Jibin Sabu</span></span></span>

This is Rs. 100/hour on E2E — so Rs. 1K/hour for say, 10 hours? I'll sponsor this.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">2:19 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:19 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="Reactions ❤, 👍 2 in total"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="❤️" draggable="false" class="bd96tnyg eipqoq80 b8 emoji apple _11JPr" tabindex="-1" style="background-position: -20px 0px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk o2es7gts i2cterl7" style="flex-shrink: 0;"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 f8m0rgwh r15c9g6i ilf8vifs gfz4du6o r7fjleex qnz2jpws kjemk6od"><div class="dn5wc90o ilf8vifs gndfcl4n p357zi0d"><span class="f8jlpxt4 qfejxiq4 ce5kru2g">2</span></div></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_E0546014C580DF8B416BAAC2203E3484_918778729707@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Anjineyulu +91 87787 29707" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/312294024_1292916711546768_6143651049984296641_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRx-uGqtZBy70dugDrKp6xg4YJ0rWqAm24DE4LiDDYJpQ&amp;oe=65EAFED5&amp;_nc_sid=e6ed6c&amp;_nc_cat=103" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d dud7p2xx" role=""><span dir="auto" aria-label="Maybe Anjineyulu" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Anjineyulu</span><span class="WJuYU" testid="author" role="button">+91 87787 29707</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:19 pm, 27/02/2024] +91 87787 29707: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g9gonq0i _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author">+91 77378 87058</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Lot of these projects are uncertainty bound than talent, money or data to be honest e.g. I have the skills to make a bhojpuri dataset + fine-tune TTS, but I don't know how to make $10K from it  — so I'll skip it completely</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Isn't this still challenging with VCs thinking only 2 of 10 startups need to be successful?</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">2:19 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:19 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB01A67E8FD6F0FC9363E_917737887058@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Nirant K +91 77378 87058" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno g9gonq0i"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">N</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author" role="button">+91 77378 87058</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:20 pm, 27/02/2024] +91 77378 87058: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="oj03u76w _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d dud7p2xx" role=""><span dir="auto" aria-label="Maybe Anjineyulu" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Anjineyulu</span><span class="WJuYU" testid="author">+91 87787 29707</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Isn't this still challenging with VCs thinking only 2 of 10 startups need to be successful?</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>I don't think this project needs a VC consideration at all — it's a hacker project, much like Krutrim</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5 hnx8ox4h mw4yctpw gj5xqxfh">Edited</span><span class="tvf2evcx oq44ahr5">2:20 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto"><span class="mw4yctpw gj5xqxfh">Edited</span>2:20 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3AC33D36CE9C7C98E124_917356725027@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Jibin Sabu +91 73567 25027" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/316930366_673815027828033_2858278428098029672_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdTlfB_H6A7cYH6jBRkZDBkKmFnVmitblzmCVTi0mkWH4w&amp;oe=65EAD6B9&amp;_nc_sid=e6ed6c&amp;_nc_cat=105" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d pfvzdcj5" role=""><span dir="auto" aria-label="Maybe Jibin Sabu" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Jibin Sabu</span><span class="WJuYU" testid="author" role="button">+91 73567 25027</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:22 pm, 27/02/2024] +91 73567 25027: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="oj03u76w _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d dud7p2xx" role=""><span dir="auto" aria-label="Maybe Anjineyulu" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Anjineyulu</span><span class="WJuYU" testid="author">+91 87787 29707</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Isn't this still challenging with VCs thinking only 2 of 10 startups need to be successful?</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Its not like that - more like they want each of their invested startups to just get the enough ROI</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">2:22 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:22 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_82CA520F51D38249C9B2900F64CAAF5E_917892792975@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Adarsh +91 78927 92975" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/363225108_857693822457811_1088711388449985548_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRcT6arSV5WnGI4aSQcbLouECrP13HSczwH_vmBb1fOmA&amp;oe=65EB07E2&amp;_nc_sid=e6ed6c&amp;_nc_cat=109" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d gvktkc8w" role=""><span dir="auto" aria-label="Maybe Adarsh" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Adarsh</span><span class="WJuYU" testid="author" role="button">+91 78927 92975</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:22 pm, 27/02/2024] +91 78927 92975: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g9gonq0i _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author">+91 77378 87058</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">cc <span class=""><span dir="auto" data-jid="917356725027@c.us" data-display="~Jibin Sabu" aria-label="Maybe Jibin Sabu" class="_11JPr">@<span dir="ltr">~Jibin Sabu</span></span></span>

This is Rs. 100/hour on E2E — so Rs. 1K/hour for say, 10 hours? I'll sponsor this.</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>We'll sponsor too! <img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="😄" draggable="false" class="b85 emoji apple _11JPr selectable-text copyable-text" data-plain-text="😄" tabindex="-1" style="background-position: 0px -80px;"></span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">2:22 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:22 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3AADD9E518DF62A2E146_19377081307@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Dr. Pratik Desai +1 (937) 708-1307" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/407052186_368942458989574_5837140328940210822_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRT7N2iCE5tWNPEgrn8zUMppMyQOh2yI-vDe0xjKk_mmA&amp;oe=65EAD8D6&amp;_nc_sid=e6ed6c&amp;_nc_cat=110" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe Dr. Pratik Desai" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Dr. Pratik Desai</span><span class="WJuYU" testid="author" role="button">+1 (937) 708-1307</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:22 pm, 27/02/2024] +1 (937) 708-1307: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g9gonq0i _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author">+91 77378 87058</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">cc <span class=""><span dir="auto" data-jid="917356725027@c.us" data-display="~Jibin Sabu" aria-label="Maybe Jibin Sabu" class="_11JPr">@<span dir="ltr">~Jibin Sabu</span></span></span>

This is Rs. 100/hour on E2E — so Rs. 1K/hour for say, 10 hours? I'll sponsor this.</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Count me in.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">2:22 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:22 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3AA36D1EC35F746DB944_919915123897@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Harveen +91 99151 23897" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/55964039_402243816992965_6637152673499971584_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdQtkZ88AbZaodP4QnfDlPwkgUdBcxY7iFDH1wr_oFccKQ&amp;oe=65EAFA23&amp;_nc_sid=e6ed6c&amp;_nc_cat=102" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d a7s2a5ab" role=""><span dir="auto" aria-label="Maybe Harveen" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Harveen</span><span class="WJuYU" testid="author" role="button">+91 99151 23897</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:22 pm, 27/02/2024] +91 99151 23897: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g9gonq0i _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author">+91 77378 87058</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">cc <span class=""><span dir="auto" data-jid="917356725027@c.us" data-display="~Jibin Sabu" aria-label="Maybe Jibin Sabu" class="_11JPr">@<span dir="ltr">~Jibin Sabu</span></span></span>

This is Rs. 100/hour on E2E — so Rs. 1K/hour for say, 10 hours? I'll sponsor this.</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Considering 1k hours of training data it would be 7 failed iterations, 96 hours of training one iteration multiply by 2 (whisper, conf) for one lang.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5 hnx8ox4h mw4yctpw gj5xqxfh">Edited</span><span class="tvf2evcx oq44ahr5">2:22 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto"><span class="mw4yctpw gj5xqxfh">Edited</span>2:22 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A1D12EBB4A5064E0742_919952465050@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Aashay +91 99524 65050" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno po56q9xr"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">A</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe Aashay" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Aashay</span><span class="WJuYU" testid="author" role="button">+91 99524 65050</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:23 pm, 27/02/2024] +91 99524 65050: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g9gonq0i _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author">+91 77378 87058</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">cc <span class=""><span dir="auto" data-jid="919952465050@c.us" data-display="~Aashay" aria-label="Maybe Aashay" class="_11JPr">@<span dir="ltr">~Aashay</span></span></span> <span class=""><span dir="auto" data-jid="919915123897@c.us" data-display="~Harveen" aria-label="Maybe Harveen" class="_11JPr">@<span dir="ltr">~Harveen</span></span></span> would you happen to know someone better than Pratik dada at Indic TTS?</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Yes we are going to put a model out on azure soon. Should be out in a week or so</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">2:23 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:23 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="Reactions 👍 2 in total"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk o2es7gts i2cterl7" style="flex-shrink: 0;"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 f8m0rgwh r15c9g6i ilf8vifs gfz4du6o r7fjleex qnz2jpws kjemk6od"><div class="dn5wc90o ilf8vifs gndfcl4n p357zi0d"><span class="f8jlpxt4 qfejxiq4 ce5kru2g">2</span></div></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A7F6DAE45C9F7AC764E_19377081307@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Dr. Pratik Desai +1 (937) 708-1307" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/407052186_368942458989574_5837140328940210822_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRT7N2iCE5tWNPEgrn8zUMppMyQOh2yI-vDe0xjKk_mmA&amp;oe=65EAD8D6&amp;_nc_sid=e6ed6c&amp;_nc_cat=110" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe Dr. Pratik Desai" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Dr. Pratik Desai</span><span class="WJuYU" testid="author" role="button">+1 (937) 708-1307</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:26 pm, 27/02/2024] +1 (937) 708-1307: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="po56q9xr _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe Aashay" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Aashay</span><span class="WJuYU" testid="author">+91 99524 65050</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Yes we are going to put a model out on azure soon. Should be out in a week or so</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Part of Azure speech services?</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">2:26 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:26 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A7CEEA8B3FEDFDE22CB_919952465050@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Aashay +91 99524 65050" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno po56q9xr"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">A</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe Aashay" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Aashay</span><span class="WJuYU" testid="author" role="button">+91 99524 65050</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:26 pm, 27/02/2024] +91 99524 65050: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="po56q9xr _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe Dr. Pratik Desai" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Dr. Pratik Desai</span><span class="WJuYU" testid="author">+1 (937) 708-1307</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Part of Azure speech services?</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Not aware of the branding specifics</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5 hnx8ox4h mw4yctpw gj5xqxfh">Edited</span><span class="tvf2evcx oq44ahr5">2:26 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto"><span class="mw4yctpw gj5xqxfh">Edited</span>2:26 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3AE0CE85825E63B71C0A_19377081307@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Dr. Pratik Desai +1 (937) 708-1307" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/407052186_368942458989574_5837140328940210822_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdRT7N2iCE5tWNPEgrn8zUMppMyQOh2yI-vDe0xjKk_mmA&amp;oe=65EAD8D6&amp;_nc_sid=e6ed6c&amp;_nc_cat=110" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe Dr. Pratik Desai" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Dr. Pratik Desai</span><span class="WJuYU" testid="author" role="button">+1 (937) 708-1307</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:27 pm, 27/02/2024] +1 (937) 708-1307: "><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Will wait then.</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">2:27 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:27 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_82BDDC64F5776F8C30E3E17B7E7E330C_918867705880@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Adithya +91 88677 05880" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/319947062_1208068306490361_762498494962268271_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdSLcJL6jiRgAQdZQsPwsIjZiltHgg3MgrJf_L012-6J_Q&amp;oe=65EAE599&amp;_nc_sid=e6ed6c&amp;_nc_cat=101" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d dud7p2xx" role=""><span dir="auto" aria-label="Maybe Adithya" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Adithya</span><span class="WJuYU" testid="author" role="button">+91 88677 05880</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:27 pm, 27/02/2024] +91 88677 05880: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g755h7wn _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d a7s2a5ab" role=""><span dir="auto" aria-label="Maybe Harveen" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Harveen</span><span class="WJuYU" testid="author">+91 99151 23897</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">On ASR: if you can sponsor compute, I will finetune conformer and whisper models and open source them. No TnC</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Do you have indic data?</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">2:27 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:27 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB075D1F6718201C34F29_917737887058@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Nirant K +91 77378 87058" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno g9gonq0i"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">N</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author" role="button">+91 77378 87058</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:27 pm, 27/02/2024] +91 77378 87058: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="po56q9xr _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe Aashay" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Aashay</span><span class="WJuYU" testid="author">+91 99524 65050</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Not aware of the technicalities</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>Not aware of branding specifics — we're the technical people broooo</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">2:27 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:27 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3AE5435F281FED13CB58_919952465050@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Aashay +91 99524 65050" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno po56q9xr"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">A</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe Aashay" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Aashay</span><span class="WJuYU" testid="author" role="button">+91 99524 65050</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[2:28 pm, 27/02/2024] +91 99524 65050: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g9gonq0i _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author">+91 77378 87058</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Not aware of branding specifics — we're the technical people broooo</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><div class="g0rxnol2 pglj95m3 p357zi0d ac2vgrno n1yiu2zv tt8xd2xn aumn88wd k0lnf8n4 htq1q0kf n3nyt3io _11JPr selectable-text copyable-text"><span class="Ov-s3"><img crossorigin="anonymous" src="/img/c5a15be93e425dcb8a26b06645ad4574_a_2171-64.png" alt="😝" draggable="false" class="URwQL YGPO2 _11JPr selectable-text copyable-text" data-plain-text="😝" tabindex="-1" style="visibility: visible;"></span></div></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">2:28 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3EB0A851E2019C66D6487C_918867852604@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe Piyush Makhija +91 88678 52604" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><img src="https://pps.whatsapp.net/v/t61.24694-24/253043599_404099507816038_9133132733458706662_n.jpg?stp=dst-jpg_s96x96&amp;ccb=11-4&amp;oh=01_AdTighSvAerv9oV6jbu0O83AhoxDpEU3KQPOEjRYfDtK7g&amp;oe=65EAFEFD&amp;_nc_sid=e6ed6c&amp;_nc_cat=101" alt="" draggable="false" class="csshhazd g0rxnol2 f804f6gw ln8gz9je ppled2lx gfz4du6o r7fjleex g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb jpthtbts lyqpd7li bs7a17vp _11JPr" tabindex="-1" style="visibility: visible;"></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d oyzge2kb" role=""><span dir="auto" aria-label="Maybe Piyush Makhija" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">Piyush Makhija</span><span class="WJuYU" testid="author" role="button">+91 88678 52604</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[3:12 pm, 27/02/2024] +91 88678 52604: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="g9gonq0i _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d as1tos1z" role=""><span dir="auto" aria-label="Maybe Nirant K" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Nirant K</span><span class="WJuYU" testid="author">+91 77378 87058</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">cc <span class=""><span dir="auto" data-jid="917356725027@c.us" data-display="~Jibin Sabu" aria-label="Maybe Jibin Sabu" class="_11JPr">@<span dir="ltr">~Jibin Sabu</span></span></span>

This is Rs. 100/hour on E2E — so Rs. 1K/hour for say, 10 hours? I'll sponsor this.</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="ltr" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>I'm in contact with some folks at E2E, they are open to sponsoring some compute for small projects</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">3:12 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">3:12 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"><button tabindex="0" class="dhq51u3o" aria-haspopup="true" aria-label="Reactions 👍, 🙃 2 in total"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 ac2vgrno sap93d0t gndfcl4n bmot90v7 i5tg98hk jfqm35v0 przvwfww bdbt56hn r18xjin9 pz6oj2k2 sa2xs5zj n58sa971 jht8oeb6 fahkg6u0 hcpeg58q p6ubcsnv avd5zzxf qqik71jn om6y7gxh gj5xqxfh"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="👍" draggable="false" class="bd96tnyg p83aruvr b39 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -60px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 tt8xd2xn jnwc1y2a mpdn4nr2 svoq16ka qfejxiq4 enbbiyaj r5qsrrlp"><div class=""><img crossorigin="anonymous" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" alt="🙃" draggable="false" class="bd96tnyg p83aruvr b88 emoji apple _11JPr" tabindex="-1" style="background-position: -20px -40px;"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk o2es7gts i2cterl7" style="flex-shrink: 0;"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 f8m0rgwh r15c9g6i ilf8vifs gfz4du6o r7fjleex qnz2jpws kjemk6od"><div class="dn5wc90o ilf8vifs gndfcl4n p357zi0d"><span class="f8jlpxt4 qfejxiq4 ce5kru2g">2</span></div></div></div></div></button></div></div></div></div><div class="" role="row"><div tabindex="-1" class="CzM4m _2zFLj" data-id="false_120363049558306142@g.us_3A1DB675F1203F36C28B_918105723869@c.us"><div class="message-in focusable-list-item _1AOLJ _1jHIY"><span class=""></span><div class="UzMP7 _1uv-a _3m5cz"><span data-icon="tail-in" class="p0s8B"><svg viewBox="0 0 8 13" height="13" width="8" preserveAspectRatio="xMidYMid meet" class="" version="1.1" x="0px" y="0px" enable-background="new 0 0 8 13"><title>tail-in</title><path opacity="0.13" fill="#0000000" d="M1.533,3.568L8,12.193V1H2.812 C1.042,1,0.474,2.156,1.533,3.568z"></path><path fill="currentColor" d="M1.533,2.568L8,11.193V0L2.812,0C1.042,0,0.474,1.156,1.533,2.568z"></path></svg></span><div tabindex="0" role="button" aria-label="Open chat details for Maybe AA +91 81057 23869" class="lhggkp7q g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb hj486dpu a2hqsskl stnyektq" style="cursor: pointer;"><div class="ln8gz9je ppled2lx g9p5wyxn i0tg5vk9 aoogvgrq o2zu3hjb octy2vkd p357zi0d gndfcl4n ac2vgrno b2boiwr0"><div dir="ltr" class="f8jlpxt4"><div class="l7jjieqr hnx8ox4h">A</div><div class="l7jjieqr hnx8ox4h"></div></div></div></div><div class="_1BOF7 _2AOIt"><div><div class="cm280p3y to2l77zo n1yiu2zv ft2m32mm oq31bsqd e1yunedv"><div class="_3IzYj _6rIWC p357zi0d mglqvgej" role=""><span dir="auto" aria-label="Maybe AA" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 ajgl1lbb edeob0r2 _11JPr" style="min-height: 0px;">AA</span><span class="WJuYU" testid="author" role="button">+91 81057 23869</span></div><div class="_1DETJ copyable-text" data-pre-plain-text="[3:37 pm, 27/02/2024] +91 81057 23869: "><div class="_1hl2r"><div class="HLjg0"><div class="_1EbXp" role="button" aria-label="Quoted Message" tabindex="0"><span class="po56q9xr _1Jh1l"></span><div class="yKTUI _3pMOs"><div class="yKTUI _1JeGx"><div class="_3IzYj p357zi0d jott5mo6" role=""><span dir="auto" aria-label="Maybe Dr. Pratik Desai" class="_2ne0X ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 _11JPr" style="min-height: 0px;">Dr. Pratik Desai</span><span class="WJuYU" testid="author">+1 (937) 708-1307</span></div><div class="_37DQv" dir="ltr" role="button"><span dir="auto" aria-label="" class="quoted-mention _11JPr" style="min-height: 0px;">Count me in.</span></div></div></div><div class="jrEtJ"><div class="kknmh"></div></div></div></div></div><div class="_21Ahp"><span dir="auto" aria-label="" class="_11JPr selectable-text copyable-text" style="min-height: 0px;"><span>+1</span></span><span class=""><span class="o38k74y6 i86elurf neme6l2y kojwoqec bbl9m3t3 i5tg98hk jfqm35v0 przvwfww bdbt56hn cr2cog7z" aria-hidden="true"><span class="tvf2evcx oq44ahr5">3:37 pm</span></span></span></div></div><div class="g0rxnol2 g2bpp9au faxx4fbg aja0i6dq jnwc1y2a bn7x0pqn qnz2jpws"><div class="gq1t1y46 o38k74y6 e4p1bexh cr2cog7z le5p0ye3 p357zi0d gndfcl4n"><span class="l7jjieqr fewfhwl7" dir="auto">3:37 pm</span></div></div></div></div><span class=""></span><div class="_1OdBf"></div></div><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd p357zi0d dnb887gk gjuq5ydh i2cterl7 fhf7t426 sap93d0t gndfcl4n FxqSn"><div class="tvf2evcx m0h2a7mj lb5m6g5c j7l1k36l ktfrpxia nu7pwgvd dnb887gk gjuq5ydh i2cterl7 rqm6ogl5 i5tg98hk folpon7g przvwfww snweb893"><div></div></div></div></div><div class="p357zi0d g0rxnol2 blj1rie1 px10qoeu ltyqj8pj okbql4zm ex3gcxaf"></div></div></div></div></div>
"""


## Step 2: Provide your Open AI token and Hugging face token.
1. This uses mistral-7b from hugging face and Gpt-4-Turbo from OpenAI.
2. The cost of running it should be under 1$ for 2 days of messages.

In [3]:
#@title Paste the token here
os.environ['HUGGINGFACE_TOKEN'] = "<hugging_face_token>"
os.environ['OPENAI_API_KEY'] = "<open_ai_token>"


## Step 3: Execute everything below.
1. At the end of it, you will get a summary of all the discussions. Validate & edit the summary as required.
2. The links are provided as one big list. You can split it into Trending News and Cool Links based on the summary.
3. You will also get threads with less messages. Review those messages and see if the summary needs to be updated.

In [246]:
# Parses the html to find the rows from it to extract information from each row. Every message on whatsapp is a row.
def find_rows(html):
    # Parse the HTML
    tree = etree.HTML(html)

    # XPath query to find elements with the specified role
    xpath_query = "//*[@role='row']"

    # Find all elements matching the XPath query
    elements = tree.xpath(xpath_query)

    # Convert found elements back to HTML/XML strings
    elements_as_string = [etree.tostring(element, pretty_print=True).decode() for element in elements]

    return elements_as_string


# This is the main parsing code. It has capability to parse quoted text find relationship b/w them.
def get_message_details(row):
    root = etree.fromstring(row)
    message_data = []
    matching_divs = root.xpath(".//div[contains(@class, 'copyable-text')][1]")
    if matching_divs:
        # Proceed with the first (and in this case, the only) matching <div>
        row_data = matching_divs[0]
        quoted_message_divs = row_data.xpath('.//div[@role="button" and @aria-label="Quoted Message"]')
        maybe_span_text = ""
        quoted_mention_text = ""
        # Check if the message has a quoted message as a part of it.
        for div in quoted_message_divs:
            # If the user is not part of your contacts, this code is invoked.
            maybe_span_text = div.xpath('.//span[@testid="author"]/text()')
            if not maybe_span_text:
                # If the user is present in your contacts, this code will be invoked.
                maybe_span_text = div.xpath('.//div[@role=""]//span[@aria-label=""]/text()')

            quoted_mention_message = div.xpath('.//span[contains(@class, "quoted-mention")]//text()')

            #This extracts the phone number (if not in contacts) or name (if in Contacts)
            maybe_span_text = maybe_span_text[0] if maybe_span_text else "Not Found"

            #The quoted message is sometime a part of multiple <span> elements. We merge all of it here.
            if quoted_mention_message:
              quoted_mention_text = " ".join(quoted_mention_message)
            else:
               quoted_mention_text = "Not Found"

        # This is the actual message sent by the user.
        data_pre_plain_text = row_data.get('data-pre-plain-text')

        #Again the text can be part of different <span> elements. So merging them.
        text_span = row_data.xpath(".//span[contains(@class, 'selectable-text') and contains(@class, 'copyable-text')]//text()")
        if text_span:
            text = " ".join(text_span)
        else:
            #TODO If the user has shared an image. This code needs to be enhanced. Right now we are just getting the alt text of the image.
            text_div = row_data.xpath(".//div[contains(@class, 'selectable-text') and contains(@class, 'copyable-text')]//img/@alt")
            if text_div:
                text = text_div[0]
            else:
                text = None
        message_data.append({'message_author': data_pre_plain_text, 'text': text, 'quoted_author': maybe_span_text, 'quoted_text':quoted_mention_text})

    return message_data

In [248]:
# Find and print elements with role="row"
rows = find_rows(html)
len(rows)



245

In [249]:
# Extracted data.
message_details = []

# This variable can be used for debugging.
valid_rows=[]

# For each row, extract the metadata and store it in an object
for index, row in enumerate(rows):
    message_metadata = get_message_details(row);
    if message_metadata:
        message_details.extend(message_metadata)
        valid_rows.append(row)
    else:
        print(f"Index: {index}, has no row metadata")


len(valid_rows)


Index: 11, has no row metadata

Index: 27, has no row metadata

Index: 48, has no row metadata

Index: 68, has no row metadata

Index: 72, has no row metadata

...

Index: 166, has no row metadata

Index: 185, has no row metadata

Index: 186, has no row metadata

Index: 228, has no row metadata

233

In [250]:
# Convert list of dictionaries to pandas DataFrame
df = pd.DataFrame(message_details)
#pd.set_option('display.max_rows', None)

# Print the DataFrame as a table
display(df)

,message_author,text,quoted_author,quoted_text
0,"[9:31 pm, 24/02/2024] +1 (206) 499-3617:",\n Can you share how you did that? Which VLM? \n,+91 97730 65092,For us we just FTed a VLM
1,"[10:07 pm, 24/02/2024] Bharat Shetty:","\n https://lightning.ai/lightning-ai/studios/understanding-using-and-finetuning-gemma \n\nBy Sebastian Raschka\n\n1) Gemma uses a really large vocabulary and consequently embedding weight matrices.\n\n2) Like OLMo and GPT-2, Gemma also uses weight tying.\n\n3) It even uses GeLU (in the form of GeGLU), similar to GPT-2, unlike Llama 2 and other LLMs.\n\n4) Interestingly, Gemma normalizes the embeddings by the square root of the hidden layer dimension.\n\n5) The RMSNorm layers are in the usual location, different from what was hinted at in the technical paper.\n\n6) However, the RMSNorm implementation comes with the unit offset that wasn't mentioned in the technical paper. \n",,
2,"[10:18 pm, 24/02/2024] +91 77378 87058:",\n What's GeGLU? \n,Bharat Shetty,"https://lightning.ai/lightning-ai/studios/understanding-using-and-finetuning-gemma\n\nBy Sebastian Raschka\n\n1) Gemma uses a really large vocabulary and consequently embedding weight matrices.\n\n2) Like OLMo and GPT-2, Gemma also uses weight tying.\n\n3) It even uses GeLU (in the form of GeGLU), similar to GPT-2, unlike Llama 2 and other LLMs.\n\n4) Interestingly, Gemma normalizes the embeddings by the square root of the hidden layer dimension.\n\n5) The RMSNorm layers are in the usual location, different from what was hinted at in the technical paper.\n\n6) However, the RMSNorm implementation comes with the unit offset that wasn't mentioned in the technical paper."
3,"[10:18 pm, 24/02/2024] +91 99861 97355:",\n Activation function \n,+91 77378 87058,What's GeGLU?
4,"[10:19 pm, 24/02/2024] +91 97730 65092:",\n Sigmoid+relu \n,+91 77378 87058,What's GeGLU?
...,...,...,...,...
228,"[2:27 pm, 27/02/2024] +91 88677 05880:",\n Do you have indic data? \n,+91 99151 23897,"On ASR: if you can sponsor compute, I will finetune conformer and whisper models and open source them. No TnC"
229,"[2:27 pm, 27/02/2024] +91 77378 87058:",\n Not aware of branding specifics — we're the technical people broooo \n,+91 99524 65050,Not aware of the technicalities
230,"[2:28 pm, 27/02/2024] +91 99524 65050:",😝,+91 77378 87058,Not aware of branding specifics — we're the technical people broooo
231,"[3:12 pm, 27/02/2024] +91 88678 52604:","\n I'm in contact with some folks at E2E, they are open to sponsoring some compute for small projects \n",+91 77378 87058,"cc @ ~Jibin Sabu \n\nThis is Rs. 100/hour on E2E — so Rs. 1K/hour for say, 10 hours? I'll sponsor this."


In [251]:
#This is just to validate that the data has been extracted properly. We do a bunch of validations such as author is present, messsage is present, and so on.

matching_rows = df[df['message_author'].str.contains("Not Found", na=False)]
if not matching_rows.empty:
  print(f"Messages where author could not be extracted")
  display(matching_rows)

matching_rows = df[df['quoted_text'].str.contains("Not Found", na=False)]
if not matching_rows.empty:
  print(f"Messages where quoted text could not be extracted")
  display(matching_rows)

matching_rows = df[df['quoted_author'].str.contains("Not Found", na=False)]
if not matching_rows.empty:
  print(f"Messages where quoted author could not be extracted")
  display(matching_rows)

# Filter condition
# Condition for quoted_text being null or empty and quoted_author having a value
condition1 = ((df['quoted_text'].isnull() | (df['quoted_text'] == '')) & df['quoted_author'].notnull() & (df['quoted_author'] != ''))

# Condition for quoted_author being null or empty and quoted_text having a value
condition2 = ((df['quoted_author'].isnull() | (df['quoted_author'] == '')) & df['quoted_text'].notnull() & (df['quoted_text'] != ''))

# Combining both conditions using the | operator for OR
filtered_df = df[condition1 | condition2]

if not filtered_df.empty:
  print(f"Messages where quoted author is present but message is not or viceversa")
  display(filtered_df)



Messages where quoted text could not be extracted

,message_author,text,quoted_author,quoted_text
190,"[10:57 am, 27/02/2024] +91 77378 87058:",\n cc @ ~Aashay @ ~Harveen would you happen to know someone better than Pratik dada at Indic TTS? \n,+1 (937) 708-1307,Not Found


In [252]:
#pd.set_option('display.max_colwidth', None)

#This is to remove "\n and spaces" in the text.
df['cleaned_text'] = df['text'].str.replace(r'\n\s*', ' ', regex=True).str.strip()
df['cleaned_quoted_text'] = df['quoted_text'].str.replace(r'\n\s*', ' ', regex=True).str.strip()

# Regular expression to extract time, date, and sender
# Assumes date is in DD/MM/YYYY format and time is in [H]H:MM {am/pm} format
sender_extraction_pattern = r'\[(\d{1,2}:\d{2} [ap]m), (\d{2}/\d{2}/\d{4})\] (.+):'

# Extracting the parts into new columns
df[['timestamp', 'date', 'sender']] = df['message_author'].str.extract(sender_extraction_pattern)

display(df)

,message_author,text,quoted_author,quoted_text,cleaned_text,cleaned_quoted_text,timestamp,date,sender
0,"[9:31 pm, 24/02/2024] +1 (206) 499-3617:",\n Can you share how you did that? Which VLM? \n,+91 97730 65092,For us we just FTed a VLM,Can you share how you did that? Which VLM?,For us we just FTed a VLM,9:31 pm,24/02/2024,+1 (206) 499-3617
1,"[10:07 pm, 24/02/2024] Bharat Shetty:","\n https://lightning.ai/lightning-ai/studios/understanding-using-and-finetuning-gemma \n\nBy Sebastian Raschka\n\n1) Gemma uses a really large vocabulary and consequently embedding weight matrices.\n\n2) Like OLMo and GPT-2, Gemma also uses weight tying.\n\n3) It even uses GeLU (in the form of GeGLU), similar to GPT-2, unlike Llama 2 and other LLMs.\n\n4) Interestingly, Gemma normalizes the embeddings by the square root of the hidden layer dimension.\n\n5) The RMSNorm layers are in the usual location, different from what was hinted at in the technical paper.\n\n6) However, the RMSNorm implementation comes with the unit offset that wasn't mentioned in the technical paper. \n",,,"https://lightning.ai/lightning-ai/studios/understanding-using-and-finetuning-gemma By Sebastian Raschka 1) Gemma uses a really large vocabulary and consequently embedding weight matrices. 2) Like OLMo and GPT-2, Gemma also uses weight tying. 3) It even uses GeLU (in the form of GeGLU), similar to GPT-2, unlike Llama 2 and other LLMs. 4) Interestingly, Gemma normalizes the embeddings by the square root of the hidden layer dimension. 5) The RMSNorm layers are in the usual location, different from what was hinted at in the technical paper. 6) However, the RMSNorm implementation comes with the unit offset that wasn't mentioned in the technical paper.",,10:07 pm,24/02/2024,Bharat Shetty
2,"[10:18 pm, 24/02/2024] +91 77378 87058:",\n What's GeGLU? \n,Bharat Shetty,"https://lightning.ai/lightning-ai/studios/understanding-using-and-finetuning-gemma\n\nBy Sebastian Raschka\n\n1) Gemma uses a really large vocabulary and consequently embedding weight matrices.\n\n2) Like OLMo and GPT-2, Gemma also uses weight tying.\n\n3) It even uses GeLU (in the form of GeGLU), similar to GPT-2, unlike Llama 2 and other LLMs.\n\n4) Interestingly, Gemma normalizes the embeddings by the square root of the hidden layer dimension.\n\n5) The RMSNorm layers are in the usual location, different from what was hinted at in the technical paper.\n\n6) However, the RMSNorm implementation comes with the unit offset that wasn't mentioned in the technical paper.",What's GeGLU?,"https://lightning.ai/lightning-ai/studios/understanding-using-and-finetuning-gemma By Sebastian Raschka 1) Gemma uses a really large vocabulary and consequently embedding weight matrices. 2) Like OLMo and GPT-2, Gemma also uses weight tying. 3) It even uses GeLU (in the form of GeGLU), similar to GPT-2, unlike Llama 2 and other LLMs. 4) Interestingly, Gemma normalizes the embeddings by the square root of the hidden layer dimension. 5) The RMSNorm layers are in the usual location, different from what was hinted at in the technical paper. 6) However, the RMSNorm implementation comes with the unit offset that wasn't mentioned in the technical paper.",10:18 pm,24/02/2024,+91 77378 87058
3,"[10:18 pm, 24/02/2024] +91 99861 97355:",\n Activation function \n,+91 77378 87058,What's GeGLU?,Activation function,What's GeGLU?,10:18 pm,24/02/2024,+91 99861 97355
4,"[10:19 pm, 24/02/2024] +91 97730 65092:",\n Sigmoid+relu \n,+91 77378 87058,What's GeGLU?,Sigmoid+relu,What's GeGLU?,10:19 pm,24/02/2024,+91 97730 65092
...,...,...,...,...,...,...,...,...,...
228,"[2:27 pm, 27/02/2024] +91 88677 05880:",\n Do you have indic data? \n,+91 99151 23897,"On ASR: if you can sponsor compute, I will finetune conformer and whisper models and open source them. No TnC",Do you have indic data?,"On ASR: if you can sponsor compute, I will finetune conformer and whisper models and open source them. No TnC",2:27 pm,27/02/2024,+91 88677 05880
229,"[2:27 pm, 27/02/2024] +91 7

In [253]:
!pip install requests
import requests

In [258]:

def mistral_huggingface_call(prompt):
  """
  Make a call to Mistral on huggingface for some simple inferences.
  """

  # Your Hugging Face API key
  api_key = os.getenv('HUGGINGFACE_TOKEN')

  # The API URL for the specific model
  api_url = "https://api-inference.huggingface.co/models/mistralai/Mixtral-8x7B-Instruct-v0.1"

  # Headers for the POST request
  headers = {
      "Authorization": f"Bearer {api_key}"
  }

  payload = {
      "inputs": f"<s>[INST] {prompt} [/INST]",
      "parameters": {
          "temperature": 0.1,
          "max_new_tokens": 100
      },
  }

  # Send the POST request
  response = requests.post(api_url, headers=headers, json=payload)
  model_response_value = None
  # Check if the request was successful
  if response.status_code == 200:
      # Get the model's response
      model_response = response.json()
      generated_text = model_response[0]['generated_text']

      if(generated_text):
        last_index = generated_text.rfind("[/INST]")
        if last_index != -1:
            model_response_value = generated_text[last_index + len("[/INST]"):]  # Extract text after the marker
            return model_response_value
  else:
      print(f"Request failed with status code {response.status_code}: {response.text}")
  return None;



def are_messages_related(message1, message2):
  """
  Used to check if a solo message is related to the previous message in any way. Using LLM for it.
  """

  prompt =  f"I have two consequtive messages in a whatsapp group chat. I want to know if the second message is a reply or in context of the first message. ONLY respond with 'Yes' and if its not respond with 'No'. message1 = '{message1}' message2 = '{message2}'"
  model_response_value = mistral_huggingface_call(prompt)

  if model_response_value:
    if "yes" in model_response_value.lower() and "no" not in  model_response_value.lower():
      #print(f"similarity b/w t1: {message1} \nt2: {message2} \nTrue --- {model_response_value}\n")
      return True;
  return False;



def get_url_summary(discussion_text):
  """
  Generate a summary for a short discussions for generating text in quick links.
  """

  prompt = f"Here is some discussion text with URLs in them. Create a summary under 10 words for it. Output is just the summary and NOTHING ELSE Discussion \n--------\n{discussion_text}"
  model_response_value = mistral_huggingface_call(prompt)
  return model_response_value


are_messages_related("I feel continual pretraining on gemma for different languages will be more effective when compared to llama+extended vocabulary","Why? I didnt seem tonfigure this out")
#are_messages_related("I want to go to office.", "How is the food in Empire?")

True

In [259]:
from difflib import SequenceMatcher
from datetime import datetime, timedelta
from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

threads = []

def find_thread_for_message(quoted_author, quoted_text, threads):
    """
    Find an existing thread that matches the quoted author and text.
    Returns the thread and parent message if a match is found, otherwise None.
    """
    for thread in threads:
        for message in thread['messages']:
            matched_message = find_if_quote_exists(quoted_author, quoted_text, message)
            if matched_message:
                return thread, matched_message
    return None, None

def find_if_quote_exists(quoted_author, quoted_text, message):
    if quoted_author == message['sender'] and similar(quoted_text, message['text']):
        return message
    for reply_message in message['replies']:
        matched_message = find_if_quote_exists(quoted_author, quoted_text, reply_message)
        if matched_message:
            return matched_message
    return None


def similar(text1, text2):
    """
    Checks if the first 100 characters of two texts are similar.
    """
    # Slice the first 100 characters of each string
    text1 = text1[:100]
    text2 = text2[:100]

    # Calculate similarity ratio
    ratio = SequenceMatcher(None, text1, text2).ratio()
    return ratio > 0.8  # Consider them similar if the ratio is greater than 0.8


def parse_datetime(row):
    """Parse the 'date' and 'time' columns to a datetime object."""
    date_str = row['date']
    time_str = row['timestamp']
    # Correct the hour part if it's '0:' with '12:' for AM times
    if time_str.startswith('0:'):
        time_str = '12' + time_str[1:]

    datetime_str = f"{date_str} {time_str}"
    # Now, parse with the corrected format
    return datetime.strptime(datetime_str, '%d/%m/%Y %I:%M %p')

def format_thread_hierarchy(thread, level=0):
    """
    Formats each thread with messages and their replies, indented according to hierarchy level, and returns the result as a string.
    """
    result = ""
    # Add the root thread marker only at the top level
    for message in thread['messages']:
      result += format_message_hierarchy(message, level + 1)
    return result

def format_message_hierarchy(message, level):
    """
    Recursively formats messages and their replies with indentation to reflect hierarchy and returns the formatted string.
    """
    # Creating indentation based on the current hierarchy level
    indent = '    ' * level  # Increase indentation for each level
    formatted_message = f"{indent}- {message['text']}\n"
    for reply in message['replies']:
        formatted_message += format_message_hierarchy(reply, level + 1)
    return formatted_message

def print_thread_hierarchy(threads, level=0):
    """
    Prints each thread with messages and their replies, indented according to hierarchy level.
    """
    for thread in threads:
        # Print the root thread marker only at the top level
        if level == 0:
            printmd("##Thread")
        for message in thread['messages']:
            print_message_hierarchy(message, level + 1)

def print_message_hierarchy(message, level):
    """
    Recursively prints messages and their replies with indentation to reflect hierarchy.
    """
    # Creating indentation based on the current hierarchy level
    indent = '    ' * level  # Increase indentation for each level
    printmd(f"{indent}- {message['sender']}: {message['text']}")
    for reply in message['replies']:
        print_message_hierarchy(reply, level + 1)



In [260]:
last_sender = None
last_datetime = None
last_thread = None
last_message = None

for index, row in df.iterrows():
    current_datetime = parse_datetime(row)
    message = {'sender': row['sender'], 'text': row['cleaned_text'], 'replies': []}
    root_thread = {'messages': [message]}
    current_thread = None;
    if row['cleaned_quoted_text'] and row['quoted_author']:
        # If there is a quoted text, find the message which is quoted and add this as a child.
        thread, parent_message = find_thread_for_message(row['quoted_author'], row['cleaned_quoted_text'], threads)
        if thread:
            # Add as a sub-message to the existing thread
            parent_message['replies'].append(message)
            current_thread = thread;
    elif last_datetime and (current_datetime - last_datetime <= timedelta(minutes=2)):
        if last_sender == row['sender']:
          # Add message as a child to the last message of the previous sender's thread if the same sender has sent more than one message in a span of 2 mins.
          last_message['replies'].append(message)
          current_thread = last_thread;
        elif are_messages_related(format_thread_hierarchy(last_thread), message['text']) == True:
          # Add message as a child to the last message of the previous sender's thread if the LLM believes they are related.
          last_message['replies'].append(message)
          current_thread = last_thread;

    if not current_thread:
        # If no matching thread is found, start a new thread
        threads.append(root_thread)
        current_thread = root_thread

    last_sender = row['sender']
    last_datetime = current_datetime
    last_message = message
    last_thread = current_thread

# Print the thread list if you want to see how it looks.
#print_thread_hierarchy(threads)



In [231]:
import re

def contains_hyperlink(text):
    # This regular expression pattern matches most URLs
    pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    if re.search(pattern, text):
        return True
    else:
        return False

def extract_urls(text):
    # Regex pattern to extract URLs
    url_pattern = r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+[\w/\-?=%.]+\b'
    # Find all matches in the text
    urls = re.findall(url_pattern, text)
    return urls

def get_thread_message_count(thread):
  message_count = 0
  for message in thread['messages']:
    message_count = message_count + get_message_count(message)
  return message_count

def get_message_count(message):
  message_count = 1
  for reply in message['replies']:
    message_count = message_count + get_message_count(reply);
  return message_count




In [232]:
#Threads with no messages
ignored_threads = []

#Threads with less than <Threshold> messages
small_threads = []

#Threads with less than <Threshold> messages but have hyperlinks in it.
small_threads_with_links = []

#Threads with more than <Threshold> messages
threads_to_summarise = []

#Find the number of messages in a thread and use it for categorising it.
for thread in threads:
  message_count = get_thread_message_count(thread)
  if message_count == 0:
    ignored_threads.append(thread)
    continue
  elif message_count <= 4:
    if contains_hyperlink(format_thread_hierarchy(thread)):
      small_threads_with_links.append(thread)
    else:
      small_threads.append(thread)
  else:
    threads_to_summarise.append(thread)

print(f"Ignored threaads - {len(ignored_threads)}\n-------------------------------")
print(f"Small threaads - {len(small_threads)}\n-------------------------------")
print(f"Small threaads with links - {len(small_threads_with_links)}\n-------------------------------")
print(f"Threads to summarise - {len(threads_to_summarise)}\n-------------------------------")


Ignored threaads - 0
-------------------------------

Small threaads - 24
-------------------------------

Small threaads with links - 16
-------------------------------

Threads to summarise - 16
-------------------------------

In [233]:
!pip install openai

In [234]:
from openai import OpenAI

# Get the OpenAI API key from the environment variable
api_key = os.getenv('OPENAI_API_KEY')

# Check if the API key is set
if not api_key:
    raise ValueError('OpenAI API key not set. Please set the OPENAI_API_KEY environment variable.')
client = OpenAI(api_key=api_key)


In [235]:
def summarise_thread(existing_summary,thread_data):
  """
  Makes a call to OpenAI using function calling. It summarises the given thread as a discussion, short qna, Link or NA.
  Only the threads that go above a threshold are sent to OpenAI.
  """
  response = client.chat.completions.create(
    model="gpt-4-turbo-preview",
    messages=[
      {
        "role": "system",
        "content": "You are an AI Expert and also an expert in summarising insights from conversations from an AI forum.\n\nINSTRUCTIONS\nThe input consists of two parts. \n1. Existing Summary\n2. Thread content.\n\n1. The existing summary contains the summary generated from the threads seen so far. Initially the summary is empty and as and when threads are summarised, it gets populated. The summary can be of two types.\na) Trending Discussion - This is used to extract learnings from long conversation threads. The summarised learning have a title (under 5 words), and crisp bullet points (7-8 max) (WITH INLINE HYPERLINKS), summarising the learnings from the Thread Content. Each bullet point should not be more than 10 words long.  It should make sense for a reader to get the learnings by reading the summary.\nb) Short QnA - If the thread is short and in a question answer format, summarise the question and the answer(s) provided.\n\n2. Thread Content\nIt is the raw thread conversation. The indentation in the thread describes replies to each message that was sent.\n\nYour task is to take the entire Thread Content and convert it into a Trending Discussions or Short QnA or Link (ONLY ONE OF THEM). You can either add it to an existing summary item or create a new summary item and output the new Existing Summary. Some threads might be short and irrelevant. You can ignore them.\n"
      },
      {
        "role": "user",
        "content": f"\nDATA\n\n{existing_summary}\n\nThread Content\n-------------------------------------------\n{thread_data}\n"
      }
    ],
    functions=[
      {
        "name": "get_summary",
        "description": "Get the summary from the Thread content given.",
        "parameters": {
          "type": "object",
          "properties": {
            "summary_text": {
              "type": "string",
              "description": "The actual summary string generated as per the instruction along with a title if its a Trending Discussion. Do not miss the hyperlinks."
            },
            "summary_type": {
              "type": "string",
              "description": "The type of the summary generated. Use NA if the thread discussions are short or irrelevant and can be ignored.",
              "enum": [
                "Trending Discussion",
                "Short QnA",
                "NA"
              ]
            }
          },
          "required": [
            "summary_text",
            "summary_type"
          ]
        }
      }
    ],
    temperature=1,
    max_tokens=4095,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
  )
  #print(response)
  if response.choices and response.choices[0].message and response.choices[0].finish_reason == 'function_call':
    function_response = response.choices[0].message.function_call.arguments
    function_name = response.choices[0].message.function_call.name
    return function_name, function_response
  else:
   return None, None


In [236]:
#Sample summary for the model to learn. Comment out if you want to give no summary.
existing_summary = "Existing Summary\n------------------------------------------\n#Trending Discussions\n\n## GitHub Copilot from India\n* This has huge potential if we can make it relelvent to existing code context.\n* Bloop, Onboard AI, sweep are already in the space. This is crowded.\n* Cursor is better than Copilot. https://cursor.sh/\n* Codeium is also an alternative to explore. https://codeium.com/\n\n# Short QnA\nQ. What is licensing model of Llava models*\nA. Code is apache. Model's is derived from base. https://github.com/haotian-liu/LLaVA/blob/main/LICENSE \n\n#Links\n* Mistrals new model. - https://twitter.com/mattshumer_/status/1758589445366763975"
#existing_summary = "Existing Summary\n------------------------------------------\n <None>\n\n"
print(f"Number of threads to summarise: {len(threads_to_summarise)}")

summary_details = []

for thread in tqdm(threads_to_summarise):
  formatted_output = format_thread_hierarchy(thread)
  function_name, summary = summarise_thread(existing_summary, formatted_output)
  if function_name == 'get_summary':
    summary_json = json.loads(summary)
    summary_details.append({'thread' : thread, 'summary_type': summary_json['summary_type'], 'summary_text': summary_json['summary_text']})

#print(summary_details)

Number of threads to summarise: 16

100%|██████████| 16/16 [01:13<00:00,  4.60s/it]


In [237]:
# For the Discussions, see if some urls have been shared in the thread but not in the summary. If so, add a point with
# all the urls that are missed in the summary. This is for completeness.
for summary in summary_details:
  thread_discussion_text = format_thread_hierarchy(summary['thread'])
  urls = extract_urls(thread_discussion_text)
  # Find URLs not present in the text
  urls_not_in_text = [url for url in urls if url not in summary['summary_text']]
  for url in urls_not_in_text:
    url_summary = "* URLs  - " + ', '.join(urls_not_in_text)
    summary['summary_text'] = summary['summary_text'] + "\n" + url_summary

In [238]:
for thread in tqdm(small_threads_with_links):
  thread_discussion_text = format_thread_hierarchy(thread)
  thread_summary = "* " + get_url_summary(thread_discussion_text) + " - " + ', '.join(extract_urls(thread_discussion_text))
  summary_details.append({'thread' : thread, 'summary_type': "Links", 'summary_text': thread_summary})



100%|██████████| 16/16 [00:03<00:00,  4.52it/s]


In [239]:
trending_discussions = "";
qna = ""
links = ""

for summary in summary_details:
  if summary['summary_type'] == "Trending Discussion":
    trending_discussions = trending_discussions + "\n\n" + summary['summary_text']
  if summary['summary_type'] == "Links":
    if contains_hyperlink(summary['summary_text']):
      links = links + "\n\n" + summary['summary_text']
  if summary['summary_type'] == "Short QnA":
    qna = qna + "\n\n" + summary['summary_text']


printmd(f"#Trending Discussions \n {trending_discussions}")
printmd(f"#Short QnA \n {qna}")
printmd(f"#Cool Links \n {links}")

#Trending Discussions 
 

## Gemma vs. Llama Comparison
* Gemma's large vocabulary and dataset outperform Llama. https://lightning.ai/lightning-ai/studios/understanding-using-and-finetuning-gemma
* GeGLU used in Gemma, akin to GPT-2's activation functions. https://paperswithcode.com/method/geglu
* Continual pretraining with Gemma potentially more effective for multilingual support.
* Concerns over vocabulary extension and efficiency in tokenizing new languages.
* Experimentation suggested to understand performance against Llama2. 
* Proposed training pipeline for integrating new tokens into Gemma's vocabulary. https://x.com/johnhewtt/status/1761056178988073319?s=46

## Efficient NLP Processing for Streaming Voice Data
* Using Whisper for STT and Mistral/GPT-3.5 every n words. 
* Mistral vs. spaCy entity extraction: unexplored by user. 
* spaCy NER 15% better than previous best. https://spacy.io/api/large-language-models#ner 
* Streaming function calling from OpenAI Models for speed.

## Compound AI Systems Insights
* Stand-alone models not the only constraint for great AI systems.
* Great systems: model + architecture + ops + sw engg. https://bair.berkeley.edu/blog/2024/02/18/compound-ai-systems/
* Dumb LLM + smart scaffolding > smart LLM without scaffolding.
* Usage of LLMs must justify gtm speed, cost, ROI.
* The importance of the system over model emphasized.
* "Thought to action" significant for future AI development.
* Success in AI: perfect alchemy of all tools.
* URLs  - http://bair.berkeley.edu/blog/2024/02/18/compound-ai-systems, https://varunshenoy.substack.com/p/natural-language-is-an-unnatural
* URLs  - http://bair.berkeley.edu/blog/2024/02/18/compound-ai-systems, https://varunshenoy.substack.com/p/natural-language-is-an-unnatural

## Krutrim Overview
* Users experiencing slow responses, questioning user load.
* Critiques label it a half-cooked product.
* Discussion on "Indian values" veers into satire.

## GitHub Copilot from India
* This has huge potential if we can make it relelvent to existing code context.
* Bloop, Onboard AI, sweep are already in the space. This is crowded.
* Cursor is better than Copilot. https://cursor.sh/
* Codeium is also an alternative to explore. https://codeium.com/
* Positive to see grassroots-level initiatives from India.
* Emphasizes adapting AI to Indian cultural contexts.
* Criticism of overhype but acknowledges potential for improvement.

## AI Rerankers Overview
* Cohere and BGE are noted as decent rerankers.
* Cohere outperforms BGE, especially with OpenAI embeddings.
* BGE's performance issue on Hugging Face due to infrastructure.
* Sentence Transformers and Jina working on MRL models.
* https://huggingface.co/nomic-ai/nomic-embed-text-v1.5 Nomic AI's MRL model.
* https://x.com/tomaarsen/status/1761020783336235186?s=20 Sentence Transformers' MRL work.
* Cohere's reranker exceptional on both MRR and Hit rate.
* URLs  - https://platform.openai.com/docs/guides/embeddings

## Hackathon Chatbot Prototype
* Create a custom GPT for game mechanics isolation. 
* Model as retrieval augmented generation system. 
* Perplexity Chat API for faster POC development. 

## Measuring Copilot Efficacy
* Character consistency key in copilot evaluation. https://www.scenario.com/
* Genid approach used for measuring experience.
* Dall-E character consistency mentioned.
* Stable diffusion alternative for cost-effective generation.
* Scenario and Dashtoon enable character consistency with Lora’s and IP adapters.
* Seed and prompt sharing boost consistency in character generation.
* Perplexity innovates in copilot UX with ChatGPT-like interface.
* Effectiveness measurement in early stages; exploratory methods dominate.

* Mistral AI is directly competing with ChatGPT. [Le Chat Mistral](https://mistral.ai/news/le-chat-mistral/) 
* Targets European languages: 'English, French, Spanish, German, Italian'.
* Surprising coherent outputs in other languages noted.
* Discussion on embedding models for European languages.

## Future of Programming Languages
* RUST replaces C++. [White House Report](https://www.whitehouse.gov/oncd/briefing-room/2024/02/26/press-release-technical-report/)
* COBOL still mentioned.
* AI code co-pilots could simplify learning Rust.
* Support for PHP's demise.

## Gemini 1.5 Pro's Game Changer
* Records, identifies & fixes bugs from videos. [Tweet](https://twitter.com/mckaywrigley/status/1762175776726663267?t=4XpMkJltHFB3yC-vHsf8Zg&s=19)
* Demo to ship speed is fastest ever.
* Predicted to disrupt the industry.
* Integration with OBS could enhance automation.

## Indic TTS Innovations
* Discussing potential in Haryanwi, Bhojpuri TTS.
* Dubverse exploring Indic TTS. https://black.dubverse.ai/p/research-to-production-neodub?subscribe_prompt=free
* A model to be released on Azure.
* Uncertainty about Azure branding specifics.

## Indian Language AI Development
* **AI TTS** for Bhojpuri lacks market incentive.
* **Eleven Labs** pricing spurs Indian TTS interest, mainly **Hindi, Tamil, Telugu**.
* Volunteer effort to create open source **TTS** and **ASR** models.
* Projects include **Swecha** collaboration, possibly an **open source college consortium**.
* **Volunteer-driven compute cluster** initiative for model finetuning.
* **Ozonetel** and individuals pledge compute resources.
* Efforts targeting **Hindi, Tamil, Telugu** could benefit from higher purchasing power regions.
* Not seen as needing VC funding; more of a **hacker project**.

## Llama Models Hosting Variances
* Minor differences noted between providers. 
* LLM config varies for structured data use cases. 
* Quantization may affect responses slightly. 
* Providers claim to use fp16 without degradation.

#Short QnA 
 

#Cool Links 
 

*  New computing paradigm: Context-aware, interchangeable mediums. - https://twitter.com/The_AI_Investor/status/1760921566408507529?s=20

*  "Image Generation Study Guide: Hexo's Resource" - https://hexoai.notion.site/Image-Generation-Study-Guide-97b81731c2c64117a54e50cda6149ed8?pvs=4

*  "Feather by OpenAI; Reddit users find it interesting." - https://feather.openai.com, https://www.reddit.com/r/OpenAI/comments/17ybes0/comment/k9tc04x

*  "Lamini AI Offers Startup Credits for GPU & DPO Implementations." - https://docs.google.com/forms/d/e/1FAIpQLSdiUGjIkzJfplThk1285D3RFyvXS8OlWeJ2Hg-HAcBciGcNxw/viewform

*  "New paper explores reasoning in artificial intelligence." - https://arxiv.org/pdf/2402.10200.pdf

*  GraphRAG Improves LLM Discovery on Private Data. - https://www.microsoft.com/en-us/research/blog/graphrag-unlocking-llm-discovery-on-narrative-private-data

*  Brett Adcock: SpaceX's Starship Prototype Explodes During Test. - https://twitter.com/adcock_brett/status/1761814066748670071

*  "Mamba: A New Programming Language; HN Discussion"
(Note: The summary is exactly 10 words long, including the ellipsis.) - https://jackcook.com/2024/02/23/mamba.html, https://news.ycombinator.com/item?id=39482428

*  Microsoft's LongRoPE Extends LLM Context Window to 2 Million Tokens. - https://medium.com/syncedreview/microsofts-longrope-breaks-the-limit-of-context-window-of-llms-extents-it-to-2-million-tokens-eb43efdbadff

*  "Ola Founder Releases LLM Program Details" - https://chat.olakrutrim.com/home

*  "Arthur Mensch Shares Thoughts on Unknown"

(Note: The summary is based on the information provided, which only mentions a link to Arthur Mensch's status update on a platform with the domain x.com. The actual content of the status update is not accessible, so the summary is necessarily limited to what is known from the context given.) - https://x.com/arthurmensch/status/1762121295330725965?t=BLYLr2dlQC0cx-uFOTs93A

*  DeepMind's Interactive Video Generation Model Training Details - https://sites.google.com/view/genie-2024/home

*  "Exploring Google's Text-to-Video Model Lumiere at Vision-Language Club Meetup" - https://twitter.com/hila_chefer/status/1761792814868750745, https://technion.zoom.us/j/98830698694

*  RAG pipeline generating responses; Prompt engineering used. - https://x.com/Krutrim/status/1762165852194189400?s=20

*  "Discussion on model fine-tuning after pretraining." - https://twitter.com/Krutrim/status/1762173810328146283?t=sO0JXYsqsVgJREhnyBpLGw

*  "LLM Time Capsule Tracks Latest Large Language Models." - https://llmtimecapsule.octo.ai

In [240]:
# These are small threads and they do not have any links in it. Needs to be manually weeded. They might also be image threads without context.
# The best way (if we find something interesting here) is to go and search for the exact message on the group to see the context around it.
for thread in small_threads:
  print_thread_hierarchy([thread])


##Thread

    - +1 (206) 499-3617: Can you share how you did that? Which VLM?

##Thread

    - +91 96386 27147: I have a question about how autoregressive models (AR) work, let's say GPT-2. We have an input string of 500 words(or 740 tokens).  AR model reads one window of words and starts predicting the output without reading the whole input, the model keeps on sequentially reading the windows and predicting the output.  AR model reads the whole input in one go and start predicting the output until EOS token is predicted.

        - +91 77378 87058: The second. That's why the input tokens can't be streamed easily.

...

##Thread

    - +1 (937) 708-1307: Do we have anyone here successful at tuning openTTS, Coqui or similar model for Indic voices and deploying efficiently for production?

##Thread

    - +91 77378 87058: Not aware of branding specifics — we're the technical people broooo

        - +91 99524 65050: 😝